In [2]:
import os
os.environ['HF_ENDPOINT']='https://hf-mirror.com'
os.environ['CUDA_VISIBLE_DEVICES']='2'
import torch
import datasets
import transformers
from transformers import AutoModelForQuestionAnswering,AutoTokenizer
from nn_pruning.patch_coordinator import ModelPatchingCoordinator
from nn_pruning.model_patcher import ModelPatcher
import torch
from nn_pruning.patch_coordinator import SparseTrainingArguments

datasets.logging.set_verbosity_error()
transformers.logging.set_verbosity_error()
print(f"Using transformers v{transformers.__version__} and datasets v{datasets.__version__} and torch v{torch.__version__}")

/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using transformers v4.34.0 and datasets v3.0.1 and torch v2.0.1+cu117


In [3]:
model_name = "gpt2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
model.config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.34.0",
  "use_cache": true,
  "vocab_size": 50257
}

In [5]:
for name,m in model.named_modules():
    print(name)
    if(isinstance(m,torch.nn.Linear)):
        print(name)


transformer
transformer.wte
transformer.wpe
transformer.drop
transformer.h
transformer.h.0
transformer.h.0.ln_1
transformer.h.0.attn
transformer.h.0.attn.c_attn
transformer.h.0.attn.c_proj
transformer.h.0.attn.attn_dropout
transformer.h.0.attn.resid_dropout
transformer.h.0.ln_2
transformer.h.0.mlp
transformer.h.0.mlp.c_fc
transformer.h.0.mlp.c_proj
transformer.h.0.mlp.act
transformer.h.0.mlp.dropout
transformer.h.1
transformer.h.1.ln_1
transformer.h.1.attn
transformer.h.1.attn.c_attn
transformer.h.1.attn.c_proj
transformer.h.1.attn.attn_dropout
transformer.h.1.attn.resid_dropout
transformer.h.1.ln_2
transformer.h.1.mlp
transformer.h.1.mlp.c_fc
transformer.h.1.mlp.c_proj
transformer.h.1.mlp.act
transformer.h.1.mlp.dropout
transformer.h.2
transformer.h.2.ln_1
transformer.h.2.attn
transformer.h.2.attn.c_attn
transformer.h.2.attn.c_proj
transformer.h.2.attn.attn_dropout
transformer.h.2.attn.resid_dropout
transformer.h.2.ln_2
transformer.h.2.mlp
transformer.h.2.mlp.c_fc
transformer.h.2.mlp

In [6]:
model

GPT2ForQuestionAnswering(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (qa_outputs): Linear(in_features=768, out_features=2, bias=True)
)

In [6]:
import torch
from transformers import GPT2LMHeadModel, GPT2Config
from transformers.pytorch_utils import Conv1D

# 加载预训练模型

def replace_conv1d_with_linear(model):
    for name, module in model.named_modules():
        if isinstance(module, Conv1D):
            # 获取原始Conv1D层的参数
            weight = module.weight
            bias = module.bias
            
            # 创建一个新的Linear层
            linear_layer = torch.nn.Linear(weight.size(0), weight.size(1), bias=True)
            
            # 复制权重和偏置
            with torch.no_grad():
                linear_layer.weight.copy_(weight.t())  # 注意这里需要转置
                linear_layer.bias.copy_(bias)
            
            # 替换模型中的Conv1D层
            parent_module, child_name = get_parent_module_and_child_name(model, name)
            setattr(parent_module, child_name, linear_layer)

def get_parent_module_and_child_name(model, full_name):
    parts = full_name.split('.')
    parent = model
    for part in parts[:-1]:
        parent = getattr(parent, part)
    return parent, parts[-1]

In [7]:
replace_conv1d_with_linear(model)

In [8]:
model

GPT2ForQuestionAnswering(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)

In [4]:
sparse_args = SparseTrainingArguments()
sparse_args

SparseTrainingArguments(mask_scores_learning_rate=0.01, dense_pruning_method='topK', attention_pruning_method='topK', ampere_pruning_method='disabled', attention_output_with_dense=True, bias_mask=True, mask_init='constant', mask_scale=0.0, dense_block_rows=1, dense_block_cols=1, attention_block_rows=1, attention_block_cols=1, initial_threshold=1.0, final_threshold=0.5, initial_warmup=1, final_warmup=2, initial_ampere_temperature=0.0, final_ampere_temperature=20.0, regularization='disabled', regularization_final_lambda=0.0, attention_lambda=1.0, dense_lambda=1.0, decoder_attention_lambda=None, decoder_dense_lambda=None, distil_teacher_name_or_path=None, distil_alpha_ce=0.5, distil_alpha_teacher=0.5, distil_temperature=2.0, final_finetune=False, layer_norm_patch=False, layer_norm_patch_steps=50000, layer_norm_patch_start_delta=0.99, gelu_patch=False, gelu_patch_steps=50000, linear_min_parameters=0.005, rewind_model_name_or_path=None, eval_with_current_patch_params=False, qat=False, qconf

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


mpc = ModelPatchingCoordinator(
    sparse_args=sparse_args, 
    device=device, 
    cache_dir="checkpoints", 
    logit_names="logits", 
    model_name_or_path=model_name,
    teacher_constructor=None)

# 对模型进行补丁
patched_model = mpc.patch_model(model)

/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
tokenizer.cls_token_id

In [9]:
from datasets import load_dataset

In [10]:
data_set = load_dataset('squad')

In [6]:
tokenizer

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
questions = ['What Was estimated amount of deaths during the war?', 'What percent of the deaths during the war were civilians?', 'What countries suffered the most casualties in the war?', "What percentage of it's population did Poland lose in the war?", 'How many deaths did the Soviet Union suffer in the war?', 'What is the Holocaust?', 'During what word event did the Holocaust happen? ', 'Who is responsible for the Holocaust? ', 'How did the Nazis conjure up anti-Semitic feelings in civilians?', 'When did the suffering of the Jews begin?', 'What was Europe split into after World War II?', 'Who battled in the cold war?', 'What does the term NATO represent?', 'What did the defeat of Japan mean for Asia?', 'What did China name themselves after the war?', "By what percentage did the world's per-capita gross domestic product grow by during the 20th century?", 'What has Modern medicine increased the average life expectancy by?', 'What happened to the income gap between the rich and the poor?', 'What did the media industry make possible in the middle of the century?', 'What did the brutal dictatorships caused by the media lead to?', 'What did increased communications lead contribute to?', 'What time period did technological advances lead to?', "what did the Soviet Union annex some of it's territory as?", 'What did the territory the Soviet Union maintained as satellite states, later became known as?', 'The United States and many Western countries began what policy to fight communism?', 'What did the soviets opposed rebuilding?', 'What actions of the Soviet Union did The United States oppose?', 'Through what time period did the Berlin Blockade  last?', 'What types of periods did the cold war see?', 'Through what time period did the Korean War last?', 'Through what time period did the Vietnam war last?', 'When did the Cuban Missile crisis happen?', 'During what time was the cold war close to happening?', 'What did  the Reagan administration increase on the Soviet Union?', "What was  the Soviet Union suffering from in the 1980's?", 'What did Mikhail Gorbachev introduce?', 'In what year did the Soviet Union collapse?', "What caused Latin America's right-wing authorities to support coup o'etats?", "Why did the Latin Americas Upper class support coup o'etats?", 'The regimes in the southern cone collaborated in killing of who?', "What did all countries restore in the 1990's ", 'Where did the United States support dictatorships in the 1970"s?', 'What is the space age?', 'Who launched  Sputnik 1?', 'How long does did it take for the Sputnik 1 to orbit the earth?', 'What did Sputnik 1 usher in?', 'How many people watched the Apollo 11 landing?', 'What types of control are considered when labeling a country with the perjorative "51st state" label?', 'Sometimes, when a country refers to themselves as the 51st state, what has happened to their culture?', 'What reasoning would cause a country to be referred to as the "51st state" in a positive way?', 'What is a positive connotation of the label "51st state"?', 'What is a reason for one negative connotation of the term?', 'If a country refers to themselves as the "51st state" after cultural changes, what can we assume has changed?', 'What does Article IV, Section three of the US Constitution outline?', 'Article IV, Section Three of th United States Constitution gives Congress the power to do what?', 'What does the states\' requirement to give "full faith and credit" help recognize?', 'What are all states guaranteed by the federal government?', 'What does Article 4, Section 4 guarantee?', 'What part of the US Constitution outlines the relationship among the states?', 'What does the concept of "full faith and credit" protect?', 'Article IV, Section Three allows Congress to do what?', 'What is guaranteed to the states by the federal government?', 'What country has been regarded as a potential new addition to the United States?', 'What percentage of voters expressed dissatisfaction with their current political relationship?', 'What percentage of voters supported statehood?', 'When did Puerto Rico resolve to request that the United States act on these results?', 'What would be an ideal reaction from the United States, according to Puerto Rico?', 'What territory has been suggested as a new addition to the United States?', 'What percentages of voters expressed unhappiness with the current political relationship between the U.S. and Puerto Rico?', 'When did the legislature of Puerto Rico request that they be considered for statehood?', 'How long has Puerto Rico had representation in Congress?', 'Who represents Puerto Rico in Congress?', 'Which Congress gave the commissioner the power to vote?', 'What is the commissioner not allowed to vote on?', 'What types of taxes do Puerto Ricans pay?', 'What makes up the federal taxes Puerto Rican citizens pay?', 'How else can they serve the U.S.?', 'How many Puerto Ricans have died in the Iraq War and War in Afghanistan?', 'In the aftermath of which war was the U.S. granted Puerto Rico?', 'What country handed over Puerto Rico to the US?', 'How long have Puerto Ricans been U.S. citizens?', 'What form of government do the Puerto Ricans have?', 'How was the Puerto Rican constitution brought about?', 'When was the Puerto Rican constitution written?', 'When was the constitution accepted by the electorate, U.S. Congress, and U.S. President?', 'What clause gives Puerto Rican citizens the same rights as US citizens?', 'How is Puerto Rico designated in its constitution?', 'What title does the country have?', 'What act was approved on April 29th, 2010?', 'What is the island under jurisdiction of?', 'What percentage of voters rejected the status of territory?', 'What percentage of voters preferred statehood?', 'What percentage of voters abstained from voting on a preferred alternative status?', 'What argument do those opposing statehood use?', "What newspaper suggested that Congress would ignore Puerto Rico's referendum?", "What newspapers have published opinion pieces expressing support for Puerto Rico's statehood?", 'When was that article published?', "What Congress members agreed with the Hill's assesment?", 'What politicians pushed for statehood for Puerto Rico?', 'What politician fought against statehood for Puerto Rico?', 'How did these politicians voice their concerns?', 'What was the White House stance?', 'When was H.R. 2000 introduced?', "What did Senator Heinrich's bill require?", 'What would happen in the event of a yes vote in this referendum?', 'What would H.R. 2000 do?', 'Who considered the implications of the definition of the seat of government in the Constitution?', 'Where are these ideas located in the Federalist Papers?', 'What limits does the constitution have for the size of Washington, D.C.?', 'What caused the size of Washington, D.C. to change?', 'What Revolutionary War motto have Washington, D.C. statehood supporters modified and used?', 'What does the phrase protest?', 'In what way has the phrase been spread wide?', 'When did  Obama have the protest plates put on his Presidential limousine?', 'What party carried the position of D.C. Statehood?', 'What party did the D.C. Statehood Party merge when?', 'When did the movement get closest to success?', 'What would be the new name of the D.C. State?', 'When did the drive for statehood falter?', 'What is another likely country for statehood?', 'What is another likely country for statehood?', 'What entity reported on Guam and the Northern Mariana Islands becoming one again?', 'What country had small grassroots movement for US Statehood?', 'What party used to be known as the Federalista Party?', 'What was the most recent year of the Phillipines pushing for statehood?', 'What is another connotation for the 51st state label?', 'When was the Canada-US Free Trade Agreement enacted?', 'When was the Charlottetown Accord signed?', 'When did the Clarity Act go into effect?', 'Who suggested that the Free Trade Agreement amounted to an American takeover of Canada?', 'When was this campaign ran?', 'Who erased the border in the ad?', 'What was the slogan of this ad?', 'When did Quebec land in British hands?', 'When did Ontario land in British hands?', 'During which the US invade Canada?', 'What did several US politicians suggest doing in the 19th century?', 'When were the last days of the Dominion of Newfoundland?', 'What did the Economic Union Party support?', 'What was the result of the 1948 referendum?', 'What does Canada being referred to as the 51st state in a positive way mean in the US, generally?', 'What is a negative connotation of the term?', 'How man votes did Parti 51 attract in 1989?', 'What did the other parties receive that year?', 'When did Panama have the US-ruled Canal Zone Territory?', 'When did El Salvador adopt the US Dollar as its currency?', 'When did the British Honduras become Belize?', 'What country was Cuba a territory of?', 'What battleship sunk in the Havana Harbor?', 'When did Fidel Castro overthrow the Cuban government?', 'What government did Castro install?', 'Why is Israel referred to as the 51st state?', 'What book did Martine Rothblatt publish?', 'When was Two Stars for Peace published?', 'Who wrote the book The American State of Canaan?', 'When did Artcle 3 of the Treaty of San Francisco come into effect?', 'How many people lived on the island of Okinawa?', 'When was Okinawa finally returned?', 'Why did the US have troops on Okinawa?', 'When did the 51st State Party attempt to register in New Zealand?', 'Who is the secretary of the 51st State Party?', 'What does the 51st State Party advocate?', 'Can people in New Zealand vote for the 51st State Party?', 'Who is the mayor of Tirana?', 'Who said that Albania is the most pro-American country in Europe?', 'When did James Baker visit Albania as Secretary of State?', 'What is the name of the largest US military base outside of US territory?', 'When did the US control Greenland?', 'How much did the US offer to pay for Greenland?', 'What university is professor Gudmundur Alfredssson from?', 'What is one reason the US would be interested in Greenland?', "What had been Poland's stance towards the US, historically?", 'What led to the creation of an independent Poland?', 'What did Poland contribute to the "Coalition of the Willing"?', 'How many members did the Party of Reconstruction in Sicily have in 1944?', 'What did the Party of Reconstruction in Sicily campaign for?', 'When did the annexation of Sicily by Italy happen?', 'When did the Allied invasion of Sicily occur?', 'What are the four categories of terra nullius?', 'What three treaties exist for unclaimed lands?', 'What do the treaties defend against?', 'Where is Bir Tawil located?', 'What device is able to change electric powerinto radio waves and also do the reverse?', 'What device is often used in conjuntion with the antenna?', 'What process associated with antennas produces a high frequency alternating current?', 'What else can radio waves be called?', 'When does an antenna catch electromagnetic waves?', 'What is frequently used to connect elements to the receiver?', 'During transmission does the electric field wave move closer to the antenna?', 'What particles are pushed through the antenna by a transmitter?', 'During what process would electrons be vacillating in the antenna?', 'What is an acceptable synonym for antenna?', 'What can sometimes be meant by the term aerial specifically?', 'What is one way of referring to more than one antenna?', 'What is the most widely accepted term for an electrical device that converts electric power into radio waves?', 'Who is most associated with the emergence of the word antenna?', 'When did experimentation with electromagnetic waves begin?', 'What is a way to increase the strength of a radio transmission?', 'How far was Marconi able to get his signal to go using this technique?', 'What is essential for the mating of the elements that create radio waves?', 'How fast are signals produced by antenna transmitted?', 'What is one system that uses electromagnetic waves?', 'What mobile locator and direction finder technology takes advantage of radio waves?', 'What often inconspicuous part of a laptop computer allows for internet usage?', 'What is the main element of an omnidirectional antenna?', 'What type of antenna would most likely be a half a wavelength long?', 'In which direction would you expect to find a dipole?', 'What is meant by the area where signals cannot be accepted well along the conductor?', 'Are basic antennas expensive?', 'What is the foundation most often used when creating new antenna models?', 'What category do vertical antennas fall under?', 'What is used to close the circuit of a dipole antenna?', 'What are different designs aimed at increasing?', 'What is different from a monopole antenna with most other antenna types?', 'Gain when referring to an antenna refers to what about radiated power?', 'Where is the power that is distributed originating from?', 'How many antenna would make up a phased array?', 'What determines if the antenna creates a broadside array or an end fire array?', 'What are the two standard antenna types?', 'How are antennas oriented when arranged with specific spacing?', 'What type of antenna formation is made up of multiple dipole elements of varying lengths?', 'What is the benefit of this formation type?', 'What is a variation of this antenna type?', 'What is the main application of these set ups?', 'For radio broadcast that one would hear in their car what type of towers are used?', 'If one wished to take an antenna to different locations, which type would be best?', 'Why is this specific antenna type used?', 'What is a problem one might encounter when using a portable antenna?', 'What is mainly responsible for the connection of the antenna to its end destinations?', 'What is the group of elements used to join the elements of an antenna?', 'What are the type of antennas used with satellite television called?', 'What can be used in place of  a standard transmission line?', 'Monopole antennas consist of what element?', 'What are small monopoles used as?', 'What the most common form? ', 'What has vertical polarization?', 'What is the most widley used antenna class?', '                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 What radiates two lobes perpendicular to the antennas axis?                                                                                                                                       ', 'Besides low gain antennas, what is also used as driven elements in complicated higher gain types of antennas?', 'How much gain does a half-wave dipole have?', 'What must be true of the antenna and transmission medium for the repiprocity rule to apply ?', 'What is one material that does not have this quality?', 'How can the lack of this quality be used in an everyday way?', 'What is called when the direction of the elctrical current does not make a difference to the current?', 'How could one deide what type of antenna is nedded for a project?', 'What would most likely be the main concern in making this decision?', 'How would you distribute power to allow for a greater gain?', 'What is often the most imprtant crediting factor of antennas?', 'Which antenna type would be best for a project requiring a specific frequency?', 'How could an antenna be made to coordinate with the needed frequency?', 'What factor would result in a desired feedback independence?', 'If you were unable to choose the antenna you wanted how could you still achieve this result?', 'What is the essential difference in radio waves transmitted by an antenna?', 'How can this be adjusted more easily than changing antenna types?', 'When might the size of an antenna cause an issue?', 'What size antenna would not be good when the antenna needs to be directional?', 'There is a relationship between the size of the antenna and the size of the wavelength with what antenna type?', "What particle's migration does the resonance principle rely on?", 'What is the basis for the way most antennas are developed?', 'What type of surface is formed by the tip of a conductor?', 'How would you place the conductor in relation to the signal you wished to obtain?', 'What part of the radio signal creates voltage?', 'What would the electric current always gravitate to?', 'What is the outcome when a curren is reflected>', 'What amount of phase change would return a signal to its originial state?', 'What is the most popular antenna type?', 'What is the current usually beng put into it or pulled out of?', 'Elements in a half wave dipole and on an identical axis are said to be what?', 'What antenna type is a portion of the half wave dipole?', 'What popular type combines more that one antenna?', 'What frequency are antennas normally designed to be?', 'What can be added to f0 to create a standing wave pattern?', 'What multiple is essential for wavelengths?', 'How are waves which are used in the ways discussed controlled?', 'How do quarter wave elements act in relation to a series relevant electrical element?', 'What frequency develops a current peak?', 'How would you describe a current that has maximum efficiency?', 'What could be the best possible output versus input ratio?', 'For use with more than one channel changes are made to increase what property?', 'What could be coupled with an antenna to form pure resistance?', 'What element would this pure resistance be coupled with?', 'What phrase describes the process of purposely using a lower standing antenna at a less than peak frequency?', ' If your were unable to match the source frequency with the design frequency of the antenna what could you use?', 'What is the most common application of resonant antennas?', 'What antenna setup type is generally used for TV viewing?', 'For use with more than one channel changes are made to increase what property?', 'What accounts for the geometry involved in the use of an antenna?', ' How is the compactness of the signal measured?', 'What type of project would call for more than one element used together?', 'What is a half wave dipole need to be coupled with in most instances?', 'What part of the current is usually strongest?', 'What does this result in?', 'Element used to provide support to the original signal are called?', 'What can take advantage of these elements to add more gain?', 'Does this device improve the operation of the antenna?', 'What is the name for passive elements nearer to the source of the signal?', 'Adding more elements to a Yagi-Uda would have what effect?', 'What can be paired with transmission lines to create phases that would support output?', 'What is the most common element used to create a single in phase signal?', 'What must be precise in order to create this signal at output?', 'What is a more complicated occurrence of the single in-phase producing antenna?', 'If you wanted to somehow clone the original signal you could use?', 'What effect would the use of a reflector have on a signal?', 'What allowance can be made for gaps without loss of performance?', 'What is the most recognized type of an antenna that has its basis in reflection?', 'What magnifies the small radiation resistance of a loop?', 'What is resonated by a capacitor along with the receiver tuning that maintains resonance over the AM broadcast band?', 'What is used in most AM broadcaster receivers?', 'When using a small loop antenna at a low frequency, what may occur?', 'What is left after antenna tuning?', 'Where does cancellation of any reactance seen?', 'What can be used to cancel a inductibe reactance or residual capacitative?', 'What is an antenna designed to have?', 'What type of antenna has impedance at a specific frequency?', 'What term can be used to refer to the usable spectrum of an antennas frequency?', 'What causes the frequencies outside of the bandwidth to be unusable?', 'What is lessened by the Yagi-Uda design?', 'What characteristic of the conductor changes the amount of reactance?', 'What characteristic would be better if it were steady?', 'What would be used to create a half wave or quarter wave dipole?', 'What type of tubes are generally used for sturdier always of antennas?', 'What effect do cages have on the spectrum of usable frequencies?', 'What type of tubes are generally used for sturdier always of antennas?', 'How could one achieve the task of creating an antenna that can be Used over various bands?', 'What is an essential for dealing with directing the flow of power?', 'What is an acknowledgement of the range of possible direction for and antenna?', 'What is another way to refer to an antennas gain?', 'What is the meaning of intensity?', 'What type of an antenna would offer the same level of power to each possible destination?', 'Antennas with higher gain have what positive feature?', 'What is a measure of how much efficient is improved by adding a device designed to make something stronger?', 'Dish network Tv takes example of what type of antenna?', 'What type of antenna would be best if placement was tricky?', 'What must the relationship of an antenna and the receiving area be due to reciprocity?', 'what could an antenna with complete electrical efficiency be said to have?', 'Another term for the effective area is?', 'hat could an antenna with complete electrical efficiency be said to have?', 'A plot of the radiation behaviors of an antenna would show what?', 'What type of pictoral aid is often used to show this?', ' A sphere shows what type of antennas radiation?', 'Dipoles are considered to be what antenna type?', 'What would an omnidirectional antenna look like if plotted?', 'What is the term that refers to areas where an antennas radiation is zero? ', 'What do radio waves do that account for maxima or nulls in an antenna pattern?', 'If you desired to project radio waves to the south, what part of the antenna would you build larger in that direction?', 'What is the main distinction of side lobes?', 'What is one piece that makes up an antenna system?', 'What is created by a portion of a radio waves energy reversing?', 'A term that refers to the highs and lows of power in electro magnetic waves is?', 'Impedance matching makes an important difference in what antenna function?', 'A measure of the power that is useable and the power absorbed by the terminals?', 'What happens to the power that is not absorbed by the antenna?', 'What can cause that reaction?', 'What can cause that reaction?', 'What would need to be placed into the transmitter to create ten W oh warmth? ', 'What would need to be investigated to determine how much power was radiated?', 'What factor may play a hand in lessened power from a transmitter?', 'What can increase the feedpoint impedance of a component?', 'The addition of Rr and Rloss equals what?', 'What equation an determine an antennas effectiveness?', 'What is the single factor that can be precisely measured?', 'What is the median level for measuring atmospheric noise?', 'In what type of programs would low efficiency antennas not make a difference in effectiveness?', 'At lesser frequencies what can account for incorrect assumptions about efficiency?', 'What is SNR?', 'What else is also known as power gain?', 'What is used to signal toward a reciever?', 'Which gain does not iclude the effect of an antenna?', 'Wats divided by the antennas efficiency?', 'Small and minimal frequency antennas are know to be what?', 'What is added to to increase ability for reception?', 'How would this antenna rate in the greater scheme of things?', 'When talking about much larger network what effect can reduced antenna effectiveness have?', 'How many polarizations will a antenna have when mounted vertically?', 'What is another name for electric-field?', 'When is a magnetic fields right angles to a electrical field?', 'Whats understood in reference of direction of an E-plane?', 'What mostly affects polarization?', 'What reflector can change the waves polarization?', "What will you be matching with the  receiving antenna's polarization?", 'What stays the same regarding polarization state at the recieving location?', 'What is the name used for signals recieved following reflection by the ionosphere?', "Whats predictable about an antenna's geometry?", 'The polarization of what antenna is essential specification?', "Most rooftop antenna's are polarized in which direction?", "What is a more complicated type of polarization from an antenna's geometry?", 'What are electrical fields projected on to?', 'What is the imagenary plane perpindicular to?', 'What is the name to describe polarization of radio waves varies over times?', 'How many directions does the E-plane of the radio waves oscillates back and forth?', 'What is the best for an recieving antenna for optimum reception?', 'What loses signal strength?', 'What is used to match vertical and horizontal linear polarizations?', 'A transmission can cause reduction in what?', 'What requires matching of the empedance to an antenna system?', 'Why would a transmitter have additional adjustments?', 'What is SWR?', 'What is the desired design value for transmmiting circuitry?', 'What can be shorter than 1/4 wavelength long?', 'What is the opposite of inductive reactance?', 'What is at the base of the antenna?', 'Where is the resistance seen on the loading foil?', 'What is not adjustable when it comes to a transformer?', 'How many ohms are in are in a standard coax?', 'A general matching network needs a minimum of how many adjustments to correct all components of impedance?', 'What is included between the transmission line and the antenna element?', 'A balun should be integrated into what?', "What antenna's are nonresonant?", "How long are the wire antenna's that the voltage and current waves travel in the same direction?", 'What are undirectional traveling wave directions terminated by?', 'What antenna does not have linear polarization?', 'What is the resistor equal to?', 'What is the radiation patter influenced by?', 'The ground is important for what antenna?', 'Whats another name for the ground?', 'What does the artificial ground plane do for the antenna current?', 'What is the net quality of the ground dependent of?', 'What is the reason for the receiver seeing both the ral antenna and image of the antenna?', 'What frequencies is associated with shorter wavelengths?', 'When will reflections not be coherent?', 'What is the refractive index of the ground?', 'When is polarized radiation reversed?', 'What is the refractive index of air?', 'Besides a ground, what has these phases apply well to?', 'Who proposed this theory?', 'Is the ground has a great conductor, how much of the wave will be reflected?', 'what is reflected and transmitted into the ground when a plane surface is struck?', 'What depends on the polarization and waves angle?', 'What is the portion of something that is reached by the radio transmission called?', 'What effects the function of signals received by an antenna?', 'What is a measure of the power of an antenna?', 'How much power will be delivered to the reciever if the antenna has an effective area of 12 m/2?', 'The properties of a usable radio frequency can be called its?', 'What is a measure of the comparison between reactance and resistance?', 'What is the largest Q that could be achieved with a slimmer band antenna?', 'What type of element would be used to encounter less reactance?', 'What can be added to allow for an antenna shorter than the needed height to produce desired results?', 'How would the resistive impedance from this scenario compare to if the antenna was the proper height?', 'The coil has lengthed the antenna to reach how much length electrically?', 'What is the major thing that sets the size of antennas at lower frequencies?', 'How big would an antenna be used to with with one m wavelengths?', 'How could the rection of output be adressed by?', ' What unit is used to measure current?', 'What effect occurs when the signal is reflected back into the antenna?', 'What sort of changes in material would a current reflect?', ' What must match between the transmission line and elements?', 'What technique is used to prevent unintentional decrease in signal?', 'How is this process usually used for transmission lines and antennas?', 'What is used to allow one to control the function of wave fronts moving through an antenna?', 'What changes the structuak traits of a wave on either side?', 'What is the main purpose of a refractor?', 'A refractor can be compared to what type of viewing?', 'What can receive a stron signal by transmitting an original wave?', 'Where will the additional current be induced?', 'What is altered because of the proximity to the ground?', 'what happens to the reflected wave which allows the antenna to reach its asympotic feedpoint impedance?', "What is between recieving and transmitting antenna's?", 'When is the emission at maximized recieved signal?', 'Where is the radiation patterns in the vertical plane shown?', "What is also increased along with the antenna's height?", 'What is another name for classical television transmissions?', 'Buildings can create ghost images because of what?', 'When is ghosting reduced?', 'What cause less problems in digital terrestrial television?', 'Where are the indiviual array elements known?', 'What induces a feedpoint to antennas nearby?', 'What are the mathmatic useful towards?', 'What cause mutual interactions between the elements electric and magnetic fields?', 'a antenna element not connected to anything is circuited how?', 'When is the element not allowed voltage?', 'What element absorbs and reradiate RF-energy?', 'What antenna can the solid rod be viewed as a dipole antenna?', 'Where can this solid rod be viewed?', 'What type of programmin relies on vertical polarization?', 'If you were to position your receiver closer to the ground what might be a negative of this placement?', 'What is one use that would require an antenna to receive signals in various ways at once?', 'If you were to develop a signal to be used with mobile phones what would be an effective antenna type?', 'What would the antenna you are using take advantage of to allow it to work in multiple locations?', 'What type of antenna can be formed by a circular segment of wire?', 'What is a loop antenna compared with to determine its beahvior?', 'If you needed to place an antenna somewhere with a large amount of interference, which type would be best?', 'Why would this type be good for receiving but not transmtting?', 'What else would be an effective application of this antenna type?', 'Are essential properties of an antenna changed based on what fungtion they are performing', 'What proposition explains the equivalency of the recieving pattern of an antenna?', 'Can the antenna serve more than one fungtion at a time?', 'What is one electrical trait of an antenna?', 'What are the most diverse group of land plants?', 'How many species of flowering plants are there about?', 'What does an angiosperm produce its seeds within?', 'What do angiosperms have in common with gymnosperms?', 'What is the Greek word for "case" or "casing"?', 'How long have higher plants lived on land?', 'How did early plants reproduce sexually?', 'Upright meisporangia allowed what to be dispersed to new habitats?', 'What are the nearest algal relatives existing today?', 'What is the common name for Selaginella?', 'What did Charles Darwin call the sudden appearance of nearly modern flowers in the fossil record?', 'What type of fossil is the recently discovered Archaefructus?', 'What fossil evidence is lacking when it comes to flowers?', 'From what period does fossilized pollen suggest an older date for angiosperm origin?', 'What relationship proposed on the basis of morphological evidence has recently been disputed?', 'How many distinct rounds of genome duplication events are suspected in the evolution of seed plants?', 'When did the first whole genome duplication event occur?', 'What type of event perhaps created the line which led to modern flowering plants?', 'How are duplication events studied?', 'Who coined the term "abominable mystery"?', 'When is the earliest known macrofossil identified as an angiosperm dated?', 'What is the earliest known angiosperm?', 'What pushes the age of angiosperm 5 million years further back?', 'How long ago has circumstantial evidence has been found for the existence of angiosperms?', 'What group of now extinct seed plants had many of the traits of what are now flowering plants?', 'When does a great diversity of angiosperms appear in the fossil record?', 'What is the appearance of a large amount of angiosperms in the fossil record known as?', 'When did the division of the eight main groups of angiosperms occur?', 'What were the angiosperms up to by the late Cretaceous?', 'Beech and maple had already appeared by what period?', 'What is one proposed explanation for the instant appearance of flowering plants?', 'What is island genetics thought to be a default source of?', 'What did radical adaptations seem to have required?', 'How did an isolated setting like an island help flowering plants evolve?', 'What are bees descended from?', 'What are animals also a part of?', 'What did the enlargement of flower parts evolve to form?', 'What does a plant get out of forming fruit?', 'What are many mutualistic relationships, thus failing to survive competition? ', 'What turned out to be unusually effective for plants to reproduce?', 'What uses a combination of genes to form new shoots?', 'What were plant parts like in their primitive days?', 'How did flowers become bixsexual?', 'What did some plant parts do as they evolved?', 'What is a term for a plant with a specific sex per flower?', 'How long has flower evolution occurred? ', 'What heritage do many modern domesticated flower species have?', 'Humans have effected some flowers so much they can no longer do what in nature?', 'Why did weeds grow with human crops?', 'What feature helped flowers not get plucked?', 'What three groups were the first to diverge from angiosperm?', 'What term refers to the first three groups to diverge from angiosperm?', 'How clear is the relationship between the broadest of the three non-basal angiosperm groups?', 'What group do some analyses seem to indicate diverged first?', 'What term did Paul Hermann come up with in 1690?', 'What did Hermann use the term angiosperm a primary division of in the plant kingdom?', 'Who restricted the application of the angiosperm term?', 'What did Robert Brown establish the existence of in Cycadeae and Coniferae?', 'What is the group-name angiosperm has been used for by botanical writers?', 'In taxonomies, what type of group are flowering plants treated?', 'What does anthophyta mean?', 'What do the Wettstein and Engler systems use at the assigned rank of subdivision?', 'How does the APG system of 1998, with its 2003 and 2009 revisions, treat flowering plants?', 'What formal subclass classification were flowering plants given in 2009?', 'What consensus is the AGP trying to reach?', 'What internal component has undergone considerable revision?', 'What system did Arthur Cronquist put forth in 1968, but not publish in full form until 1981?', 'Who published an influential reclassification of the angiosperms in 1998?', 'What type of groups do monocots form, based on a recent APG studies?', 'Eudicots or tricolpates are part of a monophyletic group formed by what species?', 'What third major clade can many dicot species be found in?', 'How many species dicot species are magnoliids?', 'Basal angiosperms are what type of grouping of primitive species?', 'What is the estimated range for the number of flowering plants?', 'How many species of moss is there?', 'What does the large number of flowering plant species indicate is true of them in general?', 'What are the minimum number of families in the APG system?', 'How many families are in AGP III?', 'What shape are the bundles in the young stem of dicotyledons arranged in?', 'What does the cambium separate?', 'What is a layer of meristem or formative tissue known as?', 'What results from the development of xylem on the inside and phloem on the outside?', "What's another name for the concentric rings in trees?", 'What is the defining feature of angiosperms?', 'What characteristic flowers show variation in?', 'What do flowers provide the best external feature for doing?', 'What is it the function of the flower to ensure fertilization of?', 'What is the name of a branch system?', "What is a willow's flower comprised of?", 'What have some structures evolved to protect?', 'What appearance are calyx of sepals typically?', 'Individual members of enveloping structures are known by what terms?', 'The features of flowers which attract pollinators also attract what other creature?', 'What sexual feature do a majority of flowers demonstrate?', 'Why did flowering plants develop numerous morphological and physiological mechanisms?', "Why are a heteromorphic flower's carpels and stamens different lengths?", 'What type of mechanism might homomorphic flowers use to tell the difference between foreign and self pollen grains?', 'How are some male and female parts separated for some species?', 'What process does double fertilization refer to?', 'What does a pollen grain adhere to, to start the process of double fertilization?', "What type of cell travels down the pollen tube while it's growing?", 'Where does the pollen tube release its sperm cells?', "What serves as the embryo's food supply?", 'The appearance of the seed coat bears a definite relation to that of what?', 'What does the seed coat protect?', 'What does the fruit of the flower secure?', "How developed is the seed coat when it's used for dissemination?", 'On what is agriculture almost completely dependent?', 'What family of plants is most important for human sustenance?', 'What is the more common name of the Fabaceae?', 'What family do potatoes, tomatoes and peppers belong to?', 'What family would you find cherries in?', 'Which city is the capital of Telangana?', 'How many square miles does Hyderabad cover?', 'What river is Hyderabad next to?', 'What is the population of Hyderabad?', 'At what altitude is Hyderabad, in meters?', 'In what year was Hyderabad established?', 'Who is the person that established Hyderabad?', 'What dynasty controlled Hyderabad until the early 18th century?', 'Which Mughal viceroy created a dynasty in early 18th century Hyderabad?', 'Which dynasty did Asif Jah I create?', 'Which individual commissioned Chaminar?', 'When did Hyderabad become important culturally in India?', 'Which two groups had an influence in making Hyderabad a cultural center?', 'What culture is most represented in the food of Hyderabad?', "Hyderabad has India's second biggest film production industry, what is it called?", 'What items had been historically traded in Hyderabad?', 'What is a nickname for Hyderabad?', 'What are three historic bazaars in Hyderabad?', 'What is the economic output of Hyderabad?', "What is the rank of Hyderabad among cities contributing to India's GDP?", 'What does the Oxford Concise Dictionary of World Place Names say Hyderabad means?', 'Hyderabad was named in honor of someone, who was it?', 'What does Andrew Petersen say that Hyderabad was originally known as?', 'Who founded what came to be known Hyderabad?', 'What person is Hyderabad said to have been renamed for?', 'There may be Iron Age findings near Hyderabad, what age are they thought to be from?', 'What time period did the Chalukya dynasty rule the Hyderabad region?', 'When did the Calukya empire split into four?', 'What was the capital of Golkonda in the mid 12th century?', 'What was the Hyderabad region known as under the Chalukya dynasty?', 'Who defeated the Kakatiya dynasty in 1310?', 'Which entity subsumed the Kakatiya dynasty?', 'What did the Malik Kafur do to the Kakatiya dynasty in 1321?', 'When was the Behmani Sultanate established?', 'In what year did Muhammad bin Tughluq succeed the Delhi sultanate?', 'What position did Sultan Quli hold?', 'When was Hyderabad founded?', 'In what year did the establishment of the Qutb dynasty occur?', 'In the late 17th century the capital of Deccan Suba was moved, to where?', 'Whom did Sultan Quli rebel against?', 'What position did Farrukhsiyar hold?', 'What official title did the Viceroy of Deccan hold?', 'Who was the Viceroy of the Deccan under Farrukhsiyar?', 'In what year was Mubariz Khan defeated by Asif Jah I?', 'What did Asif Jah I rename Deccan Suba to?', 'Whom did the Nizam ally with in the late 18th century?', 'What was the title of Baji Rao I?', 'What was the title of Hyder Ali?', 'Which military occupied Hyderabad city in a protection role?', 'In what year did Hyderabad become the capital of Nizams?', 'On what date did the Indian Army take control of Hyderabad?', 'Which uprising occurred from 1946 to 1951?', 'What was the code name for the Indian Army invasion of Hyderabad?', 'Which Nizam defeated by the Indian Army?', 'On what date did the constitution of India become active?', 'On what date was Hyderabad state divided into three?', 'Which three states was Hyderabad state divided into?', 'Hyderabad city became the capital of what state in the mid 20th century?', 'When was Telangana state announced?', 'What date was the official formation of Telangana state?', 'How far south of Delhi, in miles, is Hyderabad?', 'In what region of the Deccan Plateau is Hyderabad?', 'How large in square kilometers is Greater Hyderabad?', 'What are the highest hills in Hyderabad?', 'When was Hussain Sagar lake built?', 'What is the mean yearly temperature in Hyderabad in Celsius?', 'Köppen Aw refers to what kind of climate?', 'During what months does summer occur in Hyderabad?', 'What is the typical lowest temperature in Celsius during winter in Hyderabad?', 'What is generally the hottest month in Hyderabad?', 'What is the name of the largest zoo in Hyderabad?', "Hyderabad's largest zoo is known as India's first to have what two kinds of animals in a safari park setting?", 'How many national parks does Hyderabad have?', 'Two of the national parks in Hyderabad are Mrugavani National Park, Mahavir Harina Vanasthali National Park, what is the third?', 'Which environmental preserve in Hyderabad is known to be a home for migratory birds?', 'What is the entity that controls the infrastructure of Hyderabad?', 'How many municipal wards are within Hyderabad?', 'In what year did the GHMC form?', 'How much area, in square miles, does the GHMC oversee?', 'Which board oversees military areas within Hyderabad?', 'Which Hyderabad agency is responsible for the largest area?', 'What is the HMDA mainly responsible for?', 'What does the acronym HMDA stand for?', 'What does the acronym HMWSSB represent?', 'Hyderabad city can be referred to as something else as administrative agencies are concerned, what?', 'What dam began to provide water to Hyderabad in 2005?', 'Along with dealing with sewerage and the water supply, what is one other thing the HMWSSB regulates?', 'How long is the pipeline from the Nagarjuna Sagar Dam which the HMWSSB operates?', 'Which company is responsible for electricity in Hyderabad?', 'Which entity is responsible for the fire stations in Hyderabad?', "Where is the site where Hyderabad's solid waste is dumped?", "Which entity is responsible for disposing Hyderabad's solid waste?", 'What percentage of the air pollution in Hyderabad comes solely from vehicles?', 'How many deaths are believed to be caused in Hyderabad due to air pollution each year?', "What is the rating for the hardness of Hyderabad's water?", 'How many government hospitals did Hyderabad have in 2010?', "How many beds are available in all of Hyderabad's hospitals and nursing homes combined?", 'How many nurses per 10,000 persons are there in Hyderabad?', 'Which agency is in charge of health and wellness services in Hyderabad?', 'People choose to use what type of facility due to long distances and poor care at Government hospitals? ', 'What percentage of the children in Hyderabad city had basic vaccinations in 2005?', 'What is the only city that had a lower vaccination rate for children than Hyderabad in the 2005 National Family Health Survey?', 'What was the fertility rate of Hyderabad according to a 2005 survey?', 'What was the child vaccination rate according to a 2005 survey of Hyderabad?', 'Only one city, according to a 2005 survey, had a lower child vaccination rate than Hyderabad, what was it?', 'According to a 2005 survey what was in Hyderabad?', 'What percentage of children in a 2005 survey of people in Hyderabad were underweight?', 'What was the area of Hyderabad before the GHMC?', 'What was the population of Hyderabad in the 2011 census?', 'How many females were in Hyderabad for the 2011 census?', 'What is the literacy rate of Hyderabad?', 'What was the population density in Hyderabad in 2011?', 'What are the people who live in Hyderabad called?', 'There are two main languages spoken by the people of Hyderabad, what are they?', 'What is the main language of most Muslims from Hyderabad?', 'Which is the largest ethnic community in Hyderabad?', 'Hyderabadi is a type of what langauge?', "Which percentage of Hyderabad's population is below the poverty line?", 'What population of Hyderabad lived in slums in 2012?', 'What is the literacy rate in the slums of Hyderabad?', 'In the slums of Hyderabad how many private schools are there?', 'What percentage of children in Hyderabad are working hazardous jobs?', 'In what region of Hyderabad is Mecca Masjid?', 'South central Hyderabad is generally the home to what attractions?', 'On what side of the river is Begum Bazaar?', 'Where in Hyderabad is the Secunderabad Railway Station?', 'In what region of Hyderabad city is Jubilee Hills located?', 'Where in Hyderabad would one find defence centres?', 'What area in Hyderabad contains the Hyderabad Airport?', 'Cyberabad contains a national park, what is it?', 'After a flood in what year did Hyderabad expand?', 'Which river flooded in 1908, preceding the expansion of Hyderabad?', 'The Mir Osman Ali Khan had a title, what was it?', 'What did India declare Hyderabad om 2012?', 'What type of heritage architecture is mainly displayed in Hyderabad?', 'What influence did Qutb Shahi architecture borrow from?', 'What is the oldest piece of Qutb Shahi architecture in Hyderabad?', 'What iconic historic structure is in the center of old Hyderabad?', 'The government of India proposed that The Charminar become a UNESCO World Heritage site, what other two monuments were suggested for the same status?', 'How high are the minarets on the Charminar?', 'The Chowmahalla Palace represents what type of architecture?', 'The royal court of Chowmahalla palace is an example what style of architecture?', 'What century was Bella Vista Palace built?', 'Which family constructed the Basheer Bagh Palace?', 'Which palace in Hyderabad did the work of Andrea Palladio influence?', 'Where was Hyderabad ranked in 2012 as a producer of GDP in Talangana?', 'Where was Hyderabad ranked in India in 2012 in terms of deposits?', 'What was the GDP of Hyderabad in US dollars in 2011-2012?', 'How many people were employed by the Andhra Pradesh government in 2006?', 'What percentage of females in Hyderabad were employed in 2005?', 'In the 18th century there was one global trading hub for large diamonds, what was it?', 'When did railway expansion in Hyderabad occur?', 'What did the Hyderabad Stock Exchange come to be known as?', 'During what era was The Defence Research and Development Organisation founded?', 'At the end of 2014 what entity would handle the the transaction services for BSE-Mumbai?', 'In what year was Indian Drugs and Pharmaceuticals Limited established?', "In 2010 what percentage of India's bulk pharmaceuticals were produced in Hyderabad?", "What percentage of Andhra Pradesh's IT infrastructure exports originated from Cyberabad in 2013?", 'How many ITES and IT companies are located in HITEC city?', 'What rank within India was HITEC City given by the World Bank Group in 2009?', "What percentage of Hyderabad's employed are employed informally?", 'In 2007 how many street vendors did Hyderabad have?', 'What percentage of the street vendors in Hyderabad were male in 2007?', 'Of the street vendors in Hyderabad what percentage owned their own stall in 2007?', 'In 2007 what percentage of street vendors in Hyderabad borrowed money to finance their operations?', "What event pushed Hyderabad to the top of India's cultural centres?", 'What major event in India in 1857 caused migration of peformance artists into Hyderabad?', 'What are the official languages of Telangana?', 'Which religious group holds the Eid ul-Fitr festival in Hyderabad?', 'The festival of Diwali is a tradition of what faith?', 'What are mushairas?', 'Which dynasty supported the Deccani Urdu literature movement?', 'When was Lazzat Un Nisa completed?', 'At what location did Lazzat Un Nisa get compiled?', 'Who wrote Gulzar-e-Mahlaqa?', 'In what year did the Hyderabad Literary Festival start?', 'What is the name of the largest public library in Hyderabad?', 'What is one of the activities Telugu Academy is credited with encouraging?', 'In what year was the biggest public library in Hyderabad created?', 'What is the Deccan region known for culturally?', 'What is a tawaif?', 'What kind of music is dholak ke geet?', 'What entity is responsible for the Taramati Music Festival?', 'During the rule of what two groups did dance and music from North India become popular?', 'What type of art might one encounter at Lalithakala Thoranam?', 'What is Numaish?', 'What is a popular name for the Telugu film industry?', 'Where was the Telugu film industry ranked in 2012 in India in comparison to Bollywood?', 'What did Guinness World Records say of Ramoji Film City was in 20015?', 'Golconda is a type of what kind of painting?', 'In what century did Golconda painting develop?', 'What type of colors would one generally find in Golconda painting?', 'In what century did Hyderabad style painting develop?', 'What types of scenes does Hyderabad style painting generally contain?', 'In which century did metalwork become popular in Hyderabad?', 'Under which act is the Geographical Indication tag granted?', 'What is Zardozi an example of?', 'What is Kalamkari?', 'In what style is Kalamkari made?', 'What type of foods are generally thought of as a part of Hyderabadi cuisine?', 'What is a large influence on Hyderabadi food?', 'What is kaddu ki kheer?', 'What types of cuisine are noted within Hyderabadi haleem?', 'When was The Deccan Times founded?', 'What is The Deccan Chronicle?', 'What type of paper is The Siasat Daily?', 'What year was the first radio station in Hyderabad formed?', 'On what date did the first public radio station in Hyderabad begin broadcasting?', 'When did Doordarshan begin broadcasting in Hyderabad?', 'What satellite network began broadcasting to Hyderabad in July 1992?', 'In what year did the general public first get access to the internet in Hyderabad?', 'The first private company to offer Hyderabad internet service began offering it in what year?', 'What is Doordarshan?', 'What entity controls the schools in Hyderabad?', 'What percentage of students are in private schools in Hyderabad?', 'Which languages are used for teaching in the schools of Hyderabad?', 'What type of educational plan does the Central Board of Secondary Education use in Hyderabad?', 'Students in Hyderabad may be required to acquire one of two different certificates, one is  the Secondary School Certificate, what is the other?', 'How many universities are in Hyderabad?', 'Of the universities in Hyderabad how many are state run?', 'Maulana Azad National Urdu University is a type of university that represents what type of university?', "In 1982 the first long distance university was opened in India, what is it's name?", 'What year was Osmania University formed?', 'How many medical schools are in Hyderabad?', 'What type of medicine is taught at The Government Nizamia Tibbi College?', 'What non-profit cardiovascular educational association is headquartered in Hyderabad?', 'What type of school is Raffles Millennium International?', 'What type of is Wigan and Leigh College?', 'What is the school otherwise known as IIT-H?', 'There is a research institute in Hyderabad known as ICRISAT, what does the acronym represent?', 'Two of the most popular sports played in Hyderabad include association football, what is the other?', 'What two world championships were held in Hyderabad in 2009?', 'What sport is played at the Swarnandhra Pradesh Sports Complex?', 'Where is the G.M.C. Balayohi Stadium located?', 'What is the home stadium of the Hyderabad cricket association?', 'At what time was Secunderabad an important sport center?', 'What type of racing is the Hyderabad Race Club popular for?', 'What organization holds the TSD Rally?', 'Under whom did the Secunderabad Club and the Nizam Club form?', 'Which annual derby is held at the Hyderabad Race Club?', 'Ghulam Ahmed and Shivlal Yadav are what kind of sportsmen?', 'What sport does Sania Mirza play?', 'Who is a notable bodybuilder from Hyderabad?', 'Gagan Narang does what sporting activity?', 'Pullela Gopichand and Saina Nehwal are known in Hyderabad for being what?', 'What type of transport that is not government owned is commonly used in Hyderabad?', 'How many people are carried each day by the bus service network in Hyderabad?', 'What is the light rail system in Hyderabad known as?', 'How many passengers use the Hyderabad light rail system each day?', 'What is Setwin otherwise known as?', 'In 2012 how many vehicles were driving in Hyderabad?', 'What percentage of the vehicles in Hyderabad were cars in 2012?', 'What percentage of Hyderabad city was covered by roads in 2012?', 'In 2012 what percentage of Hyderabad freight was moved by road?', 'What is the maximum speed for buses in Hyderabad?', 'How many national highways form a junction in Hyderabad?', 'How many states are linked to from highways in Hyderabad?', 'How many miles long is NH-7?', 'How many state highways pass through or begin in Hyderabad?', 'What road connects Hyderabad to Bhopalpatnam?', 'What type of city is Santa Monica?', 'What part of Los Angeles County is it located in?', 'What is Santa Monica named after?', 'Approximately how many residents of Santa Monica rent their property?', 'What was the population in 2010 according to the Census Bureau?', 'What did indigenous people that lived in Santa Monica call it previously?', 'What was the local population called indigenous to the Santa Monica Area?', 'What was the first explorers name that came in contact with the natives?', 'What date did the explorer reach Santa Monica?', 'What day is the Santa Monica Feast held on?', 'What ethic group grew in the 20th century?', "What other ethic minority did white american's treat poorly?", 'What type of Village was located on the Long Wharf?', 'How many ethnic minorities were looked at differently in Santa Monica?', 'What role did the fishing village play in Santa Monica?', 'Who built a plant in Clover field?', 'What company was the structure at Clover Field for?', 'What did Clover Field built airplanes attempt the first of?', 'The Douglas Company had facilities in Santa Monica through what decade?', 'How Many Planes returned from the circumnavigation in 1924?', "What brought on the Douglas business's company growth?", 'How many people did the company employ in 1943?', 'The company used camouflage from what designers?', 'What company started as a project in 1945?', 'What was the camouflage designed to do?', 'What year was the Sanat Monica Pier buildt?', 'What historic landmark is located on the Pier?', 'La Monica Ballroom used to be what type of ballroom in the United States?', "Where were the Academy Awards held in the 1960's?", 'What place is still considered a prominent acoustic performance area?', 'The Third Street Promenade is located in what part of Santa Monica?', 'How large is the Third Street Promenade?', 'What is Third Street access shut off from?', 'What two major department stores are located at the Third Street Promenade?', 'What year did the mall reopen?', 'What event does the Chamber of commerce host in the fall?', 'What can visitors do at the Taste of Santa Monica?', 'Where is the Taste of Santa Monica held each year?', 'Who provides the guests with these samples at the event?', "What hotel offers a trip to the famer's market to influence dinner?", 'Roughly how many days of sunshine does Santa Monica get annually? ', 'What USDA hardiness zone is Santa Monica located in?', 'What commonly occurs on the mornings in May?', 'What have locals coined the morning time in May?', 'What is the beach temperature in Fahrenheit during the summer ? ', "In what two years were the first Sustainable City Plan's introduced?", "What percent of the city's public work's vehicles use alternate fuels?", 'Water conservation policies have been adopted by what two sectors?', 'What percent of the bus systems use alternate fuels?', 'What company provides the city with natural gas?', 'What is the name of the urban runoff facility?', 'How many gallons of water does (SMURFF) treat every week?', 'How much money do residents get back for participating in water conservation per square foot?', 'Santa Monica plans on water independence by what year?', 'If (SMURFF) did not collect water where would it go?', 'What year does the city plan on having a bicycle infrastructure?', 'The city has two bike Action Plans of how many years?', "The city's bus system is being over hauled to work with what other program?", "What percentages does the city Bike transportation mode's target?", 'What type of business certification do they plan on expanding?', 'What was the total average family size?', 'What percent of households were made of one person?', 'What percent of households had someone over 65 living in it?', 'Roughly how many same-sex couples were there?', 'What percent of household have children under 18?', "How many citizen's were in the year 2000 census?", 'How many urban families were there?', 'What is the population density per square mile?', 'From 2000 what was the average family size?', 'What percent in the 2000 census had persons under the age of 18?', 'What year was the Santa Monica junior college founded?', 'How many acres of land is the Santa Monica Junior College?', 'By what area is the Art institute in Santa Monica?', 'There are a plethora of colleges and universities within what radius of Santa Monica?', 'How many students enroll to the SMC annually?  ', 'What plan launched a Bicycle sharing system?', "When was Santa Monica's Bicycle sharing system launched?", 'What is the name of the bike trail that goes through Santa Monica?', 'When was the most recent Bicycle Friendly Community Award given to Santa Monica? ', 'What other year did Santa Monica receive the Bicycle Friendly Community award?', 'What is the name of the Santa Monica Freeway?', 'What road does state route 2 begin on?', 'How many more states aside from California does the interstate 10 run through?', 'What city and state does the interstate 10 end at?', 'Where does the Santa Monica Freeway begin around?', 'Beginning in what decade did the Santa Monica  Expo Air Line host freight and passenger services?', 'What year did the Santa Monica freight and passenger service stop running?', 'Diesel powered freight deliveries continued through what date?', 'What was the result from the discontinuation of using the line for deliveries?', 'Who bought the Expo line from Southern Pacific?', 'In 2006 what percent of the population was impacted by crime?', 'What percent of crime was strictly property in 2006?', 'In 2006 what was crime mostly comprised of?', 'How are the Santa Monica per-capita crime rates calculated?', 'The Santa Monica Police Department suggests that the per-capita crime rate sways in what way?', 'In what year was there a double homicide?', 'Where was this homicide perpetrated?', 'What gang is mentioned in connection with this crime?', 'What two people were killed inside the store?', 'Where is the getaway driver of the Culver city gang murder?', 'What is one of the oldest exterior shots in Santa Monica?', 'What does this iconic exterior shot display?', 'What 1982 movie includes Sylvester Stallone running on the Santa Monica Beach?', 'What 1996 movie was set completely in Santa Monica?', 'What movie was shot at Samohi?', 'What type of games is Santa Monica featured in?', "In what year did Tony hawk's American Wasteland release?", 'What video game features Santa Monica in 2003?', 'Grand theft Auto V was released in what year?', 'What video game featured Santa Monica in 2015?', 'When was Washington University founded?', 'How many Nobel laureates have been affiliated with Washington University?', 'Where is Washington University ranked among Academic Ranking of World Universities?', 'For whom is Washington University named?  ', 'How many different countries comprise the makeup of faculty and students of Washington University?', 'Who was the first chancellor of Washington University?', 'When did Washington University secure its charter?', 'What makes Washington University unique among other American universities?', 'Who was the first president of Washington University?', 'What local businessman help provide support for the Washington University?', 'What was one of the names that was initially considered for Washington University in St. Louis?', 'What name was selected by the Board of Trustees in 1854 for Washington University?', 'Why did the Board of Trustees choose to name Washing University after George Washington?', 'When was the name Washington University amended? ', 'When was the suffix "in St. Louis" added to Washington University?', 'Where was Washington University located? ', 'When did classes first begin at Washington University?', 'What building was first used for classes at Washington University?', 'What three departments were Washington University divided?', 'What type of school did Washington University first function?', 'When was did Washington University open its law school?', 'Who was Robert Somers Brookings?', 'When did Washington University establish its medical school? ', 'When did St. Louis Medical college affiliate itself with Washington University?', "Where was Washington University located during the first half of the 1800's. ", 'When did Washington University begin to expand west?', 'What architecture firm was hired by the Board of Directors at Washington University? ', 'Who was on the committee that found a new site for Washington University?', 'What nickname was given to the new campus site?', 'When did Washington University hold a national design contest for the new campus?', 'What firm won the Washington University national design contest?']
contexts =['It is possible that around 62 million people died in the war; estimates vary greatly. About 60% of all casualties were civilians, who died as a result of disease, starvation, genocide (in particular, the Holocaust), and aerial bombing. The former Soviet Union and China suffered the most casualties. Estimates place deaths in the Soviet Union at around 23 million, while China suffered about 10 million. No country lost a greater portion of its population than Poland: approximately 5.6 million, or 16%, of its pre-war population of 34.8 million died.', 'It is possible that around 62 million people died in the war; estimates vary greatly. About 60% of all casualties were civilians, who died as a result of disease, starvation, genocide (in particular, the Holocaust), and aerial bombing. The former Soviet Union and China suffered the most casualties. Estimates place deaths in the Soviet Union at around 23 million, while China suffered about 10 million. No country lost a greater portion of its population than Poland: approximately 5.6 million, or 16%, of its pre-war population of 34.8 million died.', 'It is possible that around 62 million people died in the war; estimates vary greatly. About 60% of all casualties were civilians, who died as a result of disease, starvation, genocide (in particular, the Holocaust), and aerial bombing. The former Soviet Union and China suffered the most casualties. Estimates place deaths in the Soviet Union at around 23 million, while China suffered about 10 million. No country lost a greater portion of its population than Poland: approximately 5.6 million, or 16%, of its pre-war population of 34.8 million died.', 'It is possible that around 62 million people died in the war; estimates vary greatly. About 60% of all casualties were civilians, who died as a result of disease, starvation, genocide (in particular, the Holocaust), and aerial bombing. The former Soviet Union and China suffered the most casualties. Estimates place deaths in the Soviet Union at around 23 million, while China suffered about 10 million. No country lost a greater portion of its population than Poland: approximately 5.6 million, or 16%, of its pre-war population of 34.8 million died.', 'It is possible that around 62 million people died in the war; estimates vary greatly. About 60% of all casualties were civilians, who died as a result of disease, starvation, genocide (in particular, the Holocaust), and aerial bombing. The former Soviet Union and China suffered the most casualties. Estimates place deaths in the Soviet Union at around 23 million, while China suffered about 10 million. No country lost a greater portion of its population than Poland: approximately 5.6 million, or 16%, of its pre-war population of 34.8 million died.', 'The Holocaust (which roughly means "burnt whole") was the deliberate and systematic murder of millions of Jews and other "unwanted" during World War II by the Nazi regime in Germany. Several differing views exist regarding whether it was intended to occur from the war\'s beginning, or if the plans for it came about later. Regardless, persecution of Jews extended well before the war even started, such as in the Kristallnacht (Night of Broken Glass). The Nazis used propaganda to great effect to stir up anti-Semitic feelings within ordinary Germans.', 'The Holocaust (which roughly means "burnt whole") was the deliberate and systematic murder of millions of Jews and other "unwanted" during World War II by the Nazi regime in Germany. Several differing views exist regarding whether it was intended to occur from the war\'s beginning, or if the plans for it came about later. Regardless, persecution of Jews extended well before the war even started, such as in the Kristallnacht (Night of Broken Glass). The Nazis used propaganda to great effect to stir up anti-Semitic feelings within ordinary Germans.', 'The Holocaust (which roughly means "burnt whole") was the deliberate and systematic murder of millions of Jews and other "unwanted" during World War II by the Nazi regime in Germany. Several differing views exist regarding whether it was intended to occur from the war\'s beginning, or if the plans for it came about later. Regardless, persecution of Jews extended well before the war even started, such as in the Kristallnacht (Night of Broken Glass). The Nazis used propaganda to great effect to stir up anti-Semitic feelings within ordinary Germans.', 'The Holocaust (which roughly means "burnt whole") was the deliberate and systematic murder of millions of Jews and other "unwanted" during World War II by the Nazi regime in Germany. Several differing views exist regarding whether it was intended to occur from the war\'s beginning, or if the plans for it came about later. Regardless, persecution of Jews extended well before the war even started, such as in the Kristallnacht (Night of Broken Glass). The Nazis used propaganda to great effect to stir up anti-Semitic feelings within ordinary Germans.', 'The Holocaust (which roughly means "burnt whole") was the deliberate and systematic murder of millions of Jews and other "unwanted" during World War II by the Nazi regime in Germany. Several differing views exist regarding whether it was intended to occur from the war\'s beginning, or if the plans for it came about later. Regardless, persecution of Jews extended well before the war even started, such as in the Kristallnacht (Night of Broken Glass). The Nazis used propaganda to great effect to stir up anti-Semitic feelings within ordinary Germans.', "After World War II, Europe was informally split into Western and Soviet spheres of influence. Western Europe later aligned as the North Atlantic Treaty Organization (NATO) and Eastern Europe as the Warsaw Pact. There was a shift in power from Western Europe and the British Empire to the two new superpowers, the United States and the Soviet Union. These two rivals would later face off in the Cold War. In Asia, the defeat of Japan led to its democratization. China's civil war continued through and after the war, resulting eventually in the establishment of the People's Republic of China. The former colonies of the European powers began their road to independence.", "After World War II, Europe was informally split into Western and Soviet spheres of influence. Western Europe later aligned as the North Atlantic Treaty Organization (NATO) and Eastern Europe as the Warsaw Pact. There was a shift in power from Western Europe and the British Empire to the two new superpowers, the United States and the Soviet Union. These two rivals would later face off in the Cold War. In Asia, the defeat of Japan led to its democratization. China's civil war continued through and after the war, resulting eventually in the establishment of the People's Republic of China. The former colonies of the European powers began their road to independence.", "After World War II, Europe was informally split into Western and Soviet spheres of influence. Western Europe later aligned as the North Atlantic Treaty Organization (NATO) and Eastern Europe as the Warsaw Pact. There was a shift in power from Western Europe and the British Empire to the two new superpowers, the United States and the Soviet Union. These two rivals would later face off in the Cold War. In Asia, the defeat of Japan led to its democratization. China's civil war continued through and after the war, resulting eventually in the establishment of the People's Republic of China. The former colonies of the European powers began their road to independence.", "After World War II, Europe was informally split into Western and Soviet spheres of influence. Western Europe later aligned as the North Atlantic Treaty Organization (NATO) and Eastern Europe as the Warsaw Pact. There was a shift in power from Western Europe and the British Empire to the two new superpowers, the United States and the Soviet Union. These two rivals would later face off in the Cold War. In Asia, the defeat of Japan led to its democratization. China's civil war continued through and after the war, resulting eventually in the establishment of the People's Republic of China. The former colonies of the European powers began their road to independence.", "After World War II, Europe was informally split into Western and Soviet spheres of influence. Western Europe later aligned as the North Atlantic Treaty Organization (NATO) and Eastern Europe as the Warsaw Pact. There was a shift in power from Western Europe and the British Empire to the two new superpowers, the United States and the Soviet Union. These two rivals would later face off in the Cold War. In Asia, the defeat of Japan led to its democratization. China's civil war continued through and after the war, resulting eventually in the establishment of the People's Republic of China. The former colonies of the European powers began their road to independence.", "Over the course of the 20th century, the world's per-capita gross domestic product grew by a factor of five, much more than all earlier centuries combined (including the 19th with its Industrial Revolution). Many economists make the case that this understates the magnitude of growth, as many of the goods and services consumed at the end of the 20th century, such as improved medicine (causing world life expectancy to increase by more than two decades) and communications technologies, were not available at any price at its beginning. However, the gulf between the world's rich and poor grew wider, and the majority of the global population remained in the poor side of the divide.", "Over the course of the 20th century, the world's per-capita gross domestic product grew by a factor of five, much more than all earlier centuries combined (including the 19th with its Industrial Revolution). Many economists make the case that this understates the magnitude of growth, as many of the goods and services consumed at the end of the 20th century, such as improved medicine (causing world life expectancy to increase by more than two decades) and communications technologies, were not available at any price at its beginning. However, the gulf between the world's rich and poor grew wider, and the majority of the global population remained in the poor side of the divide.", "Over the course of the 20th century, the world's per-capita gross domestic product grew by a factor of five, much more than all earlier centuries combined (including the 19th with its Industrial Revolution). Many economists make the case that this understates the magnitude of growth, as many of the goods and services consumed at the end of the 20th century, such as improved medicine (causing world life expectancy to increase by more than two decades) and communications technologies, were not available at any price at its beginning. However, the gulf between the world's rich and poor grew wider, and the majority of the global population remained in the poor side of the divide.", 'Still, advancing technology and medicine has had a great impact even in the Global South. Large-scale industry and more centralized media made brutal dictatorships possible on an unprecedented scale in the middle of the century, leading to wars that were also unprecedented. However, the increased communications contributed to democratization. Technological developments included the development of airplanes and space exploration, nuclear technology, advancement in genetics, and the dawning of the Information Age.', 'Still, advancing technology and medicine has had a great impact even in the Global South. Large-scale industry and more centralized media made brutal dictatorships possible on an unprecedented scale in the middle of the century, leading to wars that were also unprecedented. However, the increased communications contributed to democratization. Technological developments included the development of airplanes and space exploration, nuclear technology, advancement in genetics, and the dawning of the Information Age.', 'Still, advancing technology and medicine has had a great impact even in the Global South. Large-scale industry and more centralized media made brutal dictatorships possible on an unprecedented scale in the middle of the century, leading to wars that were also unprecedented. However, the increased communications contributed to democratization. Technological developments included the development of airplanes and space exploration, nuclear technology, advancement in genetics, and the dawning of the Information Age.', 'Still, advancing technology and medicine has had a great impact even in the Global South. Large-scale industry and more centralized media made brutal dictatorships possible on an unprecedented scale in the middle of the century, leading to wars that were also unprecedented. However, the increased communications contributed to democratization. Technological developments included the development of airplanes and space exploration, nuclear technology, advancement in genetics, and the dawning of the Information Age.', 'The Soviet Union created the Eastern Bloc of countries that it occupied, annexing some as Soviet Socialist Republics and maintaining others as satellite states that would later form the Warsaw Pact. The United States and various western European countries began a policy of "containment" of communism and forged myriad alliances to this end, including NATO. Several of these western countries also coordinated efforts regarding the rebuilding of western Europe, including western Germany, which the Soviets opposed. In other regions of the world, such as Latin America and Southeast Asia, the Soviet Union fostered communist revolutionary movements, which the United States and many of its allies opposed and, in some cases, attempted to "roll back". Many countries were prompted to align themselves with the nations that would later form either NATO or the Warsaw Pact, though other movements would also emerge.', 'The Soviet Union created the Eastern Bloc of countries that it occupied, annexing some as Soviet Socialist Republics and maintaining others as satellite states that would later form the Warsaw Pact. The United States and various western European countries began a policy of "containment" of communism and forged myriad alliances to this end, including NATO. Several of these western countries also coordinated efforts regarding the rebuilding of western Europe, including western Germany, which the Soviets opposed. In other regions of the world, such as Latin America and Southeast Asia, the Soviet Union fostered communist revolutionary movements, which the United States and many of its allies opposed and, in some cases, attempted to "roll back". Many countries were prompted to align themselves with the nations that would later form either NATO or the Warsaw Pact, though other movements would also emerge.', 'The Soviet Union created the Eastern Bloc of countries that it occupied, annexing some as Soviet Socialist Republics and maintaining others as satellite states that would later form the Warsaw Pact. The United States and various western European countries began a policy of "containment" of communism and forged myriad alliances to this end, including NATO. Several of these western countries also coordinated efforts regarding the rebuilding of western Europe, including western Germany, which the Soviets opposed. In other regions of the world, such as Latin America and Southeast Asia, the Soviet Union fostered communist revolutionary movements, which the United States and many of its allies opposed and, in some cases, attempted to "roll back". Many countries were prompted to align themselves with the nations that would later form either NATO or the Warsaw Pact, though other movements would also emerge.', 'The Soviet Union created the Eastern Bloc of countries that it occupied, annexing some as Soviet Socialist Republics and maintaining others as satellite states that would later form the Warsaw Pact. The United States and various western European countries began a policy of "containment" of communism and forged myriad alliances to this end, including NATO. Several of these western countries also coordinated efforts regarding the rebuilding of western Europe, including western Germany, which the Soviets opposed. In other regions of the world, such as Latin America and Southeast Asia, the Soviet Union fostered communist revolutionary movements, which the United States and many of its allies opposed and, in some cases, attempted to "roll back". Many countries were prompted to align themselves with the nations that would later form either NATO or the Warsaw Pact, though other movements would also emerge.', 'The Soviet Union created the Eastern Bloc of countries that it occupied, annexing some as Soviet Socialist Republics and maintaining others as satellite states that would later form the Warsaw Pact. The United States and various western European countries began a policy of "containment" of communism and forged myriad alliances to this end, including NATO. Several of these western countries also coordinated efforts regarding the rebuilding of western Europe, including western Germany, which the Soviets opposed. In other regions of the world, such as Latin America and Southeast Asia, the Soviet Union fostered communist revolutionary movements, which the United States and many of its allies opposed and, in some cases, attempted to "roll back". Many countries were prompted to align themselves with the nations that would later form either NATO or the Warsaw Pact, though other movements would also emerge.', 'The Cold War saw periods of both heightened tension and relative calm. International crises arose, such as the Berlin Blockade (1948–1949), the Korean War (1950–1953), the Berlin Crisis of 1961, the Vietnam War (1959–1975), the Cuban Missile Crisis (1962), the Soviet war in Afghanistan (1979–1989) and NATO exercises in November 1983. There were also periods of reduced tension as both sides sought détente. Direct military attacks on adversaries were deterred by the potential for mutual assured destruction using deliverable nuclear weapons. In the Cold War era, the Generation of Love and the rise of computers changed society in very different, complex ways, including higher social and local mobility.', 'The Cold War saw periods of both heightened tension and relative calm. International crises arose, such as the Berlin Blockade (1948–1949), the Korean War (1950–1953), the Berlin Crisis of 1961, the Vietnam War (1959–1975), the Cuban Missile Crisis (1962), the Soviet war in Afghanistan (1979–1989) and NATO exercises in November 1983. There were also periods of reduced tension as both sides sought détente. Direct military attacks on adversaries were deterred by the potential for mutual assured destruction using deliverable nuclear weapons. In the Cold War era, the Generation of Love and the rise of computers changed society in very different, complex ways, including higher social and local mobility.', 'The Cold War saw periods of both heightened tension and relative calm. International crises arose, such as the Berlin Blockade (1948–1949), the Korean War (1950–1953), the Berlin Crisis of 1961, the Vietnam War (1959–1975), the Cuban Missile Crisis (1962), the Soviet war in Afghanistan (1979–1989) and NATO exercises in November 1983. There were also periods of reduced tension as both sides sought détente. Direct military attacks on adversaries were deterred by the potential for mutual assured destruction using deliverable nuclear weapons. In the Cold War era, the Generation of Love and the rise of computers changed society in very different, complex ways, including higher social and local mobility.', 'The Cold War saw periods of both heightened tension and relative calm. International crises arose, such as the Berlin Blockade (1948–1949), the Korean War (1950–1953), the Berlin Crisis of 1961, the Vietnam War (1959–1975), the Cuban Missile Crisis (1962), the Soviet war in Afghanistan (1979–1989) and NATO exercises in November 1983. There were also periods of reduced tension as both sides sought détente. Direct military attacks on adversaries were deterred by the potential for mutual assured destruction using deliverable nuclear weapons. In the Cold War era, the Generation of Love and the rise of computers changed society in very different, complex ways, including higher social and local mobility.', 'The Cold War saw periods of both heightened tension and relative calm. International crises arose, such as the Berlin Blockade (1948–1949), the Korean War (1950–1953), the Berlin Crisis of 1961, the Vietnam War (1959–1975), the Cuban Missile Crisis (1962), the Soviet war in Afghanistan (1979–1989) and NATO exercises in November 1983. There were also periods of reduced tension as both sides sought détente. Direct military attacks on adversaries were deterred by the potential for mutual assured destruction using deliverable nuclear weapons. In the Cold War era, the Generation of Love and the rise of computers changed society in very different, complex ways, including higher social and local mobility.', 'The Cold War drew to a close in the late 1980s and the early 1990s. The United States under President Ronald Reagan increased diplomatic, military, and economic pressure on the Soviet Union, which was already suffering from severe economic stagnation. In the second half of the 1980s, newly appointed Soviet leader Mikhail Gorbachev introduced the perestroika and glasnost reforms. The Soviet Union collapsed in 1991, leaving the United States as the dominant military power, though Russia retained much of the massive Soviet nuclear arsenal.', 'The Cold War drew to a close in the late 1980s and the early 1990s. The United States under President Ronald Reagan increased diplomatic, military, and economic pressure on the Soviet Union, which was already suffering from severe economic stagnation. In the second half of the 1980s, newly appointed Soviet leader Mikhail Gorbachev introduced the perestroika and glasnost reforms. The Soviet Union collapsed in 1991, leaving the United States as the dominant military power, though Russia retained much of the massive Soviet nuclear arsenal.', 'The Cold War drew to a close in the late 1980s and the early 1990s. The United States under President Ronald Reagan increased diplomatic, military, and economic pressure on the Soviet Union, which was already suffering from severe economic stagnation. In the second half of the 1980s, newly appointed Soviet leader Mikhail Gorbachev introduced the perestroika and glasnost reforms. The Soviet Union collapsed in 1991, leaving the United States as the dominant military power, though Russia retained much of the massive Soviet nuclear arsenal.', 'The Cold War drew to a close in the late 1980s and the early 1990s. The United States under President Ronald Reagan increased diplomatic, military, and economic pressure on the Soviet Union, which was already suffering from severe economic stagnation. In the second half of the 1980s, newly appointed Soviet leader Mikhail Gorbachev introduced the perestroika and glasnost reforms. The Soviet Union collapsed in 1991, leaving the United States as the dominant military power, though Russia retained much of the massive Soviet nuclear arsenal.', 'The Cold War drew to a close in the late 1980s and the early 1990s. The United States under President Ronald Reagan increased diplomatic, military, and economic pressure on the Soviet Union, which was already suffering from severe economic stagnation. In the second half of the 1980s, newly appointed Soviet leader Mikhail Gorbachev introduced the perestroika and glasnost reforms. The Soviet Union collapsed in 1991, leaving the United States as the dominant military power, though Russia retained much of the massive Soviet nuclear arsenal.', "In Latin America in the 1970s, leftists acquired a significant political influence which prompted the right-wing, ecclesiastical authorities and a large portion of the individual country's upper class to support coup d'états to avoid what they perceived as a communist threat. This was further fueled by Cuban and United States intervention which led to a political polarization. Most South American countries were in some periods ruled by military dictatorships that were supported by the United States of America. In the 1970s, the regimes of the Southern Cone collaborated in Operation Condor killing many leftist dissidents, including some urban guerrillas. However, by the early 1990s all countries had restored their democracies.", "In Latin America in the 1970s, leftists acquired a significant political influence which prompted the right-wing, ecclesiastical authorities and a large portion of the individual country's upper class to support coup d'états to avoid what they perceived as a communist threat. This was further fueled by Cuban and United States intervention which led to a political polarization. Most South American countries were in some periods ruled by military dictatorships that were supported by the United States of America. In the 1970s, the regimes of the Southern Cone collaborated in Operation Condor killing many leftist dissidents, including some urban guerrillas. However, by the early 1990s all countries had restored their democracies.", "In Latin America in the 1970s, leftists acquired a significant political influence which prompted the right-wing, ecclesiastical authorities and a large portion of the individual country's upper class to support coup d'états to avoid what they perceived as a communist threat. This was further fueled by Cuban and United States intervention which led to a political polarization. Most South American countries were in some periods ruled by military dictatorships that were supported by the United States of America. In the 1970s, the regimes of the Southern Cone collaborated in Operation Condor killing many leftist dissidents, including some urban guerrillas. However, by the early 1990s all countries had restored their democracies.", "In Latin America in the 1970s, leftists acquired a significant political influence which prompted the right-wing, ecclesiastical authorities and a large portion of the individual country's upper class to support coup d'états to avoid what they perceived as a communist threat. This was further fueled by Cuban and United States intervention which led to a political polarization. Most South American countries were in some periods ruled by military dictatorships that were supported by the United States of America. In the 1970s, the regimes of the Southern Cone collaborated in Operation Condor killing many leftist dissidents, including some urban guerrillas. However, by the early 1990s all countries had restored their democracies.", "In Latin America in the 1970s, leftists acquired a significant political influence which prompted the right-wing, ecclesiastical authorities and a large portion of the individual country's upper class to support coup d'états to avoid what they perceived as a communist threat. This was further fueled by Cuban and United States intervention which led to a political polarization. Most South American countries were in some periods ruled by military dictatorships that were supported by the United States of America. In the 1970s, the regimes of the Southern Cone collaborated in Operation Condor killing many leftist dissidents, including some urban guerrillas. However, by the early 1990s all countries had restored their democracies.", "The Space Age is a period encompassing the activities related to the Space Race, space exploration, space technology, and the cultural developments influenced by these events. The Space Age began with the development of several technologies that culminated with the launch of Sputnik 1 by the Soviet Union. This was the world's first artificial satellite, orbiting the Earth in 98.1 minutes and weighing in at 83 kg. The launch of Sputnik 1 ushered a new era of political, scientific and technological achievements that became known as the Space Age. The Space Age was characterized by rapid development of new technology in a close race mostly between the United States and the Soviet Union. The Space Age brought the first human spaceflight during the Vostok programme and reached its peak with the Apollo program which captured the imagination of much of the world's population. The landing of Apollo 11 was an event watched by over 500 million people around the world and is widely recognized as one of the defining moments of the 20th century. Since then and with the end of the space race due to the dissolution of the Soviet Union, public attention has largely moved to other areas.", "The Space Age is a period encompassing the activities related to the Space Race, space exploration, space technology, and the cultural developments influenced by these events. The Space Age began with the development of several technologies that culminated with the launch of Sputnik 1 by the Soviet Union. This was the world's first artificial satellite, orbiting the Earth in 98.1 minutes and weighing in at 83 kg. The launch of Sputnik 1 ushered a new era of political, scientific and technological achievements that became known as the Space Age. The Space Age was characterized by rapid development of new technology in a close race mostly between the United States and the Soviet Union. The Space Age brought the first human spaceflight during the Vostok programme and reached its peak with the Apollo program which captured the imagination of much of the world's population. The landing of Apollo 11 was an event watched by over 500 million people around the world and is widely recognized as one of the defining moments of the 20th century. Since then and with the end of the space race due to the dissolution of the Soviet Union, public attention has largely moved to other areas.", "The Space Age is a period encompassing the activities related to the Space Race, space exploration, space technology, and the cultural developments influenced by these events. The Space Age began with the development of several technologies that culminated with the launch of Sputnik 1 by the Soviet Union. This was the world's first artificial satellite, orbiting the Earth in 98.1 minutes and weighing in at 83 kg. The launch of Sputnik 1 ushered a new era of political, scientific and technological achievements that became known as the Space Age. The Space Age was characterized by rapid development of new technology in a close race mostly between the United States and the Soviet Union. The Space Age brought the first human spaceflight during the Vostok programme and reached its peak with the Apollo program which captured the imagination of much of the world's population. The landing of Apollo 11 was an event watched by over 500 million people around the world and is widely recognized as one of the defining moments of the 20th century. Since then and with the end of the space race due to the dissolution of the Soviet Union, public attention has largely moved to other areas.", "The Space Age is a period encompassing the activities related to the Space Race, space exploration, space technology, and the cultural developments influenced by these events. The Space Age began with the development of several technologies that culminated with the launch of Sputnik 1 by the Soviet Union. This was the world's first artificial satellite, orbiting the Earth in 98.1 minutes and weighing in at 83 kg. The launch of Sputnik 1 ushered a new era of political, scientific and technological achievements that became known as the Space Age. The Space Age was characterized by rapid development of new technology in a close race mostly between the United States and the Soviet Union. The Space Age brought the first human spaceflight during the Vostok programme and reached its peak with the Apollo program which captured the imagination of much of the world's population. The landing of Apollo 11 was an event watched by over 500 million people around the world and is widely recognized as one of the defining moments of the 20th century. Since then and with the end of the space race due to the dissolution of the Soviet Union, public attention has largely moved to other areas.", "The Space Age is a period encompassing the activities related to the Space Race, space exploration, space technology, and the cultural developments influenced by these events. The Space Age began with the development of several technologies that culminated with the launch of Sputnik 1 by the Soviet Union. This was the world's first artificial satellite, orbiting the Earth in 98.1 minutes and weighing in at 83 kg. The launch of Sputnik 1 ushered a new era of political, scientific and technological achievements that became known as the Space Age. The Space Age was characterized by rapid development of new technology in a close race mostly between the United States and the Soviet Union. The Space Age brought the first human spaceflight during the Vostok programme and reached its peak with the Apollo program which captured the imagination of much of the world's population. The landing of Apollo 11 was an event watched by over 500 million people around the world and is widely recognized as one of the defining moments of the 20th century. Since then and with the end of the space race due to the dissolution of the Soviet Union, public attention has largely moved to other areas.", 'The phrase "51st state" can be used in a positive sense, meaning that a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state. It can also be used in a pejorative sense, meaning an area or region is perceived to be under excessive American cultural or military influence or control. In various countries around the world, people who believe their local or national culture has become too Americanized sometimes use the term "51st state" in reference to their own countries.', 'The phrase "51st state" can be used in a positive sense, meaning that a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state. It can also be used in a pejorative sense, meaning an area or region is perceived to be under excessive American cultural or military influence or control. In various countries around the world, people who believe their local or national culture has become too Americanized sometimes use the term "51st state" in reference to their own countries.', 'The phrase "51st state" can be used in a positive sense, meaning that a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state. It can also be used in a pejorative sense, meaning an area or region is perceived to be under excessive American cultural or military influence or control. In various countries around the world, people who believe their local or national culture has become too Americanized sometimes use the term "51st state" in reference to their own countries.', 'The phrase "51st state" can be used in a positive sense, meaning that a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state. It can also be used in a pejorative sense, meaning an area or region is perceived to be under excessive American cultural or military influence or control. In various countries around the world, people who believe their local or national culture has become too Americanized sometimes use the term "51st state" in reference to their own countries.', 'The phrase "51st state" can be used in a positive sense, meaning that a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state. It can also be used in a pejorative sense, meaning an area or region is perceived to be under excessive American cultural or military influence or control. In various countries around the world, people who believe their local or national culture has become too Americanized sometimes use the term "51st state" in reference to their own countries.', 'The phrase "51st state" can be used in a positive sense, meaning that a region or territory is so aligned, supportive, and conducive with the United States, that it is like a U.S. state. It can also be used in a pejorative sense, meaning an area or region is perceived to be under excessive American cultural or military influence or control. In various countries around the world, people who believe their local or national culture has become too Americanized sometimes use the term "51st state" in reference to their own countries.', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', 'Under Article IV, Section Three of the United States Constitution, which outlines the relationship among the states, Congress has the power to admit new states to the union. The states are required to give "full faith and credit" to the acts of each other\'s legislatures and courts, which is generally held to include the recognition of legal contracts, marriages, and criminal judgments. The states are guaranteed military and civil defense by the federal government, which is also obliged by Article IV, Section Four, to "guarantee to every state in this union a republican form of government".', "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Puerto Rico has been discussed as a potential 51st state of the United States. In a 2012 status referendum a majority of voters, 54%, expressed dissatisfaction with the current political relationship. In a separate question, 61% of voters supported statehood (excluding the 26% of voters who left this question blank). On December 11, 2012, Puerto Rico's legislature resolved to request that the President and the U.S. Congress act on the results, end the current form of territorial status and begin the process of admitting Puerto Rico to the Union as a state.", "Since 1898, Puerto Rico has had limited representation in the Congress in the form of a Resident Commissioner, a nonvoting delegate. The 110th Congress returned the Commissioner's power to vote in the Committee of the Whole, but not on matters where the vote would represent a decisive participation. Puerto Rico has elections on the United States presidential primary or caucus of the Democratic Party and the Republican Party to select delegates to the respective parties' national conventions although presidential electors are not granted on the Electoral College. As American citizens, Puerto Ricans can vote in U.S. presidential elections, provided they reside in one of the 50 states or the District of Columbia and not in Puerto Rico itself.", "Since 1898, Puerto Rico has had limited representation in the Congress in the form of a Resident Commissioner, a nonvoting delegate. The 110th Congress returned the Commissioner's power to vote in the Committee of the Whole, but not on matters where the vote would represent a decisive participation. Puerto Rico has elections on the United States presidential primary or caucus of the Democratic Party and the Republican Party to select delegates to the respective parties' national conventions although presidential electors are not granted on the Electoral College. As American citizens, Puerto Ricans can vote in U.S. presidential elections, provided they reside in one of the 50 states or the District of Columbia and not in Puerto Rico itself.", "Since 1898, Puerto Rico has had limited representation in the Congress in the form of a Resident Commissioner, a nonvoting delegate. The 110th Congress returned the Commissioner's power to vote in the Committee of the Whole, but not on matters where the vote would represent a decisive participation. Puerto Rico has elections on the United States presidential primary or caucus of the Democratic Party and the Republican Party to select delegates to the respective parties' national conventions although presidential electors are not granted on the Electoral College. As American citizens, Puerto Ricans can vote in U.S. presidential elections, provided they reside in one of the 50 states or the District of Columbia and not in Puerto Rico itself.", "Since 1898, Puerto Rico has had limited representation in the Congress in the form of a Resident Commissioner, a nonvoting delegate. The 110th Congress returned the Commissioner's power to vote in the Committee of the Whole, but not on matters where the vote would represent a decisive participation. Puerto Rico has elections on the United States presidential primary or caucus of the Democratic Party and the Republican Party to select delegates to the respective parties' national conventions although presidential electors are not granted on the Electoral College. As American citizens, Puerto Ricans can vote in U.S. presidential elections, provided they reside in one of the 50 states or the District of Columbia and not in Puerto Rico itself.", 'Residents of Puerto Rico pay U.S. federal taxes: import/export taxes, federal commodity taxes, social security taxes, therefore contributing to the American Government. Most Puerto Rico residents do not pay federal income tax but do pay federal payroll taxes (Social Security and Medicare). However, federal employees, those who do business with the federal government, Puerto Rico–based corporations that intend to send funds to the U.S. and others do pay federal income taxes. Puerto Ricans may enlist in the U.S. military. Puerto Ricans have participated in all American wars since 1898; 52 Puerto Ricans had been killed in the Iraq War and War in Afghanistan by November 2012.', 'Residents of Puerto Rico pay U.S. federal taxes: import/export taxes, federal commodity taxes, social security taxes, therefore contributing to the American Government. Most Puerto Rico residents do not pay federal income tax but do pay federal payroll taxes (Social Security and Medicare). However, federal employees, those who do business with the federal government, Puerto Rico–based corporations that intend to send funds to the U.S. and others do pay federal income taxes. Puerto Ricans may enlist in the U.S. military. Puerto Ricans have participated in all American wars since 1898; 52 Puerto Ricans had been killed in the Iraq War and War in Afghanistan by November 2012.', 'Residents of Puerto Rico pay U.S. federal taxes: import/export taxes, federal commodity taxes, social security taxes, therefore contributing to the American Government. Most Puerto Rico residents do not pay federal income tax but do pay federal payroll taxes (Social Security and Medicare). However, federal employees, those who do business with the federal government, Puerto Rico–based corporations that intend to send funds to the U.S. and others do pay federal income taxes. Puerto Ricans may enlist in the U.S. military. Puerto Ricans have participated in all American wars since 1898; 52 Puerto Ricans had been killed in the Iraq War and War in Afghanistan by November 2012.', 'Residents of Puerto Rico pay U.S. federal taxes: import/export taxes, federal commodity taxes, social security taxes, therefore contributing to the American Government. Most Puerto Rico residents do not pay federal income tax but do pay federal payroll taxes (Social Security and Medicare). However, federal employees, those who do business with the federal government, Puerto Rico–based corporations that intend to send funds to the U.S. and others do pay federal income taxes. Puerto Ricans may enlist in the U.S. military. Puerto Ricans have participated in all American wars since 1898; 52 Puerto Ricans had been killed in the Iraq War and War in Afghanistan by November 2012.', "Puerto Rico has been under U.S. sovereignty for over a century when it was ceded to the U.S. by Spain following the end of the Spanish–American War, and Puerto Ricans have been U.S. citizens since 1917. The island's ultimate status has not been determined as of 2012[update], its residents do not have voting representation in their federal government. Puerto Rico has limited representation in the U.S. Congress in the form of a Resident Commissioner, a delegate with limited no voting rights. Like the states, Puerto Rico has self-rule, a republican form of government organized pursuant to a constitution adopted by its people, and a bill of rights.", "Puerto Rico has been under U.S. sovereignty for over a century when it was ceded to the U.S. by Spain following the end of the Spanish–American War, and Puerto Ricans have been U.S. citizens since 1917. The island's ultimate status has not been determined as of 2012[update], its residents do not have voting representation in their federal government. Puerto Rico has limited representation in the U.S. Congress in the form of a Resident Commissioner, a delegate with limited no voting rights. Like the states, Puerto Rico has self-rule, a republican form of government organized pursuant to a constitution adopted by its people, and a bill of rights.", "Puerto Rico has been under U.S. sovereignty for over a century when it was ceded to the U.S. by Spain following the end of the Spanish–American War, and Puerto Ricans have been U.S. citizens since 1917. The island's ultimate status has not been determined as of 2012[update], its residents do not have voting representation in their federal government. Puerto Rico has limited representation in the U.S. Congress in the form of a Resident Commissioner, a delegate with limited no voting rights. Like the states, Puerto Rico has self-rule, a republican form of government organized pursuant to a constitution adopted by its people, and a bill of rights.", "Puerto Rico has been under U.S. sovereignty for over a century when it was ceded to the U.S. by Spain following the end of the Spanish–American War, and Puerto Ricans have been U.S. citizens since 1917. The island's ultimate status has not been determined as of 2012[update], its residents do not have voting representation in their federal government. Puerto Rico has limited representation in the U.S. Congress in the form of a Resident Commissioner, a delegate with limited no voting rights. Like the states, Puerto Rico has self-rule, a republican form of government organized pursuant to a constitution adopted by its people, and a bill of rights.", 'This constitution was created when the U.S. Congress directed local government to organize a constitutional convention to write the Puerto Rico Constitution in 1951. The acceptance of that constitution by Puerto Rico\'s electorate, the U.S. Congress, and the U.S. president occurred in 1952. In addition, the rights, privileges and immunities attendant to United States citizens are "respected in Puerto Rico to the same extent as though Puerto Rico were a state of the union" through the express extension of the Privileges and Immunities Clause of the U.S. Constitution by the U.S. Congress in 1948.', 'This constitution was created when the U.S. Congress directed local government to organize a constitutional convention to write the Puerto Rico Constitution in 1951. The acceptance of that constitution by Puerto Rico\'s electorate, the U.S. Congress, and the U.S. president occurred in 1952. In addition, the rights, privileges and immunities attendant to United States citizens are "respected in Puerto Rico to the same extent as though Puerto Rico were a state of the union" through the express extension of the Privileges and Immunities Clause of the U.S. Constitution by the U.S. Congress in 1948.', 'This constitution was created when the U.S. Congress directed local government to organize a constitutional convention to write the Puerto Rico Constitution in 1951. The acceptance of that constitution by Puerto Rico\'s electorate, the U.S. Congress, and the U.S. president occurred in 1952. In addition, the rights, privileges and immunities attendant to United States citizens are "respected in Puerto Rico to the same extent as though Puerto Rico were a state of the union" through the express extension of the Privileges and Immunities Clause of the U.S. Constitution by the U.S. Congress in 1948.', 'This constitution was created when the U.S. Congress directed local government to organize a constitutional convention to write the Puerto Rico Constitution in 1951. The acceptance of that constitution by Puerto Rico\'s electorate, the U.S. Congress, and the U.S. president occurred in 1952. In addition, the rights, privileges and immunities attendant to United States citizens are "respected in Puerto Rico to the same extent as though Puerto Rico were a state of the union" through the express extension of the Privileges and Immunities Clause of the U.S. Constitution by the U.S. Congress in 1948.', 'Puerto Rico is designated in its constitution as the "Commonwealth of Puerto Rico". The Constitution of Puerto Rico which became effective in 1952 adopted the name of Estado Libre Asociado (literally translated as "Free Associated State"), officially translated into English as Commonwealth, for its body politic. The island is under the jurisdiction of the Territorial Clause of the U.S. Constitution, which has led to doubts about the finality of the Commonwealth status for Puerto Rico. In addition, all people born in Puerto Rico become citizens of the U.S. at birth (under provisions of the Jones–Shafroth Act in 1917), but citizens residing in Puerto Rico cannot vote for president nor for full members of either house of Congress. Statehood would grant island residents full voting rights at the Federal level. The Puerto Rico Democracy Act (H.R. 2499) was approved on April 29, 2010, by the United States House of Representatives 223–169, but was not approved by the Senate before the end of the 111th Congress. It would have provided for a federally sanctioned self-determination process for the people of Puerto Rico. This act would provide for referendums to be held in Puerto Rico to determine the island\'s ultimate political status. It had also been introduced in 2007.', 'Puerto Rico is designated in its constitution as the "Commonwealth of Puerto Rico". The Constitution of Puerto Rico which became effective in 1952 adopted the name of Estado Libre Asociado (literally translated as "Free Associated State"), officially translated into English as Commonwealth, for its body politic. The island is under the jurisdiction of the Territorial Clause of the U.S. Constitution, which has led to doubts about the finality of the Commonwealth status for Puerto Rico. In addition, all people born in Puerto Rico become citizens of the U.S. at birth (under provisions of the Jones–Shafroth Act in 1917), but citizens residing in Puerto Rico cannot vote for president nor for full members of either house of Congress. Statehood would grant island residents full voting rights at the Federal level. The Puerto Rico Democracy Act (H.R. 2499) was approved on April 29, 2010, by the United States House of Representatives 223–169, but was not approved by the Senate before the end of the 111th Congress. It would have provided for a federally sanctioned self-determination process for the people of Puerto Rico. This act would provide for referendums to be held in Puerto Rico to determine the island\'s ultimate political status. It had also been introduced in 2007.', 'Puerto Rico is designated in its constitution as the "Commonwealth of Puerto Rico". The Constitution of Puerto Rico which became effective in 1952 adopted the name of Estado Libre Asociado (literally translated as "Free Associated State"), officially translated into English as Commonwealth, for its body politic. The island is under the jurisdiction of the Territorial Clause of the U.S. Constitution, which has led to doubts about the finality of the Commonwealth status for Puerto Rico. In addition, all people born in Puerto Rico become citizens of the U.S. at birth (under provisions of the Jones–Shafroth Act in 1917), but citizens residing in Puerto Rico cannot vote for president nor for full members of either house of Congress. Statehood would grant island residents full voting rights at the Federal level. The Puerto Rico Democracy Act (H.R. 2499) was approved on April 29, 2010, by the United States House of Representatives 223–169, but was not approved by the Senate before the end of the 111th Congress. It would have provided for a federally sanctioned self-determination process for the people of Puerto Rico. This act would provide for referendums to be held in Puerto Rico to determine the island\'s ultimate political status. It had also been introduced in 2007.', 'Puerto Rico is designated in its constitution as the "Commonwealth of Puerto Rico". The Constitution of Puerto Rico which became effective in 1952 adopted the name of Estado Libre Asociado (literally translated as "Free Associated State"), officially translated into English as Commonwealth, for its body politic. The island is under the jurisdiction of the Territorial Clause of the U.S. Constitution, which has led to doubts about the finality of the Commonwealth status for Puerto Rico. In addition, all people born in Puerto Rico become citizens of the U.S. at birth (under provisions of the Jones–Shafroth Act in 1917), but citizens residing in Puerto Rico cannot vote for president nor for full members of either house of Congress. Statehood would grant island residents full voting rights at the Federal level. The Puerto Rico Democracy Act (H.R. 2499) was approved on April 29, 2010, by the United States House of Representatives 223–169, but was not approved by the Senate before the end of the 111th Congress. It would have provided for a federally sanctioned self-determination process for the people of Puerto Rico. This act would provide for referendums to be held in Puerto Rico to determine the island\'s ultimate political status. It had also been introduced in 2007.', 'In November 2012, a referendum resulted in 54 percent of respondents voting to reject the current status under the territorial clause of the U.S. Constitution, while a second question resulted in 61 percent of voters identifying statehood as the preferred alternative to the current territorial status. The 2012 referendum was by far the most successful referendum for statehood advocates and support for statehood has risen in each successive popular referendum. However, more than one in four voters abstained from answering the question on the preferred alternative status. Statehood opponents have argued that the statehood option garnered only 45 percent of the votes if abstentions are included. If abstentions are considered, the result of the referendum is much closer to 44 percent for statehood, a number that falls under the 50 percent majority mark.', 'In November 2012, a referendum resulted in 54 percent of respondents voting to reject the current status under the territorial clause of the U.S. Constitution, while a second question resulted in 61 percent of voters identifying statehood as the preferred alternative to the current territorial status. The 2012 referendum was by far the most successful referendum for statehood advocates and support for statehood has risen in each successive popular referendum. However, more than one in four voters abstained from answering the question on the preferred alternative status. Statehood opponents have argued that the statehood option garnered only 45 percent of the votes if abstentions are included. If abstentions are considered, the result of the referendum is much closer to 44 percent for statehood, a number that falls under the 50 percent majority mark.', 'In November 2012, a referendum resulted in 54 percent of respondents voting to reject the current status under the territorial clause of the U.S. Constitution, while a second question resulted in 61 percent of voters identifying statehood as the preferred alternative to the current territorial status. The 2012 referendum was by far the most successful referendum for statehood advocates and support for statehood has risen in each successive popular referendum. However, more than one in four voters abstained from answering the question on the preferred alternative status. Statehood opponents have argued that the statehood option garnered only 45 percent of the votes if abstentions are included. If abstentions are considered, the result of the referendum is much closer to 44 percent for statehood, a number that falls under the 50 percent majority mark.', 'In November 2012, a referendum resulted in 54 percent of respondents voting to reject the current status under the territorial clause of the U.S. Constitution, while a second question resulted in 61 percent of voters identifying statehood as the preferred alternative to the current territorial status. The 2012 referendum was by far the most successful referendum for statehood advocates and support for statehood has risen in each successive popular referendum. However, more than one in four voters abstained from answering the question on the preferred alternative status. Statehood opponents have argued that the statehood option garnered only 45 percent of the votes if abstentions are included. If abstentions are considered, the result of the referendum is much closer to 44 percent for statehood, a number that falls under the 50 percent majority mark.', 'The Washington Post, The New York Times and the Boston Herald have published opinion pieces expressing support for the statehood of Puerto Rico. On November 8, 2012, Washington, D.C. newspaper The Hill published an article saying that Congress will likely ignore the results of the referendum due to the circumstances behind the votes. and U.S. Congressman Luis Gutiérrez U.S. Congresswoman Nydia Velázquez, both of Puerto Rican ancestry, agreed with the The Hill \'s statements. Shortly after the results were published Puerto Rico-born U.S. Congressman José Enrique Serrano commented "I was particularly impressed with the outcome of the \'status\' referendum in Puerto Rico. A majority of those voting signaled the desire to change the current territorial status. In a second question an even larger majority asked to become a state. This is an earthquake in Puerto Rican politics. It will demand the attention of Congress, and a definitive answer to the Puerto Rican request for change. This is a history-making moment where voters asked to move forward."', 'The Washington Post, The New York Times and the Boston Herald have published opinion pieces expressing support for the statehood of Puerto Rico. On November 8, 2012, Washington, D.C. newspaper The Hill published an article saying that Congress will likely ignore the results of the referendum due to the circumstances behind the votes. and U.S. Congressman Luis Gutiérrez U.S. Congresswoman Nydia Velázquez, both of Puerto Rican ancestry, agreed with the The Hill \'s statements. Shortly after the results were published Puerto Rico-born U.S. Congressman José Enrique Serrano commented "I was particularly impressed with the outcome of the \'status\' referendum in Puerto Rico. A majority of those voting signaled the desire to change the current territorial status. In a second question an even larger majority asked to become a state. This is an earthquake in Puerto Rican politics. It will demand the attention of Congress, and a definitive answer to the Puerto Rican request for change. This is a history-making moment where voters asked to move forward."', 'The Washington Post, The New York Times and the Boston Herald have published opinion pieces expressing support for the statehood of Puerto Rico. On November 8, 2012, Washington, D.C. newspaper The Hill published an article saying that Congress will likely ignore the results of the referendum due to the circumstances behind the votes. and U.S. Congressman Luis Gutiérrez U.S. Congresswoman Nydia Velázquez, both of Puerto Rican ancestry, agreed with the The Hill \'s statements. Shortly after the results were published Puerto Rico-born U.S. Congressman José Enrique Serrano commented "I was particularly impressed with the outcome of the \'status\' referendum in Puerto Rico. A majority of those voting signaled the desire to change the current territorial status. In a second question an even larger majority asked to become a state. This is an earthquake in Puerto Rican politics. It will demand the attention of Congress, and a definitive answer to the Puerto Rican request for change. This is a history-making moment where voters asked to move forward."', 'The Washington Post, The New York Times and the Boston Herald have published opinion pieces expressing support for the statehood of Puerto Rico. On November 8, 2012, Washington, D.C. newspaper The Hill published an article saying that Congress will likely ignore the results of the referendum due to the circumstances behind the votes. and U.S. Congressman Luis Gutiérrez U.S. Congresswoman Nydia Velázquez, both of Puerto Rican ancestry, agreed with the The Hill \'s statements. Shortly after the results were published Puerto Rico-born U.S. Congressman José Enrique Serrano commented "I was particularly impressed with the outcome of the \'status\' referendum in Puerto Rico. A majority of those voting signaled the desire to change the current territorial status. In a second question an even larger majority asked to become a state. This is an earthquake in Puerto Rican politics. It will demand the attention of Congress, and a definitive answer to the Puerto Rican request for change. This is a history-making moment where voters asked to move forward."', 'Several days after the referendum, the Resident Commissioner Pedro Pierluisi, Governor Luis Fortuño, and Governor-elect Alejandro García Padilla wrote separate letters to the President of the United States Barack Obama addressing the results of the voting. Pierluisi urged Obama to begin legislation in favor of the statehood of Puerto Rico, in light of its win in the referendum. Fortuño urged him to move the process forward. García Padilla asked him to reject the results because of their ambiguity. The White House stance related to the November 2012 plebiscite was that the results were clear, the people of Puerto Rico want the issue of status resolved, and a majority chose statehood in the second question. Former White House director of Hispanic media stated, "Now it is time for Congress to act and the administration will work with them on that effort, so that the people of Puerto Rico can determine their own future."', 'Several days after the referendum, the Resident Commissioner Pedro Pierluisi, Governor Luis Fortuño, and Governor-elect Alejandro García Padilla wrote separate letters to the President of the United States Barack Obama addressing the results of the voting. Pierluisi urged Obama to begin legislation in favor of the statehood of Puerto Rico, in light of its win in the referendum. Fortuño urged him to move the process forward. García Padilla asked him to reject the results because of their ambiguity. The White House stance related to the November 2012 plebiscite was that the results were clear, the people of Puerto Rico want the issue of status resolved, and a majority chose statehood in the second question. Former White House director of Hispanic media stated, "Now it is time for Congress to act and the administration will work with them on that effort, so that the people of Puerto Rico can determine their own future."', 'Several days after the referendum, the Resident Commissioner Pedro Pierluisi, Governor Luis Fortuño, and Governor-elect Alejandro García Padilla wrote separate letters to the President of the United States Barack Obama addressing the results of the voting. Pierluisi urged Obama to begin legislation in favor of the statehood of Puerto Rico, in light of its win in the referendum. Fortuño urged him to move the process forward. García Padilla asked him to reject the results because of their ambiguity. The White House stance related to the November 2012 plebiscite was that the results were clear, the people of Puerto Rico want the issue of status resolved, and a majority chose statehood in the second question. Former White House director of Hispanic media stated, "Now it is time for Congress to act and the administration will work with them on that effort, so that the people of Puerto Rico can determine their own future."', 'Several days after the referendum, the Resident Commissioner Pedro Pierluisi, Governor Luis Fortuño, and Governor-elect Alejandro García Padilla wrote separate letters to the President of the United States Barack Obama addressing the results of the voting. Pierluisi urged Obama to begin legislation in favor of the statehood of Puerto Rico, in light of its win in the referendum. Fortuño urged him to move the process forward. García Padilla asked him to reject the results because of their ambiguity. The White House stance related to the November 2012 plebiscite was that the results were clear, the people of Puerto Rico want the issue of status resolved, and a majority chose statehood in the second question. Former White House director of Hispanic media stated, "Now it is time for Congress to act and the administration will work with them on that effort, so that the people of Puerto Rico can determine their own future."', 'On May 15, 2013, Resident Commissioner Pierluisi introduced H.R. 2000 to Congress to "set forth the process for Puerto Rico to be admitted as a state of the Union," asking for Congress to vote on ratifying Puerto Rico as the 51st state. On February 12, 2014, Senator Martin Heinrich introduced a bill in the US Senate. The bill would require a binding referendum to be held in Puerto Rico asking whether the territory wants to be admitted as a state. In the event of a yes vote, the president would be asked to submit legislation to Congress to admit Puerto Rico as a state.', 'On May 15, 2013, Resident Commissioner Pierluisi introduced H.R. 2000 to Congress to "set forth the process for Puerto Rico to be admitted as a state of the Union," asking for Congress to vote on ratifying Puerto Rico as the 51st state. On February 12, 2014, Senator Martin Heinrich introduced a bill in the US Senate. The bill would require a binding referendum to be held in Puerto Rico asking whether the territory wants to be admitted as a state. In the event of a yes vote, the president would be asked to submit legislation to Congress to admit Puerto Rico as a state.', 'On May 15, 2013, Resident Commissioner Pierluisi introduced H.R. 2000 to Congress to "set forth the process for Puerto Rico to be admitted as a state of the Union," asking for Congress to vote on ratifying Puerto Rico as the 51st state. On February 12, 2014, Senator Martin Heinrich introduced a bill in the US Senate. The bill would require a binding referendum to be held in Puerto Rico asking whether the territory wants to be admitted as a state. In the event of a yes vote, the president would be asked to submit legislation to Congress to admit Puerto Rico as a state.', 'On May 15, 2013, Resident Commissioner Pierluisi introduced H.R. 2000 to Congress to "set forth the process for Puerto Rico to be admitted as a state of the Union," asking for Congress to vote on ratifying Puerto Rico as the 51st state. On February 12, 2014, Senator Martin Heinrich introduced a bill in the US Senate. The bill would require a binding referendum to be held in Puerto Rico asking whether the territory wants to be admitted as a state. In the event of a yes vote, the president would be asked to submit legislation to Congress to admit Puerto Rico as a state.', 'Washington, D.C. is often mentioned as a candidate for statehood. In Federalist No. 43 of The Federalist Papers, James Madison considered the implications of the definition of the "seat of government" found in the United States Constitution. Although he noted potential conflicts of interest, and the need for a "municipal legislature for local purposes," Madison did not address the district\'s role in national voting. Legal scholars disagree on whether a simple act of Congress can admit the District as a state, due to its status as the seat of government of the United States, which Article I, Section 8 of the Constitution requires to be under the exclusive jurisdiction of Congress; depending on the interpretation of this text, admission of the full District as a state may require a Constitutional amendment, which is much more difficult to enact. However, the Constitution does not set a minimum size for the District. Its size has already changed once before, when Virginia reclaimed the portion of the District south of the Potomac. So the constitutional requirement for a federal district can be satisfied by reducing its size to the small central core of government buildings and monuments, giving the rest of the territory to the new state.', 'Washington, D.C. is often mentioned as a candidate for statehood. In Federalist No. 43 of The Federalist Papers, James Madison considered the implications of the definition of the "seat of government" found in the United States Constitution. Although he noted potential conflicts of interest, and the need for a "municipal legislature for local purposes," Madison did not address the district\'s role in national voting. Legal scholars disagree on whether a simple act of Congress can admit the District as a state, due to its status as the seat of government of the United States, which Article I, Section 8 of the Constitution requires to be under the exclusive jurisdiction of Congress; depending on the interpretation of this text, admission of the full District as a state may require a Constitutional amendment, which is much more difficult to enact. However, the Constitution does not set a minimum size for the District. Its size has already changed once before, when Virginia reclaimed the portion of the District south of the Potomac. So the constitutional requirement for a federal district can be satisfied by reducing its size to the small central core of government buildings and monuments, giving the rest of the territory to the new state.', 'Washington, D.C. is often mentioned as a candidate for statehood. In Federalist No. 43 of The Federalist Papers, James Madison considered the implications of the definition of the "seat of government" found in the United States Constitution. Although he noted potential conflicts of interest, and the need for a "municipal legislature for local purposes," Madison did not address the district\'s role in national voting. Legal scholars disagree on whether a simple act of Congress can admit the District as a state, due to its status as the seat of government of the United States, which Article I, Section 8 of the Constitution requires to be under the exclusive jurisdiction of Congress; depending on the interpretation of this text, admission of the full District as a state may require a Constitutional amendment, which is much more difficult to enact. However, the Constitution does not set a minimum size for the District. Its size has already changed once before, when Virginia reclaimed the portion of the District south of the Potomac. So the constitutional requirement for a federal district can be satisfied by reducing its size to the small central core of government buildings and monuments, giving the rest of the territory to the new state.', 'Washington, D.C. is often mentioned as a candidate for statehood. In Federalist No. 43 of The Federalist Papers, James Madison considered the implications of the definition of the "seat of government" found in the United States Constitution. Although he noted potential conflicts of interest, and the need for a "municipal legislature for local purposes," Madison did not address the district\'s role in national voting. Legal scholars disagree on whether a simple act of Congress can admit the District as a state, due to its status as the seat of government of the United States, which Article I, Section 8 of the Constitution requires to be under the exclusive jurisdiction of Congress; depending on the interpretation of this text, admission of the full District as a state may require a Constitutional amendment, which is much more difficult to enact. However, the Constitution does not set a minimum size for the District. Its size has already changed once before, when Virginia reclaimed the portion of the District south of the Potomac. So the constitutional requirement for a federal district can be satisfied by reducing its size to the small central core of government buildings and monuments, giving the rest of the territory to the new state.', 'Washington, D.C. residents who support the statehood movement sometimes use a shortened version of the Revolutionary War protest motto "No taxation without representation", omitting the initial "No", denoting their lack of Congressional representation; the phrase is now printed on newly issued Washington, D.C. license plates (although a driver may choose to have the Washington, D.C. website address instead). President Bill Clinton\'s presidential limousine had the "Taxation without representation" license plate late in his term, while President George W. Bush had the vehicle\'s plates changed shortly after beginning his term in office. President Barack Obama had the license plates changed back to the protest style at the beginning of his second term.', 'Washington, D.C. residents who support the statehood movement sometimes use a shortened version of the Revolutionary War protest motto "No taxation without representation", omitting the initial "No", denoting their lack of Congressional representation; the phrase is now printed on newly issued Washington, D.C. license plates (although a driver may choose to have the Washington, D.C. website address instead). President Bill Clinton\'s presidential limousine had the "Taxation without representation" license plate late in his term, while President George W. Bush had the vehicle\'s plates changed shortly after beginning his term in office. President Barack Obama had the license plates changed back to the protest style at the beginning of his second term.', 'Washington, D.C. residents who support the statehood movement sometimes use a shortened version of the Revolutionary War protest motto "No taxation without representation", omitting the initial "No", denoting their lack of Congressional representation; the phrase is now printed on newly issued Washington, D.C. license plates (although a driver may choose to have the Washington, D.C. website address instead). President Bill Clinton\'s presidential limousine had the "Taxation without representation" license plate late in his term, while President George W. Bush had the vehicle\'s plates changed shortly after beginning his term in office. President Barack Obama had the license plates changed back to the protest style at the beginning of his second term.', 'Washington, D.C. residents who support the statehood movement sometimes use a shortened version of the Revolutionary War protest motto "No taxation without representation", omitting the initial "No", denoting their lack of Congressional representation; the phrase is now printed on newly issued Washington, D.C. license plates (although a driver may choose to have the Washington, D.C. website address instead). President Bill Clinton\'s presidential limousine had the "Taxation without representation" license plate late in his term, while President George W. Bush had the vehicle\'s plates changed shortly after beginning his term in office. President Barack Obama had the license plates changed back to the protest style at the beginning of his second term.', 'This position was carried by the D.C. Statehood Party, a political party; it has since merged with the local Green Party affiliate to form the D.C. Statehood Green Party. The nearest this movement ever came to success was in 1978, when Congress passed the District of Columbia Voting Rights Amendment. Two years later in 1980, local citizens passed an initiative calling for a constitutional convention for a new state. In 1982, voters ratified the constitution of the state, which was to be called New Columbia. The drive for statehood stalled in 1985, however, when the Washington, D.C. Voting Rights Amendment failed because not enough states ratified the amendment within the seven-year span specified.', 'This position was carried by the D.C. Statehood Party, a political party; it has since merged with the local Green Party affiliate to form the D.C. Statehood Green Party. The nearest this movement ever came to success was in 1978, when Congress passed the District of Columbia Voting Rights Amendment. Two years later in 1980, local citizens passed an initiative calling for a constitutional convention for a new state. In 1982, voters ratified the constitution of the state, which was to be called New Columbia. The drive for statehood stalled in 1985, however, when the Washington, D.C. Voting Rights Amendment failed because not enough states ratified the amendment within the seven-year span specified.', 'This position was carried by the D.C. Statehood Party, a political party; it has since merged with the local Green Party affiliate to form the D.C. Statehood Green Party. The nearest this movement ever came to success was in 1978, when Congress passed the District of Columbia Voting Rights Amendment. Two years later in 1980, local citizens passed an initiative calling for a constitutional convention for a new state. In 1982, voters ratified the constitution of the state, which was to be called New Columbia. The drive for statehood stalled in 1985, however, when the Washington, D.C. Voting Rights Amendment failed because not enough states ratified the amendment within the seven-year span specified.', 'This position was carried by the D.C. Statehood Party, a political party; it has since merged with the local Green Party affiliate to form the D.C. Statehood Green Party. The nearest this movement ever came to success was in 1978, when Congress passed the District of Columbia Voting Rights Amendment. Two years later in 1980, local citizens passed an initiative calling for a constitutional convention for a new state. In 1982, voters ratified the constitution of the state, which was to be called New Columbia. The drive for statehood stalled in 1985, however, when the Washington, D.C. Voting Rights Amendment failed because not enough states ratified the amendment within the seven-year span specified.', 'This position was carried by the D.C. Statehood Party, a political party; it has since merged with the local Green Party affiliate to form the D.C. Statehood Green Party. The nearest this movement ever came to success was in 1978, when Congress passed the District of Columbia Voting Rights Amendment. Two years later in 1980, local citizens passed an initiative calling for a constitutional convention for a new state. In 1982, voters ratified the constitution of the state, which was to be called New Columbia. The drive for statehood stalled in 1985, however, when the Washington, D.C. Voting Rights Amendment failed because not enough states ratified the amendment within the seven-year span specified.', 'Other less likely contenders are Guam and the United States Virgin Islands, both of which are unincorporated organized territories of the United States. Also, the Northern Mariana Islands and American Samoa, an unorganized, unincorporated territory, could both attempt to gain statehood. Some proposals call for the Virgin Islands to be admitted with Puerto Rico as one state (often known as the proposed "Commonwealth of Prusvi", for Puerto Rico/U.S. Virgin Islands, or as "Puerto Virgo"), and for the amalgamation of U.S. territories or former territories in the Pacific Ocean, in the manner of the "Greater Hawaii" concept of the 1960s. Guam and the Northern Mariana Islands would be admitted as one state, along with Palau, the Federated States of Micronesia, and the Marshall Islands (although these latter three entities are now separate sovereign nations, which have Compact of Free Association relationships with the United States). Such a state would have a population of 412,381 (slightly lower than Wyoming\'s population) and a land area of 911.82 square miles (2,361.6 km2) (slightly smaller than Rhode Island). American Samoa could possibly be part of such a state, increasing the population to 467,900 and the area to 988.65 square miles (2,560.6 km2). Radio Australia, in late May 2008, issued signs of Guam and the Northern Mariana Islands becoming one again and becoming the 51st state.', 'Other less likely contenders are Guam and the United States Virgin Islands, both of which are unincorporated organized territories of the United States. Also, the Northern Mariana Islands and American Samoa, an unorganized, unincorporated territory, could both attempt to gain statehood. Some proposals call for the Virgin Islands to be admitted with Puerto Rico as one state (often known as the proposed "Commonwealth of Prusvi", for Puerto Rico/U.S. Virgin Islands, or as "Puerto Virgo"), and for the amalgamation of U.S. territories or former territories in the Pacific Ocean, in the manner of the "Greater Hawaii" concept of the 1960s. Guam and the Northern Mariana Islands would be admitted as one state, along with Palau, the Federated States of Micronesia, and the Marshall Islands (although these latter three entities are now separate sovereign nations, which have Compact of Free Association relationships with the United States). Such a state would have a population of 412,381 (slightly lower than Wyoming\'s population) and a land area of 911.82 square miles (2,361.6 km2) (slightly smaller than Rhode Island). American Samoa could possibly be part of such a state, increasing the population to 467,900 and the area to 988.65 square miles (2,560.6 km2). Radio Australia, in late May 2008, issued signs of Guam and the Northern Mariana Islands becoming one again and becoming the 51st state.', 'Other less likely contenders are Guam and the United States Virgin Islands, both of which are unincorporated organized territories of the United States. Also, the Northern Mariana Islands and American Samoa, an unorganized, unincorporated territory, could both attempt to gain statehood. Some proposals call for the Virgin Islands to be admitted with Puerto Rico as one state (often known as the proposed "Commonwealth of Prusvi", for Puerto Rico/U.S. Virgin Islands, or as "Puerto Virgo"), and for the amalgamation of U.S. territories or former territories in the Pacific Ocean, in the manner of the "Greater Hawaii" concept of the 1960s. Guam and the Northern Mariana Islands would be admitted as one state, along with Palau, the Federated States of Micronesia, and the Marshall Islands (although these latter three entities are now separate sovereign nations, which have Compact of Free Association relationships with the United States). Such a state would have a population of 412,381 (slightly lower than Wyoming\'s population) and a land area of 911.82 square miles (2,361.6 km2) (slightly smaller than Rhode Island). American Samoa could possibly be part of such a state, increasing the population to 467,900 and the area to 988.65 square miles (2,560.6 km2). Radio Australia, in late May 2008, issued signs of Guam and the Northern Mariana Islands becoming one again and becoming the 51st state.', 'The Philippines has had small grassroots movements for U.S. statehood. Originally part of the platform of the Progressive Party, then known as the Federalista Party, the party dropped it in 1907, which coincided with the name change. As recently as 2004, the concept of the Philippines becoming a U.S. state has been part of a political platform in the Philippines. Supporters of this movement include Filipinos who believe that the quality of life in the Philippines would be higher and that there would be less poverty there if the Philippines were an American state or territory. Supporters also include Filipinos that had fought as members of the United States Armed Forces in various wars during the Commonwealth period.', 'The Philippines has had small grassroots movements for U.S. statehood. Originally part of the platform of the Progressive Party, then known as the Federalista Party, the party dropped it in 1907, which coincided with the name change. As recently as 2004, the concept of the Philippines becoming a U.S. state has been part of a political platform in the Philippines. Supporters of this movement include Filipinos who believe that the quality of life in the Philippines would be higher and that there would be less poverty there if the Philippines were an American state or territory. Supporters also include Filipinos that had fought as members of the United States Armed Forces in various wars during the Commonwealth period.', 'The Philippines has had small grassroots movements for U.S. statehood. Originally part of the platform of the Progressive Party, then known as the Federalista Party, the party dropped it in 1907, which coincided with the name change. As recently as 2004, the concept of the Philippines becoming a U.S. state has been part of a political platform in the Philippines. Supporters of this movement include Filipinos who believe that the quality of life in the Philippines would be higher and that there would be less poverty there if the Philippines were an American state or territory. Supporters also include Filipinos that had fought as members of the United States Armed Forces in various wars during the Commonwealth period.', 'In Canada, "the 51st state" is a phrase generally used in such a way as to imply that if a certain political course is taken, Canada\'s destiny will be as little more than a part of the United States. Examples include the Canada-United States Free Trade Agreement in 1988, the debate over the creation of a common defense perimeter, and as a potential consequence of not adopting proposals intended to resolve the issue of Quebec sovereignty, the Charlottetown Accord in 1992 and the Clarity Act in 1999.', 'In Canada, "the 51st state" is a phrase generally used in such a way as to imply that if a certain political course is taken, Canada\'s destiny will be as little more than a part of the United States. Examples include the Canada-United States Free Trade Agreement in 1988, the debate over the creation of a common defense perimeter, and as a potential consequence of not adopting proposals intended to resolve the issue of Quebec sovereignty, the Charlottetown Accord in 1992 and the Clarity Act in 1999.', 'In Canada, "the 51st state" is a phrase generally used in such a way as to imply that if a certain political course is taken, Canada\'s destiny will be as little more than a part of the United States. Examples include the Canada-United States Free Trade Agreement in 1988, the debate over the creation of a common defense perimeter, and as a potential consequence of not adopting proposals intended to resolve the issue of Quebec sovereignty, the Charlottetown Accord in 1992 and the Clarity Act in 1999.', 'In Canada, "the 51st state" is a phrase generally used in such a way as to imply that if a certain political course is taken, Canada\'s destiny will be as little more than a part of the United States. Examples include the Canada-United States Free Trade Agreement in 1988, the debate over the creation of a common defense perimeter, and as a potential consequence of not adopting proposals intended to resolve the issue of Quebec sovereignty, the Charlottetown Accord in 1992 and the Clarity Act in 1999.', 'The phrase is usually used in local political debates, in polemic writing or in private conversations. It is rarely used by politicians themselves in a public context, although at certain times in Canadian history political parties have used other similarly loaded imagery. In the 1988 federal election, the Liberals asserted that the proposed Free Trade Agreement amounted to an American takeover of Canada—notably, the party ran an ad in which Progressive Conservative (PC) strategists, upon the adoption of the agreement, slowly erased the Canada-U.S. border from a desktop map of North America. Within days, however, the PCs responded with an ad which featured the border being drawn back on with a permanent marker, as an announcer intoned "Here\'s where we draw the line."', 'The phrase is usually used in local political debates, in polemic writing or in private conversations. It is rarely used by politicians themselves in a public context, although at certain times in Canadian history political parties have used other similarly loaded imagery. In the 1988 federal election, the Liberals asserted that the proposed Free Trade Agreement amounted to an American takeover of Canada—notably, the party ran an ad in which Progressive Conservative (PC) strategists, upon the adoption of the agreement, slowly erased the Canada-U.S. border from a desktop map of North America. Within days, however, the PCs responded with an ad which featured the border being drawn back on with a permanent marker, as an announcer intoned "Here\'s where we draw the line."', 'The phrase is usually used in local political debates, in polemic writing or in private conversations. It is rarely used by politicians themselves in a public context, although at certain times in Canadian history political parties have used other similarly loaded imagery. In the 1988 federal election, the Liberals asserted that the proposed Free Trade Agreement amounted to an American takeover of Canada—notably, the party ran an ad in which Progressive Conservative (PC) strategists, upon the adoption of the agreement, slowly erased the Canada-U.S. border from a desktop map of North America. Within days, however, the PCs responded with an ad which featured the border being drawn back on with a permanent marker, as an announcer intoned "Here\'s where we draw the line."', 'The phrase is usually used in local political debates, in polemic writing or in private conversations. It is rarely used by politicians themselves in a public context, although at certain times in Canadian history political parties have used other similarly loaded imagery. In the 1988 federal election, the Liberals asserted that the proposed Free Trade Agreement amounted to an American takeover of Canada—notably, the party ran an ad in which Progressive Conservative (PC) strategists, upon the adoption of the agreement, slowly erased the Canada-U.S. border from a desktop map of North America. Within days, however, the PCs responded with an ad which featured the border being drawn back on with a permanent marker, as an announcer intoned "Here\'s where we draw the line."', "The implication has historical basis and dates to the breakup of British America during the American Revolution. The colonies that had confederated to form the United States invaded Canada (at the time a term referring specifically to the modern-day provinces of Quebec and Ontario, which had only been in British hands since 1763) at least twice, neither time succeeding in taking control of the territory. The first invasion was during the Revolution, under the assumption that French-speaking Canadians' presumed hostility towards British colonial rule combined with the Franco-American alliance would make them natural allies to the American cause; the Continental Army successfully recruited two Canadian regiments for the invasion. That invasion's failure forced the members of those regiments into exile, and they settled mostly in upstate New York. The Articles of Confederation, written during the Revolution, included a provision for Canada to join the United States, should they ever decide to do so, without needing to seek U.S. permission as other states would. The United States again invaded Canada during the War of 1812, but this effort was made more difficult due to the large number of Loyalist Americans that had fled to what is now Ontario and still resisted joining the republic. The Hunter Patriots in the 1830s and the Fenian raids after the American Civil War were private attacks on Canada from the U.S. Several U.S. politicians in the 19th century also spoke in favour of annexing Canada.", "The implication has historical basis and dates to the breakup of British America during the American Revolution. The colonies that had confederated to form the United States invaded Canada (at the time a term referring specifically to the modern-day provinces of Quebec and Ontario, which had only been in British hands since 1763) at least twice, neither time succeeding in taking control of the territory. The first invasion was during the Revolution, under the assumption that French-speaking Canadians' presumed hostility towards British colonial rule combined with the Franco-American alliance would make them natural allies to the American cause; the Continental Army successfully recruited two Canadian regiments for the invasion. That invasion's failure forced the members of those regiments into exile, and they settled mostly in upstate New York. The Articles of Confederation, written during the Revolution, included a provision for Canada to join the United States, should they ever decide to do so, without needing to seek U.S. permission as other states would. The United States again invaded Canada during the War of 1812, but this effort was made more difficult due to the large number of Loyalist Americans that had fled to what is now Ontario and still resisted joining the republic. The Hunter Patriots in the 1830s and the Fenian raids after the American Civil War were private attacks on Canada from the U.S. Several U.S. politicians in the 19th century also spoke in favour of annexing Canada.", "The implication has historical basis and dates to the breakup of British America during the American Revolution. The colonies that had confederated to form the United States invaded Canada (at the time a term referring specifically to the modern-day provinces of Quebec and Ontario, which had only been in British hands since 1763) at least twice, neither time succeeding in taking control of the territory. The first invasion was during the Revolution, under the assumption that French-speaking Canadians' presumed hostility towards British colonial rule combined with the Franco-American alliance would make them natural allies to the American cause; the Continental Army successfully recruited two Canadian regiments for the invasion. That invasion's failure forced the members of those regiments into exile, and they settled mostly in upstate New York. The Articles of Confederation, written during the Revolution, included a provision for Canada to join the United States, should they ever decide to do so, without needing to seek U.S. permission as other states would. The United States again invaded Canada during the War of 1812, but this effort was made more difficult due to the large number of Loyalist Americans that had fled to what is now Ontario and still resisted joining the republic. The Hunter Patriots in the 1830s and the Fenian raids after the American Civil War were private attacks on Canada from the U.S. Several U.S. politicians in the 19th century also spoke in favour of annexing Canada.", "The implication has historical basis and dates to the breakup of British America during the American Revolution. The colonies that had confederated to form the United States invaded Canada (at the time a term referring specifically to the modern-day provinces of Quebec and Ontario, which had only been in British hands since 1763) at least twice, neither time succeeding in taking control of the territory. The first invasion was during the Revolution, under the assumption that French-speaking Canadians' presumed hostility towards British colonial rule combined with the Franco-American alliance would make them natural allies to the American cause; the Continental Army successfully recruited two Canadian regiments for the invasion. That invasion's failure forced the members of those regiments into exile, and they settled mostly in upstate New York. The Articles of Confederation, written during the Revolution, included a provision for Canada to join the United States, should they ever decide to do so, without needing to seek U.S. permission as other states would. The United States again invaded Canada during the War of 1812, but this effort was made more difficult due to the large number of Loyalist Americans that had fled to what is now Ontario and still resisted joining the republic. The Hunter Patriots in the 1830s and the Fenian raids after the American Civil War were private attacks on Canada from the U.S. Several U.S. politicians in the 19th century also spoke in favour of annexing Canada.", 'In the late 1940s, during the last days of the Dominion of Newfoundland (at the time a dominion-dependency in the Commonwealth and independent of Canada), there was mainstream support, although not majority, for Newfoundland to form an economic union with the United States, thanks to the efforts of the Economic Union Party and significant U.S. investment in Newfoundland stemming from the U.S.-British alliance in World War II. The movement ultimately failed when, in a 1948 referendum, voters narrowly chose to confederate with Canada (the Economic Union Party supported an independent "responsible government" that they would then push toward their goals).', 'In the late 1940s, during the last days of the Dominion of Newfoundland (at the time a dominion-dependency in the Commonwealth and independent of Canada), there was mainstream support, although not majority, for Newfoundland to form an economic union with the United States, thanks to the efforts of the Economic Union Party and significant U.S. investment in Newfoundland stemming from the U.S.-British alliance in World War II. The movement ultimately failed when, in a 1948 referendum, voters narrowly chose to confederate with Canada (the Economic Union Party supported an independent "responsible government" that they would then push toward their goals).', 'In the late 1940s, during the last days of the Dominion of Newfoundland (at the time a dominion-dependency in the Commonwealth and independent of Canada), there was mainstream support, although not majority, for Newfoundland to form an economic union with the United States, thanks to the efforts of the Economic Union Party and significant U.S. investment in Newfoundland stemming from the U.S.-British alliance in World War II. The movement ultimately failed when, in a 1948 referendum, voters narrowly chose to confederate with Canada (the Economic Union Party supported an independent "responsible government" that they would then push toward their goals).', 'In the United States, the term "the 51st state" when applied to Canada can serve to highlight the similarities and close relationship between the United States and Canada. Sometimes the term is used disparagingly, intended to deride Canada as an unimportant neighbor. In the Quebec general election, 1989, the political party Parti 51 ran 11 candidates on a platform of Quebec seceding from Canada to join the United States (with its leader, André Perron, claiming Quebec could not survive as an independent nation). The party attracted just 3,846 votes across the province, 0.11% of the total votes cast. In comparison, the other parties in favour of sovereignty of Quebec in that election got 40.16% (PQ) and 1.22% (NPDQ).', 'In the United States, the term "the 51st state" when applied to Canada can serve to highlight the similarities and close relationship between the United States and Canada. Sometimes the term is used disparagingly, intended to deride Canada as an unimportant neighbor. In the Quebec general election, 1989, the political party Parti 51 ran 11 candidates on a platform of Quebec seceding from Canada to join the United States (with its leader, André Perron, claiming Quebec could not survive as an independent nation). The party attracted just 3,846 votes across the province, 0.11% of the total votes cast. In comparison, the other parties in favour of sovereignty of Quebec in that election got 40.16% (PQ) and 1.22% (NPDQ).', 'In the United States, the term "the 51st state" when applied to Canada can serve to highlight the similarities and close relationship between the United States and Canada. Sometimes the term is used disparagingly, intended to deride Canada as an unimportant neighbor. In the Quebec general election, 1989, the political party Parti 51 ran 11 candidates on a platform of Quebec seceding from Canada to join the United States (with its leader, André Perron, claiming Quebec could not survive as an independent nation). The party attracted just 3,846 votes across the province, 0.11% of the total votes cast. In comparison, the other parties in favour of sovereignty of Quebec in that election got 40.16% (PQ) and 1.22% (NPDQ).', 'In the United States, the term "the 51st state" when applied to Canada can serve to highlight the similarities and close relationship between the United States and Canada. Sometimes the term is used disparagingly, intended to deride Canada as an unimportant neighbor. In the Quebec general election, 1989, the political party Parti 51 ran 11 candidates on a platform of Quebec seceding from Canada to join the United States (with its leader, André Perron, claiming Quebec could not survive as an independent nation). The party attracted just 3,846 votes across the province, 0.11% of the total votes cast. In comparison, the other parties in favour of sovereignty of Quebec in that election got 40.16% (PQ) and 1.22% (NPDQ).', 'Due to geographical proximity of the Central American countries to the U.S. which has powerful military, economic, and political influences, there were several movements and proposals by the United States during the 19th and 20th centuries to annex some or all of the Central American republics (Costa Rica, El Salvador, Guatemala, Honduras with the formerly British-ruled Bay Islands, Nicaragua, Panama which had the U.S.-ruled Canal Zone territory from 1903 to 1979, and formerly British Honduras or Belize since 1981). However, the U.S. never acted on these proposals from some U.S. politicians; some of which were never delivered or considered seriously. In 2001, El Salvador adopted the U.S. dollar as its currency, while Panama has used it for decades due to its ties to the Canal Zone.', 'Due to geographical proximity of the Central American countries to the U.S. which has powerful military, economic, and political influences, there were several movements and proposals by the United States during the 19th and 20th centuries to annex some or all of the Central American republics (Costa Rica, El Salvador, Guatemala, Honduras with the formerly British-ruled Bay Islands, Nicaragua, Panama which had the U.S.-ruled Canal Zone territory from 1903 to 1979, and formerly British Honduras or Belize since 1981). However, the U.S. never acted on these proposals from some U.S. politicians; some of which were never delivered or considered seriously. In 2001, El Salvador adopted the U.S. dollar as its currency, while Panama has used it for decades due to its ties to the Canal Zone.', 'Due to geographical proximity of the Central American countries to the U.S. which has powerful military, economic, and political influences, there were several movements and proposals by the United States during the 19th and 20th centuries to annex some or all of the Central American republics (Costa Rica, El Salvador, Guatemala, Honduras with the formerly British-ruled Bay Islands, Nicaragua, Panama which had the U.S.-ruled Canal Zone territory from 1903 to 1979, and formerly British Honduras or Belize since 1981). However, the U.S. never acted on these proposals from some U.S. politicians; some of which were never delivered or considered seriously. In 2001, El Salvador adopted the U.S. dollar as its currency, while Panama has used it for decades due to its ties to the Canal Zone.', 'Cuba, like many Spanish territories, wanted to break free from Spain. A pro-independence movement in Cuba was supported by the U.S., and Cuban guerrilla leaders wanted annexation to the United States, but Cuban revolutionary leader José Martí called for Cuban nationhood. When the U.S. battleship Maine sank in Havana Harbor, the U.S. blamed Spain and the Spanish–American War broke out in 1898. After the U.S. won, Spain relinquished claim of sovereignty over territories, including Cuba. The U.S. administered Cuba as a protectorate until 1902. Several decades later in 1959, the corrupt Cuban government of U.S.-backed Fulgencio Batista was overthrown by Fidel Castro. Castro installed a Marxist–Leninist government allied with the Soviet Union, which has been in power ever since.', 'Cuba, like many Spanish territories, wanted to break free from Spain. A pro-independence movement in Cuba was supported by the U.S., and Cuban guerrilla leaders wanted annexation to the United States, but Cuban revolutionary leader José Martí called for Cuban nationhood. When the U.S. battleship Maine sank in Havana Harbor, the U.S. blamed Spain and the Spanish–American War broke out in 1898. After the U.S. won, Spain relinquished claim of sovereignty over territories, including Cuba. The U.S. administered Cuba as a protectorate until 1902. Several decades later in 1959, the corrupt Cuban government of U.S.-backed Fulgencio Batista was overthrown by Fidel Castro. Castro installed a Marxist–Leninist government allied with the Soviet Union, which has been in power ever since.', 'Cuba, like many Spanish territories, wanted to break free from Spain. A pro-independence movement in Cuba was supported by the U.S., and Cuban guerrilla leaders wanted annexation to the United States, but Cuban revolutionary leader José Martí called for Cuban nationhood. When the U.S. battleship Maine sank in Havana Harbor, the U.S. blamed Spain and the Spanish–American War broke out in 1898. After the U.S. won, Spain relinquished claim of sovereignty over territories, including Cuba. The U.S. administered Cuba as a protectorate until 1902. Several decades later in 1959, the corrupt Cuban government of U.S.-backed Fulgencio Batista was overthrown by Fidel Castro. Castro installed a Marxist–Leninist government allied with the Soviet Union, which has been in power ever since.', 'Cuba, like many Spanish territories, wanted to break free from Spain. A pro-independence movement in Cuba was supported by the U.S., and Cuban guerrilla leaders wanted annexation to the United States, but Cuban revolutionary leader José Martí called for Cuban nationhood. When the U.S. battleship Maine sank in Havana Harbor, the U.S. blamed Spain and the Spanish–American War broke out in 1898. After the U.S. won, Spain relinquished claim of sovereignty over territories, including Cuba. The U.S. administered Cuba as a protectorate until 1902. Several decades later in 1959, the corrupt Cuban government of U.S.-backed Fulgencio Batista was overthrown by Fidel Castro. Castro installed a Marxist–Leninist government allied with the Soviet Union, which has been in power ever since.', 'Several websites assert that Israel is the 51st state due to the annual funding and defense support it receives from the United States. An example of this concept can be found in 2003 when Martine Rothblatt published a book called Two Stars for Peace that argued for the addition of Israel and the Palestinian territories surrounding it as the 51st state in the Union. The American State of Canaan, is a book published by Prof. Alfred de Grazia, political science and sociologist, in March 2009, proposing the creation of the 51st and 52nd states from Israel and the Palestinian territories.', 'Several websites assert that Israel is the 51st state due to the annual funding and defense support it receives from the United States. An example of this concept can be found in 2003 when Martine Rothblatt published a book called Two Stars for Peace that argued for the addition of Israel and the Palestinian territories surrounding it as the 51st state in the Union. The American State of Canaan, is a book published by Prof. Alfred de Grazia, political science and sociologist, in March 2009, proposing the creation of the 51st and 52nd states from Israel and the Palestinian territories.', 'Several websites assert that Israel is the 51st state due to the annual funding and defense support it receives from the United States. An example of this concept can be found in 2003 when Martine Rothblatt published a book called Two Stars for Peace that argued for the addition of Israel and the Palestinian territories surrounding it as the 51st state in the Union. The American State of Canaan, is a book published by Prof. Alfred de Grazia, political science and sociologist, in March 2009, proposing the creation of the 51st and 52nd states from Israel and the Palestinian territories.', 'Several websites assert that Israel is the 51st state due to the annual funding and defense support it receives from the United States. An example of this concept can be found in 2003 when Martine Rothblatt published a book called Two Stars for Peace that argued for the addition of Israel and the Palestinian territories surrounding it as the 51st state in the Union. The American State of Canaan, is a book published by Prof. Alfred de Grazia, political science and sociologist, in March 2009, proposing the creation of the 51st and 52nd states from Israel and the Palestinian territories.', "In Article 3 of the Treaty of San Francisco between the Allied Powers and Japan, which came into force in April 1952, the U.S. put the outlying islands of the Ryukyus, including the island of Okinawa—home to over 1 million Okinawans related to the Japanese—and the Bonin Islands, the Volcano Islands, and Iwo Jima into U.S. trusteeship. All these trusteeships were slowly returned to Japanese rule. Okinawa was returned on May 15, 1972, but the U.S. stations troops in the island's bases as a defense for Japan.", "In Article 3 of the Treaty of San Francisco between the Allied Powers and Japan, which came into force in April 1952, the U.S. put the outlying islands of the Ryukyus, including the island of Okinawa—home to over 1 million Okinawans related to the Japanese—and the Bonin Islands, the Volcano Islands, and Iwo Jima into U.S. trusteeship. All these trusteeships were slowly returned to Japanese rule. Okinawa was returned on May 15, 1972, but the U.S. stations troops in the island's bases as a defense for Japan.", "In Article 3 of the Treaty of San Francisco between the Allied Powers and Japan, which came into force in April 1952, the U.S. put the outlying islands of the Ryukyus, including the island of Okinawa—home to over 1 million Okinawans related to the Japanese—and the Bonin Islands, the Volcano Islands, and Iwo Jima into U.S. trusteeship. All these trusteeships were slowly returned to Japanese rule. Okinawa was returned on May 15, 1972, but the U.S. stations troops in the island's bases as a defense for Japan.", "In Article 3 of the Treaty of San Francisco between the Allied Powers and Japan, which came into force in April 1952, the U.S. put the outlying islands of the Ryukyus, including the island of Okinawa—home to over 1 million Okinawans related to the Japanese—and the Bonin Islands, the Volcano Islands, and Iwo Jima into U.S. trusteeship. All these trusteeships were slowly returned to Japanese rule. Okinawa was returned on May 15, 1972, but the U.S. stations troops in the island's bases as a defense for Japan.", "In 2010 there was an attempt to register a 51st State Party with the New Zealand Electoral Commission. The party advocates New Zealand becoming the 51st state of the United States of America. The party's secretary is Paulus Telfer, a former Christchurch mayoral candidate. On February 5, 2010, the party applied to register a logo with the Electoral Commission. The logo – a US flag with 51 stars – was rejected by the Electoral Commission on the grounds that it was likely to cause confusion or mislead electors. As of 2014[update], the party remains unregistered and cannot appear on a ballot.", "In 2010 there was an attempt to register a 51st State Party with the New Zealand Electoral Commission. The party advocates New Zealand becoming the 51st state of the United States of America. The party's secretary is Paulus Telfer, a former Christchurch mayoral candidate. On February 5, 2010, the party applied to register a logo with the Electoral Commission. The logo – a US flag with 51 stars – was rejected by the Electoral Commission on the grounds that it was likely to cause confusion or mislead electors. As of 2014[update], the party remains unregistered and cannot appear on a ballot.", "In 2010 there was an attempt to register a 51st State Party with the New Zealand Electoral Commission. The party advocates New Zealand becoming the 51st state of the United States of America. The party's secretary is Paulus Telfer, a former Christchurch mayoral candidate. On February 5, 2010, the party applied to register a logo with the Electoral Commission. The logo – a US flag with 51 stars – was rejected by the Electoral Commission on the grounds that it was likely to cause confusion or mislead electors. As of 2014[update], the party remains unregistered and cannot appear on a ballot.", "In 2010 there was an attempt to register a 51st State Party with the New Zealand Electoral Commission. The party advocates New Zealand becoming the 51st state of the United States of America. The party's secretary is Paulus Telfer, a former Christchurch mayoral candidate. On February 5, 2010, the party applied to register a logo with the Electoral Commission. The logo – a US flag with 51 stars – was rejected by the Electoral Commission on the grounds that it was likely to cause confusion or mislead electors. As of 2014[update], the party remains unregistered and cannot appear on a ballot.", 'Albania has often been called the 51st state for its perceived strongly pro-American positions, mainly because of the United States\' policies towards it. In reference to President George W. Bush\'s 2007 European tour, Edi Rama, Tirana\'s mayor and leader of the opposition Socialists, said: "Albania is for sure the most pro-American country in Europe, maybe even in the world ... Nowhere else can you find such respect and hospitality for the President of the United States. Even in Michigan, he wouldn\'t be as welcome." At the time of ex-Secretary of State James Baker\'s visit in 1992, there was even a move to hold a referendum declaring the country as the 51st American state. In addition to Albania, Kosovo which is predominately Albanian is seen as a 51st state due to the heavily presence and influence of the United States. The US has had troops and the largest base outside US territory, Camp Bondsteel in the territory since 1999.', 'Albania has often been called the 51st state for its perceived strongly pro-American positions, mainly because of the United States\' policies towards it. In reference to President George W. Bush\'s 2007 European tour, Edi Rama, Tirana\'s mayor and leader of the opposition Socialists, said: "Albania is for sure the most pro-American country in Europe, maybe even in the world ... Nowhere else can you find such respect and hospitality for the President of the United States. Even in Michigan, he wouldn\'t be as welcome." At the time of ex-Secretary of State James Baker\'s visit in 1992, there was even a move to hold a referendum declaring the country as the 51st American state. In addition to Albania, Kosovo which is predominately Albanian is seen as a 51st state due to the heavily presence and influence of the United States. The US has had troops and the largest base outside US territory, Camp Bondsteel in the territory since 1999.', 'Albania has often been called the 51st state for its perceived strongly pro-American positions, mainly because of the United States\' policies towards it. In reference to President George W. Bush\'s 2007 European tour, Edi Rama, Tirana\'s mayor and leader of the opposition Socialists, said: "Albania is for sure the most pro-American country in Europe, maybe even in the world ... Nowhere else can you find such respect and hospitality for the President of the United States. Even in Michigan, he wouldn\'t be as welcome." At the time of ex-Secretary of State James Baker\'s visit in 1992, there was even a move to hold a referendum declaring the country as the 51st American state. In addition to Albania, Kosovo which is predominately Albanian is seen as a 51st state due to the heavily presence and influence of the United States. The US has had troops and the largest base outside US territory, Camp Bondsteel in the territory since 1999.', 'Albania has often been called the 51st state for its perceived strongly pro-American positions, mainly because of the United States\' policies towards it. In reference to President George W. Bush\'s 2007 European tour, Edi Rama, Tirana\'s mayor and leader of the opposition Socialists, said: "Albania is for sure the most pro-American country in Europe, maybe even in the world ... Nowhere else can you find such respect and hospitality for the President of the United States. Even in Michigan, he wouldn\'t be as welcome." At the time of ex-Secretary of State James Baker\'s visit in 1992, there was even a move to hold a referendum declaring the country as the 51st American state. In addition to Albania, Kosovo which is predominately Albanian is seen as a 51st state due to the heavily presence and influence of the United States. The US has had troops and the largest base outside US territory, Camp Bondsteel in the territory since 1999.', 'During World War II, when Denmark was occupied by Nazi Germany, the United States briefly controlled Greenland for battlefields and protection. In 1946, the United States offered to buy Greenland from Denmark for $100 million ($1.2 billion today) but Denmark refused to sell it. Several politicians and others have in recent years argued that Greenland could hypothetically be in a better financial situation as a part of the United States; for instance mentioned by professor Gudmundur Alfredsson at University of Akureyri in 2014. One of the actual reasons behind US interest in Greenland could be the vast natural resources of the island. According to Wikileaks, the U.S. appears to be highly interested in investing in the resource base of the island and in tapping the vast expected hydrocarbons off the Greenlandic coast.', 'During World War II, when Denmark was occupied by Nazi Germany, the United States briefly controlled Greenland for battlefields and protection. In 1946, the United States offered to buy Greenland from Denmark for $100 million ($1.2 billion today) but Denmark refused to sell it. Several politicians and others have in recent years argued that Greenland could hypothetically be in a better financial situation as a part of the United States; for instance mentioned by professor Gudmundur Alfredsson at University of Akureyri in 2014. One of the actual reasons behind US interest in Greenland could be the vast natural resources of the island. According to Wikileaks, the U.S. appears to be highly interested in investing in the resource base of the island and in tapping the vast expected hydrocarbons off the Greenlandic coast.', 'During World War II, when Denmark was occupied by Nazi Germany, the United States briefly controlled Greenland for battlefields and protection. In 1946, the United States offered to buy Greenland from Denmark for $100 million ($1.2 billion today) but Denmark refused to sell it. Several politicians and others have in recent years argued that Greenland could hypothetically be in a better financial situation as a part of the United States; for instance mentioned by professor Gudmundur Alfredsson at University of Akureyri in 2014. One of the actual reasons behind US interest in Greenland could be the vast natural resources of the island. According to Wikileaks, the U.S. appears to be highly interested in investing in the resource base of the island and in tapping the vast expected hydrocarbons off the Greenlandic coast.', 'During World War II, when Denmark was occupied by Nazi Germany, the United States briefly controlled Greenland for battlefields and protection. In 1946, the United States offered to buy Greenland from Denmark for $100 million ($1.2 billion today) but Denmark refused to sell it. Several politicians and others have in recent years argued that Greenland could hypothetically be in a better financial situation as a part of the United States; for instance mentioned by professor Gudmundur Alfredsson at University of Akureyri in 2014. One of the actual reasons behind US interest in Greenland could be the vast natural resources of the island. According to Wikileaks, the U.S. appears to be highly interested in investing in the resource base of the island and in tapping the vast expected hydrocarbons off the Greenlandic coast.', 'Poland has historically been staunchly pro-American, dating back to General Tadeusz Kościuszko and Casimir Pulaski\'s involvement in the American Revolution. This pro-American stance was reinforced following favorable American intervention in World War I (leading to the creation of an independent Poland) and the Cold War (culminating in a Polish state independent of Soviet influence). Poland contributed a large force to the "Coalition of the Willing" in Iraq. A quote referring to Poland as "the 51st state" has been attributed to James Pavitt, then Central Intelligence Agency Deputy Director for Operations, especially in connection to extraordinary rendition.', 'Poland has historically been staunchly pro-American, dating back to General Tadeusz Kościuszko and Casimir Pulaski\'s involvement in the American Revolution. This pro-American stance was reinforced following favorable American intervention in World War I (leading to the creation of an independent Poland) and the Cold War (culminating in a Polish state independent of Soviet influence). Poland contributed a large force to the "Coalition of the Willing" in Iraq. A quote referring to Poland as "the 51st state" has been attributed to James Pavitt, then Central Intelligence Agency Deputy Director for Operations, especially in connection to extraordinary rendition.', 'Poland has historically been staunchly pro-American, dating back to General Tadeusz Kościuszko and Casimir Pulaski\'s involvement in the American Revolution. This pro-American stance was reinforced following favorable American intervention in World War I (leading to the creation of an independent Poland) and the Cold War (culminating in a Polish state independent of Soviet influence). Poland contributed a large force to the "Coalition of the Willing" in Iraq. A quote referring to Poland as "the 51st state" has been attributed to James Pavitt, then Central Intelligence Agency Deputy Director for Operations, especially in connection to extraordinary rendition.', 'The Party of Reconstruction in Sicily, which claimed 40,000 members in 1944, campaigned for Sicily to be admitted as a U.S. state. This party was one of several Sicilian separatist movements active after the downfall of Italian Fascism. Sicilians felt neglected or underrepresented by the Italian government after the annexation of 1861 that ended the rule of the Kingdom of the Two Sicilies based in Naples. The large population of Sicilians in America and the American-led Allied invasion of Sicily in July–August 1943 may have contributed to the sentiment.', 'The Party of Reconstruction in Sicily, which claimed 40,000 members in 1944, campaigned for Sicily to be admitted as a U.S. state. This party was one of several Sicilian separatist movements active after the downfall of Italian Fascism. Sicilians felt neglected or underrepresented by the Italian government after the annexation of 1861 that ended the rule of the Kingdom of the Two Sicilies based in Naples. The large population of Sicilians in America and the American-led Allied invasion of Sicily in July–August 1943 may have contributed to the sentiment.', 'The Party of Reconstruction in Sicily, which claimed 40,000 members in 1944, campaigned for Sicily to be admitted as a U.S. state. This party was one of several Sicilian separatist movements active after the downfall of Italian Fascism. Sicilians felt neglected or underrepresented by the Italian government after the annexation of 1861 that ended the rule of the Kingdom of the Two Sicilies based in Naples. The large population of Sicilians in America and the American-led Allied invasion of Sicily in July–August 1943 may have contributed to the sentiment.', 'The Party of Reconstruction in Sicily, which claimed 40,000 members in 1944, campaigned for Sicily to be admitted as a U.S. state. This party was one of several Sicilian separatist movements active after the downfall of Italian Fascism. Sicilians felt neglected or underrepresented by the Italian government after the annexation of 1861 that ended the rule of the Kingdom of the Two Sicilies based in Naples. The large population of Sicilians in America and the American-led Allied invasion of Sicily in July–August 1943 may have contributed to the sentiment.', 'There are four categories of terra nullius, land that is unclaimed by any state: the small unclaimed territory of Bir Tawil between Egypt and Sudan, Antarctica, the oceans, and celestial bodies such as the Moon or Mars. In the last three of these, international treaties (the Antarctic Treaty, the United Nations Convention on the Law of the Sea, and the Outer Space Treaty respectively) prevent colonization and potential statehood of any of these uninhabited (and, given current technology, not permanently inhabitable) territories.', 'There are four categories of terra nullius, land that is unclaimed by any state: the small unclaimed territory of Bir Tawil between Egypt and Sudan, Antarctica, the oceans, and celestial bodies such as the Moon or Mars. In the last three of these, international treaties (the Antarctic Treaty, the United Nations Convention on the Law of the Sea, and the Outer Space Treaty respectively) prevent colonization and potential statehood of any of these uninhabited (and, given current technology, not permanently inhabitable) territories.', 'There are four categories of terra nullius, land that is unclaimed by any state: the small unclaimed territory of Bir Tawil between Egypt and Sudan, Antarctica, the oceans, and celestial bodies such as the Moon or Mars. In the last three of these, international treaties (the Antarctic Treaty, the United Nations Convention on the Law of the Sea, and the Outer Space Treaty respectively) prevent colonization and potential statehood of any of these uninhabited (and, given current technology, not permanently inhabitable) territories.', 'There are four categories of terra nullius, land that is unclaimed by any state: the small unclaimed territory of Bir Tawil between Egypt and Sudan, Antarctica, the oceans, and celestial bodies such as the Moon or Mars. In the last three of these, international treaties (the Antarctic Treaty, the United Nations Convention on the Law of the Sea, and the Outer Space Treaty respectively) prevent colonization and potential statehood of any of these uninhabited (and, given current technology, not permanently inhabitable) territories.', "An antenna (plural antennae or antennas), or aerial, is an electrical device which converts electric power into radio waves, and vice versa. It is usually used with a radio transmitter or radio receiver. In transmission, a radio transmitter supplies an electric current oscillating at radio frequency (i.e. a high frequency alternating current (AC)) to the antenna's terminals, and the antenna radiates the energy from the current as electromagnetic waves (radio waves). In reception, an antenna intercepts some of the power of an electromagnetic wave in order to produce a tiny voltage at its terminals, that is applied to a receiver to be amplified.", "An antenna (plural antennae or antennas), or aerial, is an electrical device which converts electric power into radio waves, and vice versa. It is usually used with a radio transmitter or radio receiver. In transmission, a radio transmitter supplies an electric current oscillating at radio frequency (i.e. a high frequency alternating current (AC)) to the antenna's terminals, and the antenna radiates the energy from the current as electromagnetic waves (radio waves). In reception, an antenna intercepts some of the power of an electromagnetic wave in order to produce a tiny voltage at its terminals, that is applied to a receiver to be amplified.", "An antenna (plural antennae or antennas), or aerial, is an electrical device which converts electric power into radio waves, and vice versa. It is usually used with a radio transmitter or radio receiver. In transmission, a radio transmitter supplies an electric current oscillating at radio frequency (i.e. a high frequency alternating current (AC)) to the antenna's terminals, and the antenna radiates the energy from the current as electromagnetic waves (radio waves). In reception, an antenna intercepts some of the power of an electromagnetic wave in order to produce a tiny voltage at its terminals, that is applied to a receiver to be amplified.", "An antenna (plural antennae or antennas), or aerial, is an electrical device which converts electric power into radio waves, and vice versa. It is usually used with a radio transmitter or radio receiver. In transmission, a radio transmitter supplies an electric current oscillating at radio frequency (i.e. a high frequency alternating current (AC)) to the antenna's terminals, and the antenna radiates the energy from the current as electromagnetic waves (radio waves). In reception, an antenna intercepts some of the power of an electromagnetic wave in order to produce a tiny voltage at its terminals, that is applied to a receiver to be amplified.", "An antenna (plural antennae or antennas), or aerial, is an electrical device which converts electric power into radio waves, and vice versa. It is usually used with a radio transmitter or radio receiver. In transmission, a radio transmitter supplies an electric current oscillating at radio frequency (i.e. a high frequency alternating current (AC)) to the antenna's terminals, and the antenna radiates the energy from the current as electromagnetic waves (radio waves). In reception, an antenna intercepts some of the power of an electromagnetic wave in order to produce a tiny voltage at its terminals, that is applied to a receiver to be amplified.", 'Typically an antenna consists of an arrangement of metallic conductors (elements), electrically connected (often through a transmission line) to the receiver or transmitter. An oscillating current of electrons forced through the antenna by a transmitter will create an oscillating magnetic field around the antenna elements, while the charge of the electrons also creates an oscillating electric field along the elements. These time-varying fields radiate away from the antenna into space as a moving transverse electromagnetic field wave. Conversely, during reception, the oscillating electric and magnetic fields of an incoming radio wave exert force on the electrons in the antenna elements, causing them to move back and forth, creating oscillating currents in the antenna.', 'Typically an antenna consists of an arrangement of metallic conductors (elements), electrically connected (often through a transmission line) to the receiver or transmitter. An oscillating current of electrons forced through the antenna by a transmitter will create an oscillating magnetic field around the antenna elements, while the charge of the electrons also creates an oscillating electric field along the elements. These time-varying fields radiate away from the antenna into space as a moving transverse electromagnetic field wave. Conversely, during reception, the oscillating electric and magnetic fields of an incoming radio wave exert force on the electrons in the antenna elements, causing them to move back and forth, creating oscillating currents in the antenna.', 'Typically an antenna consists of an arrangement of metallic conductors (elements), electrically connected (often through a transmission line) to the receiver or transmitter. An oscillating current of electrons forced through the antenna by a transmitter will create an oscillating magnetic field around the antenna elements, while the charge of the electrons also creates an oscillating electric field along the elements. These time-varying fields radiate away from the antenna into space as a moving transverse electromagnetic field wave. Conversely, during reception, the oscillating electric and magnetic fields of an incoming radio wave exert force on the electrons in the antenna elements, causing them to move back and forth, creating oscillating currents in the antenna.', 'Typically an antenna consists of an arrangement of metallic conductors (elements), electrically connected (often through a transmission line) to the receiver or transmitter. An oscillating current of electrons forced through the antenna by a transmitter will create an oscillating magnetic field around the antenna elements, while the charge of the electrons also creates an oscillating electric field along the elements. These time-varying fields radiate away from the antenna into space as a moving transverse electromagnetic field wave. Conversely, during reception, the oscillating electric and magnetic fields of an incoming radio wave exert force on the electrons in the antenna elements, causing them to move back and forth, creating oscillating currents in the antenna.', 'The words antenna (plural: antennas in US English, although both "antennas" and "antennae" are used in International English) and aerial are used interchangeably. Occasionally the term "aerial" is used to mean a wire antenna. However, note the important international technical journal, the IEEE Transactions on Antennas and Propagation. In the United Kingdom and other areas where British English is used, the term aerial is sometimes used although \'antenna\' has been universal in professional use for many years.', 'The words antenna (plural: antennas in US English, although both "antennas" and "antennae" are used in International English) and aerial are used interchangeably. Occasionally the term "aerial" is used to mean a wire antenna. However, note the important international technical journal, the IEEE Transactions on Antennas and Propagation. In the United Kingdom and other areas where British English is used, the term aerial is sometimes used although \'antenna\' has been universal in professional use for many years.', 'The words antenna (plural: antennas in US English, although both "antennas" and "antennae" are used in International English) and aerial are used interchangeably. Occasionally the term "aerial" is used to mean a wire antenna. However, note the important international technical journal, the IEEE Transactions on Antennas and Propagation. In the United Kingdom and other areas where British English is used, the term aerial is sometimes used although \'antenna\' has been universal in professional use for many years.', 'The words antenna (plural: antennas in US English, although both "antennas" and "antennae" are used in International English) and aerial are used interchangeably. Occasionally the term "aerial" is used to mean a wire antenna. However, note the important international technical journal, the IEEE Transactions on Antennas and Propagation. In the United Kingdom and other areas where British English is used, the term aerial is sometimes used although \'antenna\' has been universal in professional use for many years.', 'The origin of the word antenna relative to wireless apparatus is attributed to Italian radio pioneer Guglielmo Marconi. In the summer of 1895, Marconi began testing his wireless system outdoors on his father\'s estate near Bologna and soon began to experiment with long wire "aerials". Marconi discovered that by raising the "aerial" wire above the ground and connecting the other side of his transmitter to ground, the transmission range was increased. Soon he was able to transmit signals over a hill, a distance of approximately 2.4 kilometres (1.5 mi). In Italian a tent pole is known as l\'antenna centrale, and the pole with the wire was simply called l\'antenna. Until then wireless radiating transmitting and receiving elements were known simply as aerials or terminals.', 'The origin of the word antenna relative to wireless apparatus is attributed to Italian radio pioneer Guglielmo Marconi. In the summer of 1895, Marconi began testing his wireless system outdoors on his father\'s estate near Bologna and soon began to experiment with long wire "aerials". Marconi discovered that by raising the "aerial" wire above the ground and connecting the other side of his transmitter to ground, the transmission range was increased. Soon he was able to transmit signals over a hill, a distance of approximately 2.4 kilometres (1.5 mi). In Italian a tent pole is known as l\'antenna centrale, and the pole with the wire was simply called l\'antenna. Until then wireless radiating transmitting and receiving elements were known simply as aerials or terminals.', 'The origin of the word antenna relative to wireless apparatus is attributed to Italian radio pioneer Guglielmo Marconi. In the summer of 1895, Marconi began testing his wireless system outdoors on his father\'s estate near Bologna and soon began to experiment with long wire "aerials". Marconi discovered that by raising the "aerial" wire above the ground and connecting the other side of his transmitter to ground, the transmission range was increased. Soon he was able to transmit signals over a hill, a distance of approximately 2.4 kilometres (1.5 mi). In Italian a tent pole is known as l\'antenna centrale, and the pole with the wire was simply called l\'antenna. Until then wireless radiating transmitting and receiving elements were known simply as aerials or terminals.', 'The origin of the word antenna relative to wireless apparatus is attributed to Italian radio pioneer Guglielmo Marconi. In the summer of 1895, Marconi began testing his wireless system outdoors on his father\'s estate near Bologna and soon began to experiment with long wire "aerials". Marconi discovered that by raising the "aerial" wire above the ground and connecting the other side of his transmitter to ground, the transmission range was increased. Soon he was able to transmit signals over a hill, a distance of approximately 2.4 kilometres (1.5 mi). In Italian a tent pole is known as l\'antenna centrale, and the pole with the wire was simply called l\'antenna. Until then wireless radiating transmitting and receiving elements were known simply as aerials or terminals.', 'Antennas are required by any radio receiver or transmitter to couple its electrical connection to the electromagnetic field. Radio waves are electromagnetic waves which carry signals through the air (or through space) at the speed of light with almost no transmission loss. Radio transmitters and receivers are used to convey signals (information) in systems including broadcast (audio) radio, television, mobile telephones, Wi-Fi (WLAN) data networks, trunk lines and point-to-point communications links (telephone, data networks), satellite links, many remote controlled devices such as garage door openers, and wireless remote sensors, among many others. Radio waves are also used directly for measurements in technologies including radar, GPS, and radio astronomy. In each and every case, the transmitters and receivers involved require antennas, although these are sometimes hidden (such as the antenna inside an AM radio or inside a laptop computer equipped with Wi-Fi).', 'Antennas are required by any radio receiver or transmitter to couple its electrical connection to the electromagnetic field. Radio waves are electromagnetic waves which carry signals through the air (or through space) at the speed of light with almost no transmission loss. Radio transmitters and receivers are used to convey signals (information) in systems including broadcast (audio) radio, television, mobile telephones, Wi-Fi (WLAN) data networks, trunk lines and point-to-point communications links (telephone, data networks), satellite links, many remote controlled devices such as garage door openers, and wireless remote sensors, among many others. Radio waves are also used directly for measurements in technologies including radar, GPS, and radio astronomy. In each and every case, the transmitters and receivers involved require antennas, although these are sometimes hidden (such as the antenna inside an AM radio or inside a laptop computer equipped with Wi-Fi).', 'Antennas are required by any radio receiver or transmitter to couple its electrical connection to the electromagnetic field. Radio waves are electromagnetic waves which carry signals through the air (or through space) at the speed of light with almost no transmission loss. Radio transmitters and receivers are used to convey signals (information) in systems including broadcast (audio) radio, television, mobile telephones, Wi-Fi (WLAN) data networks, trunk lines and point-to-point communications links (telephone, data networks), satellite links, many remote controlled devices such as garage door openers, and wireless remote sensors, among many others. Radio waves are also used directly for measurements in technologies including radar, GPS, and radio astronomy. In each and every case, the transmitters and receivers involved require antennas, although these are sometimes hidden (such as the antenna inside an AM radio or inside a laptop computer equipped with Wi-Fi).', 'Antennas are required by any radio receiver or transmitter to couple its electrical connection to the electromagnetic field. Radio waves are electromagnetic waves which carry signals through the air (or through space) at the speed of light with almost no transmission loss. Radio transmitters and receivers are used to convey signals (information) in systems including broadcast (audio) radio, television, mobile telephones, Wi-Fi (WLAN) data networks, trunk lines and point-to-point communications links (telephone, data networks), satellite links, many remote controlled devices such as garage door openers, and wireless remote sensors, among many others. Radio waves are also used directly for measurements in technologies including radar, GPS, and radio astronomy. In each and every case, the transmitters and receivers involved require antennas, although these are sometimes hidden (such as the antenna inside an AM radio or inside a laptop computer equipped with Wi-Fi).', 'Antennas are required by any radio receiver or transmitter to couple its electrical connection to the electromagnetic field. Radio waves are electromagnetic waves which carry signals through the air (or through space) at the speed of light with almost no transmission loss. Radio transmitters and receivers are used to convey signals (information) in systems including broadcast (audio) radio, television, mobile telephones, Wi-Fi (WLAN) data networks, trunk lines and point-to-point communications links (telephone, data networks), satellite links, many remote controlled devices such as garage door openers, and wireless remote sensors, among many others. Radio waves are also used directly for measurements in technologies including radar, GPS, and radio astronomy. In each and every case, the transmitters and receivers involved require antennas, although these are sometimes hidden (such as the antenna inside an AM radio or inside a laptop computer equipped with Wi-Fi).', 'One example of omnidirectional antennas is the very common vertical antenna or whip antenna consisting of a metal rod (often, but not always, a quarter of a wavelength long). A dipole antenna is similar but consists of two such conductors extending in opposite directions, with a total length that is often, but not always, a half of a wavelength long. Dipoles are typically oriented horizontally in which case they are weakly directional: signals are reasonably well radiated toward or received from all directions with the exception of the direction along the conductor itself; this region is called the antenna blind cone or null.', 'One example of omnidirectional antennas is the very common vertical antenna or whip antenna consisting of a metal rod (often, but not always, a quarter of a wavelength long). A dipole antenna is similar but consists of two such conductors extending in opposite directions, with a total length that is often, but not always, a half of a wavelength long. Dipoles are typically oriented horizontally in which case they are weakly directional: signals are reasonably well radiated toward or received from all directions with the exception of the direction along the conductor itself; this region is called the antenna blind cone or null.', 'One example of omnidirectional antennas is the very common vertical antenna or whip antenna consisting of a metal rod (often, but not always, a quarter of a wavelength long). A dipole antenna is similar but consists of two such conductors extending in opposite directions, with a total length that is often, but not always, a half of a wavelength long. Dipoles are typically oriented horizontally in which case they are weakly directional: signals are reasonably well radiated toward or received from all directions with the exception of the direction along the conductor itself; this region is called the antenna blind cone or null.', 'One example of omnidirectional antennas is the very common vertical antenna or whip antenna consisting of a metal rod (often, but not always, a quarter of a wavelength long). A dipole antenna is similar but consists of two such conductors extending in opposite directions, with a total length that is often, but not always, a half of a wavelength long. Dipoles are typically oriented horizontally in which case they are weakly directional: signals are reasonably well radiated toward or received from all directions with the exception of the direction along the conductor itself; this region is called the antenna blind cone or null.', 'Both the vertical and dipole antennas are simple in construction and relatively inexpensive. The dipole antenna, which is the basis for most antenna designs, is a balanced component, with equal but opposite voltages and currents applied at its two terminals through a balanced transmission line (or to a coaxial transmission line through a so-called balun). The vertical antenna, on the other hand, is a monopole antenna. It is typically connected to the inner conductor of a coaxial transmission line (or a matching network); the shield of the transmission line is connected to ground. In this way, the ground (or any large conductive surface) plays the role of the second conductor of a dipole, thereby forming a complete circuit. Since monopole antennas rely on a conductive ground, a so-called grounding structure may be employed to provide a better ground contact to the earth or which itself acts as a ground plane to perform that function regardless of (or in absence of) an actual contact with the earth.', 'Both the vertical and dipole antennas are simple in construction and relatively inexpensive. The dipole antenna, which is the basis for most antenna designs, is a balanced component, with equal but opposite voltages and currents applied at its two terminals through a balanced transmission line (or to a coaxial transmission line through a so-called balun). The vertical antenna, on the other hand, is a monopole antenna. It is typically connected to the inner conductor of a coaxial transmission line (or a matching network); the shield of the transmission line is connected to ground. In this way, the ground (or any large conductive surface) plays the role of the second conductor of a dipole, thereby forming a complete circuit. Since monopole antennas rely on a conductive ground, a so-called grounding structure may be employed to provide a better ground contact to the earth or which itself acts as a ground plane to perform that function regardless of (or in absence of) an actual contact with the earth.', 'Both the vertical and dipole antennas are simple in construction and relatively inexpensive. The dipole antenna, which is the basis for most antenna designs, is a balanced component, with equal but opposite voltages and currents applied at its two terminals through a balanced transmission line (or to a coaxial transmission line through a so-called balun). The vertical antenna, on the other hand, is a monopole antenna. It is typically connected to the inner conductor of a coaxial transmission line (or a matching network); the shield of the transmission line is connected to ground. In this way, the ground (or any large conductive surface) plays the role of the second conductor of a dipole, thereby forming a complete circuit. Since monopole antennas rely on a conductive ground, a so-called grounding structure may be employed to provide a better ground contact to the earth or which itself acts as a ground plane to perform that function regardless of (or in absence of) an actual contact with the earth.', 'Both the vertical and dipole antennas are simple in construction and relatively inexpensive. The dipole antenna, which is the basis for most antenna designs, is a balanced component, with equal but opposite voltages and currents applied at its two terminals through a balanced transmission line (or to a coaxial transmission line through a so-called balun). The vertical antenna, on the other hand, is a monopole antenna. It is typically connected to the inner conductor of a coaxial transmission line (or a matching network); the shield of the transmission line is connected to ground. In this way, the ground (or any large conductive surface) plays the role of the second conductor of a dipole, thereby forming a complete circuit. Since monopole antennas rely on a conductive ground, a so-called grounding structure may be employed to provide a better ground contact to the earth or which itself acts as a ground plane to perform that function regardless of (or in absence of) an actual contact with the earth.', 'Antennas more complex than the dipole or vertical designs are usually intended to increase the directivity and consequently the gain of the antenna. This can be accomplished in many different ways leading to a plethora of antenna designs. The vast majority of designs are fed with a balanced line (unlike a monopole antenna) and are based on the dipole antenna with additional components (or elements) which increase its directionality. Antenna "gain" in this instance describes the concentration of radiated power into a particular solid angle of space, as opposed to the spherically uniform radiation of the ideal radiator. The increased power in the desired direction is at the expense of that in the undesired directions. Power is conserved, and there is no net power increase over that delivered from the power source (the transmitter.)', 'Antennas more complex than the dipole or vertical designs are usually intended to increase the directivity and consequently the gain of the antenna. This can be accomplished in many different ways leading to a plethora of antenna designs. The vast majority of designs are fed with a balanced line (unlike a monopole antenna) and are based on the dipole antenna with additional components (or elements) which increase its directionality. Antenna "gain" in this instance describes the concentration of radiated power into a particular solid angle of space, as opposed to the spherically uniform radiation of the ideal radiator. The increased power in the desired direction is at the expense of that in the undesired directions. Power is conserved, and there is no net power increase over that delivered from the power source (the transmitter.)', 'Antennas more complex than the dipole or vertical designs are usually intended to increase the directivity and consequently the gain of the antenna. This can be accomplished in many different ways leading to a plethora of antenna designs. The vast majority of designs are fed with a balanced line (unlike a monopole antenna) and are based on the dipole antenna with additional components (or elements) which increase its directionality. Antenna "gain" in this instance describes the concentration of radiated power into a particular solid angle of space, as opposed to the spherically uniform radiation of the ideal radiator. The increased power in the desired direction is at the expense of that in the undesired directions. Power is conserved, and there is no net power increase over that delivered from the power source (the transmitter.)', 'Antennas more complex than the dipole or vertical designs are usually intended to increase the directivity and consequently the gain of the antenna. This can be accomplished in many different ways leading to a plethora of antenna designs. The vast majority of designs are fed with a balanced line (unlike a monopole antenna) and are based on the dipole antenna with additional components (or elements) which increase its directionality. Antenna "gain" in this instance describes the concentration of radiated power into a particular solid angle of space, as opposed to the spherically uniform radiation of the ideal radiator. The increased power in the desired direction is at the expense of that in the undesired directions. Power is conserved, and there is no net power increase over that delivered from the power source (the transmitter.)', 'For instance, a phased array consists of two or more simple antennas which are connected together through an electrical network. This often involves a number of parallel dipole antennas with a certain spacing. Depending on the relative phase introduced by the network, the same combination of dipole antennas can operate as a "broadside array" (directional normal to a line connecting the elements) or as an "end-fire array" (directional along the line connecting the elements). Antenna arrays may employ any basic (omnidirectional or weakly directional) antenna type, such as dipole, loop or slot antennas. These elements are often identical.', 'For instance, a phased array consists of two or more simple antennas which are connected together through an electrical network. This often involves a number of parallel dipole antennas with a certain spacing. Depending on the relative phase introduced by the network, the same combination of dipole antennas can operate as a "broadside array" (directional normal to a line connecting the elements) or as an "end-fire array" (directional along the line connecting the elements). Antenna arrays may employ any basic (omnidirectional or weakly directional) antenna type, such as dipole, loop or slot antennas. These elements are often identical.', 'For instance, a phased array consists of two or more simple antennas which are connected together through an electrical network. This often involves a number of parallel dipole antennas with a certain spacing. Depending on the relative phase introduced by the network, the same combination of dipole antennas can operate as a "broadside array" (directional normal to a line connecting the elements) or as an "end-fire array" (directional along the line connecting the elements). Antenna arrays may employ any basic (omnidirectional or weakly directional) antenna type, such as dipole, loop or slot antennas. These elements are often identical.', 'For instance, a phased array consists of two or more simple antennas which are connected together through an electrical network. This often involves a number of parallel dipole antennas with a certain spacing. Depending on the relative phase introduced by the network, the same combination of dipole antennas can operate as a "broadside array" (directional normal to a line connecting the elements) or as an "end-fire array" (directional along the line connecting the elements). Antenna arrays may employ any basic (omnidirectional or weakly directional) antenna type, such as dipole, loop or slot antennas. These elements are often identical.', 'However a log-periodic dipole array consists of a number of dipole elements of different lengths in order to obtain a somewhat directional antenna having an extremely wide bandwidth: these are frequently used for television reception in fringe areas. The dipole antennas composing it are all considered "active elements" since they are all electrically connected together (and to the transmission line). On the other hand, a superficially similar dipole array, the Yagi-Uda Antenna (or simply "Yagi"), has only one dipole element with an electrical connection; the other so-called parasitic elements interact with the electromagnetic field in order to realize a fairly directional antenna but one which is limited to a rather narrow bandwidth. The Yagi antenna has similar looking parasitic dipole elements but which act differently due to their somewhat different lengths. There may be a number of so-called "directors" in front of the active element in the direction of propagation, and usually a single (but possibly more) "reflector" on the opposite side of the active element.', 'However a log-periodic dipole array consists of a number of dipole elements of different lengths in order to obtain a somewhat directional antenna having an extremely wide bandwidth: these are frequently used for television reception in fringe areas. The dipole antennas composing it are all considered "active elements" since they are all electrically connected together (and to the transmission line). On the other hand, a superficially similar dipole array, the Yagi-Uda Antenna (or simply "Yagi"), has only one dipole element with an electrical connection; the other so-called parasitic elements interact with the electromagnetic field in order to realize a fairly directional antenna but one which is limited to a rather narrow bandwidth. The Yagi antenna has similar looking parasitic dipole elements but which act differently due to their somewhat different lengths. There may be a number of so-called "directors" in front of the active element in the direction of propagation, and usually a single (but possibly more) "reflector" on the opposite side of the active element.', 'However a log-periodic dipole array consists of a number of dipole elements of different lengths in order to obtain a somewhat directional antenna having an extremely wide bandwidth: these are frequently used for television reception in fringe areas. The dipole antennas composing it are all considered "active elements" since they are all electrically connected together (and to the transmission line). On the other hand, a superficially similar dipole array, the Yagi-Uda Antenna (or simply "Yagi"), has only one dipole element with an electrical connection; the other so-called parasitic elements interact with the electromagnetic field in order to realize a fairly directional antenna but one which is limited to a rather narrow bandwidth. The Yagi antenna has similar looking parasitic dipole elements but which act differently due to their somewhat different lengths. There may be a number of so-called "directors" in front of the active element in the direction of propagation, and usually a single (but possibly more) "reflector" on the opposite side of the active element.', 'However a log-periodic dipole array consists of a number of dipole elements of different lengths in order to obtain a somewhat directional antenna having an extremely wide bandwidth: these are frequently used for television reception in fringe areas. The dipole antennas composing it are all considered "active elements" since they are all electrically connected together (and to the transmission line). On the other hand, a superficially similar dipole array, the Yagi-Uda Antenna (or simply "Yagi"), has only one dipole element with an electrical connection; the other so-called parasitic elements interact with the electromagnetic field in order to realize a fairly directional antenna but one which is limited to a rather narrow bandwidth. The Yagi antenna has similar looking parasitic dipole elements but which act differently due to their somewhat different lengths. There may be a number of so-called "directors" in front of the active element in the direction of propagation, and usually a single (but possibly more) "reflector" on the opposite side of the active element.', 'At low frequencies (such as AM broadcast), arrays of vertical towers are used to achieve directionality  and they will occupy large areas of land. For reception, a long Beverage antenna can have significant directivity. For non directional portable use, a short vertical antenna or small loop antenna works well, with the main design challenge being that of impedance matching. With a vertical antenna a loading coil at the base of the antenna may be employed to cancel the reactive component of impedance; small loop antennas are tuned with parallel capacitors for this purpose.', 'At low frequencies (such as AM broadcast), arrays of vertical towers are used to achieve directionality  and they will occupy large areas of land. For reception, a long Beverage antenna can have significant directivity. For non directional portable use, a short vertical antenna or small loop antenna works well, with the main design challenge being that of impedance matching. With a vertical antenna a loading coil at the base of the antenna may be employed to cancel the reactive component of impedance; small loop antennas are tuned with parallel capacitors for this purpose.', 'At low frequencies (such as AM broadcast), arrays of vertical towers are used to achieve directionality  and they will occupy large areas of land. For reception, a long Beverage antenna can have significant directivity. For non directional portable use, a short vertical antenna or small loop antenna works well, with the main design challenge being that of impedance matching. With a vertical antenna a loading coil at the base of the antenna may be employed to cancel the reactive component of impedance; small loop antennas are tuned with parallel capacitors for this purpose.', 'At low frequencies (such as AM broadcast), arrays of vertical towers are used to achieve directionality  and they will occupy large areas of land. For reception, a long Beverage antenna can have significant directivity. For non directional portable use, a short vertical antenna or small loop antenna works well, with the main design challenge being that of impedance matching. With a vertical antenna a loading coil at the base of the antenna may be employed to cancel the reactive component of impedance; small loop antennas are tuned with parallel capacitors for this purpose.', 'An antenna lead-in is the transmission line (or feed line) which connects the antenna to a transmitter or receiver. The antenna feed may refer to all components connecting the antenna to the transmitter or receiver, such as an impedance matching network in addition to the transmission line. In a so-called aperture antenna, such as a horn or parabolic dish, the "feed" may also refer to a basic antenna inside the entire system (normally at the focus of the parabolic dish or at the throat of a horn) which could be considered the one active element in that antenna system. A microwave antenna may also be fed directly from a waveguide in place of a (conductive) transmission line.', 'An antenna lead-in is the transmission line (or feed line) which connects the antenna to a transmitter or receiver. The antenna feed may refer to all components connecting the antenna to the transmitter or receiver, such as an impedance matching network in addition to the transmission line. In a so-called aperture antenna, such as a horn or parabolic dish, the "feed" may also refer to a basic antenna inside the entire system (normally at the focus of the parabolic dish or at the throat of a horn) which could be considered the one active element in that antenna system. A microwave antenna may also be fed directly from a waveguide in place of a (conductive) transmission line.', 'An antenna lead-in is the transmission line (or feed line) which connects the antenna to a transmitter or receiver. The antenna feed may refer to all components connecting the antenna to the transmitter or receiver, such as an impedance matching network in addition to the transmission line. In a so-called aperture antenna, such as a horn or parabolic dish, the "feed" may also refer to a basic antenna inside the entire system (normally at the focus of the parabolic dish or at the throat of a horn) which could be considered the one active element in that antenna system. A microwave antenna may also be fed directly from a waveguide in place of a (conductive) transmission line.', 'An antenna lead-in is the transmission line (or feed line) which connects the antenna to a transmitter or receiver. The antenna feed may refer to all components connecting the antenna to the transmitter or receiver, such as an impedance matching network in addition to the transmission line. In a so-called aperture antenna, such as a horn or parabolic dish, the "feed" may also refer to a basic antenna inside the entire system (normally at the focus of the parabolic dish or at the throat of a horn) which could be considered the one active element in that antenna system. A microwave antenna may also be fed directly from a waveguide in place of a (conductive) transmission line.', 'Monopole antennas consist of a single radiating element such as a metal rod, often mounted over a conducting surface, a ground plane. One side of the feedline from the receiver or transmitter is connected to the rod, and the other side to the ground plane, which may be the Earth. The most common form is the quarter-wave monopole which is one-quarter of a wavelength long and has a gain of 5.12 dBi when mounted over a ground plane. Monopoles have an omnidirectional radiation pattern, so they are used for broad coverage of an area, and have vertical polarization. The ground waves used for broadcasting at low frequencies must be vertically polarized, so large vertical monopole antennas are used for broadcasting in the MF, LF, and VLF bands. Small monopoles are used as nondirectional antennas on portable radios in the HF, VHF, and UHF bands.', 'Monopole antennas consist of a single radiating element such as a metal rod, often mounted over a conducting surface, a ground plane. One side of the feedline from the receiver or transmitter is connected to the rod, and the other side to the ground plane, which may be the Earth. The most common form is the quarter-wave monopole which is one-quarter of a wavelength long and has a gain of 5.12 dBi when mounted over a ground plane. Monopoles have an omnidirectional radiation pattern, so they are used for broad coverage of an area, and have vertical polarization. The ground waves used for broadcasting at low frequencies must be vertically polarized, so large vertical monopole antennas are used for broadcasting in the MF, LF, and VLF bands. Small monopoles are used as nondirectional antennas on portable radios in the HF, VHF, and UHF bands.', 'Monopole antennas consist of a single radiating element such as a metal rod, often mounted over a conducting surface, a ground plane. One side of the feedline from the receiver or transmitter is connected to the rod, and the other side to the ground plane, which may be the Earth. The most common form is the quarter-wave monopole which is one-quarter of a wavelength long and has a gain of 5.12 dBi when mounted over a ground plane. Monopoles have an omnidirectional radiation pattern, so they are used for broad coverage of an area, and have vertical polarization. The ground waves used for broadcasting at low frequencies must be vertically polarized, so large vertical monopole antennas are used for broadcasting in the MF, LF, and VLF bands. Small monopoles are used as nondirectional antennas on portable radios in the HF, VHF, and UHF bands.', 'Monopole antennas consist of a single radiating element such as a metal rod, often mounted over a conducting surface, a ground plane. One side of the feedline from the receiver or transmitter is connected to the rod, and the other side to the ground plane, which may be the Earth. The most common form is the quarter-wave monopole which is one-quarter of a wavelength long and has a gain of 5.12 dBi when mounted over a ground plane. Monopoles have an omnidirectional radiation pattern, so they are used for broad coverage of an area, and have vertical polarization. The ground waves used for broadcasting at low frequencies must be vertically polarized, so large vertical monopole antennas are used for broadcasting in the MF, LF, and VLF bands. Small monopoles are used as nondirectional antennas on portable radios in the HF, VHF, and UHF bands.', "The most widely used class of antenna, a dipole antenna consists of two symmetrical radiators such as metal rods or wires, with one side of the balanced feedline from the transmitter or receiver attached to each. A horizontal dipole radiates in two lobes perpendicular to the antenna's axis. A half-wave dipole the most common type, has two collinear elements each a quarter wavelength long and a gain of 2.15 dBi. Used individually as low gain antennas, dipoles are also used as driven elements in many more complicated higher gain types of antennas.", "The most widely used class of antenna, a dipole antenna consists of two symmetrical radiators such as metal rods or wires, with one side of the balanced feedline from the transmitter or receiver attached to each. A horizontal dipole radiates in two lobes perpendicular to the antenna's axis. A half-wave dipole the most common type, has two collinear elements each a quarter wavelength long and a gain of 2.15 dBi. Used individually as low gain antennas, dipoles are also used as driven elements in many more complicated higher gain types of antennas.", "The most widely used class of antenna, a dipole antenna consists of two symmetrical radiators such as metal rods or wires, with one side of the balanced feedline from the transmitter or receiver attached to each. A horizontal dipole radiates in two lobes perpendicular to the antenna's axis. A half-wave dipole the most common type, has two collinear elements each a quarter wavelength long and a gain of 2.15 dBi. Used individually as low gain antennas, dipoles are also used as driven elements in many more complicated higher gain types of antennas.", "The most widely used class of antenna, a dipole antenna consists of two symmetrical radiators such as metal rods or wires, with one side of the balanced feedline from the transmitter or receiver attached to each. A horizontal dipole radiates in two lobes perpendicular to the antenna's axis. A half-wave dipole the most common type, has two collinear elements each a quarter wavelength long and a gain of 2.15 dBi. Used individually as low gain antennas, dipoles are also used as driven elements in many more complicated higher gain types of antennas.", 'A necessary condition for the aforementioned reciprocity property is that the materials in the antenna and transmission medium are linear and reciprocal. Reciprocal (or bilateral) means that the material has the same response to an electric current or magnetic field in one direction, as it has to the field or current in the opposite direction. Most materials used in antennas meet these conditions, but some microwave antennas use high-tech components such as isolators and circulators, made of nonreciprocal materials such as ferrite. These can be used to give the antenna a different behavior on receiving than it has on transmitting, which can be useful in applications like radar.', 'A necessary condition for the aforementioned reciprocity property is that the materials in the antenna and transmission medium are linear and reciprocal. Reciprocal (or bilateral) means that the material has the same response to an electric current or magnetic field in one direction, as it has to the field or current in the opposite direction. Most materials used in antennas meet these conditions, but some microwave antennas use high-tech components such as isolators and circulators, made of nonreciprocal materials such as ferrite. These can be used to give the antenna a different behavior on receiving than it has on transmitting, which can be useful in applications like radar.', 'A necessary condition for the aforementioned reciprocity property is that the materials in the antenna and transmission medium are linear and reciprocal. Reciprocal (or bilateral) means that the material has the same response to an electric current or magnetic field in one direction, as it has to the field or current in the opposite direction. Most materials used in antennas meet these conditions, but some microwave antennas use high-tech components such as isolators and circulators, made of nonreciprocal materials such as ferrite. These can be used to give the antenna a different behavior on receiving than it has on transmitting, which can be useful in applications like radar.', 'A necessary condition for the aforementioned reciprocity property is that the materials in the antenna and transmission medium are linear and reciprocal. Reciprocal (or bilateral) means that the material has the same response to an electric current or magnetic field in one direction, as it has to the field or current in the opposite direction. Most materials used in antennas meet these conditions, but some microwave antennas use high-tech components such as isolators and circulators, made of nonreciprocal materials such as ferrite. These can be used to give the antenna a different behavior on receiving than it has on transmitting, which can be useful in applications like radar.', 'Antennas are characterized by a number of performance measures which a user would be concerned with in selecting or designing an antenna for a particular application. Chief among these relate to the directional characteristics (as depicted in the antenna\'s radiation pattern) and the resulting gain. Even in omnidirectional (or weakly directional) antennas, the gain can often be increased by concentrating more of its power in the horizontal directions, sacrificing power radiated toward the sky and ground. The antenna\'s power gain (or simply "gain") also takes into account the antenna\'s efficiency, and is often the primary figure of merit.', 'Antennas are characterized by a number of performance measures which a user would be concerned with in selecting or designing an antenna for a particular application. Chief among these relate to the directional characteristics (as depicted in the antenna\'s radiation pattern) and the resulting gain. Even in omnidirectional (or weakly directional) antennas, the gain can often be increased by concentrating more of its power in the horizontal directions, sacrificing power radiated toward the sky and ground. The antenna\'s power gain (or simply "gain") also takes into account the antenna\'s efficiency, and is often the primary figure of merit.', 'Antennas are characterized by a number of performance measures which a user would be concerned with in selecting or designing an antenna for a particular application. Chief among these relate to the directional characteristics (as depicted in the antenna\'s radiation pattern) and the resulting gain. Even in omnidirectional (or weakly directional) antennas, the gain can often be increased by concentrating more of its power in the horizontal directions, sacrificing power radiated toward the sky and ground. The antenna\'s power gain (or simply "gain") also takes into account the antenna\'s efficiency, and is often the primary figure of merit.', 'Antennas are characterized by a number of performance measures which a user would be concerned with in selecting or designing an antenna for a particular application. Chief among these relate to the directional characteristics (as depicted in the antenna\'s radiation pattern) and the resulting gain. Even in omnidirectional (or weakly directional) antennas, the gain can often be increased by concentrating more of its power in the horizontal directions, sacrificing power radiated toward the sky and ground. The antenna\'s power gain (or simply "gain") also takes into account the antenna\'s efficiency, and is often the primary figure of merit.', "Resonant antennas are expected to be used around a particular resonant frequency; an antenna must therefore be built or ordered to match the frequency range of the intended application. A particular antenna design will present a particular feedpoint impedance. While this may affect the choice of an antenna, an antenna's impedance can also be adapted to the desired impedance level of a system using a matching network while maintaining the other characteristics (except for a possible loss of efficiency).", "Resonant antennas are expected to be used around a particular resonant frequency; an antenna must therefore be built or ordered to match the frequency range of the intended application. A particular antenna design will present a particular feedpoint impedance. While this may affect the choice of an antenna, an antenna's impedance can also be adapted to the desired impedance level of a system using a matching network while maintaining the other characteristics (except for a possible loss of efficiency).", "Resonant antennas are expected to be used around a particular resonant frequency; an antenna must therefore be built or ordered to match the frequency range of the intended application. A particular antenna design will present a particular feedpoint impedance. While this may affect the choice of an antenna, an antenna's impedance can also be adapted to the desired impedance level of a system using a matching network while maintaining the other characteristics (except for a possible loss of efficiency).", "Resonant antennas are expected to be used around a particular resonant frequency; an antenna must therefore be built or ordered to match the frequency range of the intended application. A particular antenna design will present a particular feedpoint impedance. While this may affect the choice of an antenna, an antenna's impedance can also be adapted to the desired impedance level of a system using a matching network while maintaining the other characteristics (except for a possible loss of efficiency).", 'An antenna transmits and receives radio waves with a particular polarization which can be reoriented by tilting the axis of the antenna in many (but not all) cases. The physical size of an antenna is often a practical issue, particularly at lower frequencies (longer wavelengths). Highly directional antennas need to be significantly larger than the wavelength. Resonant antennas usually use a linear conductor (or element), or pair of such elements, each of which is about a quarter of the wavelength in length (an odd multiple of quarter wavelengths will also be resonant). Antennas that are required to be small compared to the wavelength sacrifice efficiency and cannot be very directional. Fortunately at higher frequencies (UHF, microwaves) trading off performance to obtain a smaller physical size is usually not required.', 'An antenna transmits and receives radio waves with a particular polarization which can be reoriented by tilting the axis of the antenna in many (but not all) cases. The physical size of an antenna is often a practical issue, particularly at lower frequencies (longer wavelengths). Highly directional antennas need to be significantly larger than the wavelength. Resonant antennas usually use a linear conductor (or element), or pair of such elements, each of which is about a quarter of the wavelength in length (an odd multiple of quarter wavelengths will also be resonant). Antennas that are required to be small compared to the wavelength sacrifice efficiency and cannot be very directional. Fortunately at higher frequencies (UHF, microwaves) trading off performance to obtain a smaller physical size is usually not required.', 'An antenna transmits and receives radio waves with a particular polarization which can be reoriented by tilting the axis of the antenna in many (but not all) cases. The physical size of an antenna is often a practical issue, particularly at lower frequencies (longer wavelengths). Highly directional antennas need to be significantly larger than the wavelength. Resonant antennas usually use a linear conductor (or element), or pair of such elements, each of which is about a quarter of the wavelength in length (an odd multiple of quarter wavelengths will also be resonant). Antennas that are required to be small compared to the wavelength sacrifice efficiency and cannot be very directional. Fortunately at higher frequencies (UHF, microwaves) trading off performance to obtain a smaller physical size is usually not required.', 'An antenna transmits and receives radio waves with a particular polarization which can be reoriented by tilting the axis of the antenna in many (but not all) cases. The physical size of an antenna is often a practical issue, particularly at lower frequencies (longer wavelengths). Highly directional antennas need to be significantly larger than the wavelength. Resonant antennas usually use a linear conductor (or element), or pair of such elements, each of which is about a quarter of the wavelength in length (an odd multiple of quarter wavelengths will also be resonant). Antennas that are required to be small compared to the wavelength sacrifice efficiency and cannot be very directional. Fortunately at higher frequencies (UHF, microwaves) trading off performance to obtain a smaller physical size is usually not required.', 'An antenna transmits and receives radio waves with a particular polarization which can be reoriented by tilting the axis of the antenna in many (but not all) cases. The physical size of an antenna is often a practical issue, particularly at lower frequencies (longer wavelengths). Highly directional antennas need to be significantly larger than the wavelength. Resonant antennas usually use a linear conductor (or element), or pair of such elements, each of which is about a quarter of the wavelength in length (an odd multiple of quarter wavelengths will also be resonant). Antennas that are required to be small compared to the wavelength sacrifice efficiency and cannot be very directional. Fortunately at higher frequencies (UHF, microwaves) trading off performance to obtain a smaller physical size is usually not required.', 'The majority of antenna designs are based on the resonance principle. This relies on the behaviour of moving electrons, which reflect off surfaces where the dielectric constant changes, in a fashion similar to the way light reflects when optical properties change. In these designs, the reflective surface is created by the end of a conductor, normally a thin metal wire or rod, which in the simplest case has a feed point at one end where it is connected to a transmission line. The conductor, or element, is aligned with the electrical field of the desired signal, normally meaning it is perpendicular to the line from the antenna to the source (or receiver in the case of a broadcast antenna).', 'The majority of antenna designs are based on the resonance principle. This relies on the behaviour of moving electrons, which reflect off surfaces where the dielectric constant changes, in a fashion similar to the way light reflects when optical properties change. In these designs, the reflective surface is created by the end of a conductor, normally a thin metal wire or rod, which in the simplest case has a feed point at one end where it is connected to a transmission line. The conductor, or element, is aligned with the electrical field of the desired signal, normally meaning it is perpendicular to the line from the antenna to the source (or receiver in the case of a broadcast antenna).', 'The majority of antenna designs are based on the resonance principle. This relies on the behaviour of moving electrons, which reflect off surfaces where the dielectric constant changes, in a fashion similar to the way light reflects when optical properties change. In these designs, the reflective surface is created by the end of a conductor, normally a thin metal wire or rod, which in the simplest case has a feed point at one end where it is connected to a transmission line. The conductor, or element, is aligned with the electrical field of the desired signal, normally meaning it is perpendicular to the line from the antenna to the source (or receiver in the case of a broadcast antenna).', 'The majority of antenna designs are based on the resonance principle. This relies on the behaviour of moving electrons, which reflect off surfaces where the dielectric constant changes, in a fashion similar to the way light reflects when optical properties change. In these designs, the reflective surface is created by the end of a conductor, normally a thin metal wire or rod, which in the simplest case has a feed point at one end where it is connected to a transmission line. The conductor, or element, is aligned with the electrical field of the desired signal, normally meaning it is perpendicular to the line from the antenna to the source (or receiver in the case of a broadcast antenna).', "The radio signal's electrical component induces a voltage in the conductor. This causes an electrical current to begin flowing in the direction of the signal's instantaneous field. When the resulting current reaches the end of the conductor, it reflects, which is equivalent to a 180 degree change in phase. If the conductor is 1⁄4 of a wavelength long, current from the feed point will undergo 90 degree phase change by the time it reaches the end of the conductor, reflect through 180 degrees, and then another 90 degrees as it travels back. That means it has undergone a total 360 degree phase change, returning it to the original signal. The current in the element thus adds to the current being created from the source at that instant. This process creates a standing wave in the conductor, with the maximum current at the feed.", "The radio signal's electrical component induces a voltage in the conductor. This causes an electrical current to begin flowing in the direction of the signal's instantaneous field. When the resulting current reaches the end of the conductor, it reflects, which is equivalent to a 180 degree change in phase. If the conductor is 1⁄4 of a wavelength long, current from the feed point will undergo 90 degree phase change by the time it reaches the end of the conductor, reflect through 180 degrees, and then another 90 degrees as it travels back. That means it has undergone a total 360 degree phase change, returning it to the original signal. The current in the element thus adds to the current being created from the source at that instant. This process creates a standing wave in the conductor, with the maximum current at the feed.", "The radio signal's electrical component induces a voltage in the conductor. This causes an electrical current to begin flowing in the direction of the signal's instantaneous field. When the resulting current reaches the end of the conductor, it reflects, which is equivalent to a 180 degree change in phase. If the conductor is 1⁄4 of a wavelength long, current from the feed point will undergo 90 degree phase change by the time it reaches the end of the conductor, reflect through 180 degrees, and then another 90 degrees as it travels back. That means it has undergone a total 360 degree phase change, returning it to the original signal. The current in the element thus adds to the current being created from the source at that instant. This process creates a standing wave in the conductor, with the maximum current at the feed.", "The radio signal's electrical component induces a voltage in the conductor. This causes an electrical current to begin flowing in the direction of the signal's instantaneous field. When the resulting current reaches the end of the conductor, it reflects, which is equivalent to a 180 degree change in phase. If the conductor is 1⁄4 of a wavelength long, current from the feed point will undergo 90 degree phase change by the time it reaches the end of the conductor, reflect through 180 degrees, and then another 90 degrees as it travels back. That means it has undergone a total 360 degree phase change, returning it to the original signal. The current in the element thus adds to the current being created from the source at that instant. This process creates a standing wave in the conductor, with the maximum current at the feed.", 'The ordinary half-wave dipole is probably the most widely used antenna design. This consists of two 1⁄4-wavelength elements arranged end-to-end, and lying along essentially the same axis (or collinear), each feeding one side of a two-conductor transmission wire. The physical arrangement of the two elements places them 180 degrees out of phase, which means that at any given instant one of the elements is driving current into the transmission line while the other is pulling it out. The monopole antenna is essentially one half of the half-wave dipole, a single 1⁄4-wavelength element with the other side connected to ground or an equivalent ground plane (or counterpoise). Monopoles, which are one-half the size of a dipole, are common for long-wavelength radio signals where a dipole would be impractically large. Another common design is the folded dipole, which is essentially two dipoles placed side-by-side and connected at their ends to make a single one-wavelength antenna.', 'The ordinary half-wave dipole is probably the most widely used antenna design. This consists of two 1⁄4-wavelength elements arranged end-to-end, and lying along essentially the same axis (or collinear), each feeding one side of a two-conductor transmission wire. The physical arrangement of the two elements places them 180 degrees out of phase, which means that at any given instant one of the elements is driving current into the transmission line while the other is pulling it out. The monopole antenna is essentially one half of the half-wave dipole, a single 1⁄4-wavelength element with the other side connected to ground or an equivalent ground plane (or counterpoise). Monopoles, which are one-half the size of a dipole, are common for long-wavelength radio signals where a dipole would be impractically large. Another common design is the folded dipole, which is essentially two dipoles placed side-by-side and connected at their ends to make a single one-wavelength antenna.', 'The ordinary half-wave dipole is probably the most widely used antenna design. This consists of two 1⁄4-wavelength elements arranged end-to-end, and lying along essentially the same axis (or collinear), each feeding one side of a two-conductor transmission wire. The physical arrangement of the two elements places them 180 degrees out of phase, which means that at any given instant one of the elements is driving current into the transmission line while the other is pulling it out. The monopole antenna is essentially one half of the half-wave dipole, a single 1⁄4-wavelength element with the other side connected to ground or an equivalent ground plane (or counterpoise). Monopoles, which are one-half the size of a dipole, are common for long-wavelength radio signals where a dipole would be impractically large. Another common design is the folded dipole, which is essentially two dipoles placed side-by-side and connected at their ends to make a single one-wavelength antenna.', 'The ordinary half-wave dipole is probably the most widely used antenna design. This consists of two 1⁄4-wavelength elements arranged end-to-end, and lying along essentially the same axis (or collinear), each feeding one side of a two-conductor transmission wire. The physical arrangement of the two elements places them 180 degrees out of phase, which means that at any given instant one of the elements is driving current into the transmission line while the other is pulling it out. The monopole antenna is essentially one half of the half-wave dipole, a single 1⁄4-wavelength element with the other side connected to ground or an equivalent ground plane (or counterpoise). Monopoles, which are one-half the size of a dipole, are common for long-wavelength radio signals where a dipole would be impractically large. Another common design is the folded dipole, which is essentially two dipoles placed side-by-side and connected at their ends to make a single one-wavelength antenna.', 'The ordinary half-wave dipole is probably the most widely used antenna design. This consists of two 1⁄4-wavelength elements arranged end-to-end, and lying along essentially the same axis (or collinear), each feeding one side of a two-conductor transmission wire. The physical arrangement of the two elements places them 180 degrees out of phase, which means that at any given instant one of the elements is driving current into the transmission line while the other is pulling it out. The monopole antenna is essentially one half of the half-wave dipole, a single 1⁄4-wavelength element with the other side connected to ground or an equivalent ground plane (or counterpoise). Monopoles, which are one-half the size of a dipole, are common for long-wavelength radio signals where a dipole would be impractically large. Another common design is the folded dipole, which is essentially two dipoles placed side-by-side and connected at their ends to make a single one-wavelength antenna.', 'The standing wave forms with this desired pattern at the design frequency, f0, and antennas are normally designed to be this size. However, feeding that element with 3f0 (whose wavelength is 1⁄3 that of f0) will also lead to a standing wave pattern. Thus, an antenna element is also resonant when its length is 3⁄4 of a wavelength. This is true for all odd multiples of 1⁄4 wavelength. This allows some flexibility of design in terms of antenna lengths and feed points. Antennas used in such a fashion are known to be harmonically operated.', 'The standing wave forms with this desired pattern at the design frequency, f0, and antennas are normally designed to be this size. However, feeding that element with 3f0 (whose wavelength is 1⁄3 that of f0) will also lead to a standing wave pattern. Thus, an antenna element is also resonant when its length is 3⁄4 of a wavelength. This is true for all odd multiples of 1⁄4 wavelength. This allows some flexibility of design in terms of antenna lengths and feed points. Antennas used in such a fashion are known to be harmonically operated.', 'The standing wave forms with this desired pattern at the design frequency, f0, and antennas are normally designed to be this size. However, feeding that element with 3f0 (whose wavelength is 1⁄3 that of f0) will also lead to a standing wave pattern. Thus, an antenna element is also resonant when its length is 3⁄4 of a wavelength. This is true for all odd multiples of 1⁄4 wavelength. This allows some flexibility of design in terms of antenna lengths and feed points. Antennas used in such a fashion are known to be harmonically operated.', 'The standing wave forms with this desired pattern at the design frequency, f0, and antennas are normally designed to be this size. However, feeding that element with 3f0 (whose wavelength is 1⁄3 that of f0) will also lead to a standing wave pattern. Thus, an antenna element is also resonant when its length is 3⁄4 of a wavelength. This is true for all odd multiples of 1⁄4 wavelength. This allows some flexibility of design in terms of antenna lengths and feed points. Antennas used in such a fashion are known to be harmonically operated.', "The quarter-wave elements imitate a series-resonant electrical element due to the standing wave present along the conductor. At the resonant frequency, the standing wave has a current peak and voltage node (minimum) at the feed. In electrical terms, this means the element has minimum reactance, generating the maximum current for minimum voltage. This is the ideal situation, because it produces the maximum output for the minimum input, producing the highest possible efficiency. Contrary to an ideal (lossless) series-resonant circuit, a finite resistance remains (corresponding to the relatively small voltage at the feed-point) due to the antenna's radiation resistance as well as any actual electrical losses.", "The quarter-wave elements imitate a series-resonant electrical element due to the standing wave present along the conductor. At the resonant frequency, the standing wave has a current peak and voltage node (minimum) at the feed. In electrical terms, this means the element has minimum reactance, generating the maximum current for minimum voltage. This is the ideal situation, because it produces the maximum output for the minimum input, producing the highest possible efficiency. Contrary to an ideal (lossless) series-resonant circuit, a finite resistance remains (corresponding to the relatively small voltage at the feed-point) due to the antenna's radiation resistance as well as any actual electrical losses.", "The quarter-wave elements imitate a series-resonant electrical element due to the standing wave present along the conductor. At the resonant frequency, the standing wave has a current peak and voltage node (minimum) at the feed. In electrical terms, this means the element has minimum reactance, generating the maximum current for minimum voltage. This is the ideal situation, because it produces the maximum output for the minimum input, producing the highest possible efficiency. Contrary to an ideal (lossless) series-resonant circuit, a finite resistance remains (corresponding to the relatively small voltage at the feed-point) due to the antenna's radiation resistance as well as any actual electrical losses.", "The quarter-wave elements imitate a series-resonant electrical element due to the standing wave present along the conductor. At the resonant frequency, the standing wave has a current peak and voltage node (minimum) at the feed. In electrical terms, this means the element has minimum reactance, generating the maximum current for minimum voltage. This is the ideal situation, because it produces the maximum output for the minimum input, producing the highest possible efficiency. Contrary to an ideal (lossless) series-resonant circuit, a finite resistance remains (corresponding to the relatively small voltage at the feed-point) due to the antenna's radiation resistance as well as any actual electrical losses.", 'It is possible to use the impedance matching concepts to construct vertical antennas substantially shorter than the 1⁄4 wavelength at which the antenna is resonant. By adding an inductance in series with the antenna, a so-called loading coil, the capacitive reactance of this antenna can be cancelled leaving a pure resistance which can then be matched to the transmission line. Sometimes the resulting resonant frequency of such a system (antenna plus matching network) is described using the construct of electrical length and the use of a shorter antenna at a lower frequency than its resonant frequency is termed electrical lengthening.', 'It is possible to use the impedance matching concepts to construct vertical antennas substantially shorter than the 1⁄4 wavelength at which the antenna is resonant. By adding an inductance in series with the antenna, a so-called loading coil, the capacitive reactance of this antenna can be cancelled leaving a pure resistance which can then be matched to the transmission line. Sometimes the resulting resonant frequency of such a system (antenna plus matching network) is described using the construct of electrical length and the use of a shorter antenna at a lower frequency than its resonant frequency is termed electrical lengthening.', 'It is possible to use the impedance matching concepts to construct vertical antennas substantially shorter than the 1⁄4 wavelength at which the antenna is resonant. By adding an inductance in series with the antenna, a so-called loading coil, the capacitive reactance of this antenna can be cancelled leaving a pure resistance which can then be matched to the transmission line. Sometimes the resulting resonant frequency of such a system (antenna plus matching network) is described using the construct of electrical length and the use of a shorter antenna at a lower frequency than its resonant frequency is termed electrical lengthening.', 'It is possible to use the impedance matching concepts to construct vertical antennas substantially shorter than the 1⁄4 wavelength at which the antenna is resonant. By adding an inductance in series with the antenna, a so-called loading coil, the capacitive reactance of this antenna can be cancelled leaving a pure resistance which can then be matched to the transmission line. Sometimes the resulting resonant frequency of such a system (antenna plus matching network) is described using the construct of electrical length and the use of a shorter antenna at a lower frequency than its resonant frequency is termed electrical lengthening.', "The end result is that the resonant antenna will efficiently feed a signal into the transmission line only when the source signal's frequency is close to that of the design frequency of the antenna, or one of the resonant multiples. This makes resonant antenna designs inherently narrowband, and they are most commonly used with a single target signal. They are particularly common on radar systems, where the same antenna is used for both broadcast and reception, or for radio and television broadcasts, where the antenna is working with a single frequency. They are less commonly used for reception where multiple channels are present, in which case additional modifications are used to increase the bandwidth, or entirely different antenna designs are used.", "The end result is that the resonant antenna will efficiently feed a signal into the transmission line only when the source signal's frequency is close to that of the design frequency of the antenna, or one of the resonant multiples. This makes resonant antenna designs inherently narrowband, and they are most commonly used with a single target signal. They are particularly common on radar systems, where the same antenna is used for both broadcast and reception, or for radio and television broadcasts, where the antenna is working with a single frequency. They are less commonly used for reception where multiple channels are present, in which case additional modifications are used to increase the bandwidth, or entirely different antenna designs are used.", "The end result is that the resonant antenna will efficiently feed a signal into the transmission line only when the source signal's frequency is close to that of the design frequency of the antenna, or one of the resonant multiples. This makes resonant antenna designs inherently narrowband, and they are most commonly used with a single target signal. They are particularly common on radar systems, where the same antenna is used for both broadcast and reception, or for radio and television broadcasts, where the antenna is working with a single frequency. They are less commonly used for reception where multiple channels are present, in which case additional modifications are used to increase the bandwidth, or entirely different antenna designs are used.", "The end result is that the resonant antenna will efficiently feed a signal into the transmission line only when the source signal's frequency is close to that of the design frequency of the antenna, or one of the resonant multiples. This makes resonant antenna designs inherently narrowband, and they are most commonly used with a single target signal. They are particularly common on radar systems, where the same antenna is used for both broadcast and reception, or for radio and television broadcasts, where the antenna is working with a single frequency. They are less commonly used for reception where multiple channels are present, in which case additional modifications are used to increase the bandwidth, or entirely different antenna designs are used.", "The amount of signal received from a distant transmission source is essentially geometric in nature due to the inverse square law, and this leads to the concept of effective area. This measures the performance of an antenna by comparing the amount of power it generates to the amount of power in the original signal, measured in terms of the signal's power density in Watts per square metre. A half-wave dipole has an effective area of 0.13 2. If more performance is needed, one cannot simply make the antenna larger. Although this would intercept more energy from the signal, due to the considerations above, it would decrease the output significantly due to it moving away from the resonant length. In roles where higher performance is needed, designers often use multiple elements combined together.", "The amount of signal received from a distant transmission source is essentially geometric in nature due to the inverse square law, and this leads to the concept of effective area. This measures the performance of an antenna by comparing the amount of power it generates to the amount of power in the original signal, measured in terms of the signal's power density in Watts per square metre. A half-wave dipole has an effective area of 0.13 2. If more performance is needed, one cannot simply make the antenna larger. Although this would intercept more energy from the signal, due to the considerations above, it would decrease the output significantly due to it moving away from the resonant length. In roles where higher performance is needed, designers often use multiple elements combined together.", "The amount of signal received from a distant transmission source is essentially geometric in nature due to the inverse square law, and this leads to the concept of effective area. This measures the performance of an antenna by comparing the amount of power it generates to the amount of power in the original signal, measured in terms of the signal's power density in Watts per square metre. A half-wave dipole has an effective area of 0.13 2. If more performance is needed, one cannot simply make the antenna larger. Although this would intercept more energy from the signal, due to the considerations above, it would decrease the output significantly due to it moving away from the resonant length. In roles where higher performance is needed, designers often use multiple elements combined together.", 'Returning to the basic concept of current flows in a conductor, consider what happens if a half-wave dipole is not connected to a feed point, but instead shorted out. Electrically this forms a single 1⁄2-wavelength element. But the overall current pattern is the same; the current will be zero at the two ends, and reach a maximum in the center. Thus signals near the design frequency will continue to create a standing wave pattern. Any varying electrical current, like the standing wave in the element, will radiate a signal. In this case, aside from resistive losses in the element, the rebroadcast signal will be significantly similar to the original signal in both magnitude and shape. If this element is placed so its signal reaches the main dipole in-phase, it will reinforce the original signal, and increase the current in the dipole. Elements used in this way are known as passive elements.', 'Returning to the basic concept of current flows in a conductor, consider what happens if a half-wave dipole is not connected to a feed point, but instead shorted out. Electrically this forms a single 1⁄2-wavelength element. But the overall current pattern is the same; the current will be zero at the two ends, and reach a maximum in the center. Thus signals near the design frequency will continue to create a standing wave pattern. Any varying electrical current, like the standing wave in the element, will radiate a signal. In this case, aside from resistive losses in the element, the rebroadcast signal will be significantly similar to the original signal in both magnitude and shape. If this element is placed so its signal reaches the main dipole in-phase, it will reinforce the original signal, and increase the current in the dipole. Elements used in this way are known as passive elements.', 'Returning to the basic concept of current flows in a conductor, consider what happens if a half-wave dipole is not connected to a feed point, but instead shorted out. Electrically this forms a single 1⁄2-wavelength element. But the overall current pattern is the same; the current will be zero at the two ends, and reach a maximum in the center. Thus signals near the design frequency will continue to create a standing wave pattern. Any varying electrical current, like the standing wave in the element, will radiate a signal. In this case, aside from resistive losses in the element, the rebroadcast signal will be significantly similar to the original signal in both magnitude and shape. If this element is placed so its signal reaches the main dipole in-phase, it will reinforce the original signal, and increase the current in the dipole. Elements used in this way are known as passive elements.', 'Returning to the basic concept of current flows in a conductor, consider what happens if a half-wave dipole is not connected to a feed point, but instead shorted out. Electrically this forms a single 1⁄2-wavelength element. But the overall current pattern is the same; the current will be zero at the two ends, and reach a maximum in the center. Thus signals near the design frequency will continue to create a standing wave pattern. Any varying electrical current, like the standing wave in the element, will radiate a signal. In this case, aside from resistive losses in the element, the rebroadcast signal will be significantly similar to the original signal in both magnitude and shape. If this element is placed so its signal reaches the main dipole in-phase, it will reinforce the original signal, and increase the current in the dipole. Elements used in this way are known as passive elements.', "A Yagi-Uda array uses passive elements to greatly increase gain. It is built along a support boom that is pointed toward the signal, and thus sees no induced signal and does not contribute to the antenna's operation. The end closer to the source is referred to as the front. Near the rear is a single active element, typically a half-wave dipole or folded dipole. Passive elements are arranged in front (directors) and behind (reflectors) the active element along the boom. The Yagi has the inherent quality that it becomes increasingly directional, and thus has higher gain, as the number of elements increases. However, this also makes it increasingly sensitive to changes in frequency; if the signal frequency changes, not only does the active element receive less energy directly, but all of the passive elements adding to that signal also decrease their output as well and their signals no longer reach the active element in-phase.", "A Yagi-Uda array uses passive elements to greatly increase gain. It is built along a support boom that is pointed toward the signal, and thus sees no induced signal and does not contribute to the antenna's operation. The end closer to the source is referred to as the front. Near the rear is a single active element, typically a half-wave dipole or folded dipole. Passive elements are arranged in front (directors) and behind (reflectors) the active element along the boom. The Yagi has the inherent quality that it becomes increasingly directional, and thus has higher gain, as the number of elements increases. However, this also makes it increasingly sensitive to changes in frequency; if the signal frequency changes, not only does the active element receive less energy directly, but all of the passive elements adding to that signal also decrease their output as well and their signals no longer reach the active element in-phase.", "A Yagi-Uda array uses passive elements to greatly increase gain. It is built along a support boom that is pointed toward the signal, and thus sees no induced signal and does not contribute to the antenna's operation. The end closer to the source is referred to as the front. Near the rear is a single active element, typically a half-wave dipole or folded dipole. Passive elements are arranged in front (directors) and behind (reflectors) the active element along the boom. The Yagi has the inherent quality that it becomes increasingly directional, and thus has higher gain, as the number of elements increases. However, this also makes it increasingly sensitive to changes in frequency; if the signal frequency changes, not only does the active element receive less energy directly, but all of the passive elements adding to that signal also decrease their output as well and their signals no longer reach the active element in-phase.", "A Yagi-Uda array uses passive elements to greatly increase gain. It is built along a support boom that is pointed toward the signal, and thus sees no induced signal and does not contribute to the antenna's operation. The end closer to the source is referred to as the front. Near the rear is a single active element, typically a half-wave dipole or folded dipole. Passive elements are arranged in front (directors) and behind (reflectors) the active element along the boom. The Yagi has the inherent quality that it becomes increasingly directional, and thus has higher gain, as the number of elements increases. However, this also makes it increasingly sensitive to changes in frequency; if the signal frequency changes, not only does the active element receive less energy directly, but all of the passive elements adding to that signal also decrease their output as well and their signals no longer reach the active element in-phase.", 'It is also possible to use multiple active elements and combine them together with transmission lines to produce a similar system where the phases add up to reinforce the output. The antenna array and very similar reflective array antenna consist of multiple elements, often half-wave dipoles, spaced out on a plane and wired together with transmission lines with specific phase lengths to produce a single in-phase signal at the output. The log-periodic antenna is a more complex design that uses multiple in-line elements similar in appearance to the Yagi-Uda but using transmission lines between the elements to produce the output.', 'It is also possible to use multiple active elements and combine them together with transmission lines to produce a similar system where the phases add up to reinforce the output. The antenna array and very similar reflective array antenna consist of multiple elements, often half-wave dipoles, spaced out on a plane and wired together with transmission lines with specific phase lengths to produce a single in-phase signal at the output. The log-periodic antenna is a more complex design that uses multiple in-line elements similar in appearance to the Yagi-Uda but using transmission lines between the elements to produce the output.', 'It is also possible to use multiple active elements and combine them together with transmission lines to produce a similar system where the phases add up to reinforce the output. The antenna array and very similar reflective array antenna consist of multiple elements, often half-wave dipoles, spaced out on a plane and wired together with transmission lines with specific phase lengths to produce a single in-phase signal at the output. The log-periodic antenna is a more complex design that uses multiple in-line elements similar in appearance to the Yagi-Uda but using transmission lines between the elements to produce the output.', 'It is also possible to use multiple active elements and combine them together with transmission lines to produce a similar system where the phases add up to reinforce the output. The antenna array and very similar reflective array antenna consist of multiple elements, often half-wave dipoles, spaced out on a plane and wired together with transmission lines with specific phase lengths to produce a single in-phase signal at the output. The log-periodic antenna is a more complex design that uses multiple in-line elements similar in appearance to the Yagi-Uda but using transmission lines between the elements to produce the output.', 'Reflection of the original signal also occurs when it hits an extended conductive surface, in a fashion similar to a mirror. This effect can also be used to increase signal through the use of a reflector, normally placed behind the active element and spaced so the reflected signal reaches the element in-phase. Generally the reflector will remain highly reflective even if it is not solid; gaps less than 1⁄10 generally have little effect on the outcome. For this reason, reflectors often take the form of wire meshes or rows of passive elements, which makes them lighter and less subject to wind. The parabolic reflector is perhaps the best known example of a reflector-based antenna, which has an effective area far greater than the active element alone.', 'Reflection of the original signal also occurs when it hits an extended conductive surface, in a fashion similar to a mirror. This effect can also be used to increase signal through the use of a reflector, normally placed behind the active element and spaced so the reflected signal reaches the element in-phase. Generally the reflector will remain highly reflective even if it is not solid; gaps less than 1⁄10 generally have little effect on the outcome. For this reason, reflectors often take the form of wire meshes or rows of passive elements, which makes them lighter and less subject to wind. The parabolic reflector is perhaps the best known example of a reflector-based antenna, which has an effective area far greater than the active element alone.', 'Reflection of the original signal also occurs when it hits an extended conductive surface, in a fashion similar to a mirror. This effect can also be used to increase signal through the use of a reflector, normally placed behind the active element and spaced so the reflected signal reaches the element in-phase. Generally the reflector will remain highly reflective even if it is not solid; gaps less than 1⁄10 generally have little effect on the outcome. For this reason, reflectors often take the form of wire meshes or rows of passive elements, which makes them lighter and less subject to wind. The parabolic reflector is perhaps the best known example of a reflector-based antenna, which has an effective area far greater than the active element alone.', 'Reflection of the original signal also occurs when it hits an extended conductive surface, in a fashion similar to a mirror. This effect can also be used to increase signal through the use of a reflector, normally placed behind the active element and spaced so the reflected signal reaches the element in-phase. Generally the reflector will remain highly reflective even if it is not solid; gaps less than 1⁄10 generally have little effect on the outcome. For this reason, reflectors often take the form of wire meshes or rows of passive elements, which makes them lighter and less subject to wind. The parabolic reflector is perhaps the best known example of a reflector-based antenna, which has an effective area far greater than the active element alone.', 'Another extreme case of impedance matching occurs when using a small loop antenna (usually, but not always, for receiving) at a relatively low frequency where it appears almost as a pure inductor. Resonating such an inductor with a capacitor at the frequency of operation not only cancels the reactance but greatly magnifies the very small radiation resistance of such a loop.[citation needed] This is implemented in most AM broadcast receivers, with a small ferrite loop antenna resonated by a capacitor which is varied along with the receiver tuning in order to maintain resonance over the AM broadcast band', 'Another extreme case of impedance matching occurs when using a small loop antenna (usually, but not always, for receiving) at a relatively low frequency where it appears almost as a pure inductor. Resonating such an inductor with a capacitor at the frequency of operation not only cancels the reactance but greatly magnifies the very small radiation resistance of such a loop.[citation needed] This is implemented in most AM broadcast receivers, with a small ferrite loop antenna resonated by a capacitor which is varied along with the receiver tuning in order to maintain resonance over the AM broadcast band', 'Another extreme case of impedance matching occurs when using a small loop antenna (usually, but not always, for receiving) at a relatively low frequency where it appears almost as a pure inductor. Resonating such an inductor with a capacitor at the frequency of operation not only cancels the reactance but greatly magnifies the very small radiation resistance of such a loop.[citation needed] This is implemented in most AM broadcast receivers, with a small ferrite loop antenna resonated by a capacitor which is varied along with the receiver tuning in order to maintain resonance over the AM broadcast band', 'Another extreme case of impedance matching occurs when using a small loop antenna (usually, but not always, for receiving) at a relatively low frequency where it appears almost as a pure inductor. Resonating such an inductor with a capacitor at the frequency of operation not only cancels the reactance but greatly magnifies the very small radiation resistance of such a loop.[citation needed] This is implemented in most AM broadcast receivers, with a small ferrite loop antenna resonated by a capacitor which is varied along with the receiver tuning in order to maintain resonance over the AM broadcast band', 'Antenna tuning generally refers to cancellation of any reactance seen at the antenna terminals, leaving only a resistive impedance which might or might not be exactly the desired impedance (that of the transmission line). Although an antenna may be designed to have a purely resistive feedpoint impedance (such as a dipole 97% of a half wavelength long) this might not be exactly true at the frequency that it is eventually used at. In some cases the physical length of the antenna can be "trimmed" to obtain a pure resistance. On the other hand, the addition of a series inductance or parallel capacitance can be used to cancel a residual capacitative or inductive reactance, respectively.', 'Antenna tuning generally refers to cancellation of any reactance seen at the antenna terminals, leaving only a resistive impedance which might or might not be exactly the desired impedance (that of the transmission line). Although an antenna may be designed to have a purely resistive feedpoint impedance (such as a dipole 97% of a half wavelength long) this might not be exactly true at the frequency that it is eventually used at. In some cases the physical length of the antenna can be "trimmed" to obtain a pure resistance. On the other hand, the addition of a series inductance or parallel capacitance can be used to cancel a residual capacitative or inductive reactance, respectively.', 'Antenna tuning generally refers to cancellation of any reactance seen at the antenna terminals, leaving only a resistive impedance which might or might not be exactly the desired impedance (that of the transmission line). Although an antenna may be designed to have a purely resistive feedpoint impedance (such as a dipole 97% of a half wavelength long) this might not be exactly true at the frequency that it is eventually used at. In some cases the physical length of the antenna can be "trimmed" to obtain a pure resistance. On the other hand, the addition of a series inductance or parallel capacitance can be used to cancel a residual capacitative or inductive reactance, respectively.', 'Antenna tuning generally refers to cancellation of any reactance seen at the antenna terminals, leaving only a resistive impedance which might or might not be exactly the desired impedance (that of the transmission line). Although an antenna may be designed to have a purely resistive feedpoint impedance (such as a dipole 97% of a half wavelength long) this might not be exactly true at the frequency that it is eventually used at. In some cases the physical length of the antenna can be "trimmed" to obtain a pure resistance. On the other hand, the addition of a series inductance or parallel capacitance can be used to cancel a residual capacitative or inductive reactance, respectively.', "Although a resonant antenna has a purely resistive feed-point impedance at a particular frequency, many (if not most) applications require using an antenna over a range of frequencies. An antenna's bandwidth specifies the range of frequencies over which its performance does not suffer due to a poor impedance match. Also in the case of a Yagi-Uda array, the use of the antenna very far away from its design frequency reduces the antenna's directivity, thus reducing the usable bandwidth regardless of impedance matching.", "Although a resonant antenna has a purely resistive feed-point impedance at a particular frequency, many (if not most) applications require using an antenna over a range of frequencies. An antenna's bandwidth specifies the range of frequencies over which its performance does not suffer due to a poor impedance match. Also in the case of a Yagi-Uda array, the use of the antenna very far away from its design frequency reduces the antenna's directivity, thus reducing the usable bandwidth regardless of impedance matching.", "Although a resonant antenna has a purely resistive feed-point impedance at a particular frequency, many (if not most) applications require using an antenna over a range of frequencies. An antenna's bandwidth specifies the range of frequencies over which its performance does not suffer due to a poor impedance match. Also in the case of a Yagi-Uda array, the use of the antenna very far away from its design frequency reduces the antenna's directivity, thus reducing the usable bandwidth regardless of impedance matching.", "Although a resonant antenna has a purely resistive feed-point impedance at a particular frequency, many (if not most) applications require using an antenna over a range of frequencies. An antenna's bandwidth specifies the range of frequencies over which its performance does not suffer due to a poor impedance match. Also in the case of a Yagi-Uda array, the use of the antenna very far away from its design frequency reduces the antenna's directivity, thus reducing the usable bandwidth regardless of impedance matching.", "Instead, it is often desired to have an antenna whose impedance does not vary so greatly over a certain bandwidth. It turns out that the amount of reactance seen at the terminals of a resonant antenna when the frequency is shifted, say, by 5%, depends very much on the diameter of the conductor used. A long thin wire used as a half-wave dipole (or quarter wave monopole) will have a reactance significantly greater than the resistive impedance it has at resonance, leading to a poor match and generally unacceptable performance. Making the element using a tube of a diameter perhaps 1/50 of its length, however, results in a reactance at this altered frequency which is not so great, and a much less serious mismatch which will only modestly damage the antenna's net performance. Thus rather thick tubes are typically used for the solid elements of such antennas, including Yagi-Uda arrays.", "Instead, it is often desired to have an antenna whose impedance does not vary so greatly over a certain bandwidth. It turns out that the amount of reactance seen at the terminals of a resonant antenna when the frequency is shifted, say, by 5%, depends very much on the diameter of the conductor used. A long thin wire used as a half-wave dipole (or quarter wave monopole) will have a reactance significantly greater than the resistive impedance it has at resonance, leading to a poor match and generally unacceptable performance. Making the element using a tube of a diameter perhaps 1/50 of its length, however, results in a reactance at this altered frequency which is not so great, and a much less serious mismatch which will only modestly damage the antenna's net performance. Thus rather thick tubes are typically used for the solid elements of such antennas, including Yagi-Uda arrays.", "Instead, it is often desired to have an antenna whose impedance does not vary so greatly over a certain bandwidth. It turns out that the amount of reactance seen at the terminals of a resonant antenna when the frequency is shifted, say, by 5%, depends very much on the diameter of the conductor used. A long thin wire used as a half-wave dipole (or quarter wave monopole) will have a reactance significantly greater than the resistive impedance it has at resonance, leading to a poor match and generally unacceptable performance. Making the element using a tube of a diameter perhaps 1/50 of its length, however, results in a reactance at this altered frequency which is not so great, and a much less serious mismatch which will only modestly damage the antenna's net performance. Thus rather thick tubes are typically used for the solid elements of such antennas, including Yagi-Uda arrays.", "Instead, it is often desired to have an antenna whose impedance does not vary so greatly over a certain bandwidth. It turns out that the amount of reactance seen at the terminals of a resonant antenna when the frequency is shifted, say, by 5%, depends very much on the diameter of the conductor used. A long thin wire used as a half-wave dipole (or quarter wave monopole) will have a reactance significantly greater than the resistive impedance it has at resonance, leading to a poor match and generally unacceptable performance. Making the element using a tube of a diameter perhaps 1/50 of its length, however, results in a reactance at this altered frequency which is not so great, and a much less serious mismatch which will only modestly damage the antenna's net performance. Thus rather thick tubes are typically used for the solid elements of such antennas, including Yagi-Uda arrays.", "Rather than just using a thick tube, there are similar techniques used to the same effect such as replacing thin wire elements with cages to simulate a thicker element. This widens the bandwidth of the resonance. On the other hand, amateur radio antennas need to operate over several bands which are widely separated from each other. This can often be accomplished simply by connecting resonant elements for the different bands in parallel. Most of the transmitter's power will flow into the resonant element while the others present a high (reactive) impedance and draw little current from the same voltage. A popular solution uses so-called traps consisting of parallel resonant circuits which are strategically placed in breaks along each antenna element. When used at one particular frequency band the trap presents a very high impedance (parallel resonance) effectively truncating the element at that length, making it a proper resonant antenna. At a lower frequency the trap allows the full length of the element to be employed, albeit with a shifted resonant frequency due to the inclusion of the trap's net reactance at that lower frequency.", "Rather than just using a thick tube, there are similar techniques used to the same effect such as replacing thin wire elements with cages to simulate a thicker element. This widens the bandwidth of the resonance. On the other hand, amateur radio antennas need to operate over several bands which are widely separated from each other. This can often be accomplished simply by connecting resonant elements for the different bands in parallel. Most of the transmitter's power will flow into the resonant element while the others present a high (reactive) impedance and draw little current from the same voltage. A popular solution uses so-called traps consisting of parallel resonant circuits which are strategically placed in breaks along each antenna element. When used at one particular frequency band the trap presents a very high impedance (parallel resonance) effectively truncating the element at that length, making it a proper resonant antenna. At a lower frequency the trap allows the full length of the element to be employed, albeit with a shifted resonant frequency due to the inclusion of the trap's net reactance at that lower frequency.", "Rather than just using a thick tube, there are similar techniques used to the same effect such as replacing thin wire elements with cages to simulate a thicker element. This widens the bandwidth of the resonance. On the other hand, amateur radio antennas need to operate over several bands which are widely separated from each other. This can often be accomplished simply by connecting resonant elements for the different bands in parallel. Most of the transmitter's power will flow into the resonant element while the others present a high (reactive) impedance and draw little current from the same voltage. A popular solution uses so-called traps consisting of parallel resonant circuits which are strategically placed in breaks along each antenna element. When used at one particular frequency band the trap presents a very high impedance (parallel resonance) effectively truncating the element at that length, making it a proper resonant antenna. At a lower frequency the trap allows the full length of the element to be employed, albeit with a shifted resonant frequency due to the inclusion of the trap's net reactance at that lower frequency.", "Rather than just using a thick tube, there are similar techniques used to the same effect such as replacing thin wire elements with cages to simulate a thicker element. This widens the bandwidth of the resonance. On the other hand, amateur radio antennas need to operate over several bands which are widely separated from each other. This can often be accomplished simply by connecting resonant elements for the different bands in parallel. Most of the transmitter's power will flow into the resonant element while the others present a high (reactive) impedance and draw little current from the same voltage. A popular solution uses so-called traps consisting of parallel resonant circuits which are strategically placed in breaks along each antenna element. When used at one particular frequency band the trap presents a very high impedance (parallel resonance) effectively truncating the element at that length, making it a proper resonant antenna. At a lower frequency the trap allows the full length of the element to be employed, albeit with a shifted resonant frequency due to the inclusion of the trap's net reactance at that lower frequency.", 'Gain is a parameter which measures the degree of directivity of the antenna\'s radiation pattern. A high-gain antenna will radiate most of its power in a particular direction, while a low-gain antenna will radiate over a wider angle. The antenna gain, or power gain of an antenna is defined as the ratio of the intensity (power per unit surface area)  radiated by the antenna in the direction of its maximum output, at an arbitrary distance, divided by the intensity  radiated at the same distance by a hypothetical isotropic antenna which radiates equal power in all directions. This dimensionless ratio is usually expressed logarithmically in decibels, these units are called "decibels-isotropic" (dBi)', 'Gain is a parameter which measures the degree of directivity of the antenna\'s radiation pattern. A high-gain antenna will radiate most of its power in a particular direction, while a low-gain antenna will radiate over a wider angle. The antenna gain, or power gain of an antenna is defined as the ratio of the intensity (power per unit surface area)  radiated by the antenna in the direction of its maximum output, at an arbitrary distance, divided by the intensity  radiated at the same distance by a hypothetical isotropic antenna which radiates equal power in all directions. This dimensionless ratio is usually expressed logarithmically in decibels, these units are called "decibels-isotropic" (dBi)', 'Gain is a parameter which measures the degree of directivity of the antenna\'s radiation pattern. A high-gain antenna will radiate most of its power in a particular direction, while a low-gain antenna will radiate over a wider angle. The antenna gain, or power gain of an antenna is defined as the ratio of the intensity (power per unit surface area)  radiated by the antenna in the direction of its maximum output, at an arbitrary distance, divided by the intensity  radiated at the same distance by a hypothetical isotropic antenna which radiates equal power in all directions. This dimensionless ratio is usually expressed logarithmically in decibels, these units are called "decibels-isotropic" (dBi)', 'Gain is a parameter which measures the degree of directivity of the antenna\'s radiation pattern. A high-gain antenna will radiate most of its power in a particular direction, while a low-gain antenna will radiate over a wider angle. The antenna gain, or power gain of an antenna is defined as the ratio of the intensity (power per unit surface area)  radiated by the antenna in the direction of its maximum output, at an arbitrary distance, divided by the intensity  radiated at the same distance by a hypothetical isotropic antenna which radiates equal power in all directions. This dimensionless ratio is usually expressed logarithmically in decibels, these units are called "decibels-isotropic" (dBi)', 'High-gain antennas have the advantage of longer range and better signal quality, but must be aimed carefully at the other antenna. An example of a high-gain antenna is a parabolic dish such as a satellite television antenna. Low-gain antennas have shorter range, but the orientation of the antenna is relatively unimportant. An example of a low-gain antenna is the whip antenna found on portable radios and cordless phones. Antenna gain should not be confused with amplifier gain, a separate parameter measuring the increase in signal power due to an amplifying device.', 'High-gain antennas have the advantage of longer range and better signal quality, but must be aimed carefully at the other antenna. An example of a high-gain antenna is a parabolic dish such as a satellite television antenna. Low-gain antennas have shorter range, but the orientation of the antenna is relatively unimportant. An example of a low-gain antenna is the whip antenna found on portable radios and cordless phones. Antenna gain should not be confused with amplifier gain, a separate parameter measuring the increase in signal power due to an amplifying device.', 'High-gain antennas have the advantage of longer range and better signal quality, but must be aimed carefully at the other antenna. An example of a high-gain antenna is a parabolic dish such as a satellite television antenna. Low-gain antennas have shorter range, but the orientation of the antenna is relatively unimportant. An example of a low-gain antenna is the whip antenna found on portable radios and cordless phones. Antenna gain should not be confused with amplifier gain, a separate parameter measuring the increase in signal power due to an amplifying device.', 'High-gain antennas have the advantage of longer range and better signal quality, but must be aimed carefully at the other antenna. An example of a high-gain antenna is a parabolic dish such as a satellite television antenna. Low-gain antennas have shorter range, but the orientation of the antenna is relatively unimportant. An example of a low-gain antenna is the whip antenna found on portable radios and cordless phones. Antenna gain should not be confused with amplifier gain, a separate parameter measuring the increase in signal power due to an amplifying device.', 'Due to reciprocity (discussed above) the gain of an antenna used for transmitting must be proportional to its effective area when used for receiving. Consider an antenna with no loss, that is, one whose electrical efficiency is 100%. It can be shown that its effective area averaged over all directions must be equal to λ2/4π, the wavelength squared divided by 4π. Gain is defined such that the average gain over all directions for an antenna with 100% electrical efficiency is equal to 1. Therefore, the effective area Aeff in terms of the gain G in a given direction is given by:', 'Due to reciprocity (discussed above) the gain of an antenna used for transmitting must be proportional to its effective area when used for receiving. Consider an antenna with no loss, that is, one whose electrical efficiency is 100%. It can be shown that its effective area averaged over all directions must be equal to λ2/4π, the wavelength squared divided by 4π. Gain is defined such that the average gain over all directions for an antenna with 100% electrical efficiency is equal to 1. Therefore, the effective area Aeff in terms of the gain G in a given direction is given by:', 'Due to reciprocity (discussed above) the gain of an antenna used for transmitting must be proportional to its effective area when used for receiving. Consider an antenna with no loss, that is, one whose electrical efficiency is 100%. It can be shown that its effective area averaged over all directions must be equal to λ2/4π, the wavelength squared divided by 4π. Gain is defined such that the average gain over all directions for an antenna with 100% electrical efficiency is equal to 1. Therefore, the effective area Aeff in terms of the gain G in a given direction is given by:', 'Due to reciprocity (discussed above) the gain of an antenna used for transmitting must be proportional to its effective area when used for receiving. Consider an antenna with no loss, that is, one whose electrical efficiency is 100%. It can be shown that its effective area averaged over all directions must be equal to λ2/4π, the wavelength squared divided by 4π. Gain is defined such that the average gain over all directions for an antenna with 100% electrical efficiency is equal to 1. Therefore, the effective area Aeff in terms of the gain G in a given direction is given by:', 'The radiation pattern of an antenna is a plot of the relative field strength of the radio waves emitted by the antenna at different angles. It is typically represented by a three-dimensional graph, or polar plots of the horizontal and vertical cross sections. The pattern of an ideal isotropic antenna, which radiates equally in all directions, would look like a sphere. Many nondirectional antennas, such as monopoles and dipoles, emit equal power in all horizontal directions, with the power dropping off at higher and lower angles; this is called an omnidirectional pattern and when plotted looks like a torus or donut.', 'The radiation pattern of an antenna is a plot of the relative field strength of the radio waves emitted by the antenna at different angles. It is typically represented by a three-dimensional graph, or polar plots of the horizontal and vertical cross sections. The pattern of an ideal isotropic antenna, which radiates equally in all directions, would look like a sphere. Many nondirectional antennas, such as monopoles and dipoles, emit equal power in all horizontal directions, with the power dropping off at higher and lower angles; this is called an omnidirectional pattern and when plotted looks like a torus or donut.', 'The radiation pattern of an antenna is a plot of the relative field strength of the radio waves emitted by the antenna at different angles. It is typically represented by a three-dimensional graph, or polar plots of the horizontal and vertical cross sections. The pattern of an ideal isotropic antenna, which radiates equally in all directions, would look like a sphere. Many nondirectional antennas, such as monopoles and dipoles, emit equal power in all horizontal directions, with the power dropping off at higher and lower angles; this is called an omnidirectional pattern and when plotted looks like a torus or donut.', 'The radiation pattern of an antenna is a plot of the relative field strength of the radio waves emitted by the antenna at different angles. It is typically represented by a three-dimensional graph, or polar plots of the horizontal and vertical cross sections. The pattern of an ideal isotropic antenna, which radiates equally in all directions, would look like a sphere. Many nondirectional antennas, such as monopoles and dipoles, emit equal power in all horizontal directions, with the power dropping off at higher and lower angles; this is called an omnidirectional pattern and when plotted looks like a torus or donut.', 'The radiation pattern of an antenna is a plot of the relative field strength of the radio waves emitted by the antenna at different angles. It is typically represented by a three-dimensional graph, or polar plots of the horizontal and vertical cross sections. The pattern of an ideal isotropic antenna, which radiates equally in all directions, would look like a sphere. Many nondirectional antennas, such as monopoles and dipoles, emit equal power in all horizontal directions, with the power dropping off at higher and lower angles; this is called an omnidirectional pattern and when plotted looks like a torus or donut.', 'The radiation of many antennas shows a pattern of maxima or "lobes" at various angles, separated by "nulls", angles where the radiation falls to zero. This is because the radio waves emitted by different parts of the antenna typically interfere, causing maxima at angles where the radio waves arrive at distant points in phase, and zero radiation at other angles where the radio waves arrive out of phase. In a directional antenna designed to project radio waves in a particular direction, the lobe in that direction is designed larger than the others and is called the "main lobe". The other lobes usually represent unwanted radiation and are called "sidelobes". The axis through the main lobe is called the "principal axis" or "boresight axis".', 'The radiation of many antennas shows a pattern of maxima or "lobes" at various angles, separated by "nulls", angles where the radiation falls to zero. This is because the radio waves emitted by different parts of the antenna typically interfere, causing maxima at angles where the radio waves arrive at distant points in phase, and zero radiation at other angles where the radio waves arrive out of phase. In a directional antenna designed to project radio waves in a particular direction, the lobe in that direction is designed larger than the others and is called the "main lobe". The other lobes usually represent unwanted radiation and are called "sidelobes". The axis through the main lobe is called the "principal axis" or "boresight axis".', 'The radiation of many antennas shows a pattern of maxima or "lobes" at various angles, separated by "nulls", angles where the radiation falls to zero. This is because the radio waves emitted by different parts of the antenna typically interfere, causing maxima at angles where the radio waves arrive at distant points in phase, and zero radiation at other angles where the radio waves arrive out of phase. In a directional antenna designed to project radio waves in a particular direction, the lobe in that direction is designed larger than the others and is called the "main lobe". The other lobes usually represent unwanted radiation and are called "sidelobes". The axis through the main lobe is called the "principal axis" or "boresight axis".', 'The radiation of many antennas shows a pattern of maxima or "lobes" at various angles, separated by "nulls", angles where the radiation falls to zero. This is because the radio waves emitted by different parts of the antenna typically interfere, causing maxima at angles where the radio waves arrive at distant points in phase, and zero radiation at other angles where the radio waves arrive out of phase. In a directional antenna designed to project radio waves in a particular direction, the lobe in that direction is designed larger than the others and is called the "main lobe". The other lobes usually represent unwanted radiation and are called "sidelobes". The axis through the main lobe is called the "principal axis" or "boresight axis".', "As an electro-magnetic wave travels through the different parts of the antenna system (radio, feed line, antenna, free space) it may encounter differences in impedance (E/H, V/I, etc.). At each interface, depending on the impedance match, some fraction of the wave's energy will reflect back to the source, forming a standing wave in the feed line. The ratio of maximum power to minimum power in the wave can be measured and is called the standing wave ratio (SWR). A SWR of 1:1 is ideal. A SWR of 1.5:1 is considered to be marginally acceptable in low power applications where power loss is more critical, although an SWR as high as 6:1 may still be usable with the right equipment. Minimizing impedance differences at each interface (impedance matching) will reduce SWR and maximize power transfer through each part of the antenna system.", "As an electro-magnetic wave travels through the different parts of the antenna system (radio, feed line, antenna, free space) it may encounter differences in impedance (E/H, V/I, etc.). At each interface, depending on the impedance match, some fraction of the wave's energy will reflect back to the source, forming a standing wave in the feed line. The ratio of maximum power to minimum power in the wave can be measured and is called the standing wave ratio (SWR). A SWR of 1:1 is ideal. A SWR of 1.5:1 is considered to be marginally acceptable in low power applications where power loss is more critical, although an SWR as high as 6:1 may still be usable with the right equipment. Minimizing impedance differences at each interface (impedance matching) will reduce SWR and maximize power transfer through each part of the antenna system.", "As an electro-magnetic wave travels through the different parts of the antenna system (radio, feed line, antenna, free space) it may encounter differences in impedance (E/H, V/I, etc.). At each interface, depending on the impedance match, some fraction of the wave's energy will reflect back to the source, forming a standing wave in the feed line. The ratio of maximum power to minimum power in the wave can be measured and is called the standing wave ratio (SWR). A SWR of 1:1 is ideal. A SWR of 1.5:1 is considered to be marginally acceptable in low power applications where power loss is more critical, although an SWR as high as 6:1 may still be usable with the right equipment. Minimizing impedance differences at each interface (impedance matching) will reduce SWR and maximize power transfer through each part of the antenna system.", "As an electro-magnetic wave travels through the different parts of the antenna system (radio, feed line, antenna, free space) it may encounter differences in impedance (E/H, V/I, etc.). At each interface, depending on the impedance match, some fraction of the wave's energy will reflect back to the source, forming a standing wave in the feed line. The ratio of maximum power to minimum power in the wave can be measured and is called the standing wave ratio (SWR). A SWR of 1:1 is ideal. A SWR of 1.5:1 is considered to be marginally acceptable in low power applications where power loss is more critical, although an SWR as high as 6:1 may still be usable with the right equipment. Minimizing impedance differences at each interface (impedance matching) will reduce SWR and maximize power transfer through each part of the antenna system.", "Efficiency of a transmitting antenna is the ratio of power actually radiated (in all directions) to the power absorbed by the antenna terminals. The power supplied to the antenna terminals which is not radiated is converted into heat. This is usually through loss resistance in the antenna's conductors, but can also be due to dielectric or magnetic core losses in antennas (or antenna systems) using such components. Such loss effectively robs power from the transmitter, requiring a stronger transmitter in order to transmit a signal of a given strength.", "Efficiency of a transmitting antenna is the ratio of power actually radiated (in all directions) to the power absorbed by the antenna terminals. The power supplied to the antenna terminals which is not radiated is converted into heat. This is usually through loss resistance in the antenna's conductors, but can also be due to dielectric or magnetic core losses in antennas (or antenna systems) using such components. Such loss effectively robs power from the transmitter, requiring a stronger transmitter in order to transmit a signal of a given strength.", "Efficiency of a transmitting antenna is the ratio of power actually radiated (in all directions) to the power absorbed by the antenna terminals. The power supplied to the antenna terminals which is not radiated is converted into heat. This is usually through loss resistance in the antenna's conductors, but can also be due to dielectric or magnetic core losses in antennas (or antenna systems) using such components. Such loss effectively robs power from the transmitter, requiring a stronger transmitter in order to transmit a signal of a given strength.", "Efficiency of a transmitting antenna is the ratio of power actually radiated (in all directions) to the power absorbed by the antenna terminals. The power supplied to the antenna terminals which is not radiated is converted into heat. This is usually through loss resistance in the antenna's conductors, but can also be due to dielectric or magnetic core losses in antennas (or antenna systems) using such components. Such loss effectively robs power from the transmitter, requiring a stronger transmitter in order to transmit a signal of a given strength.", 'For instance, if a transmitter delivers 100 W into an antenna having an efficiency of 80%, then the antenna will radiate 80 W as radio waves and produce 20 W of heat. In order to radiate 100 W of power, one would need to use a transmitter capable of supplying 125 W to the antenna. Note that antenna efficiency is a separate issue from impedance matching, which may also reduce the amount of power radiated using a given transmitter. If an SWR meter reads 150 W of incident power and 50 W of reflected power, that means that 100 W have actually been absorbed by the antenna (ignoring transmission line losses). How much of that power has actually been radiated cannot be directly determined through electrical measurements at (or before) the antenna terminals, but would require (for instance) careful measurement of field strength. Fortunately the loss resistance of antenna conductors such as aluminum rods can be calculated and the efficiency of an antenna using such materials predicted.', 'For instance, if a transmitter delivers 100 W into an antenna having an efficiency of 80%, then the antenna will radiate 80 W as radio waves and produce 20 W of heat. In order to radiate 100 W of power, one would need to use a transmitter capable of supplying 125 W to the antenna. Note that antenna efficiency is a separate issue from impedance matching, which may also reduce the amount of power radiated using a given transmitter. If an SWR meter reads 150 W of incident power and 50 W of reflected power, that means that 100 W have actually been absorbed by the antenna (ignoring transmission line losses). How much of that power has actually been radiated cannot be directly determined through electrical measurements at (or before) the antenna terminals, but would require (for instance) careful measurement of field strength. Fortunately the loss resistance of antenna conductors such as aluminum rods can be calculated and the efficiency of an antenna using such materials predicted.', 'For instance, if a transmitter delivers 100 W into an antenna having an efficiency of 80%, then the antenna will radiate 80 W as radio waves and produce 20 W of heat. In order to radiate 100 W of power, one would need to use a transmitter capable of supplying 125 W to the antenna. Note that antenna efficiency is a separate issue from impedance matching, which may also reduce the amount of power radiated using a given transmitter. If an SWR meter reads 150 W of incident power and 50 W of reflected power, that means that 100 W have actually been absorbed by the antenna (ignoring transmission line losses). How much of that power has actually been radiated cannot be directly determined through electrical measurements at (or before) the antenna terminals, but would require (for instance) careful measurement of field strength. Fortunately the loss resistance of antenna conductors such as aluminum rods can be calculated and the efficiency of an antenna using such materials predicted.', 'However loss resistance will generally affect the feedpoint impedance, adding to its resistive (real) component. That resistance will consist of the sum of the radiation resistance Rr and the loss resistance Rloss. If an rms current I is delivered to the terminals of an antenna, then a power of I2Rr will be radiated and a power of I2Rloss will be lost as heat. Therefore, the efficiency of an antenna is equal to Rr / (Rr + Rloss). Of course only the total resistance Rr + Rloss can be directly measured.', 'However loss resistance will generally affect the feedpoint impedance, adding to its resistive (real) component. That resistance will consist of the sum of the radiation resistance Rr and the loss resistance Rloss. If an rms current I is delivered to the terminals of an antenna, then a power of I2Rr will be radiated and a power of I2Rloss will be lost as heat. Therefore, the efficiency of an antenna is equal to Rr / (Rr + Rloss). Of course only the total resistance Rr + Rloss can be directly measured.', 'However loss resistance will generally affect the feedpoint impedance, adding to its resistive (real) component. That resistance will consist of the sum of the radiation resistance Rr and the loss resistance Rloss. If an rms current I is delivered to the terminals of an antenna, then a power of I2Rr will be radiated and a power of I2Rloss will be lost as heat. Therefore, the efficiency of an antenna is equal to Rr / (Rr + Rloss). Of course only the total resistance Rr + Rloss can be directly measured.', 'However loss resistance will generally affect the feedpoint impedance, adding to its resistive (real) component. That resistance will consist of the sum of the radiation resistance Rr and the loss resistance Rloss. If an rms current I is delivered to the terminals of an antenna, then a power of I2Rr will be radiated and a power of I2Rloss will be lost as heat. Therefore, the efficiency of an antenna is equal to Rr / (Rr + Rloss). Of course only the total resistance Rr + Rloss can be directly measured.', 'According to reciprocity, the efficiency of an antenna used as a receiving antenna is identical to the efficiency as defined above. The power that an antenna will deliver to a receiver (with a proper impedance match) is reduced by the same amount. In some receiving applications, the very inefficient antennas may have little impact on performance. At low frequencies, for example, atmospheric or man-made noise can mask antenna inefficiency. For example, CCIR Rep. 258-3 indicates man-made noise in a residential setting at 40 MHz is about 28 dB above the thermal noise floor. Consequently, an antenna with a 20 dB loss (due to inefficiency) would have little impact on system noise performance. The loss within the antenna will affect the intended signal and the noise/interference identically, leading to no reduction in signal to noise ratio (SNR).', 'According to reciprocity, the efficiency of an antenna used as a receiving antenna is identical to the efficiency as defined above. The power that an antenna will deliver to a receiver (with a proper impedance match) is reduced by the same amount. In some receiving applications, the very inefficient antennas may have little impact on performance. At low frequencies, for example, atmospheric or man-made noise can mask antenna inefficiency. For example, CCIR Rep. 258-3 indicates man-made noise in a residential setting at 40 MHz is about 28 dB above the thermal noise floor. Consequently, an antenna with a 20 dB loss (due to inefficiency) would have little impact on system noise performance. The loss within the antenna will affect the intended signal and the noise/interference identically, leading to no reduction in signal to noise ratio (SNR).', 'According to reciprocity, the efficiency of an antenna used as a receiving antenna is identical to the efficiency as defined above. The power that an antenna will deliver to a receiver (with a proper impedance match) is reduced by the same amount. In some receiving applications, the very inefficient antennas may have little impact on performance. At low frequencies, for example, atmospheric or man-made noise can mask antenna inefficiency. For example, CCIR Rep. 258-3 indicates man-made noise in a residential setting at 40 MHz is about 28 dB above the thermal noise floor. Consequently, an antenna with a 20 dB loss (due to inefficiency) would have little impact on system noise performance. The loss within the antenna will affect the intended signal and the noise/interference identically, leading to no reduction in signal to noise ratio (SNR).', 'According to reciprocity, the efficiency of an antenna used as a receiving antenna is identical to the efficiency as defined above. The power that an antenna will deliver to a receiver (with a proper impedance match) is reduced by the same amount. In some receiving applications, the very inefficient antennas may have little impact on performance. At low frequencies, for example, atmospheric or man-made noise can mask antenna inefficiency. For example, CCIR Rep. 258-3 indicates man-made noise in a residential setting at 40 MHz is about 28 dB above the thermal noise floor. Consequently, an antenna with a 20 dB loss (due to inefficiency) would have little impact on system noise performance. The loss within the antenna will affect the intended signal and the noise/interference identically, leading to no reduction in signal to noise ratio (SNR).', "The definition of antenna gain or power gain already includes the effect of the antenna's efficiency. Therefore, if one is trying to radiate a signal toward a receiver using a transmitter of a given power, one need only compare the gain of various antennas rather than considering the efficiency as well. This is likewise true for a receiving antenna at very high (especially microwave) frequencies, where the point is to receive a signal which is strong compared to the receiver's noise temperature. However, in the case of a directional antenna used for receiving signals with the intention of rejecting interference from different directions, one is no longer concerned with the antenna efficiency, as discussed above. In this case, rather than quoting the antenna gain, one would be more concerned with the directive gain which does not include the effect of antenna (in)efficiency. The directive gain of an antenna can be computed from the published gain divided by the antenna's efficiency.", "The definition of antenna gain or power gain already includes the effect of the antenna's efficiency. Therefore, if one is trying to radiate a signal toward a receiver using a transmitter of a given power, one need only compare the gain of various antennas rather than considering the efficiency as well. This is likewise true for a receiving antenna at very high (especially microwave) frequencies, where the point is to receive a signal which is strong compared to the receiver's noise temperature. However, in the case of a directional antenna used for receiving signals with the intention of rejecting interference from different directions, one is no longer concerned with the antenna efficiency, as discussed above. In this case, rather than quoting the antenna gain, one would be more concerned with the directive gain which does not include the effect of antenna (in)efficiency. The directive gain of an antenna can be computed from the published gain divided by the antenna's efficiency.", "The definition of antenna gain or power gain already includes the effect of the antenna's efficiency. Therefore, if one is trying to radiate a signal toward a receiver using a transmitter of a given power, one need only compare the gain of various antennas rather than considering the efficiency as well. This is likewise true for a receiving antenna at very high (especially microwave) frequencies, where the point is to receive a signal which is strong compared to the receiver's noise temperature. However, in the case of a directional antenna used for receiving signals with the intention of rejecting interference from different directions, one is no longer concerned with the antenna efficiency, as discussed above. In this case, rather than quoting the antenna gain, one would be more concerned with the directive gain which does not include the effect of antenna (in)efficiency. The directive gain of an antenna can be computed from the published gain divided by the antenna's efficiency.", "The definition of antenna gain or power gain already includes the effect of the antenna's efficiency. Therefore, if one is trying to radiate a signal toward a receiver using a transmitter of a given power, one need only compare the gain of various antennas rather than considering the efficiency as well. This is likewise true for a receiving antenna at very high (especially microwave) frequencies, where the point is to receive a signal which is strong compared to the receiver's noise temperature. However, in the case of a directional antenna used for receiving signals with the intention of rejecting interference from different directions, one is no longer concerned with the antenna efficiency, as discussed above. In this case, rather than quoting the antenna gain, one would be more concerned with the directive gain which does not include the effect of antenna (in)efficiency. The directive gain of an antenna can be computed from the published gain divided by the antenna's efficiency.", "This is fortunate, since antennas at lower frequencies which are not rather large (a good fraction of a wavelength in size) are inevitably inefficient (due to the small radiation resistance Rr of small antennas). Most AM broadcast radios (except for car radios) take advantage of this principle by including a small loop antenna for reception which has an extremely poor efficiency. Using such an inefficient antenna at this low frequency (530–1650 kHz) thus has little effect on the receiver's net performance, but simply requires greater amplification by the receiver's electronics. Contrast this tiny component to the massive and very tall towers used at AM broadcast stations for transmitting at the very same frequency, where every percentage point of reduced antenna efficiency entails a substantial cost.", "This is fortunate, since antennas at lower frequencies which are not rather large (a good fraction of a wavelength in size) are inevitably inefficient (due to the small radiation resistance Rr of small antennas). Most AM broadcast radios (except for car radios) take advantage of this principle by including a small loop antenna for reception which has an extremely poor efficiency. Using such an inefficient antenna at this low frequency (530–1650 kHz) thus has little effect on the receiver's net performance, but simply requires greater amplification by the receiver's electronics. Contrast this tiny component to the massive and very tall towers used at AM broadcast stations for transmitting at the very same frequency, where every percentage point of reduced antenna efficiency entails a substantial cost.", "This is fortunate, since antennas at lower frequencies which are not rather large (a good fraction of a wavelength in size) are inevitably inefficient (due to the small radiation resistance Rr of small antennas). Most AM broadcast radios (except for car radios) take advantage of this principle by including a small loop antenna for reception which has an extremely poor efficiency. Using such an inefficient antenna at this low frequency (530–1650 kHz) thus has little effect on the receiver's net performance, but simply requires greater amplification by the receiver's electronics. Contrast this tiny component to the massive and very tall towers used at AM broadcast stations for transmitting at the very same frequency, where every percentage point of reduced antenna efficiency entails a substantial cost.", "This is fortunate, since antennas at lower frequencies which are not rather large (a good fraction of a wavelength in size) are inevitably inefficient (due to the small radiation resistance Rr of small antennas). Most AM broadcast radios (except for car radios) take advantage of this principle by including a small loop antenna for reception which has an extremely poor efficiency. Using such an inefficient antenna at this low frequency (530–1650 kHz) thus has little effect on the receiver's net performance, but simply requires greater amplification by the receiver's electronics. Contrast this tiny component to the massive and very tall towers used at AM broadcast stations for transmitting at the very same frequency, where every percentage point of reduced antenna efficiency entails a substantial cost.", 'The polarization of an antenna refers to the orientation of the electric field (E-plane) of the radio wave with respect to the Earth\'s surface and is determined by the physical structure of the antenna and by its orientation; note that this designation is totally distinct from the antenna\'s directionality. Thus, a simple straight wire antenna will have one polarization when mounted vertically, and a different polarization when mounted horizontally. As a transverse wave, the magnetic field of a radio wave is at right angles to that of the electric field, but by convention, talk of an antenna\'s "polarization" is understood to refer to the direction of the electric field.', 'The polarization of an antenna refers to the orientation of the electric field (E-plane) of the radio wave with respect to the Earth\'s surface and is determined by the physical structure of the antenna and by its orientation; note that this designation is totally distinct from the antenna\'s directionality. Thus, a simple straight wire antenna will have one polarization when mounted vertically, and a different polarization when mounted horizontally. As a transverse wave, the magnetic field of a radio wave is at right angles to that of the electric field, but by convention, talk of an antenna\'s "polarization" is understood to refer to the direction of the electric field.', 'The polarization of an antenna refers to the orientation of the electric field (E-plane) of the radio wave with respect to the Earth\'s surface and is determined by the physical structure of the antenna and by its orientation; note that this designation is totally distinct from the antenna\'s directionality. Thus, a simple straight wire antenna will have one polarization when mounted vertically, and a different polarization when mounted horizontally. As a transverse wave, the magnetic field of a radio wave is at right angles to that of the electric field, but by convention, talk of an antenna\'s "polarization" is understood to refer to the direction of the electric field.', 'The polarization of an antenna refers to the orientation of the electric field (E-plane) of the radio wave with respect to the Earth\'s surface and is determined by the physical structure of the antenna and by its orientation; note that this designation is totally distinct from the antenna\'s directionality. Thus, a simple straight wire antenna will have one polarization when mounted vertically, and a different polarization when mounted horizontally. As a transverse wave, the magnetic field of a radio wave is at right angles to that of the electric field, but by convention, talk of an antenna\'s "polarization" is understood to refer to the direction of the electric field.', "Reflections generally affect polarization. For radio waves, one important reflector is the ionosphere which can change the wave's polarization. Thus for signals received following reflection by the ionosphere (a skywave), a consistent polarization cannot be expected. For line-of-sight communications or ground wave propagation, horizontally or vertically polarized transmissions generally remain in about the same polarization state at the receiving location. Matching the receiving antenna's polarization to that of the transmitter can make a very substantial difference in received signal strength.", "Reflections generally affect polarization. For radio waves, one important reflector is the ionosphere which can change the wave's polarization. Thus for signals received following reflection by the ionosphere (a skywave), a consistent polarization cannot be expected. For line-of-sight communications or ground wave propagation, horizontally or vertically polarized transmissions generally remain in about the same polarization state at the receiving location. Matching the receiving antenna's polarization to that of the transmitter can make a very substantial difference in received signal strength.", "Reflections generally affect polarization. For radio waves, one important reflector is the ionosphere which can change the wave's polarization. Thus for signals received following reflection by the ionosphere (a skywave), a consistent polarization cannot be expected. For line-of-sight communications or ground wave propagation, horizontally or vertically polarized transmissions generally remain in about the same polarization state at the receiving location. Matching the receiving antenna's polarization to that of the transmitter can make a very substantial difference in received signal strength.", "Reflections generally affect polarization. For radio waves, one important reflector is the ionosphere which can change the wave's polarization. Thus for signals received following reflection by the ionosphere (a skywave), a consistent polarization cannot be expected. For line-of-sight communications or ground wave propagation, horizontally or vertically polarized transmissions generally remain in about the same polarization state at the receiving location. Matching the receiving antenna's polarization to that of the transmitter can make a very substantial difference in received signal strength.", "Reflections generally affect polarization. For radio waves, one important reflector is the ionosphere which can change the wave's polarization. Thus for signals received following reflection by the ionosphere (a skywave), a consistent polarization cannot be expected. For line-of-sight communications or ground wave propagation, horizontally or vertically polarized transmissions generally remain in about the same polarization state at the receiving location. Matching the receiving antenna's polarization to that of the transmitter can make a very substantial difference in received signal strength.", "Polarization is predictable from an antenna's geometry, although in some cases it is not at all obvious (such as for the quad antenna). An antenna's linear polarization is generally along the direction (as viewed from the receiving location) of the antenna's currents when such a direction can be defined. For instance, a vertical whip antenna or Wi-Fi antenna vertically oriented will transmit and receive in the vertical polarization. Antennas with horizontal elements, such as most rooftop TV antennas in the United States, are horizontally polarized (broadcast TV in the U.S. usually uses horizontal polarization). Even when the antenna system has a vertical orientation, such as an array of horizontal dipole antennas, the polarization is in the horizontal direction corresponding to the current flow. The polarization of a commercial antenna is an essential specification.", "Polarization is predictable from an antenna's geometry, although in some cases it is not at all obvious (such as for the quad antenna). An antenna's linear polarization is generally along the direction (as viewed from the receiving location) of the antenna's currents when such a direction can be defined. For instance, a vertical whip antenna or Wi-Fi antenna vertically oriented will transmit and receive in the vertical polarization. Antennas with horizontal elements, such as most rooftop TV antennas in the United States, are horizontally polarized (broadcast TV in the U.S. usually uses horizontal polarization). Even when the antenna system has a vertical orientation, such as an array of horizontal dipole antennas, the polarization is in the horizontal direction corresponding to the current flow. The polarization of a commercial antenna is an essential specification.", "Polarization is predictable from an antenna's geometry, although in some cases it is not at all obvious (such as for the quad antenna). An antenna's linear polarization is generally along the direction (as viewed from the receiving location) of the antenna's currents when such a direction can be defined. For instance, a vertical whip antenna or Wi-Fi antenna vertically oriented will transmit and receive in the vertical polarization. Antennas with horizontal elements, such as most rooftop TV antennas in the United States, are horizontally polarized (broadcast TV in the U.S. usually uses horizontal polarization). Even when the antenna system has a vertical orientation, such as an array of horizontal dipole antennas, the polarization is in the horizontal direction corresponding to the current flow. The polarization of a commercial antenna is an essential specification.", "Polarization is predictable from an antenna's geometry, although in some cases it is not at all obvious (such as for the quad antenna). An antenna's linear polarization is generally along the direction (as viewed from the receiving location) of the antenna's currents when such a direction can be defined. For instance, a vertical whip antenna or Wi-Fi antenna vertically oriented will transmit and receive in the vertical polarization. Antennas with horizontal elements, such as most rooftop TV antennas in the United States, are horizontally polarized (broadcast TV in the U.S. usually uses horizontal polarization). Even when the antenna system has a vertical orientation, such as an array of horizontal dipole antennas, the polarization is in the horizontal direction corresponding to the current flow. The polarization of a commercial antenna is an essential specification.", 'Polarization is the sum of the E-plane orientations over time projected onto an imaginary plane perpendicular to the direction of motion of the radio wave. In the most general case, polarization is elliptical, meaning that the polarization of the radio waves varies over time. Two special cases are linear polarization (the ellipse collapses into a line) as we have discussed above, and circular polarization (in which the two axes of the ellipse are equal). In linear polarization the electric field of the radio wave oscillates back and forth along one direction; this can be affected by the mounting of the antenna but usually the desired direction is either horizontal or vertical polarization. In circular polarization, the electric field (and magnetic field) of the radio wave rotates at the radio frequency circularly around the axis of propagation. Circular or elliptically polarized radio waves are designated as right-handed or left-handed using the "thumb in the direction of the propagation" rule. Note that for circular polarization, optical researchers use the opposite right hand rule from the one used by radio engineers.', 'Polarization is the sum of the E-plane orientations over time projected onto an imaginary plane perpendicular to the direction of motion of the radio wave. In the most general case, polarization is elliptical, meaning that the polarization of the radio waves varies over time. Two special cases are linear polarization (the ellipse collapses into a line) as we have discussed above, and circular polarization (in which the two axes of the ellipse are equal). In linear polarization the electric field of the radio wave oscillates back and forth along one direction; this can be affected by the mounting of the antenna but usually the desired direction is either horizontal or vertical polarization. In circular polarization, the electric field (and magnetic field) of the radio wave rotates at the radio frequency circularly around the axis of propagation. Circular or elliptically polarized radio waves are designated as right-handed or left-handed using the "thumb in the direction of the propagation" rule. Note that for circular polarization, optical researchers use the opposite right hand rule from the one used by radio engineers.', 'Polarization is the sum of the E-plane orientations over time projected onto an imaginary plane perpendicular to the direction of motion of the radio wave. In the most general case, polarization is elliptical, meaning that the polarization of the radio waves varies over time. Two special cases are linear polarization (the ellipse collapses into a line) as we have discussed above, and circular polarization (in which the two axes of the ellipse are equal). In linear polarization the electric field of the radio wave oscillates back and forth along one direction; this can be affected by the mounting of the antenna but usually the desired direction is either horizontal or vertical polarization. In circular polarization, the electric field (and magnetic field) of the radio wave rotates at the radio frequency circularly around the axis of propagation. Circular or elliptically polarized radio waves are designated as right-handed or left-handed using the "thumb in the direction of the propagation" rule. Note that for circular polarization, optical researchers use the opposite right hand rule from the one used by radio engineers.', 'Polarization is the sum of the E-plane orientations over time projected onto an imaginary plane perpendicular to the direction of motion of the radio wave. In the most general case, polarization is elliptical, meaning that the polarization of the radio waves varies over time. Two special cases are linear polarization (the ellipse collapses into a line) as we have discussed above, and circular polarization (in which the two axes of the ellipse are equal). In linear polarization the electric field of the radio wave oscillates back and forth along one direction; this can be affected by the mounting of the antenna but usually the desired direction is either horizontal or vertical polarization. In circular polarization, the electric field (and magnetic field) of the radio wave rotates at the radio frequency circularly around the axis of propagation. Circular or elliptically polarized radio waves are designated as right-handed or left-handed using the "thumb in the direction of the propagation" rule. Note that for circular polarization, optical researchers use the opposite right hand rule from the one used by radio engineers.', 'It is best for the receiving antenna to match the polarization of the transmitted wave for optimum reception. Intermediate matchings will lose some signal strength, but not as much as a complete mismatch. A circularly polarized antenna can be used to equally well match vertical or horizontal linear polarizations. Transmission from a circularly polarized antenna received by a linearly polarized antenna (or vice versa) entails a 3 dB reduction in signal-to-noise ratio as the received power has thereby been cut in half.', 'It is best for the receiving antenna to match the polarization of the transmitted wave for optimum reception. Intermediate matchings will lose some signal strength, but not as much as a complete mismatch. A circularly polarized antenna can be used to equally well match vertical or horizontal linear polarizations. Transmission from a circularly polarized antenna received by a linearly polarized antenna (or vice versa) entails a 3 dB reduction in signal-to-noise ratio as the received power has thereby been cut in half.', 'It is best for the receiving antenna to match the polarization of the transmitted wave for optimum reception. Intermediate matchings will lose some signal strength, but not as much as a complete mismatch. A circularly polarized antenna can be used to equally well match vertical or horizontal linear polarizations. Transmission from a circularly polarized antenna received by a linearly polarized antenna (or vice versa) entails a 3 dB reduction in signal-to-noise ratio as the received power has thereby been cut in half.', 'It is best for the receiving antenna to match the polarization of the transmitted wave for optimum reception. Intermediate matchings will lose some signal strength, but not as much as a complete mismatch. A circularly polarized antenna can be used to equally well match vertical or horizontal linear polarizations. Transmission from a circularly polarized antenna received by a linearly polarized antenna (or vice versa) entails a 3 dB reduction in signal-to-noise ratio as the received power has thereby been cut in half.', 'Maximum power transfer requires matching the impedance of an antenna system (as seen looking into the transmission line) to the complex conjugate of the impedance of the receiver or transmitter. In the case of a transmitter, however, the desired matching impedance might not correspond to the dynamic output impedance of the transmitter as analyzed as a source impedance but rather the design value (typically 50 ohms) required for efficient and safe operation of the transmitting circuitry. The intended impedance is normally resistive but a transmitter (and some receivers) may have additional adjustments to cancel a certain amount of reactance in order to "tweak" the match. When a transmission line is used in between the antenna and the transmitter (or receiver) one generally would like an antenna system whose impedance is resistive and near the characteristic impedance of that transmission line in order to minimize the standing wave ratio (SWR) and the increase in transmission line losses it entails, in addition to supplying a good match at the transmitter or receiver itself.', 'Maximum power transfer requires matching the impedance of an antenna system (as seen looking into the transmission line) to the complex conjugate of the impedance of the receiver or transmitter. In the case of a transmitter, however, the desired matching impedance might not correspond to the dynamic output impedance of the transmitter as analyzed as a source impedance but rather the design value (typically 50 ohms) required for efficient and safe operation of the transmitting circuitry. The intended impedance is normally resistive but a transmitter (and some receivers) may have additional adjustments to cancel a certain amount of reactance in order to "tweak" the match. When a transmission line is used in between the antenna and the transmitter (or receiver) one generally would like an antenna system whose impedance is resistive and near the characteristic impedance of that transmission line in order to minimize the standing wave ratio (SWR) and the increase in transmission line losses it entails, in addition to supplying a good match at the transmitter or receiver itself.', 'Maximum power transfer requires matching the impedance of an antenna system (as seen looking into the transmission line) to the complex conjugate of the impedance of the receiver or transmitter. In the case of a transmitter, however, the desired matching impedance might not correspond to the dynamic output impedance of the transmitter as analyzed as a source impedance but rather the design value (typically 50 ohms) required for efficient and safe operation of the transmitting circuitry. The intended impedance is normally resistive but a transmitter (and some receivers) may have additional adjustments to cancel a certain amount of reactance in order to "tweak" the match. When a transmission line is used in between the antenna and the transmitter (or receiver) one generally would like an antenna system whose impedance is resistive and near the characteristic impedance of that transmission line in order to minimize the standing wave ratio (SWR) and the increase in transmission line losses it entails, in addition to supplying a good match at the transmitter or receiver itself.', 'Maximum power transfer requires matching the impedance of an antenna system (as seen looking into the transmission line) to the complex conjugate of the impedance of the receiver or transmitter. In the case of a transmitter, however, the desired matching impedance might not correspond to the dynamic output impedance of the transmitter as analyzed as a source impedance but rather the design value (typically 50 ohms) required for efficient and safe operation of the transmitting circuitry. The intended impedance is normally resistive but a transmitter (and some receivers) may have additional adjustments to cancel a certain amount of reactance in order to "tweak" the match. When a transmission line is used in between the antenna and the transmitter (or receiver) one generally would like an antenna system whose impedance is resistive and near the characteristic impedance of that transmission line in order to minimize the standing wave ratio (SWR) and the increase in transmission line losses it entails, in addition to supplying a good match at the transmitter or receiver itself.', 'In some cases this is done in a more extreme manner, not simply to cancel a small amount of residual reactance, but to resonate an antenna whose resonance frequency is quite different from the intended frequency of operation. For instance, a "whip antenna" can be made significantly shorter than 1/4 wavelength long, for practical reasons, and then resonated using a so-called loading coil. This physically large inductor at the base of the antenna has an inductive reactance which is the opposite of the capacitative reactance that such a vertical antenna has at the desired operating frequency. The result is a pure resistance seen at feedpoint of the loading coil; unfortunately that resistance is somewhat lower than would be desired to match commercial coax.[citation needed]', 'In some cases this is done in a more extreme manner, not simply to cancel a small amount of residual reactance, but to resonate an antenna whose resonance frequency is quite different from the intended frequency of operation. For instance, a "whip antenna" can be made significantly shorter than 1/4 wavelength long, for practical reasons, and then resonated using a so-called loading coil. This physically large inductor at the base of the antenna has an inductive reactance which is the opposite of the capacitative reactance that such a vertical antenna has at the desired operating frequency. The result is a pure resistance seen at feedpoint of the loading coil; unfortunately that resistance is somewhat lower than would be desired to match commercial coax.[citation needed]', 'In some cases this is done in a more extreme manner, not simply to cancel a small amount of residual reactance, but to resonate an antenna whose resonance frequency is quite different from the intended frequency of operation. For instance, a "whip antenna" can be made significantly shorter than 1/4 wavelength long, for practical reasons, and then resonated using a so-called loading coil. This physically large inductor at the base of the antenna has an inductive reactance which is the opposite of the capacitative reactance that such a vertical antenna has at the desired operating frequency. The result is a pure resistance seen at feedpoint of the loading coil; unfortunately that resistance is somewhat lower than would be desired to match commercial coax.[citation needed]', 'In some cases this is done in a more extreme manner, not simply to cancel a small amount of residual reactance, but to resonate an antenna whose resonance frequency is quite different from the intended frequency of operation. For instance, a "whip antenna" can be made significantly shorter than 1/4 wavelength long, for practical reasons, and then resonated using a so-called loading coil. This physically large inductor at the base of the antenna has an inductive reactance which is the opposite of the capacitative reactance that such a vertical antenna has at the desired operating frequency. The result is a pure resistance seen at feedpoint of the loading coil; unfortunately that resistance is somewhat lower than would be desired to match commercial coax.[citation needed]', 'So an additional problem beyond canceling the unwanted reactance is of matching the remaining resistive impedance to the characteristic impedance of the transmission line. In principle this can always be done with a transformer, however the turns ratio of a transformer is not adjustable. A general matching network with at least two adjustments can be made to correct both components of impedance. Matching networks using discrete inductors and capacitors will have losses associated with those components, and will have power restrictions when used for transmitting. Avoiding these difficulties, commercial antennas are generally designed with fixed matching elements or feeding strategies to get an approximate match to standard coax, such as 50 or 75 Ohms. Antennas based on the dipole (rather than vertical antennas) should include a balun in between the transmission line and antenna element, which may be integrated into any such matching network.', 'So an additional problem beyond canceling the unwanted reactance is of matching the remaining resistive impedance to the characteristic impedance of the transmission line. In principle this can always be done with a transformer, however the turns ratio of a transformer is not adjustable. A general matching network with at least two adjustments can be made to correct both components of impedance. Matching networks using discrete inductors and capacitors will have losses associated with those components, and will have power restrictions when used for transmitting. Avoiding these difficulties, commercial antennas are generally designed with fixed matching elements or feeding strategies to get an approximate match to standard coax, such as 50 or 75 Ohms. Antennas based on the dipole (rather than vertical antennas) should include a balun in between the transmission line and antenna element, which may be integrated into any such matching network.', 'So an additional problem beyond canceling the unwanted reactance is of matching the remaining resistive impedance to the characteristic impedance of the transmission line. In principle this can always be done with a transformer, however the turns ratio of a transformer is not adjustable. A general matching network with at least two adjustments can be made to correct both components of impedance. Matching networks using discrete inductors and capacitors will have losses associated with those components, and will have power restrictions when used for transmitting. Avoiding these difficulties, commercial antennas are generally designed with fixed matching elements or feeding strategies to get an approximate match to standard coax, such as 50 or 75 Ohms. Antennas based on the dipole (rather than vertical antennas) should include a balun in between the transmission line and antenna element, which may be integrated into any such matching network.', 'So an additional problem beyond canceling the unwanted reactance is of matching the remaining resistive impedance to the characteristic impedance of the transmission line. In principle this can always be done with a transformer, however the turns ratio of a transformer is not adjustable. A general matching network with at least two adjustments can be made to correct both components of impedance. Matching networks using discrete inductors and capacitors will have losses associated with those components, and will have power restrictions when used for transmitting. Avoiding these difficulties, commercial antennas are generally designed with fixed matching elements or feeding strategies to get an approximate match to standard coax, such as 50 or 75 Ohms. Antennas based on the dipole (rather than vertical antennas) should include a balun in between the transmission line and antenna element, which may be integrated into any such matching network.', 'So an additional problem beyond canceling the unwanted reactance is of matching the remaining resistive impedance to the characteristic impedance of the transmission line. In principle this can always be done with a transformer, however the turns ratio of a transformer is not adjustable. A general matching network with at least two adjustments can be made to correct both components of impedance. Matching networks using discrete inductors and capacitors will have losses associated with those components, and will have power restrictions when used for transmitting. Avoiding these difficulties, commercial antennas are generally designed with fixed matching elements or feeding strategies to get an approximate match to standard coax, such as 50 or 75 Ohms. Antennas based on the dipole (rather than vertical antennas) should include a balun in between the transmission line and antenna element, which may be integrated into any such matching network.', "Unlike the above antennas, traveling wave antennas are nonresonant so they have inherently broad bandwidth. They are typically wire antennas multiple wavelengths long, through which the voltage and current waves travel in one direction, instead of bouncing back and forth to form standing waves as in resonant antennas. They have linear polarization (except for the helical antenna). Unidirectional traveling wave antennas are terminated by a resistor at one end equal to the antenna's characteristic resistance, to absorb the waves from one direction. This makes them inefficient as transmitting antennas.", "Unlike the above antennas, traveling wave antennas are nonresonant so they have inherently broad bandwidth. They are typically wire antennas multiple wavelengths long, through which the voltage and current waves travel in one direction, instead of bouncing back and forth to form standing waves as in resonant antennas. They have linear polarization (except for the helical antenna). Unidirectional traveling wave antennas are terminated by a resistor at one end equal to the antenna's characteristic resistance, to absorb the waves from one direction. This makes them inefficient as transmitting antennas.", "Unlike the above antennas, traveling wave antennas are nonresonant so they have inherently broad bandwidth. They are typically wire antennas multiple wavelengths long, through which the voltage and current waves travel in one direction, instead of bouncing back and forth to form standing waves as in resonant antennas. They have linear polarization (except for the helical antenna). Unidirectional traveling wave antennas are terminated by a resistor at one end equal to the antenna's characteristic resistance, to absorb the waves from one direction. This makes them inefficient as transmitting antennas.", "Unlike the above antennas, traveling wave antennas are nonresonant so they have inherently broad bandwidth. They are typically wire antennas multiple wavelengths long, through which the voltage and current waves travel in one direction, instead of bouncing back and forth to form standing waves as in resonant antennas. They have linear polarization (except for the helical antenna). Unidirectional traveling wave antennas are terminated by a resistor at one end equal to the antenna's characteristic resistance, to absorb the waves from one direction. This makes them inefficient as transmitting antennas.", "Unlike the above antennas, traveling wave antennas are nonresonant so they have inherently broad bandwidth. They are typically wire antennas multiple wavelengths long, through which the voltage and current waves travel in one direction, instead of bouncing back and forth to form standing waves as in resonant antennas. They have linear polarization (except for the helical antenna). Unidirectional traveling wave antennas are terminated by a resistor at one end equal to the antenna's characteristic resistance, to absorb the waves from one direction. This makes them inefficient as transmitting antennas.", "The radiation pattern and even the driving point impedance of an antenna can be influenced by the dielectric constant and especially conductivity of nearby objects. For a terrestrial antenna, the ground is usually one such object of importance. The antenna's height above the ground, as well as the electrical properties (permittivity and conductivity) of the ground, can then be important. Also, in the particular case of a monopole antenna, the ground (or an artificial ground plane) serves as the return connection for the antenna current thus having an additional effect, particularly on the impedance seen by the feed line.", "The radiation pattern and even the driving point impedance of an antenna can be influenced by the dielectric constant and especially conductivity of nearby objects. For a terrestrial antenna, the ground is usually one such object of importance. The antenna's height above the ground, as well as the electrical properties (permittivity and conductivity) of the ground, can then be important. Also, in the particular case of a monopole antenna, the ground (or an artificial ground plane) serves as the return connection for the antenna current thus having an additional effect, particularly on the impedance seen by the feed line.", "The radiation pattern and even the driving point impedance of an antenna can be influenced by the dielectric constant and especially conductivity of nearby objects. For a terrestrial antenna, the ground is usually one such object of importance. The antenna's height above the ground, as well as the electrical properties (permittivity and conductivity) of the ground, can then be important. Also, in the particular case of a monopole antenna, the ground (or an artificial ground plane) serves as the return connection for the antenna current thus having an additional effect, particularly on the impedance seen by the feed line.", "The radiation pattern and even the driving point impedance of an antenna can be influenced by the dielectric constant and especially conductivity of nearby objects. For a terrestrial antenna, the ground is usually one such object of importance. The antenna's height above the ground, as well as the electrical properties (permittivity and conductivity) of the ground, can then be important. Also, in the particular case of a monopole antenna, the ground (or an artificial ground plane) serves as the return connection for the antenna current thus having an additional effect, particularly on the impedance seen by the feed line.", 'The net quality of a ground reflection depends on the topography of the surface. When the irregularities of the surface are much smaller than the wavelength, we are in the regime of specular reflection, and the receiver sees both the real antenna and an image of the antenna under the ground due to reflection. But if the ground has irregularities not small compared to the wavelength, reflections will not be coherent but shifted by random phases. With shorter wavelengths (higher frequencies), this is generally the case.', 'The net quality of a ground reflection depends on the topography of the surface. When the irregularities of the surface are much smaller than the wavelength, we are in the regime of specular reflection, and the receiver sees both the real antenna and an image of the antenna under the ground due to reflection. But if the ground has irregularities not small compared to the wavelength, reflections will not be coherent but shifted by random phases. With shorter wavelengths (higher frequencies), this is generally the case.', 'The net quality of a ground reflection depends on the topography of the surface. When the irregularities of the surface are much smaller than the wavelength, we are in the regime of specular reflection, and the receiver sees both the real antenna and an image of the antenna under the ground due to reflection. But if the ground has irregularities not small compared to the wavelength, reflections will not be coherent but shifted by random phases. With shorter wavelengths (higher frequencies), this is generally the case.', 'The net quality of a ground reflection depends on the topography of the surface. When the irregularities of the surface are much smaller than the wavelength, we are in the regime of specular reflection, and the receiver sees both the real antenna and an image of the antenna under the ground due to reflection. But if the ground has irregularities not small compared to the wavelength, reflections will not be coherent but shifted by random phases. With shorter wavelengths (higher frequencies), this is generally the case.', "The phase of reflection of electromagnetic waves depends on the polarization of the incident wave. Given the larger refractive index of the ground (typically n=2) compared to air (n=1), the phase of horizontally polarized radiation is reversed upon reflection (a phase shift of  radians or 180°). On the other hand, the vertical component of the wave's electric field is reflected at grazing angles of incidence approximately in phase. These phase shifts apply as well to a ground modelled as a good electrical conductor.", "The phase of reflection of electromagnetic waves depends on the polarization of the incident wave. Given the larger refractive index of the ground (typically n=2) compared to air (n=1), the phase of horizontally polarized radiation is reversed upon reflection (a phase shift of  radians or 180°). On the other hand, the vertical component of the wave's electric field is reflected at grazing angles of incidence approximately in phase. These phase shifts apply as well to a ground modelled as a good electrical conductor.", "The phase of reflection of electromagnetic waves depends on the polarization of the incident wave. Given the larger refractive index of the ground (typically n=2) compared to air (n=1), the phase of horizontally polarized radiation is reversed upon reflection (a phase shift of  radians or 180°). On the other hand, the vertical component of the wave's electric field is reflected at grazing angles of incidence approximately in phase. These phase shifts apply as well to a ground modelled as a good electrical conductor.", "The phase of reflection of electromagnetic waves depends on the polarization of the incident wave. Given the larger refractive index of the ground (typically n=2) compared to air (n=1), the phase of horizontally polarized radiation is reversed upon reflection (a phase shift of  radians or 180°). On the other hand, the vertical component of the wave's electric field is reflected at grazing angles of incidence approximately in phase. These phase shifts apply as well to a ground modelled as a good electrical conductor.", "When an electromagnetic wave strikes a plane surface such as the ground, part of the wave is transmitted into the ground and part of it is reflected, according to the Fresnel coefficients. If the ground is a very good conductor then almost all of the wave is reflected (180° out of phase), whereas a ground modeled as a (lossy) dielectric can absorb a large amount of the wave's power. The power remaining in the reflected wave, and the phase shift upon reflection, strongly depend on the wave's angle of incidence and polarization. The dielectric constant and conductivity (or simply the complex dielectric constant) is dependent on the soil type and is a function of frequency.", "When an electromagnetic wave strikes a plane surface such as the ground, part of the wave is transmitted into the ground and part of it is reflected, according to the Fresnel coefficients. If the ground is a very good conductor then almost all of the wave is reflected (180° out of phase), whereas a ground modeled as a (lossy) dielectric can absorb a large amount of the wave's power. The power remaining in the reflected wave, and the phase shift upon reflection, strongly depend on the wave's angle of incidence and polarization. The dielectric constant and conductivity (or simply the complex dielectric constant) is dependent on the soil type and is a function of frequency.", "When an electromagnetic wave strikes a plane surface such as the ground, part of the wave is transmitted into the ground and part of it is reflected, according to the Fresnel coefficients. If the ground is a very good conductor then almost all of the wave is reflected (180° out of phase), whereas a ground modeled as a (lossy) dielectric can absorb a large amount of the wave's power. The power remaining in the reflected wave, and the phase shift upon reflection, strongly depend on the wave's angle of incidence and polarization. The dielectric constant and conductivity (or simply the complex dielectric constant) is dependent on the soil type and is a function of frequency.", "When an electromagnetic wave strikes a plane surface such as the ground, part of the wave is transmitted into the ground and part of it is reflected, according to the Fresnel coefficients. If the ground is a very good conductor then almost all of the wave is reflected (180° out of phase), whereas a ground modeled as a (lossy) dielectric can absorb a large amount of the wave's power. The power remaining in the reflected wave, and the phase shift upon reflection, strongly depend on the wave's angle of incidence and polarization. The dielectric constant and conductivity (or simply the complex dielectric constant) is dependent on the soil type and is a function of frequency.", 'The effective area or effective aperture of a receiving antenna expresses the portion of the power of a passing electromagnetic wave which it delivers to its terminals, expressed in terms of an equivalent area. For instance, if a radio wave passing a given location has a flux of 1 pW / m2 (10−12 watts per square meter) and an antenna has an effective area of 12 m2, then the antenna would deliver 12 pW of RF power to the receiver (30 microvolts rms at 75 ohms). Since the receiving antenna is not equally sensitive to signals received from all directions, the effective area is a function of the direction to the source.', 'The effective area or effective aperture of a receiving antenna expresses the portion of the power of a passing electromagnetic wave which it delivers to its terminals, expressed in terms of an equivalent area. For instance, if a radio wave passing a given location has a flux of 1 pW / m2 (10−12 watts per square meter) and an antenna has an effective area of 12 m2, then the antenna would deliver 12 pW of RF power to the receiver (30 microvolts rms at 75 ohms). Since the receiving antenna is not equally sensitive to signals received from all directions, the effective area is a function of the direction to the source.', 'The effective area or effective aperture of a receiving antenna expresses the portion of the power of a passing electromagnetic wave which it delivers to its terminals, expressed in terms of an equivalent area. For instance, if a radio wave passing a given location has a flux of 1 pW / m2 (10−12 watts per square meter) and an antenna has an effective area of 12 m2, then the antenna would deliver 12 pW of RF power to the receiver (30 microvolts rms at 75 ohms). Since the receiving antenna is not equally sensitive to signals received from all directions, the effective area is a function of the direction to the source.', 'The effective area or effective aperture of a receiving antenna expresses the portion of the power of a passing electromagnetic wave which it delivers to its terminals, expressed in terms of an equivalent area. For instance, if a radio wave passing a given location has a flux of 1 pW / m2 (10−12 watts per square meter) and an antenna has an effective area of 12 m2, then the antenna would deliver 12 pW of RF power to the receiver (30 microvolts rms at 75 ohms). Since the receiving antenna is not equally sensitive to signals received from all directions, the effective area is a function of the direction to the source.', 'The bandwidth characteristics of a resonant antenna element can be characterized according to its Q, just as one uses to characterize the sharpness of an L-C resonant circuit. However it is often assumed that there is an advantage in an antenna having a high Q. After all, Q is short for "quality factor" and a low Q typically signifies excessive loss (due to unwanted resistance) in a resonant L-C circuit. However this understanding does not apply to resonant antennas where the resistance involved is the radiation resistance, a desired quantity which removes energy from the resonant element in order to radiate it (the purpose of an antenna, after all!). The Q is a measure of the ratio of reactance to resistance, so with a fixed radiation resistance (an element\'s radiation resistance is almost independent of its diameter) a greater reactance off-resonance corresponds to the poorer bandwidth of a very thin conductor. The Q of such a narrowband antenna can be as high as 15. On the other hand, a thick element presents less reactance at an off-resonant frequency, and consequently a Q as low as 5. These two antennas will perform equivalently at the resonant frequency, but the second antenna will perform over a bandwidth 3 times as wide as the "hi-Q" antenna consisting of a thin conductor.', 'The bandwidth characteristics of a resonant antenna element can be characterized according to its Q, just as one uses to characterize the sharpness of an L-C resonant circuit. However it is often assumed that there is an advantage in an antenna having a high Q. After all, Q is short for "quality factor" and a low Q typically signifies excessive loss (due to unwanted resistance) in a resonant L-C circuit. However this understanding does not apply to resonant antennas where the resistance involved is the radiation resistance, a desired quantity which removes energy from the resonant element in order to radiate it (the purpose of an antenna, after all!). The Q is a measure of the ratio of reactance to resistance, so with a fixed radiation resistance (an element\'s radiation resistance is almost independent of its diameter) a greater reactance off-resonance corresponds to the poorer bandwidth of a very thin conductor. The Q of such a narrowband antenna can be as high as 15. On the other hand, a thick element presents less reactance at an off-resonant frequency, and consequently a Q as low as 5. These two antennas will perform equivalently at the resonant frequency, but the second antenna will perform over a bandwidth 3 times as wide as the "hi-Q" antenna consisting of a thin conductor.', 'The bandwidth characteristics of a resonant antenna element can be characterized according to its Q, just as one uses to characterize the sharpness of an L-C resonant circuit. However it is often assumed that there is an advantage in an antenna having a high Q. After all, Q is short for "quality factor" and a low Q typically signifies excessive loss (due to unwanted resistance) in a resonant L-C circuit. However this understanding does not apply to resonant antennas where the resistance involved is the radiation resistance, a desired quantity which removes energy from the resonant element in order to radiate it (the purpose of an antenna, after all!). The Q is a measure of the ratio of reactance to resistance, so with a fixed radiation resistance (an element\'s radiation resistance is almost independent of its diameter) a greater reactance off-resonance corresponds to the poorer bandwidth of a very thin conductor. The Q of such a narrowband antenna can be as high as 15. On the other hand, a thick element presents less reactance at an off-resonant frequency, and consequently a Q as low as 5. These two antennas will perform equivalently at the resonant frequency, but the second antenna will perform over a bandwidth 3 times as wide as the "hi-Q" antenna consisting of a thin conductor.', 'The bandwidth characteristics of a resonant antenna element can be characterized according to its Q, just as one uses to characterize the sharpness of an L-C resonant circuit. However it is often assumed that there is an advantage in an antenna having a high Q. After all, Q is short for "quality factor" and a low Q typically signifies excessive loss (due to unwanted resistance) in a resonant L-C circuit. However this understanding does not apply to resonant antennas where the resistance involved is the radiation resistance, a desired quantity which removes energy from the resonant element in order to radiate it (the purpose of an antenna, after all!). The Q is a measure of the ratio of reactance to resistance, so with a fixed radiation resistance (an element\'s radiation resistance is almost independent of its diameter) a greater reactance off-resonance corresponds to the poorer bandwidth of a very thin conductor. The Q of such a narrowband antenna can be as high as 15. On the other hand, a thick element presents less reactance at an off-resonant frequency, and consequently a Q as low as 5. These two antennas will perform equivalently at the resonant frequency, but the second antenna will perform over a bandwidth 3 times as wide as the "hi-Q" antenna consisting of a thin conductor.', 'For example, at 30 MHz (10 m wavelength) a true resonant 1⁄4-wavelength monopole would be almost 2.5 meters long, and using an antenna only 1.5 meters tall would require the addition of a loading coil. Then it may be said that the coil has lengthened the antenna to achieve an electrical length of 2.5 meters. However, the resulting resistive impedance achieved will be quite a bit lower than the impedance of a resonant monopole, likely requiring further impedance matching. In addition to a lower radiation resistance, the reactance becomes higher as the antenna size is reduced, and the resonant circuit formed by the antenna and the tuning coil has a Q factor that rises and eventually causes the bandwidth of the antenna to be inadequate for the signal being transmitted. This is the major factor that sets the size of antennas at 1 MHz and lower frequencies.', 'For example, at 30 MHz (10 m wavelength) a true resonant 1⁄4-wavelength monopole would be almost 2.5 meters long, and using an antenna only 1.5 meters tall would require the addition of a loading coil. Then it may be said that the coil has lengthened the antenna to achieve an electrical length of 2.5 meters. However, the resulting resistive impedance achieved will be quite a bit lower than the impedance of a resonant monopole, likely requiring further impedance matching. In addition to a lower radiation resistance, the reactance becomes higher as the antenna size is reduced, and the resonant circuit formed by the antenna and the tuning coil has a Q factor that rises and eventually causes the bandwidth of the antenna to be inadequate for the signal being transmitted. This is the major factor that sets the size of antennas at 1 MHz and lower frequencies.', 'For example, at 30 MHz (10 m wavelength) a true resonant 1⁄4-wavelength monopole would be almost 2.5 meters long, and using an antenna only 1.5 meters tall would require the addition of a loading coil. Then it may be said that the coil has lengthened the antenna to achieve an electrical length of 2.5 meters. However, the resulting resistive impedance achieved will be quite a bit lower than the impedance of a resonant monopole, likely requiring further impedance matching. In addition to a lower radiation resistance, the reactance becomes higher as the antenna size is reduced, and the resonant circuit formed by the antenna and the tuning coil has a Q factor that rises and eventually causes the bandwidth of the antenna to be inadequate for the signal being transmitted. This is the major factor that sets the size of antennas at 1 MHz and lower frequencies.', 'For example, at 30 MHz (10 m wavelength) a true resonant 1⁄4-wavelength monopole would be almost 2.5 meters long, and using an antenna only 1.5 meters tall would require the addition of a loading coil. Then it may be said that the coil has lengthened the antenna to achieve an electrical length of 2.5 meters. However, the resulting resistive impedance achieved will be quite a bit lower than the impedance of a resonant monopole, likely requiring further impedance matching. In addition to a lower radiation resistance, the reactance becomes higher as the antenna size is reduced, and the resonant circuit formed by the antenna and the tuning coil has a Q factor that rises and eventually causes the bandwidth of the antenna to be inadequate for the signal being transmitted. This is the major factor that sets the size of antennas at 1 MHz and lower frequencies.', 'Consider a half-wave dipole designed to work with signals 1 m wavelength, meaning the antenna would be approximately 50 cm across. If the element has a length-to-diameter ratio of 1000, it will have an inherent resistance of about 63 ohms. Using the appropriate transmission wire or balun, we match that resistance to ensure minimum signal loss. Feeding that antenna with a current of 1 ampere will require 63 volts of RF, and the antenna will radiate 63 watts (ignoring losses) of radio frequency power. Now consider the case when the antenna is fed a signal with a wavelength of 1.25 m; in this case the reflected current would arrive at the feed out-of-phase with the signal, causing the net current to drop while the voltage remains the same. Electrically this appears to be a very high impedance. The antenna and transmission line no longer have the same impedance, and the signal will be reflected back into the antenna, reducing output. This could be addressed by changing the matching system between the antenna and transmission line, but that solution only works well at the new design frequency.', 'Consider a half-wave dipole designed to work with signals 1 m wavelength, meaning the antenna would be approximately 50 cm across. If the element has a length-to-diameter ratio of 1000, it will have an inherent resistance of about 63 ohms. Using the appropriate transmission wire or balun, we match that resistance to ensure minimum signal loss. Feeding that antenna with a current of 1 ampere will require 63 volts of RF, and the antenna will radiate 63 watts (ignoring losses) of radio frequency power. Now consider the case when the antenna is fed a signal with a wavelength of 1.25 m; in this case the reflected current would arrive at the feed out-of-phase with the signal, causing the net current to drop while the voltage remains the same. Electrically this appears to be a very high impedance. The antenna and transmission line no longer have the same impedance, and the signal will be reflected back into the antenna, reducing output. This could be addressed by changing the matching system between the antenna and transmission line, but that solution only works well at the new design frequency.', 'Consider a half-wave dipole designed to work with signals 1 m wavelength, meaning the antenna would be approximately 50 cm across. If the element has a length-to-diameter ratio of 1000, it will have an inherent resistance of about 63 ohms. Using the appropriate transmission wire or balun, we match that resistance to ensure minimum signal loss. Feeding that antenna with a current of 1 ampere will require 63 volts of RF, and the antenna will radiate 63 watts (ignoring losses) of radio frequency power. Now consider the case when the antenna is fed a signal with a wavelength of 1.25 m; in this case the reflected current would arrive at the feed out-of-phase with the signal, causing the net current to drop while the voltage remains the same. Electrically this appears to be a very high impedance. The antenna and transmission line no longer have the same impedance, and the signal will be reflected back into the antenna, reducing output. This could be addressed by changing the matching system between the antenna and transmission line, but that solution only works well at the new design frequency.', 'Consider a half-wave dipole designed to work with signals 1 m wavelength, meaning the antenna would be approximately 50 cm across. If the element has a length-to-diameter ratio of 1000, it will have an inherent resistance of about 63 ohms. Using the appropriate transmission wire or balun, we match that resistance to ensure minimum signal loss. Feeding that antenna with a current of 1 ampere will require 63 volts of RF, and the antenna will radiate 63 watts (ignoring losses) of radio frequency power. Now consider the case when the antenna is fed a signal with a wavelength of 1.25 m; in this case the reflected current would arrive at the feed out-of-phase with the signal, causing the net current to drop while the voltage remains the same. Electrically this appears to be a very high impedance. The antenna and transmission line no longer have the same impedance, and the signal will be reflected back into the antenna, reducing output. This could be addressed by changing the matching system between the antenna and transmission line, but that solution only works well at the new design frequency.', 'Recall that a current will reflect when there are changes in the electrical properties of the material. In order to efficiently send the signal into the transmission line, it is important that the transmission line has the same impedance as the elements, otherwise some of the signal will be reflected back into the antenna. This leads to the concept of impedance matching, the design of the overall system of antenna and transmission line so the impedance is as close as possible, thereby reducing these losses. Impedance matching between antennas and transmission lines is commonly handled through the use of a balun, although other solutions are also used in certain roles. An important measure of this basic concept is the standing wave ratio, which measures the magnitude of the reflected signal.', 'Recall that a current will reflect when there are changes in the electrical properties of the material. In order to efficiently send the signal into the transmission line, it is important that the transmission line has the same impedance as the elements, otherwise some of the signal will be reflected back into the antenna. This leads to the concept of impedance matching, the design of the overall system of antenna and transmission line so the impedance is as close as possible, thereby reducing these losses. Impedance matching between antennas and transmission lines is commonly handled through the use of a balun, although other solutions are also used in certain roles. An important measure of this basic concept is the standing wave ratio, which measures the magnitude of the reflected signal.', 'Recall that a current will reflect when there are changes in the electrical properties of the material. In order to efficiently send the signal into the transmission line, it is important that the transmission line has the same impedance as the elements, otherwise some of the signal will be reflected back into the antenna. This leads to the concept of impedance matching, the design of the overall system of antenna and transmission line so the impedance is as close as possible, thereby reducing these losses. Impedance matching between antennas and transmission lines is commonly handled through the use of a balun, although other solutions are also used in certain roles. An important measure of this basic concept is the standing wave ratio, which measures the magnitude of the reflected signal.', 'Recall that a current will reflect when there are changes in the electrical properties of the material. In order to efficiently send the signal into the transmission line, it is important that the transmission line has the same impedance as the elements, otherwise some of the signal will be reflected back into the antenna. This leads to the concept of impedance matching, the design of the overall system of antenna and transmission line so the impedance is as close as possible, thereby reducing these losses. Impedance matching between antennas and transmission lines is commonly handled through the use of a balun, although other solutions are also used in certain roles. An important measure of this basic concept is the standing wave ratio, which measures the magnitude of the reflected signal.', 'An electromagnetic wave refractor in some aperture antennas is a component which due to its shape and position functions to selectively delay or advance portions of the electromagnetic wavefront passing through it. The refractor alters the spatial characteristics of the wave on one side relative to the other side. It can, for instance, bring the wave to a focus or alter the wave front in other ways, generally in order to maximize the directivity of the antenna system. This is the radio equivalent of an optical lens.', 'An electromagnetic wave refractor in some aperture antennas is a component which due to its shape and position functions to selectively delay or advance portions of the electromagnetic wavefront passing through it. The refractor alters the spatial characteristics of the wave on one side relative to the other side. It can, for instance, bring the wave to a focus or alter the wave front in other ways, generally in order to maximize the directivity of the antenna system. This is the radio equivalent of an optical lens.', 'An electromagnetic wave refractor in some aperture antennas is a component which due to its shape and position functions to selectively delay or advance portions of the electromagnetic wavefront passing through it. The refractor alters the spatial characteristics of the wave on one side relative to the other side. It can, for instance, bring the wave to a focus or alter the wave front in other ways, generally in order to maximize the directivity of the antenna system. This is the radio equivalent of an optical lens.', 'An electromagnetic wave refractor in some aperture antennas is a component which due to its shape and position functions to selectively delay or advance portions of the electromagnetic wavefront passing through it. The refractor alters the spatial characteristics of the wave on one side relative to the other side. It can, for instance, bring the wave to a focus or alter the wave front in other ways, generally in order to maximize the directivity of the antenna system. This is the radio equivalent of an optical lens.', "The actual antenna which is transmitting the original wave then also may receive a strong signal from its own image from the ground. This will induce an additional current in the antenna element, changing the current at the feedpoint for a given feedpoint voltage. Thus the antenna's impedance, given by the ratio of feedpoint voltage to current, is altered due to the antenna's proximity to the ground. This can be quite a significant effect when the antenna is within a wavelength or two of the ground. But as the antenna height is increased, the reduced power of the reflected wave (due to the inverse square law) allows the antenna to approach its asymptotic feedpoint impedance given by theory. At lower heights, the effect on the antenna's impedance is very sensitive to the exact distance from the ground, as this affects the phase of the reflected wave relative to the currents in the antenna. Changing the antenna's height by a quarter wavelength, then changes the phase of the reflection by 180°, with a completely different effect on the antenna's impedance.", "The actual antenna which is transmitting the original wave then also may receive a strong signal from its own image from the ground. This will induce an additional current in the antenna element, changing the current at the feedpoint for a given feedpoint voltage. Thus the antenna's impedance, given by the ratio of feedpoint voltage to current, is altered due to the antenna's proximity to the ground. This can be quite a significant effect when the antenna is within a wavelength or two of the ground. But as the antenna height is increased, the reduced power of the reflected wave (due to the inverse square law) allows the antenna to approach its asymptotic feedpoint impedance given by theory. At lower heights, the effect on the antenna's impedance is very sensitive to the exact distance from the ground, as this affects the phase of the reflected wave relative to the currents in the antenna. Changing the antenna's height by a quarter wavelength, then changes the phase of the reflection by 180°, with a completely different effect on the antenna's impedance.", "The actual antenna which is transmitting the original wave then also may receive a strong signal from its own image from the ground. This will induce an additional current in the antenna element, changing the current at the feedpoint for a given feedpoint voltage. Thus the antenna's impedance, given by the ratio of feedpoint voltage to current, is altered due to the antenna's proximity to the ground. This can be quite a significant effect when the antenna is within a wavelength or two of the ground. But as the antenna height is increased, the reduced power of the reflected wave (due to the inverse square law) allows the antenna to approach its asymptotic feedpoint impedance given by theory. At lower heights, the effect on the antenna's impedance is very sensitive to the exact distance from the ground, as this affects the phase of the reflected wave relative to the currents in the antenna. Changing the antenna's height by a quarter wavelength, then changes the phase of the reflection by 180°, with a completely different effect on the antenna's impedance.", "The actual antenna which is transmitting the original wave then also may receive a strong signal from its own image from the ground. This will induce an additional current in the antenna element, changing the current at the feedpoint for a given feedpoint voltage. Thus the antenna's impedance, given by the ratio of feedpoint voltage to current, is altered due to the antenna's proximity to the ground. This can be quite a significant effect when the antenna is within a wavelength or two of the ground. But as the antenna height is increased, the reduced power of the reflected wave (due to the inverse square law) allows the antenna to approach its asymptotic feedpoint impedance given by theory. At lower heights, the effect on the antenna's impedance is very sensitive to the exact distance from the ground, as this affects the phase of the reflected wave relative to the currents in the antenna. Changing the antenna's height by a quarter wavelength, then changes the phase of the reflection by 180°, with a completely different effect on the antenna's impedance.", 'For horizontal propagation between transmitting and receiving antennas situated near the ground reasonably far from each other, the distances traveled by tne direct and reflected rays are nearly the same. There is almost no relative phase shift. If the emission is polarized vertically, the two fields (direct and reflected) add and there is maximum of received signal. If the signal is polarized horizontally, the two signals subtract and the received signal is largely cancelled. The vertical plane radiation patterns are shown in the image at right. With vertical polarization there is always a maximum for θ=0, horizontal propagation (left pattern). For horizontal polarization, there is cancellation at that angle. Note that the above formulae and these plots assume the ground as a perfect conductor. These plots of the radiation pattern correspond to a distance between the antenna and its image of 2.5λ. As the antenna height is increased, the number of lobes increases as well.', 'For horizontal propagation between transmitting and receiving antennas situated near the ground reasonably far from each other, the distances traveled by tne direct and reflected rays are nearly the same. There is almost no relative phase shift. If the emission is polarized vertically, the two fields (direct and reflected) add and there is maximum of received signal. If the signal is polarized horizontally, the two signals subtract and the received signal is largely cancelled. The vertical plane radiation patterns are shown in the image at right. With vertical polarization there is always a maximum for θ=0, horizontal propagation (left pattern). For horizontal polarization, there is cancellation at that angle. Note that the above formulae and these plots assume the ground as a perfect conductor. These plots of the radiation pattern correspond to a distance between the antenna and its image of 2.5λ. As the antenna height is increased, the number of lobes increases as well.', 'For horizontal propagation between transmitting and receiving antennas situated near the ground reasonably far from each other, the distances traveled by tne direct and reflected rays are nearly the same. There is almost no relative phase shift. If the emission is polarized vertically, the two fields (direct and reflected) add and there is maximum of received signal. If the signal is polarized horizontally, the two signals subtract and the received signal is largely cancelled. The vertical plane radiation patterns are shown in the image at right. With vertical polarization there is always a maximum for θ=0, horizontal propagation (left pattern). For horizontal polarization, there is cancellation at that angle. Note that the above formulae and these plots assume the ground as a perfect conductor. These plots of the radiation pattern correspond to a distance between the antenna and its image of 2.5λ. As the antenna height is increased, the number of lobes increases as well.', 'For horizontal propagation between transmitting and receiving antennas situated near the ground reasonably far from each other, the distances traveled by tne direct and reflected rays are nearly the same. There is almost no relative phase shift. If the emission is polarized vertically, the two fields (direct and reflected) add and there is maximum of received signal. If the signal is polarized horizontally, the two signals subtract and the received signal is largely cancelled. The vertical plane radiation patterns are shown in the image at right. With vertical polarization there is always a maximum for θ=0, horizontal propagation (left pattern). For horizontal polarization, there is cancellation at that angle. Note that the above formulae and these plots assume the ground as a perfect conductor. These plots of the radiation pattern correspond to a distance between the antenna and its image of 2.5λ. As the antenna height is increased, the number of lobes increases as well.', 'On the other hand, classical (analog) television transmissions are usually horizontally polarized, because in urban areas buildings can reflect the electromagnetic waves and create ghost images due to multipath propagation. Using horizontal polarization, ghosting is reduced because the amount of reflection of electromagnetic waves in the p polarization (horizontal polarization off the side of a building) is generally less than s (vertical, in this case) polarization. Vertically polarized analog television has nevertheless been used in some rural areas. In digital terrestrial television such reflections are less problematic, due to robustness of binary transmissions and error correction.', 'On the other hand, classical (analog) television transmissions are usually horizontally polarized, because in urban areas buildings can reflect the electromagnetic waves and create ghost images due to multipath propagation. Using horizontal polarization, ghosting is reduced because the amount of reflection of electromagnetic waves in the p polarization (horizontal polarization off the side of a building) is generally less than s (vertical, in this case) polarization. Vertically polarized analog television has nevertheless been used in some rural areas. In digital terrestrial television such reflections are less problematic, due to robustness of binary transmissions and error correction.', 'On the other hand, classical (analog) television transmissions are usually horizontally polarized, because in urban areas buildings can reflect the electromagnetic waves and create ghost images due to multipath propagation. Using horizontal polarization, ghosting is reduced because the amount of reflection of electromagnetic waves in the p polarization (horizontal polarization off the side of a building) is generally less than s (vertical, in this case) polarization. Vertically polarized analog television has nevertheless been used in some rural areas. In digital terrestrial television such reflections are less problematic, due to robustness of binary transmissions and error correction.', 'On the other hand, classical (analog) television transmissions are usually horizontally polarized, because in urban areas buildings can reflect the electromagnetic waves and create ghost images due to multipath propagation. Using horizontal polarization, ghosting is reduced because the amount of reflection of electromagnetic waves in the p polarization (horizontal polarization off the side of a building) is generally less than s (vertical, in this case) polarization. Vertically polarized analog television has nevertheless been used in some rural areas. In digital terrestrial television such reflections are less problematic, due to robustness of binary transmissions and error correction.', 'Current circulating in one antenna generally induces a voltage across the feedpoint of nearby antennas or antenna elements. The mathematics presented below are useful in analyzing the electrical behaviour of antenna arrays, where the properties of the individual array elements (such as half wave dipoles) are already known. If those elements were widely separated and driven in a certain amplitude and phase, then each would act independently as that element is known to. However, because of the mutual interaction between their electric and magnetic fields due to proximity, the currents in each element are not simply a function of the applied voltage (according to its driving point impedance), but depend on the currents in the other nearby elements. Note that this now is a near field phenomenon which could not be properly accounted for using the Friis transmission equation for instance.', 'Current circulating in one antenna generally induces a voltage across the feedpoint of nearby antennas or antenna elements. The mathematics presented below are useful in analyzing the electrical behaviour of antenna arrays, where the properties of the individual array elements (such as half wave dipoles) are already known. If those elements were widely separated and driven in a certain amplitude and phase, then each would act independently as that element is known to. However, because of the mutual interaction between their electric and magnetic fields due to proximity, the currents in each element are not simply a function of the applied voltage (according to its driving point impedance), but depend on the currents in the other nearby elements. Note that this now is a near field phenomenon which could not be properly accounted for using the Friis transmission equation for instance.', 'Current circulating in one antenna generally induces a voltage across the feedpoint of nearby antennas or antenna elements. The mathematics presented below are useful in analyzing the electrical behaviour of antenna arrays, where the properties of the individual array elements (such as half wave dipoles) are already known. If those elements were widely separated and driven in a certain amplitude and phase, then each would act independently as that element is known to. However, because of the mutual interaction between their electric and magnetic fields due to proximity, the currents in each element are not simply a function of the applied voltage (according to its driving point impedance), but depend on the currents in the other nearby elements. Note that this now is a near field phenomenon which could not be properly accounted for using the Friis transmission equation for instance.', 'Current circulating in one antenna generally induces a voltage across the feedpoint of nearby antennas or antenna elements. The mathematics presented below are useful in analyzing the electrical behaviour of antenna arrays, where the properties of the individual array elements (such as half wave dipoles) are already known. If those elements were widely separated and driven in a certain amplitude and phase, then each would act independently as that element is known to. However, because of the mutual interaction between their electric and magnetic fields due to proximity, the currents in each element are not simply a function of the applied voltage (according to its driving point impedance), but depend on the currents in the other nearby elements. Note that this now is a near field phenomenon which could not be properly accounted for using the Friis transmission equation for instance.', 'This is a consequence of Lorentz reciprocity. For an antenna element  not connected to anything (open circuited) one can write . But for an element  which is short circuited, a current is generated across that short but no voltage is allowed, so the corresponding . This is the case, for instance, with the so-called parasitic elements of a Yagi-Uda antenna where the solid rod can be viewed as a dipole antenna shorted across its feedpoint. Parasitic elements are unpowered elements that absorb and reradiate RF energy according to the induced current calculated using such a system of equations.', 'This is a consequence of Lorentz reciprocity. For an antenna element  not connected to anything (open circuited) one can write . But for an element  which is short circuited, a current is generated across that short but no voltage is allowed, so the corresponding . This is the case, for instance, with the so-called parasitic elements of a Yagi-Uda antenna where the solid rod can be viewed as a dipole antenna shorted across its feedpoint. Parasitic elements are unpowered elements that absorb and reradiate RF energy according to the induced current calculated using such a system of equations.', 'This is a consequence of Lorentz reciprocity. For an antenna element  not connected to anything (open circuited) one can write . But for an element  which is short circuited, a current is generated across that short but no voltage is allowed, so the corresponding . This is the case, for instance, with the so-called parasitic elements of a Yagi-Uda antenna where the solid rod can be viewed as a dipole antenna shorted across its feedpoint. Parasitic elements are unpowered elements that absorb and reradiate RF energy according to the induced current calculated using such a system of equations.', 'This is a consequence of Lorentz reciprocity. For an antenna element  not connected to anything (open circuited) one can write . But for an element  which is short circuited, a current is generated across that short but no voltage is allowed, so the corresponding . This is the case, for instance, with the so-called parasitic elements of a Yagi-Uda antenna where the solid rod can be viewed as a dipole antenna shorted across its feedpoint. Parasitic elements are unpowered elements that absorb and reradiate RF energy according to the induced current calculated using such a system of equations.', 'This is a consequence of Lorentz reciprocity. For an antenna element  not connected to anything (open circuited) one can write . But for an element  which is short circuited, a current is generated across that short but no voltage is allowed, so the corresponding . This is the case, for instance, with the so-called parasitic elements of a Yagi-Uda antenna where the solid rod can be viewed as a dipole antenna shorted across its feedpoint. Parasitic elements are unpowered elements that absorb and reradiate RF energy according to the induced current calculated using such a system of equations.', 'The difference in the above factors for the case of θ=0 is the reason that most broadcasting (transmissions intended for the public) uses vertical polarization. For receivers near the ground, horizontally polarized transmissions suffer cancellation. For best reception the receiving antennas for these signals are likewise vertically polarized. In some applications where the receiving antenna must work in any position, as in mobile phones, the base station antennas use mixed polarization, such as linear polarization at an angle (with both vertical and horizontal components) or circular polarization.', 'The difference in the above factors for the case of θ=0 is the reason that most broadcasting (transmissions intended for the public) uses vertical polarization. For receivers near the ground, horizontally polarized transmissions suffer cancellation. For best reception the receiving antennas for these signals are likewise vertically polarized. In some applications where the receiving antenna must work in any position, as in mobile phones, the base station antennas use mixed polarization, such as linear polarization at an angle (with both vertical and horizontal components) or circular polarization.', 'The difference in the above factors for the case of θ=0 is the reason that most broadcasting (transmissions intended for the public) uses vertical polarization. For receivers near the ground, horizontally polarized transmissions suffer cancellation. For best reception the receiving antennas for these signals are likewise vertically polarized. In some applications where the receiving antenna must work in any position, as in mobile phones, the base station antennas use mixed polarization, such as linear polarization at an angle (with both vertical and horizontal components) or circular polarization.', 'The difference in the above factors for the case of θ=0 is the reason that most broadcasting (transmissions intended for the public) uses vertical polarization. For receivers near the ground, horizontally polarized transmissions suffer cancellation. For best reception the receiving antennas for these signals are likewise vertically polarized. In some applications where the receiving antenna must work in any position, as in mobile phones, the base station antennas use mixed polarization, such as linear polarization at an angle (with both vertical and horizontal components) or circular polarization.', 'The difference in the above factors for the case of θ=0 is the reason that most broadcasting (transmissions intended for the public) uses vertical polarization. For receivers near the ground, horizontally polarized transmissions suffer cancellation. For best reception the receiving antennas for these signals are likewise vertically polarized. In some applications where the receiving antenna must work in any position, as in mobile phones, the base station antennas use mixed polarization, such as linear polarization at an angle (with both vertical and horizontal components) or circular polarization.', 'Loop antennas consist of a loop or coil of wire. Loops with circumference of a wavelength or larger act similarly to dipole antennas. However loops small in comparison to a wavelength act differently. They interact with the magnetic field of the radio wave instead of the electric field as other antennas do, and so are relatively insensitive to nearby electrical noise. However they have low radiation resistance, and so are inefficient for transmitting. They are used as receiving antennas at low frequencies, and also as direction finding antennas.', 'Loop antennas consist of a loop or coil of wire. Loops with circumference of a wavelength or larger act similarly to dipole antennas. However loops small in comparison to a wavelength act differently. They interact with the magnetic field of the radio wave instead of the electric field as other antennas do, and so are relatively insensitive to nearby electrical noise. However they have low radiation resistance, and so are inefficient for transmitting. They are used as receiving antennas at low frequencies, and also as direction finding antennas.', 'Loop antennas consist of a loop or coil of wire. Loops with circumference of a wavelength or larger act similarly to dipole antennas. However loops small in comparison to a wavelength act differently. They interact with the magnetic field of the radio wave instead of the electric field as other antennas do, and so are relatively insensitive to nearby electrical noise. However they have low radiation resistance, and so are inefficient for transmitting. They are used as receiving antennas at low frequencies, and also as direction finding antennas.', 'Loop antennas consist of a loop or coil of wire. Loops with circumference of a wavelength or larger act similarly to dipole antennas. However loops small in comparison to a wavelength act differently. They interact with the magnetic field of the radio wave instead of the electric field as other antennas do, and so are relatively insensitive to nearby electrical noise. However they have low radiation resistance, and so are inefficient for transmitting. They are used as receiving antennas at low frequencies, and also as direction finding antennas.', 'Loop antennas consist of a loop or coil of wire. Loops with circumference of a wavelength or larger act similarly to dipole antennas. However loops small in comparison to a wavelength act differently. They interact with the magnetic field of the radio wave instead of the electric field as other antennas do, and so are relatively insensitive to nearby electrical noise. However they have low radiation resistance, and so are inefficient for transmitting. They are used as receiving antennas at low frequencies, and also as direction finding antennas.', 'It is a fundamental property of antennas that the electrical characteristics of an antenna described in the next section, such as gain, radiation pattern, impedance, bandwidth, resonant frequency and polarization, are the same whether the antenna is transmitting or receiving. For example, the "receiving pattern" (sensitivity as a function of direction) of an antenna when used for reception is identical to the radiation pattern of the antenna when it is driven and functions as a radiator. This is a consequence of the reciprocity theorem of electromagnetics. Therefore, in discussions of antenna properties no distinction is usually made between receiving and transmitting terminology, and the antenna can be viewed as either transmitting or receiving, whichever is more convenient.', 'It is a fundamental property of antennas that the electrical characteristics of an antenna described in the next section, such as gain, radiation pattern, impedance, bandwidth, resonant frequency and polarization, are the same whether the antenna is transmitting or receiving. For example, the "receiving pattern" (sensitivity as a function of direction) of an antenna when used for reception is identical to the radiation pattern of the antenna when it is driven and functions as a radiator. This is a consequence of the reciprocity theorem of electromagnetics. Therefore, in discussions of antenna properties no distinction is usually made between receiving and transmitting terminology, and the antenna can be viewed as either transmitting or receiving, whichever is more convenient.', 'It is a fundamental property of antennas that the electrical characteristics of an antenna described in the next section, such as gain, radiation pattern, impedance, bandwidth, resonant frequency and polarization, are the same whether the antenna is transmitting or receiving. For example, the "receiving pattern" (sensitivity as a function of direction) of an antenna when used for reception is identical to the radiation pattern of the antenna when it is driven and functions as a radiator. This is a consequence of the reciprocity theorem of electromagnetics. Therefore, in discussions of antenna properties no distinction is usually made between receiving and transmitting terminology, and the antenna can be viewed as either transmitting or receiving, whichever is more convenient.', 'It is a fundamental property of antennas that the electrical characteristics of an antenna described in the next section, such as gain, radiation pattern, impedance, bandwidth, resonant frequency and polarization, are the same whether the antenna is transmitting or receiving. For example, the "receiving pattern" (sensitivity as a function of direction) of an antenna when used for reception is identical to the radiation pattern of the antenna when it is driven and functions as a radiator. This is a consequence of the reciprocity theorem of electromagnetics. Therefore, in discussions of antenna properties no distinction is usually made between receiving and transmitting terminology, and the antenna can be viewed as either transmitting or receiving, whichever is more convenient.', 'The flowering plants (angiosperms), also known as Angiospermae or Magnoliophyta, are the most diverse group of land plants, with about 350,000 species. Like gymnosperms, angiosperms are seed-producing plants; they are distinguished from gymnosperms by characteristics including flowers, endosperm within the seeds, and the production of fruits that contain the seeds. Etymologically, angiosperm means a plant that produces seeds within an enclosure, in other words, a fruiting plant. The term "angiosperm" comes from the Greek composite word (angeion-, "case" or "casing", and sperma, "seed") meaning "enclosed seeds", after the enclosed condition of the seeds.', 'The flowering plants (angiosperms), also known as Angiospermae or Magnoliophyta, are the most diverse group of land plants, with about 350,000 species. Like gymnosperms, angiosperms are seed-producing plants; they are distinguished from gymnosperms by characteristics including flowers, endosperm within the seeds, and the production of fruits that contain the seeds. Etymologically, angiosperm means a plant that produces seeds within an enclosure, in other words, a fruiting plant. The term "angiosperm" comes from the Greek composite word (angeion-, "case" or "casing", and sperma, "seed") meaning "enclosed seeds", after the enclosed condition of the seeds.', 'The flowering plants (angiosperms), also known as Angiospermae or Magnoliophyta, are the most diverse group of land plants, with about 350,000 species. Like gymnosperms, angiosperms are seed-producing plants; they are distinguished from gymnosperms by characteristics including flowers, endosperm within the seeds, and the production of fruits that contain the seeds. Etymologically, angiosperm means a plant that produces seeds within an enclosure, in other words, a fruiting plant. The term "angiosperm" comes from the Greek composite word (angeion-, "case" or "casing", and sperma, "seed") meaning "enclosed seeds", after the enclosed condition of the seeds.', 'The flowering plants (angiosperms), also known as Angiospermae or Magnoliophyta, are the most diverse group of land plants, with about 350,000 species. Like gymnosperms, angiosperms are seed-producing plants; they are distinguished from gymnosperms by characteristics including flowers, endosperm within the seeds, and the production of fruits that contain the seeds. Etymologically, angiosperm means a plant that produces seeds within an enclosure, in other words, a fruiting plant. The term "angiosperm" comes from the Greek composite word (angeion-, "case" or "casing", and sperma, "seed") meaning "enclosed seeds", after the enclosed condition of the seeds.', 'The flowering plants (angiosperms), also known as Angiospermae or Magnoliophyta, are the most diverse group of land plants, with about 350,000 species. Like gymnosperms, angiosperms are seed-producing plants; they are distinguished from gymnosperms by characteristics including flowers, endosperm within the seeds, and the production of fruits that contain the seeds. Etymologically, angiosperm means a plant that produces seeds within an enclosure, in other words, a fruiting plant. The term "angiosperm" comes from the Greek composite word (angeion-, "case" or "casing", and sperma, "seed") meaning "enclosed seeds", after the enclosed condition of the seeds.', 'Fossilized spores suggest that higher plants (embryophytes) have lived on land for at least 475 million years. Early land plants reproduced sexually with flagellated, swimming sperm, like the green algae from which they evolved. An adaptation to terrestrialization was the development of upright meiosporangia for dispersal by spores to new habitats. This feature is lacking in the descendants of their nearest algal relatives, the Charophycean green algae. A later terrestrial adaptation took place with retention of the delicate, avascular sexual stage, the gametophyte, within the tissues of the vascular sporophyte. This occurred by spore germination within sporangia rather than spore release, as in non-seed plants. A current example of how this might have happened can be seen in the precocious spore germination in Selaginella, the spike-moss. The result for the ancestors of angiosperms was enclosing them in a case, the seed. The first seed bearing plants, like the ginkgo, and conifers (such as pines and firs), did not produce flowers. The pollen grains (males) of Ginkgo and cycads produce a pair of flagellated, mobile sperm cells that "swim" down the developing pollen tube to the female and her eggs.', 'Fossilized spores suggest that higher plants (embryophytes) have lived on land for at least 475 million years. Early land plants reproduced sexually with flagellated, swimming sperm, like the green algae from which they evolved. An adaptation to terrestrialization was the development of upright meiosporangia for dispersal by spores to new habitats. This feature is lacking in the descendants of their nearest algal relatives, the Charophycean green algae. A later terrestrial adaptation took place with retention of the delicate, avascular sexual stage, the gametophyte, within the tissues of the vascular sporophyte. This occurred by spore germination within sporangia rather than spore release, as in non-seed plants. A current example of how this might have happened can be seen in the precocious spore germination in Selaginella, the spike-moss. The result for the ancestors of angiosperms was enclosing them in a case, the seed. The first seed bearing plants, like the ginkgo, and conifers (such as pines and firs), did not produce flowers. The pollen grains (males) of Ginkgo and cycads produce a pair of flagellated, mobile sperm cells that "swim" down the developing pollen tube to the female and her eggs.', 'Fossilized spores suggest that higher plants (embryophytes) have lived on land for at least 475 million years. Early land plants reproduced sexually with flagellated, swimming sperm, like the green algae from which they evolved. An adaptation to terrestrialization was the development of upright meiosporangia for dispersal by spores to new habitats. This feature is lacking in the descendants of their nearest algal relatives, the Charophycean green algae. A later terrestrial adaptation took place with retention of the delicate, avascular sexual stage, the gametophyte, within the tissues of the vascular sporophyte. This occurred by spore germination within sporangia rather than spore release, as in non-seed plants. A current example of how this might have happened can be seen in the precocious spore germination in Selaginella, the spike-moss. The result for the ancestors of angiosperms was enclosing them in a case, the seed. The first seed bearing plants, like the ginkgo, and conifers (such as pines and firs), did not produce flowers. The pollen grains (males) of Ginkgo and cycads produce a pair of flagellated, mobile sperm cells that "swim" down the developing pollen tube to the female and her eggs.', 'Fossilized spores suggest that higher plants (embryophytes) have lived on land for at least 475 million years. Early land plants reproduced sexually with flagellated, swimming sperm, like the green algae from which they evolved. An adaptation to terrestrialization was the development of upright meiosporangia for dispersal by spores to new habitats. This feature is lacking in the descendants of their nearest algal relatives, the Charophycean green algae. A later terrestrial adaptation took place with retention of the delicate, avascular sexual stage, the gametophyte, within the tissues of the vascular sporophyte. This occurred by spore germination within sporangia rather than spore release, as in non-seed plants. A current example of how this might have happened can be seen in the precocious spore germination in Selaginella, the spike-moss. The result for the ancestors of angiosperms was enclosing them in a case, the seed. The first seed bearing plants, like the ginkgo, and conifers (such as pines and firs), did not produce flowers. The pollen grains (males) of Ginkgo and cycads produce a pair of flagellated, mobile sperm cells that "swim" down the developing pollen tube to the female and her eggs.', 'Fossilized spores suggest that higher plants (embryophytes) have lived on land for at least 475 million years. Early land plants reproduced sexually with flagellated, swimming sperm, like the green algae from which they evolved. An adaptation to terrestrialization was the development of upright meiosporangia for dispersal by spores to new habitats. This feature is lacking in the descendants of their nearest algal relatives, the Charophycean green algae. A later terrestrial adaptation took place with retention of the delicate, avascular sexual stage, the gametophyte, within the tissues of the vascular sporophyte. This occurred by spore germination within sporangia rather than spore release, as in non-seed plants. A current example of how this might have happened can be seen in the precocious spore germination in Selaginella, the spike-moss. The result for the ancestors of angiosperms was enclosing them in a case, the seed. The first seed bearing plants, like the ginkgo, and conifers (such as pines and firs), did not produce flowers. The pollen grains (males) of Ginkgo and cycads produce a pair of flagellated, mobile sperm cells that "swim" down the developing pollen tube to the female and her eggs.', 'The apparently sudden appearance of nearly modern flowers in the fossil record initially posed such a problem for the theory of evolution that it was called an "abominable mystery" by Charles Darwin. However, the fossil record has considerably grown since the time of Darwin, and recently discovered angiosperm fossils such as Archaefructus, along with further discoveries of fossil gymnosperms, suggest how angiosperm characteristics may have been acquired in a series of steps. Several groups of extinct gymnosperms, in particular seed ferns, have been proposed as the ancestors of flowering plants, but there is no continuous fossil evidence showing exactly how flowers evolved. Some older fossils, such as the upper Triassic Sanmiguelia, have been suggested. Based on current evidence, some propose that the ancestors of the angiosperms diverged from an unknown group of gymnosperms in the Triassic period (245–202 million years ago). Fossil angiosperm-like pollen from the Middle Triassic (247.2–242.0 Ma) suggests an older date for their origin. A close relationship between angiosperms and gnetophytes, proposed on the basis of morphological evidence, has more recently been disputed on the basis of molecular evidence that suggest gnetophytes are instead more closely related to other gymnosperms.[citation needed]', 'The apparently sudden appearance of nearly modern flowers in the fossil record initially posed such a problem for the theory of evolution that it was called an "abominable mystery" by Charles Darwin. However, the fossil record has considerably grown since the time of Darwin, and recently discovered angiosperm fossils such as Archaefructus, along with further discoveries of fossil gymnosperms, suggest how angiosperm characteristics may have been acquired in a series of steps. Several groups of extinct gymnosperms, in particular seed ferns, have been proposed as the ancestors of flowering plants, but there is no continuous fossil evidence showing exactly how flowers evolved. Some older fossils, such as the upper Triassic Sanmiguelia, have been suggested. Based on current evidence, some propose that the ancestors of the angiosperms diverged from an unknown group of gymnosperms in the Triassic period (245–202 million years ago). Fossil angiosperm-like pollen from the Middle Triassic (247.2–242.0 Ma) suggests an older date for their origin. A close relationship between angiosperms and gnetophytes, proposed on the basis of morphological evidence, has more recently been disputed on the basis of molecular evidence that suggest gnetophytes are instead more closely related to other gymnosperms.[citation needed]', 'The apparently sudden appearance of nearly modern flowers in the fossil record initially posed such a problem for the theory of evolution that it was called an "abominable mystery" by Charles Darwin. However, the fossil record has considerably grown since the time of Darwin, and recently discovered angiosperm fossils such as Archaefructus, along with further discoveries of fossil gymnosperms, suggest how angiosperm characteristics may have been acquired in a series of steps. Several groups of extinct gymnosperms, in particular seed ferns, have been proposed as the ancestors of flowering plants, but there is no continuous fossil evidence showing exactly how flowers evolved. Some older fossils, such as the upper Triassic Sanmiguelia, have been suggested. Based on current evidence, some propose that the ancestors of the angiosperms diverged from an unknown group of gymnosperms in the Triassic period (245–202 million years ago). Fossil angiosperm-like pollen from the Middle Triassic (247.2–242.0 Ma) suggests an older date for their origin. A close relationship between angiosperms and gnetophytes, proposed on the basis of morphological evidence, has more recently been disputed on the basis of molecular evidence that suggest gnetophytes are instead more closely related to other gymnosperms.[citation needed]', 'The apparently sudden appearance of nearly modern flowers in the fossil record initially posed such a problem for the theory of evolution that it was called an "abominable mystery" by Charles Darwin. However, the fossil record has considerably grown since the time of Darwin, and recently discovered angiosperm fossils such as Archaefructus, along with further discoveries of fossil gymnosperms, suggest how angiosperm characteristics may have been acquired in a series of steps. Several groups of extinct gymnosperms, in particular seed ferns, have been proposed as the ancestors of flowering plants, but there is no continuous fossil evidence showing exactly how flowers evolved. Some older fossils, such as the upper Triassic Sanmiguelia, have been suggested. Based on current evidence, some propose that the ancestors of the angiosperms diverged from an unknown group of gymnosperms in the Triassic period (245–202 million years ago). Fossil angiosperm-like pollen from the Middle Triassic (247.2–242.0 Ma) suggests an older date for their origin. A close relationship between angiosperms and gnetophytes, proposed on the basis of morphological evidence, has more recently been disputed on the basis of molecular evidence that suggest gnetophytes are instead more closely related to other gymnosperms.[citation needed]', 'The apparently sudden appearance of nearly modern flowers in the fossil record initially posed such a problem for the theory of evolution that it was called an "abominable mystery" by Charles Darwin. However, the fossil record has considerably grown since the time of Darwin, and recently discovered angiosperm fossils such as Archaefructus, along with further discoveries of fossil gymnosperms, suggest how angiosperm characteristics may have been acquired in a series of steps. Several groups of extinct gymnosperms, in particular seed ferns, have been proposed as the ancestors of flowering plants, but there is no continuous fossil evidence showing exactly how flowers evolved. Some older fossils, such as the upper Triassic Sanmiguelia, have been suggested. Based on current evidence, some propose that the ancestors of the angiosperms diverged from an unknown group of gymnosperms in the Triassic period (245–202 million years ago). Fossil angiosperm-like pollen from the Middle Triassic (247.2–242.0 Ma) suggests an older date for their origin. A close relationship between angiosperms and gnetophytes, proposed on the basis of morphological evidence, has more recently been disputed on the basis of molecular evidence that suggest gnetophytes are instead more closely related to other gymnosperms.[citation needed]', 'The evolution of seed plants and later angiosperms appears to be the result of two distinct rounds of whole genome duplication events. These occurred at 319 million years ago and 192 million years ago. Another possible whole genome duplication event at 160 million years ago perhaps created the ancestral line that led to all modern flowering plants. That event was studied by sequencing the genome of an ancient flowering plant, Amborella trichopoda, and directly addresses Darwin\'s "abominable mystery."', 'The evolution of seed plants and later angiosperms appears to be the result of two distinct rounds of whole genome duplication events. These occurred at 319 million years ago and 192 million years ago. Another possible whole genome duplication event at 160 million years ago perhaps created the ancestral line that led to all modern flowering plants. That event was studied by sequencing the genome of an ancient flowering plant, Amborella trichopoda, and directly addresses Darwin\'s "abominable mystery."', 'The evolution of seed plants and later angiosperms appears to be the result of two distinct rounds of whole genome duplication events. These occurred at 319 million years ago and 192 million years ago. Another possible whole genome duplication event at 160 million years ago perhaps created the ancestral line that led to all modern flowering plants. That event was studied by sequencing the genome of an ancient flowering plant, Amborella trichopoda, and directly addresses Darwin\'s "abominable mystery."', 'The evolution of seed plants and later angiosperms appears to be the result of two distinct rounds of whole genome duplication events. These occurred at 319 million years ago and 192 million years ago. Another possible whole genome duplication event at 160 million years ago perhaps created the ancestral line that led to all modern flowering plants. That event was studied by sequencing the genome of an ancient flowering plant, Amborella trichopoda, and directly addresses Darwin\'s "abominable mystery."', 'The evolution of seed plants and later angiosperms appears to be the result of two distinct rounds of whole genome duplication events. These occurred at 319 million years ago and 192 million years ago. Another possible whole genome duplication event at 160 million years ago perhaps created the ancestral line that led to all modern flowering plants. That event was studied by sequencing the genome of an ancient flowering plant, Amborella trichopoda, and directly addresses Darwin\'s "abominable mystery."', 'The earliest known macrofossil confidently identified as an angiosperm, Archaefructus liaoningensis, is dated to about 125 million years BP (the Cretaceous period), whereas pollen considered to be of angiosperm origin takes the fossil record back to about 130 million years BP. However, one study has suggested that the early-middle Jurassic plant Schmeissneria, traditionally considered a type of ginkgo, may be the earliest known angiosperm, or at least a close relative. In addition, circumstantial chemical evidence has been found for the existence of angiosperms as early as 250 million years ago. Oleanane, a secondary metabolite produced by many flowering plants, has been found in Permian deposits of that age together with fossils of gigantopterids. Gigantopterids are a group of extinct seed plants that share many morphological traits with flowering plants, although they are not known to have been flowering plants themselves.', 'The earliest known macrofossil confidently identified as an angiosperm, Archaefructus liaoningensis, is dated to about 125 million years BP (the Cretaceous period), whereas pollen considered to be of angiosperm origin takes the fossil record back to about 130 million years BP. However, one study has suggested that the early-middle Jurassic plant Schmeissneria, traditionally considered a type of ginkgo, may be the earliest known angiosperm, or at least a close relative. In addition, circumstantial chemical evidence has been found for the existence of angiosperms as early as 250 million years ago. Oleanane, a secondary metabolite produced by many flowering plants, has been found in Permian deposits of that age together with fossils of gigantopterids. Gigantopterids are a group of extinct seed plants that share many morphological traits with flowering plants, although they are not known to have been flowering plants themselves.', 'The earliest known macrofossil confidently identified as an angiosperm, Archaefructus liaoningensis, is dated to about 125 million years BP (the Cretaceous period), whereas pollen considered to be of angiosperm origin takes the fossil record back to about 130 million years BP. However, one study has suggested that the early-middle Jurassic plant Schmeissneria, traditionally considered a type of ginkgo, may be the earliest known angiosperm, or at least a close relative. In addition, circumstantial chemical evidence has been found for the existence of angiosperms as early as 250 million years ago. Oleanane, a secondary metabolite produced by many flowering plants, has been found in Permian deposits of that age together with fossils of gigantopterids. Gigantopterids are a group of extinct seed plants that share many morphological traits with flowering plants, although they are not known to have been flowering plants themselves.', 'The earliest known macrofossil confidently identified as an angiosperm, Archaefructus liaoningensis, is dated to about 125 million years BP (the Cretaceous period), whereas pollen considered to be of angiosperm origin takes the fossil record back to about 130 million years BP. However, one study has suggested that the early-middle Jurassic plant Schmeissneria, traditionally considered a type of ginkgo, may be the earliest known angiosperm, or at least a close relative. In addition, circumstantial chemical evidence has been found for the existence of angiosperms as early as 250 million years ago. Oleanane, a secondary metabolite produced by many flowering plants, has been found in Permian deposits of that age together with fossils of gigantopterids. Gigantopterids are a group of extinct seed plants that share many morphological traits with flowering plants, although they are not known to have been flowering plants themselves.', 'The earliest known macrofossil confidently identified as an angiosperm, Archaefructus liaoningensis, is dated to about 125 million years BP (the Cretaceous period), whereas pollen considered to be of angiosperm origin takes the fossil record back to about 130 million years BP. However, one study has suggested that the early-middle Jurassic plant Schmeissneria, traditionally considered a type of ginkgo, may be the earliest known angiosperm, or at least a close relative. In addition, circumstantial chemical evidence has been found for the existence of angiosperms as early as 250 million years ago. Oleanane, a secondary metabolite produced by many flowering plants, has been found in Permian deposits of that age together with fossils of gigantopterids. Gigantopterids are a group of extinct seed plants that share many morphological traits with flowering plants, although they are not known to have been flowering plants themselves.', 'The great angiosperm radiation, when a great diversity of angiosperms appears in the fossil record, occurred in the mid-Cretaceous (approximately 100 million years ago). However, a study in 2007 estimated that the division of the five most recent (the genus Ceratophyllum, the family Chloranthaceae, the eudicots, the magnoliids, and the monocots) of the eight main groups occurred around 140 million years ago. By the late Cretaceous, angiosperms appear to have dominated environments formerly occupied by ferns and cycadophytes, but large canopy-forming trees replaced conifers as the dominant trees only close to the end of the Cretaceous 66 million years ago or even later, at the beginning of the Tertiary. The radiation of herbaceous angiosperms occurred much later. Yet, many fossil plants recognizable as belonging to modern families (including beech, oak, maple, and magnolia) had already appeared by the late Cretaceous.', 'The great angiosperm radiation, when a great diversity of angiosperms appears in the fossil record, occurred in the mid-Cretaceous (approximately 100 million years ago). However, a study in 2007 estimated that the division of the five most recent (the genus Ceratophyllum, the family Chloranthaceae, the eudicots, the magnoliids, and the monocots) of the eight main groups occurred around 140 million years ago. By the late Cretaceous, angiosperms appear to have dominated environments formerly occupied by ferns and cycadophytes, but large canopy-forming trees replaced conifers as the dominant trees only close to the end of the Cretaceous 66 million years ago or even later, at the beginning of the Tertiary. The radiation of herbaceous angiosperms occurred much later. Yet, many fossil plants recognizable as belonging to modern families (including beech, oak, maple, and magnolia) had already appeared by the late Cretaceous.', 'The great angiosperm radiation, when a great diversity of angiosperms appears in the fossil record, occurred in the mid-Cretaceous (approximately 100 million years ago). However, a study in 2007 estimated that the division of the five most recent (the genus Ceratophyllum, the family Chloranthaceae, the eudicots, the magnoliids, and the monocots) of the eight main groups occurred around 140 million years ago. By the late Cretaceous, angiosperms appear to have dominated environments formerly occupied by ferns and cycadophytes, but large canopy-forming trees replaced conifers as the dominant trees only close to the end of the Cretaceous 66 million years ago or even later, at the beginning of the Tertiary. The radiation of herbaceous angiosperms occurred much later. Yet, many fossil plants recognizable as belonging to modern families (including beech, oak, maple, and magnolia) had already appeared by the late Cretaceous.', 'The great angiosperm radiation, when a great diversity of angiosperms appears in the fossil record, occurred in the mid-Cretaceous (approximately 100 million years ago). However, a study in 2007 estimated that the division of the five most recent (the genus Ceratophyllum, the family Chloranthaceae, the eudicots, the magnoliids, and the monocots) of the eight main groups occurred around 140 million years ago. By the late Cretaceous, angiosperms appear to have dominated environments formerly occupied by ferns and cycadophytes, but large canopy-forming trees replaced conifers as the dominant trees only close to the end of the Cretaceous 66 million years ago or even later, at the beginning of the Tertiary. The radiation of herbaceous angiosperms occurred much later. Yet, many fossil plants recognizable as belonging to modern families (including beech, oak, maple, and magnolia) had already appeared by the late Cretaceous.', 'The great angiosperm radiation, when a great diversity of angiosperms appears in the fossil record, occurred in the mid-Cretaceous (approximately 100 million years ago). However, a study in 2007 estimated that the division of the five most recent (the genus Ceratophyllum, the family Chloranthaceae, the eudicots, the magnoliids, and the monocots) of the eight main groups occurred around 140 million years ago. By the late Cretaceous, angiosperms appear to have dominated environments formerly occupied by ferns and cycadophytes, but large canopy-forming trees replaced conifers as the dominant trees only close to the end of the Cretaceous 66 million years ago or even later, at the beginning of the Tertiary. The radiation of herbaceous angiosperms occurred much later. Yet, many fossil plants recognizable as belonging to modern families (including beech, oak, maple, and magnolia) had already appeared by the late Cretaceous.', 'Island genetics provides one proposed explanation for the sudden, fully developed appearance of flowering plants. Island genetics is believed to be a common source of speciation in general, especially when it comes to radical adaptations that seem to have required inferior transitional forms. Flowering plants may have evolved in an isolated setting like an island or island chain, where the plants bearing them were able to develop a highly specialized relationship with some specific animal (a wasp, for example). Such a relationship, with a hypothetical wasp carrying pollen from one plant to another much the way fig wasps do today, could result in the development of a high degree of specialization in both the plant(s) and their partners. Note that the wasp example is not incidental; bees, which, it is postulated, evolved specifically due to mutualistic plant relationships, are descended from wasps.', 'Island genetics provides one proposed explanation for the sudden, fully developed appearance of flowering plants. Island genetics is believed to be a common source of speciation in general, especially when it comes to radical adaptations that seem to have required inferior transitional forms. Flowering plants may have evolved in an isolated setting like an island or island chain, where the plants bearing them were able to develop a highly specialized relationship with some specific animal (a wasp, for example). Such a relationship, with a hypothetical wasp carrying pollen from one plant to another much the way fig wasps do today, could result in the development of a high degree of specialization in both the plant(s) and their partners. Note that the wasp example is not incidental; bees, which, it is postulated, evolved specifically due to mutualistic plant relationships, are descended from wasps.', 'Island genetics provides one proposed explanation for the sudden, fully developed appearance of flowering plants. Island genetics is believed to be a common source of speciation in general, especially when it comes to radical adaptations that seem to have required inferior transitional forms. Flowering plants may have evolved in an isolated setting like an island or island chain, where the plants bearing them were able to develop a highly specialized relationship with some specific animal (a wasp, for example). Such a relationship, with a hypothetical wasp carrying pollen from one plant to another much the way fig wasps do today, could result in the development of a high degree of specialization in both the plant(s) and their partners. Note that the wasp example is not incidental; bees, which, it is postulated, evolved specifically due to mutualistic plant relationships, are descended from wasps.', 'Island genetics provides one proposed explanation for the sudden, fully developed appearance of flowering plants. Island genetics is believed to be a common source of speciation in general, especially when it comes to radical adaptations that seem to have required inferior transitional forms. Flowering plants may have evolved in an isolated setting like an island or island chain, where the plants bearing them were able to develop a highly specialized relationship with some specific animal (a wasp, for example). Such a relationship, with a hypothetical wasp carrying pollen from one plant to another much the way fig wasps do today, could result in the development of a high degree of specialization in both the plant(s) and their partners. Note that the wasp example is not incidental; bees, which, it is postulated, evolved specifically due to mutualistic plant relationships, are descended from wasps.', 'Island genetics provides one proposed explanation for the sudden, fully developed appearance of flowering plants. Island genetics is believed to be a common source of speciation in general, especially when it comes to radical adaptations that seem to have required inferior transitional forms. Flowering plants may have evolved in an isolated setting like an island or island chain, where the plants bearing them were able to develop a highly specialized relationship with some specific animal (a wasp, for example). Such a relationship, with a hypothetical wasp carrying pollen from one plant to another much the way fig wasps do today, could result in the development of a high degree of specialization in both the plant(s) and their partners. Note that the wasp example is not incidental; bees, which, it is postulated, evolved specifically due to mutualistic plant relationships, are descended from wasps.', 'Animals are also involved in the distribution of seeds. Fruit, which is formed by the enlargement of flower parts, is frequently a seed-dispersal tool that attracts animals to eat or otherwise disturb it, incidentally scattering the seeds it contains (see frugivory). Although many such mutualistic relationships remain too fragile to survive competition and to spread widely, flowering proved to be an unusually effective means of reproduction, spreading (whatever its origin) to become the dominant form of land plant life.', 'Animals are also involved in the distribution of seeds. Fruit, which is formed by the enlargement of flower parts, is frequently a seed-dispersal tool that attracts animals to eat or otherwise disturb it, incidentally scattering the seeds it contains (see frugivory). Although many such mutualistic relationships remain too fragile to survive competition and to spread widely, flowering proved to be an unusually effective means of reproduction, spreading (whatever its origin) to become the dominant form of land plant life.', 'Animals are also involved in the distribution of seeds. Fruit, which is formed by the enlargement of flower parts, is frequently a seed-dispersal tool that attracts animals to eat or otherwise disturb it, incidentally scattering the seeds it contains (see frugivory). Although many such mutualistic relationships remain too fragile to survive competition and to spread widely, flowering proved to be an unusually effective means of reproduction, spreading (whatever its origin) to become the dominant form of land plant life.', 'Animals are also involved in the distribution of seeds. Fruit, which is formed by the enlargement of flower parts, is frequently a seed-dispersal tool that attracts animals to eat or otherwise disturb it, incidentally scattering the seeds it contains (see frugivory). Although many such mutualistic relationships remain too fragile to survive competition and to spread widely, flowering proved to be an unusually effective means of reproduction, spreading (whatever its origin) to become the dominant form of land plant life.', 'Animals are also involved in the distribution of seeds. Fruit, which is formed by the enlargement of flower parts, is frequently a seed-dispersal tool that attracts animals to eat or otherwise disturb it, incidentally scattering the seeds it contains (see frugivory). Although many such mutualistic relationships remain too fragile to survive competition and to spread widely, flowering proved to be an unusually effective means of reproduction, spreading (whatever its origin) to become the dominant form of land plant life.', 'Flower ontogeny uses a combination of genes normally responsible for forming new shoots. The most primitive flowers probably had a variable number of flower parts, often separate from (but in contact with) each other. The flowers tended to grow in a spiral pattern, to be bisexual (in plants, this means both male and female parts on the same flower), and to be dominated by the ovary (female part). As flowers evolved, some variations developed parts fused together, with a much more specific number and design, and with either specific sexes per flower or plant or at least "ovary-inferior".', 'Flower ontogeny uses a combination of genes normally responsible for forming new shoots. The most primitive flowers probably had a variable number of flower parts, often separate from (but in contact with) each other. The flowers tended to grow in a spiral pattern, to be bisexual (in plants, this means both male and female parts on the same flower), and to be dominated by the ovary (female part). As flowers evolved, some variations developed parts fused together, with a much more specific number and design, and with either specific sexes per flower or plant or at least "ovary-inferior".', 'Flower ontogeny uses a combination of genes normally responsible for forming new shoots. The most primitive flowers probably had a variable number of flower parts, often separate from (but in contact with) each other. The flowers tended to grow in a spiral pattern, to be bisexual (in plants, this means both male and female parts on the same flower), and to be dominated by the ovary (female part). As flowers evolved, some variations developed parts fused together, with a much more specific number and design, and with either specific sexes per flower or plant or at least "ovary-inferior".', 'Flower ontogeny uses a combination of genes normally responsible for forming new shoots. The most primitive flowers probably had a variable number of flower parts, often separate from (but in contact with) each other. The flowers tended to grow in a spiral pattern, to be bisexual (in plants, this means both male and female parts on the same flower), and to be dominated by the ovary (female part). As flowers evolved, some variations developed parts fused together, with a much more specific number and design, and with either specific sexes per flower or plant or at least "ovary-inferior".', 'Flower ontogeny uses a combination of genes normally responsible for forming new shoots. The most primitive flowers probably had a variable number of flower parts, often separate from (but in contact with) each other. The flowers tended to grow in a spiral pattern, to be bisexual (in plants, this means both male and female parts on the same flower), and to be dominated by the ovary (female part). As flowers evolved, some variations developed parts fused together, with a much more specific number and design, and with either specific sexes per flower or plant or at least "ovary-inferior".', 'Flower evolution continues to the present day; modern flowers have been so profoundly influenced by humans that some of them cannot be pollinated in nature. Many modern domesticated flower species were formerly simple weeds, which sprouted only when the ground was disturbed. Some of them tended to grow with human crops, perhaps already having symbiotic companion plant relationships with them, and the prettiest did not get plucked because of their beauty, developing a dependence upon and special adaptation to human affection.', 'Flower evolution continues to the present day; modern flowers have been so profoundly influenced by humans that some of them cannot be pollinated in nature. Many modern domesticated flower species were formerly simple weeds, which sprouted only when the ground was disturbed. Some of them tended to grow with human crops, perhaps already having symbiotic companion plant relationships with them, and the prettiest did not get plucked because of their beauty, developing a dependence upon and special adaptation to human affection.', 'Flower evolution continues to the present day; modern flowers have been so profoundly influenced by humans that some of them cannot be pollinated in nature. Many modern domesticated flower species were formerly simple weeds, which sprouted only when the ground was disturbed. Some of them tended to grow with human crops, perhaps already having symbiotic companion plant relationships with them, and the prettiest did not get plucked because of their beauty, developing a dependence upon and special adaptation to human affection.', 'Flower evolution continues to the present day; modern flowers have been so profoundly influenced by humans that some of them cannot be pollinated in nature. Many modern domesticated flower species were formerly simple weeds, which sprouted only when the ground was disturbed. Some of them tended to grow with human crops, perhaps already having symbiotic companion plant relationships with them, and the prettiest did not get plucked because of their beauty, developing a dependence upon and special adaptation to human affection.', 'Flower evolution continues to the present day; modern flowers have been so profoundly influenced by humans that some of them cannot be pollinated in nature. Many modern domesticated flower species were formerly simple weeds, which sprouted only when the ground was disturbed. Some of them tended to grow with human crops, perhaps already having symbiotic companion plant relationships with them, and the prettiest did not get plucked because of their beauty, developing a dependence upon and special adaptation to human affection.', 'The exact relationship between these eight groups is not yet clear, although there is agreement that the first three groups to diverge from the ancestral angiosperm were Amborellales, Nymphaeales, and Austrobaileyales. The term basal angiosperms refers to these three groups. Among the rest, the relationship between the three broadest of these groups (magnoliids, monocots, and eudicots) remains unclear. Some analyses make the magnoliids the first to diverge, others the monocots. Ceratophyllum seems to group with the eudicots rather than with the monocots.', 'The exact relationship between these eight groups is not yet clear, although there is agreement that the first three groups to diverge from the ancestral angiosperm were Amborellales, Nymphaeales, and Austrobaileyales. The term basal angiosperms refers to these three groups. Among the rest, the relationship between the three broadest of these groups (magnoliids, monocots, and eudicots) remains unclear. Some analyses make the magnoliids the first to diverge, others the monocots. Ceratophyllum seems to group with the eudicots rather than with the monocots.', 'The exact relationship between these eight groups is not yet clear, although there is agreement that the first three groups to diverge from the ancestral angiosperm were Amborellales, Nymphaeales, and Austrobaileyales. The term basal angiosperms refers to these three groups. Among the rest, the relationship between the three broadest of these groups (magnoliids, monocots, and eudicots) remains unclear. Some analyses make the magnoliids the first to diverge, others the monocots. Ceratophyllum seems to group with the eudicots rather than with the monocots.', 'The exact relationship between these eight groups is not yet clear, although there is agreement that the first three groups to diverge from the ancestral angiosperm were Amborellales, Nymphaeales, and Austrobaileyales. The term basal angiosperms refers to these three groups. Among the rest, the relationship between the three broadest of these groups (magnoliids, monocots, and eudicots) remains unclear. Some analyses make the magnoliids the first to diverge, others the monocots. Ceratophyllum seems to group with the eudicots rather than with the monocots.', 'The botanical term "Angiosperm", from the Ancient Greek αγγείον, angeíon (bottle, vessel) and σπέρμα, (seed), was coined in the form Angiospermae by Paul Hermann in 1690, as the name of one of his primary divisions of the plant kingdom. This included flowering plants possessing seeds enclosed in capsules, distinguished from his Gymnospermae, or flowering plants with achenial or schizo-carpic fruits, the whole fruit or each of its pieces being here regarded as a seed and naked. The term and its antonym were maintained by Carl Linnaeus with the same sense, but with restricted application, in the names of the orders of his class Didynamia. Its use with any approach to its modern scope became possible only after 1827, when Robert Brown established the existence of truly naked ovules in the Cycadeae and Coniferae, and applied to them the name Gymnosperms.[citation needed] From that time onward, as long as these Gymnosperms were, as was usual, reckoned as dicotyledonous flowering plants, the term Angiosperm was used antithetically by botanical writers, with varying scope, as a group-name for other dicotyledonous plants.', 'The botanical term "Angiosperm", from the Ancient Greek αγγείον, angeíon (bottle, vessel) and σπέρμα, (seed), was coined in the form Angiospermae by Paul Hermann in 1690, as the name of one of his primary divisions of the plant kingdom. This included flowering plants possessing seeds enclosed in capsules, distinguished from his Gymnospermae, or flowering plants with achenial or schizo-carpic fruits, the whole fruit or each of its pieces being here regarded as a seed and naked. The term and its antonym were maintained by Carl Linnaeus with the same sense, but with restricted application, in the names of the orders of his class Didynamia. Its use with any approach to its modern scope became possible only after 1827, when Robert Brown established the existence of truly naked ovules in the Cycadeae and Coniferae, and applied to them the name Gymnosperms.[citation needed] From that time onward, as long as these Gymnosperms were, as was usual, reckoned as dicotyledonous flowering plants, the term Angiosperm was used antithetically by botanical writers, with varying scope, as a group-name for other dicotyledonous plants.', 'The botanical term "Angiosperm", from the Ancient Greek αγγείον, angeíon (bottle, vessel) and σπέρμα, (seed), was coined in the form Angiospermae by Paul Hermann in 1690, as the name of one of his primary divisions of the plant kingdom. This included flowering plants possessing seeds enclosed in capsules, distinguished from his Gymnospermae, or flowering plants with achenial or schizo-carpic fruits, the whole fruit or each of its pieces being here regarded as a seed and naked. The term and its antonym were maintained by Carl Linnaeus with the same sense, but with restricted application, in the names of the orders of his class Didynamia. Its use with any approach to its modern scope became possible only after 1827, when Robert Brown established the existence of truly naked ovules in the Cycadeae and Coniferae, and applied to them the name Gymnosperms.[citation needed] From that time onward, as long as these Gymnosperms were, as was usual, reckoned as dicotyledonous flowering plants, the term Angiosperm was used antithetically by botanical writers, with varying scope, as a group-name for other dicotyledonous plants.', 'The botanical term "Angiosperm", from the Ancient Greek αγγείον, angeíon (bottle, vessel) and σπέρμα, (seed), was coined in the form Angiospermae by Paul Hermann in 1690, as the name of one of his primary divisions of the plant kingdom. This included flowering plants possessing seeds enclosed in capsules, distinguished from his Gymnospermae, or flowering plants with achenial or schizo-carpic fruits, the whole fruit or each of its pieces being here regarded as a seed and naked. The term and its antonym were maintained by Carl Linnaeus with the same sense, but with restricted application, in the names of the orders of his class Didynamia. Its use with any approach to its modern scope became possible only after 1827, when Robert Brown established the existence of truly naked ovules in the Cycadeae and Coniferae, and applied to them the name Gymnosperms.[citation needed] From that time onward, as long as these Gymnosperms were, as was usual, reckoned as dicotyledonous flowering plants, the term Angiosperm was used antithetically by botanical writers, with varying scope, as a group-name for other dicotyledonous plants.', 'The botanical term "Angiosperm", from the Ancient Greek αγγείον, angeíon (bottle, vessel) and σπέρμα, (seed), was coined in the form Angiospermae by Paul Hermann in 1690, as the name of one of his primary divisions of the plant kingdom. This included flowering plants possessing seeds enclosed in capsules, distinguished from his Gymnospermae, or flowering plants with achenial or schizo-carpic fruits, the whole fruit or each of its pieces being here regarded as a seed and naked. The term and its antonym were maintained by Carl Linnaeus with the same sense, but with restricted application, in the names of the orders of his class Didynamia. Its use with any approach to its modern scope became possible only after 1827, when Robert Brown established the existence of truly naked ovules in the Cycadeae and Coniferae, and applied to them the name Gymnosperms.[citation needed] From that time onward, as long as these Gymnosperms were, as was usual, reckoned as dicotyledonous flowering plants, the term Angiosperm was used antithetically by botanical writers, with varying scope, as a group-name for other dicotyledonous plants.', 'In most taxonomies, the flowering plants are treated as a coherent group. The most popular descriptive name has been Angiospermae (Angiosperms), with Anthophyta ("flowering plants") a second choice. These names are not linked to any rank. The Wettstein system and the Engler system use the name Angiospermae, at the assigned rank of subdivision. The Reveal system treated flowering plants as subdivision Magnoliophytina (Frohne & U. Jensen ex Reveal, Phytologia 79: 70 1996), but later split it to Magnoliopsida, Liliopsida, and Rosopsida. The Takhtajan system and Cronquist system treat this group at the rank of division, leading to the name Magnoliophyta (from the family name Magnoliaceae). The Dahlgren system and Thorne system (1992) treat this group at the rank of class, leading to the name Magnoliopsida. The APG system of 1998, and the later 2003 and 2009 revisions, treat the flowering plants as a clade called angiosperms without a formal botanical name. However, a formal classification was published alongside the 2009 revision in which the flowering plants form the Subclass Magnoliidae.', 'In most taxonomies, the flowering plants are treated as a coherent group. The most popular descriptive name has been Angiospermae (Angiosperms), with Anthophyta ("flowering plants") a second choice. These names are not linked to any rank. The Wettstein system and the Engler system use the name Angiospermae, at the assigned rank of subdivision. The Reveal system treated flowering plants as subdivision Magnoliophytina (Frohne & U. Jensen ex Reveal, Phytologia 79: 70 1996), but later split it to Magnoliopsida, Liliopsida, and Rosopsida. The Takhtajan system and Cronquist system treat this group at the rank of division, leading to the name Magnoliophyta (from the family name Magnoliaceae). The Dahlgren system and Thorne system (1992) treat this group at the rank of class, leading to the name Magnoliopsida. The APG system of 1998, and the later 2003 and 2009 revisions, treat the flowering plants as a clade called angiosperms without a formal botanical name. However, a formal classification was published alongside the 2009 revision in which the flowering plants form the Subclass Magnoliidae.', 'In most taxonomies, the flowering plants are treated as a coherent group. The most popular descriptive name has been Angiospermae (Angiosperms), with Anthophyta ("flowering plants") a second choice. These names are not linked to any rank. The Wettstein system and the Engler system use the name Angiospermae, at the assigned rank of subdivision. The Reveal system treated flowering plants as subdivision Magnoliophytina (Frohne & U. Jensen ex Reveal, Phytologia 79: 70 1996), but later split it to Magnoliopsida, Liliopsida, and Rosopsida. The Takhtajan system and Cronquist system treat this group at the rank of division, leading to the name Magnoliophyta (from the family name Magnoliaceae). The Dahlgren system and Thorne system (1992) treat this group at the rank of class, leading to the name Magnoliopsida. The APG system of 1998, and the later 2003 and 2009 revisions, treat the flowering plants as a clade called angiosperms without a formal botanical name. However, a formal classification was published alongside the 2009 revision in which the flowering plants form the Subclass Magnoliidae.', 'In most taxonomies, the flowering plants are treated as a coherent group. The most popular descriptive name has been Angiospermae (Angiosperms), with Anthophyta ("flowering plants") a second choice. These names are not linked to any rank. The Wettstein system and the Engler system use the name Angiospermae, at the assigned rank of subdivision. The Reveal system treated flowering plants as subdivision Magnoliophytina (Frohne & U. Jensen ex Reveal, Phytologia 79: 70 1996), but later split it to Magnoliopsida, Liliopsida, and Rosopsida. The Takhtajan system and Cronquist system treat this group at the rank of division, leading to the name Magnoliophyta (from the family name Magnoliaceae). The Dahlgren system and Thorne system (1992) treat this group at the rank of class, leading to the name Magnoliopsida. The APG system of 1998, and the later 2003 and 2009 revisions, treat the flowering plants as a clade called angiosperms without a formal botanical name. However, a formal classification was published alongside the 2009 revision in which the flowering plants form the Subclass Magnoliidae.', 'In most taxonomies, the flowering plants are treated as a coherent group. The most popular descriptive name has been Angiospermae (Angiosperms), with Anthophyta ("flowering plants") a second choice. These names are not linked to any rank. The Wettstein system and the Engler system use the name Angiospermae, at the assigned rank of subdivision. The Reveal system treated flowering plants as subdivision Magnoliophytina (Frohne & U. Jensen ex Reveal, Phytologia 79: 70 1996), but later split it to Magnoliopsida, Liliopsida, and Rosopsida. The Takhtajan system and Cronquist system treat this group at the rank of division, leading to the name Magnoliophyta (from the family name Magnoliaceae). The Dahlgren system and Thorne system (1992) treat this group at the rank of class, leading to the name Magnoliopsida. The APG system of 1998, and the later 2003 and 2009 revisions, treat the flowering plants as a clade called angiosperms without a formal botanical name. However, a formal classification was published alongside the 2009 revision in which the flowering plants form the Subclass Magnoliidae.', 'The internal classification of this group has undergone considerable revision. The Cronquist system, proposed by Arthur Cronquist in 1968 and published in its full form in 1981, is still widely used but is no longer believed to accurately reflect phylogeny. A consensus about how the flowering plants should be arranged has recently begun to emerge through the work of the Angiosperm Phylogeny Group (APG), which published an influential reclassification of the angiosperms in 1998. Updates incorporating more recent research were published as APG II in 2003 and as APG III in 2009.', 'The internal classification of this group has undergone considerable revision. The Cronquist system, proposed by Arthur Cronquist in 1968 and published in its full form in 1981, is still widely used but is no longer believed to accurately reflect phylogeny. A consensus about how the flowering plants should be arranged has recently begun to emerge through the work of the Angiosperm Phylogeny Group (APG), which published an influential reclassification of the angiosperms in 1998. Updates incorporating more recent research were published as APG II in 2003 and as APG III in 2009.', 'The internal classification of this group has undergone considerable revision. The Cronquist system, proposed by Arthur Cronquist in 1968 and published in its full form in 1981, is still widely used but is no longer believed to accurately reflect phylogeny. A consensus about how the flowering plants should be arranged has recently begun to emerge through the work of the Angiosperm Phylogeny Group (APG), which published an influential reclassification of the angiosperms in 1998. Updates incorporating more recent research were published as APG II in 2003 and as APG III in 2009.', 'The internal classification of this group has undergone considerable revision. The Cronquist system, proposed by Arthur Cronquist in 1968 and published in its full form in 1981, is still widely used but is no longer believed to accurately reflect phylogeny. A consensus about how the flowering plants should be arranged has recently begun to emerge through the work of the Angiosperm Phylogeny Group (APG), which published an influential reclassification of the angiosperms in 1998. Updates incorporating more recent research were published as APG II in 2003 and as APG III in 2009.', 'Recent studies, as by the APG, show that the monocots form a monophyletic group (clade) but that the dicots do not (they are paraphyletic). Nevertheless, the majority of dicot species do form a monophyletic group, called the eudicots or tricolpates. Of the remaining dicot species, most belong to a third major clade known as the magnoliids, containing about 9,000 species. The rest include a paraphyletic grouping of primitive species known collectively as the basal angiosperms, plus the families Ceratophyllaceae and Chloranthaceae.', 'Recent studies, as by the APG, show that the monocots form a monophyletic group (clade) but that the dicots do not (they are paraphyletic). Nevertheless, the majority of dicot species do form a monophyletic group, called the eudicots or tricolpates. Of the remaining dicot species, most belong to a third major clade known as the magnoliids, containing about 9,000 species. The rest include a paraphyletic grouping of primitive species known collectively as the basal angiosperms, plus the families Ceratophyllaceae and Chloranthaceae.', 'Recent studies, as by the APG, show that the monocots form a monophyletic group (clade) but that the dicots do not (they are paraphyletic). Nevertheless, the majority of dicot species do form a monophyletic group, called the eudicots or tricolpates. Of the remaining dicot species, most belong to a third major clade known as the magnoliids, containing about 9,000 species. The rest include a paraphyletic grouping of primitive species known collectively as the basal angiosperms, plus the families Ceratophyllaceae and Chloranthaceae.', 'Recent studies, as by the APG, show that the monocots form a monophyletic group (clade) but that the dicots do not (they are paraphyletic). Nevertheless, the majority of dicot species do form a monophyletic group, called the eudicots or tricolpates. Of the remaining dicot species, most belong to a third major clade known as the magnoliids, containing about 9,000 species. The rest include a paraphyletic grouping of primitive species known collectively as the basal angiosperms, plus the families Ceratophyllaceae and Chloranthaceae.', 'Recent studies, as by the APG, show that the monocots form a monophyletic group (clade) but that the dicots do not (they are paraphyletic). Nevertheless, the majority of dicot species do form a monophyletic group, called the eudicots or tricolpates. Of the remaining dicot species, most belong to a third major clade known as the magnoliids, containing about 9,000 species. The rest include a paraphyletic grouping of primitive species known collectively as the basal angiosperms, plus the families Ceratophyllaceae and Chloranthaceae.', 'The number of species of flowering plants is estimated to be in the range of 250,000 to 400,000. This compares to around 12,000 species of moss or 11,000 species of pteridophytes, showing that the flowering plants are much more diverse. The number of families in APG (1998) was 462. In APG II (2003) it is not settled; at maximum it is 457, but within this number there are 55 optional segregates, so that the minimum number of families in this system is 402. In APG III (2009) there are 415 families.', 'The number of species of flowering plants is estimated to be in the range of 250,000 to 400,000. This compares to around 12,000 species of moss or 11,000 species of pteridophytes, showing that the flowering plants are much more diverse. The number of families in APG (1998) was 462. In APG II (2003) it is not settled; at maximum it is 457, but within this number there are 55 optional segregates, so that the minimum number of families in this system is 402. In APG III (2009) there are 415 families.', 'The number of species of flowering plants is estimated to be in the range of 250,000 to 400,000. This compares to around 12,000 species of moss or 11,000 species of pteridophytes, showing that the flowering plants are much more diverse. The number of families in APG (1998) was 462. In APG II (2003) it is not settled; at maximum it is 457, but within this number there are 55 optional segregates, so that the minimum number of families in this system is 402. In APG III (2009) there are 415 families.', 'The number of species of flowering plants is estimated to be in the range of 250,000 to 400,000. This compares to around 12,000 species of moss or 11,000 species of pteridophytes, showing that the flowering plants are much more diverse. The number of families in APG (1998) was 462. In APG II (2003) it is not settled; at maximum it is 457, but within this number there are 55 optional segregates, so that the minimum number of families in this system is 402. In APG III (2009) there are 415 families.', 'The number of species of flowering plants is estimated to be in the range of 250,000 to 400,000. This compares to around 12,000 species of moss or 11,000 species of pteridophytes, showing that the flowering plants are much more diverse. The number of families in APG (1998) was 462. In APG II (2003) it is not settled; at maximum it is 457, but within this number there are 55 optional segregates, so that the minimum number of families in this system is 402. In APG III (2009) there are 415 families.', 'In the dicotyledons, the bundles in the very young stem are arranged in an open ring, separating a central pith from an outer cortex. In each bundle, separating the xylem and phloem, is a layer of meristem or active formative tissue known as cambium. By the formation of a layer of cambium between the bundles (interfascicular cambium), a complete ring is formed, and a regular periodical increase in thickness results from the development of xylem on the inside and phloem on the outside. The soft phloem becomes crushed, but the hard wood persists and forms the bulk of the stem and branches of the woody perennial. Owing to differences in the character of the elements produced at the beginning and end of the season, the wood is marked out in transverse section into concentric rings, one for each season of growth, called annual rings.', 'In the dicotyledons, the bundles in the very young stem are arranged in an open ring, separating a central pith from an outer cortex. In each bundle, separating the xylem and phloem, is a layer of meristem or active formative tissue known as cambium. By the formation of a layer of cambium between the bundles (interfascicular cambium), a complete ring is formed, and a regular periodical increase in thickness results from the development of xylem on the inside and phloem on the outside. The soft phloem becomes crushed, but the hard wood persists and forms the bulk of the stem and branches of the woody perennial. Owing to differences in the character of the elements produced at the beginning and end of the season, the wood is marked out in transverse section into concentric rings, one for each season of growth, called annual rings.', 'In the dicotyledons, the bundles in the very young stem are arranged in an open ring, separating a central pith from an outer cortex. In each bundle, separating the xylem and phloem, is a layer of meristem or active formative tissue known as cambium. By the formation of a layer of cambium between the bundles (interfascicular cambium), a complete ring is formed, and a regular periodical increase in thickness results from the development of xylem on the inside and phloem on the outside. The soft phloem becomes crushed, but the hard wood persists and forms the bulk of the stem and branches of the woody perennial. Owing to differences in the character of the elements produced at the beginning and end of the season, the wood is marked out in transverse section into concentric rings, one for each season of growth, called annual rings.', 'In the dicotyledons, the bundles in the very young stem are arranged in an open ring, separating a central pith from an outer cortex. In each bundle, separating the xylem and phloem, is a layer of meristem or active formative tissue known as cambium. By the formation of a layer of cambium between the bundles (interfascicular cambium), a complete ring is formed, and a regular periodical increase in thickness results from the development of xylem on the inside and phloem on the outside. The soft phloem becomes crushed, but the hard wood persists and forms the bulk of the stem and branches of the woody perennial. Owing to differences in the character of the elements produced at the beginning and end of the season, the wood is marked out in transverse section into concentric rings, one for each season of growth, called annual rings.', 'In the dicotyledons, the bundles in the very young stem are arranged in an open ring, separating a central pith from an outer cortex. In each bundle, separating the xylem and phloem, is a layer of meristem or active formative tissue known as cambium. By the formation of a layer of cambium between the bundles (interfascicular cambium), a complete ring is formed, and a regular periodical increase in thickness results from the development of xylem on the inside and phloem on the outside. The soft phloem becomes crushed, but the hard wood persists and forms the bulk of the stem and branches of the woody perennial. Owing to differences in the character of the elements produced at the beginning and end of the season, the wood is marked out in transverse section into concentric rings, one for each season of growth, called annual rings.', 'The characteristic feature of angiosperms is the flower. Flowers show remarkable variation in form and elaboration, and provide the most trustworthy external characteristics for establishing relationships among angiosperm species. The function of the flower is to ensure fertilization of the ovule and development of fruit containing seeds. The floral apparatus may arise terminally on a shoot or from the axil of a leaf (where the petiole attaches to the stem). Occasionally, as in violets, a flower arises singly in the axil of an ordinary foliage-leaf. More typically, the flower-bearing portion of the plant is sharply distinguished from the foliage-bearing or vegetative portion, and forms a more or less elaborate branch-system called an inflorescence.', 'The characteristic feature of angiosperms is the flower. Flowers show remarkable variation in form and elaboration, and provide the most trustworthy external characteristics for establishing relationships among angiosperm species. The function of the flower is to ensure fertilization of the ovule and development of fruit containing seeds. The floral apparatus may arise terminally on a shoot or from the axil of a leaf (where the petiole attaches to the stem). Occasionally, as in violets, a flower arises singly in the axil of an ordinary foliage-leaf. More typically, the flower-bearing portion of the plant is sharply distinguished from the foliage-bearing or vegetative portion, and forms a more or less elaborate branch-system called an inflorescence.', 'The characteristic feature of angiosperms is the flower. Flowers show remarkable variation in form and elaboration, and provide the most trustworthy external characteristics for establishing relationships among angiosperm species. The function of the flower is to ensure fertilization of the ovule and development of fruit containing seeds. The floral apparatus may arise terminally on a shoot or from the axil of a leaf (where the petiole attaches to the stem). Occasionally, as in violets, a flower arises singly in the axil of an ordinary foliage-leaf. More typically, the flower-bearing portion of the plant is sharply distinguished from the foliage-bearing or vegetative portion, and forms a more or less elaborate branch-system called an inflorescence.', 'The characteristic feature of angiosperms is the flower. Flowers show remarkable variation in form and elaboration, and provide the most trustworthy external characteristics for establishing relationships among angiosperm species. The function of the flower is to ensure fertilization of the ovule and development of fruit containing seeds. The floral apparatus may arise terminally on a shoot or from the axil of a leaf (where the petiole attaches to the stem). Occasionally, as in violets, a flower arises singly in the axil of an ordinary foliage-leaf. More typically, the flower-bearing portion of the plant is sharply distinguished from the foliage-bearing or vegetative portion, and forms a more or less elaborate branch-system called an inflorescence.', 'The characteristic feature of angiosperms is the flower. Flowers show remarkable variation in form and elaboration, and provide the most trustworthy external characteristics for establishing relationships among angiosperm species. The function of the flower is to ensure fertilization of the ovule and development of fruit containing seeds. The floral apparatus may arise terminally on a shoot or from the axil of a leaf (where the petiole attaches to the stem). Occasionally, as in violets, a flower arises singly in the axil of an ordinary foliage-leaf. More typically, the flower-bearing portion of the plant is sharply distinguished from the foliage-bearing or vegetative portion, and forms a more or less elaborate branch-system called an inflorescence.', 'The flower may consist only of these parts, as in willow, where each flower comprises only a few stamens or two carpels. Usually, other structures are present and serve to protect the sporophylls and to form an envelope attractive to pollinators. The individual members of these surrounding structures are known as sepals and petals (or tepals in flowers such as Magnolia where sepals and petals are not distinguishable from each other). The outer series (calyx of sepals) is usually green and leaf-like, and functions to protect the rest of the flower, especially the bud. The inner series (corolla of petals) is, in general, white or brightly colored, and is more delicate in structure. It functions to attract insect or bird pollinators. Attraction is effected by color, scent, and nectar, which may be secreted in some part of the flower. The characteristics that attract pollinators account for the popularity of flowers and flowering plants among humans.', 'The flower may consist only of these parts, as in willow, where each flower comprises only a few stamens or two carpels. Usually, other structures are present and serve to protect the sporophylls and to form an envelope attractive to pollinators. The individual members of these surrounding structures are known as sepals and petals (or tepals in flowers such as Magnolia where sepals and petals are not distinguishable from each other). The outer series (calyx of sepals) is usually green and leaf-like, and functions to protect the rest of the flower, especially the bud. The inner series (corolla of petals) is, in general, white or brightly colored, and is more delicate in structure. It functions to attract insect or bird pollinators. Attraction is effected by color, scent, and nectar, which may be secreted in some part of the flower. The characteristics that attract pollinators account for the popularity of flowers and flowering plants among humans.', 'The flower may consist only of these parts, as in willow, where each flower comprises only a few stamens or two carpels. Usually, other structures are present and serve to protect the sporophylls and to form an envelope attractive to pollinators. The individual members of these surrounding structures are known as sepals and petals (or tepals in flowers such as Magnolia where sepals and petals are not distinguishable from each other). The outer series (calyx of sepals) is usually green and leaf-like, and functions to protect the rest of the flower, especially the bud. The inner series (corolla of petals) is, in general, white or brightly colored, and is more delicate in structure. It functions to attract insect or bird pollinators. Attraction is effected by color, scent, and nectar, which may be secreted in some part of the flower. The characteristics that attract pollinators account for the popularity of flowers and flowering plants among humans.', 'The flower may consist only of these parts, as in willow, where each flower comprises only a few stamens or two carpels. Usually, other structures are present and serve to protect the sporophylls and to form an envelope attractive to pollinators. The individual members of these surrounding structures are known as sepals and petals (or tepals in flowers such as Magnolia where sepals and petals are not distinguishable from each other). The outer series (calyx of sepals) is usually green and leaf-like, and functions to protect the rest of the flower, especially the bud. The inner series (corolla of petals) is, in general, white or brightly colored, and is more delicate in structure. It functions to attract insect or bird pollinators. Attraction is effected by color, scent, and nectar, which may be secreted in some part of the flower. The characteristics that attract pollinators account for the popularity of flowers and flowering plants among humans.', 'The flower may consist only of these parts, as in willow, where each flower comprises only a few stamens or two carpels. Usually, other structures are present and serve to protect the sporophylls and to form an envelope attractive to pollinators. The individual members of these surrounding structures are known as sepals and petals (or tepals in flowers such as Magnolia where sepals and petals are not distinguishable from each other). The outer series (calyx of sepals) is usually green and leaf-like, and functions to protect the rest of the flower, especially the bud. The inner series (corolla of petals) is, in general, white or brightly colored, and is more delicate in structure. It functions to attract insect or bird pollinators. Attraction is effected by color, scent, and nectar, which may be secreted in some part of the flower. The characteristics that attract pollinators account for the popularity of flowers and flowering plants among humans.', 'While the majority of flowers are perfect or hermaphrodite (having both pollen and ovule producing parts in the same flower structure), flowering plants have developed numerous morphological and physiological mechanisms to reduce or prevent self-fertilization. Heteromorphic flowers have short carpels and long stamens, or vice versa, so animal pollinators cannot easily transfer pollen to the pistil (receptive part of the carpel). Homomorphic flowers may employ a biochemical (physiological) mechanism called self-incompatibility to discriminate between self and non-self pollen grains. In other species, the male and female parts are morphologically separated, developing on different flowers.', 'While the majority of flowers are perfect or hermaphrodite (having both pollen and ovule producing parts in the same flower structure), flowering plants have developed numerous morphological and physiological mechanisms to reduce or prevent self-fertilization. Heteromorphic flowers have short carpels and long stamens, or vice versa, so animal pollinators cannot easily transfer pollen to the pistil (receptive part of the carpel). Homomorphic flowers may employ a biochemical (physiological) mechanism called self-incompatibility to discriminate between self and non-self pollen grains. In other species, the male and female parts are morphologically separated, developing on different flowers.', 'While the majority of flowers are perfect or hermaphrodite (having both pollen and ovule producing parts in the same flower structure), flowering plants have developed numerous morphological and physiological mechanisms to reduce or prevent self-fertilization. Heteromorphic flowers have short carpels and long stamens, or vice versa, so animal pollinators cannot easily transfer pollen to the pistil (receptive part of the carpel). Homomorphic flowers may employ a biochemical (physiological) mechanism called self-incompatibility to discriminate between self and non-self pollen grains. In other species, the male and female parts are morphologically separated, developing on different flowers.', 'While the majority of flowers are perfect or hermaphrodite (having both pollen and ovule producing parts in the same flower structure), flowering plants have developed numerous morphological and physiological mechanisms to reduce or prevent self-fertilization. Heteromorphic flowers have short carpels and long stamens, or vice versa, so animal pollinators cannot easily transfer pollen to the pistil (receptive part of the carpel). Homomorphic flowers may employ a biochemical (physiological) mechanism called self-incompatibility to discriminate between self and non-self pollen grains. In other species, the male and female parts are morphologically separated, developing on different flowers.', 'While the majority of flowers are perfect or hermaphrodite (having both pollen and ovule producing parts in the same flower structure), flowering plants have developed numerous morphological and physiological mechanisms to reduce or prevent self-fertilization. Heteromorphic flowers have short carpels and long stamens, or vice versa, so animal pollinators cannot easily transfer pollen to the pistil (receptive part of the carpel). Homomorphic flowers may employ a biochemical (physiological) mechanism called self-incompatibility to discriminate between self and non-self pollen grains. In other species, the male and female parts are morphologically separated, developing on different flowers.', "Double fertilization refers to a process in which two sperm cells fertilize cells in the ovary. This process begins when a pollen grain adheres to the stigma of the pistil (female reproductive structure), germinates, and grows a long pollen tube. While this pollen tube is growing, a haploid generative cell travels down the tube behind the tube nucleus. The generative cell divides by mitosis to produce two haploid (n) sperm cells. As the pollen tube grows, it makes its way from the stigma, down the style and into the ovary. Here the pollen tube reaches the micropyle of the ovule and digests its way into one of the synergids, releasing its contents (which include the sperm cells). The synergid that the cells were released into degenerates and one sperm makes its way to fertilize the egg cell, producing a diploid (2n) zygote. The second sperm cell fuses with both central cell nuclei, producing a triploid (3n) cell. As the zygote develops into an embryo, the triploid cell develops into the endosperm, which serves as the embryo's food supply. The ovary now will develop into fruit and the ovule will develop into seed.", "Double fertilization refers to a process in which two sperm cells fertilize cells in the ovary. This process begins when a pollen grain adheres to the stigma of the pistil (female reproductive structure), germinates, and grows a long pollen tube. While this pollen tube is growing, a haploid generative cell travels down the tube behind the tube nucleus. The generative cell divides by mitosis to produce two haploid (n) sperm cells. As the pollen tube grows, it makes its way from the stigma, down the style and into the ovary. Here the pollen tube reaches the micropyle of the ovule and digests its way into one of the synergids, releasing its contents (which include the sperm cells). The synergid that the cells were released into degenerates and one sperm makes its way to fertilize the egg cell, producing a diploid (2n) zygote. The second sperm cell fuses with both central cell nuclei, producing a triploid (3n) cell. As the zygote develops into an embryo, the triploid cell develops into the endosperm, which serves as the embryo's food supply. The ovary now will develop into fruit and the ovule will develop into seed.", "Double fertilization refers to a process in which two sperm cells fertilize cells in the ovary. This process begins when a pollen grain adheres to the stigma of the pistil (female reproductive structure), germinates, and grows a long pollen tube. While this pollen tube is growing, a haploid generative cell travels down the tube behind the tube nucleus. The generative cell divides by mitosis to produce two haploid (n) sperm cells. As the pollen tube grows, it makes its way from the stigma, down the style and into the ovary. Here the pollen tube reaches the micropyle of the ovule and digests its way into one of the synergids, releasing its contents (which include the sperm cells). The synergid that the cells were released into degenerates and one sperm makes its way to fertilize the egg cell, producing a diploid (2n) zygote. The second sperm cell fuses with both central cell nuclei, producing a triploid (3n) cell. As the zygote develops into an embryo, the triploid cell develops into the endosperm, which serves as the embryo's food supply. The ovary now will develop into fruit and the ovule will develop into seed.", "Double fertilization refers to a process in which two sperm cells fertilize cells in the ovary. This process begins when a pollen grain adheres to the stigma of the pistil (female reproductive structure), germinates, and grows a long pollen tube. While this pollen tube is growing, a haploid generative cell travels down the tube behind the tube nucleus. The generative cell divides by mitosis to produce two haploid (n) sperm cells. As the pollen tube grows, it makes its way from the stigma, down the style and into the ovary. Here the pollen tube reaches the micropyle of the ovule and digests its way into one of the synergids, releasing its contents (which include the sperm cells). The synergid that the cells were released into degenerates and one sperm makes its way to fertilize the egg cell, producing a diploid (2n) zygote. The second sperm cell fuses with both central cell nuclei, producing a triploid (3n) cell. As the zygote develops into an embryo, the triploid cell develops into the endosperm, which serves as the embryo's food supply. The ovary now will develop into fruit and the ovule will develop into seed.", "Double fertilization refers to a process in which two sperm cells fertilize cells in the ovary. This process begins when a pollen grain adheres to the stigma of the pistil (female reproductive structure), germinates, and grows a long pollen tube. While this pollen tube is growing, a haploid generative cell travels down the tube behind the tube nucleus. The generative cell divides by mitosis to produce two haploid (n) sperm cells. As the pollen tube grows, it makes its way from the stigma, down the style and into the ovary. Here the pollen tube reaches the micropyle of the ovule and digests its way into one of the synergids, releasing its contents (which include the sperm cells). The synergid that the cells were released into degenerates and one sperm makes its way to fertilize the egg cell, producing a diploid (2n) zygote. The second sperm cell fuses with both central cell nuclei, producing a triploid (3n) cell. As the zygote develops into an embryo, the triploid cell develops into the endosperm, which serves as the embryo's food supply. The ovary now will develop into fruit and the ovule will develop into seed.", 'The character of the seed coat bears a definite relation to that of the fruit. They protect the embryo and aid in dissemination; they may also directly promote germination. Among plants with indehiscent fruits, in general, the fruit provides protection for the embryo and secures dissemination. In this case, the seed coat is only slightly developed. If the fruit is dehiscent and the seed is exposed, in general, the seed-coat is well developed, and must discharge the functions otherwise executed by the fruit.', 'The character of the seed coat bears a definite relation to that of the fruit. They protect the embryo and aid in dissemination; they may also directly promote germination. Among plants with indehiscent fruits, in general, the fruit provides protection for the embryo and secures dissemination. In this case, the seed coat is only slightly developed. If the fruit is dehiscent and the seed is exposed, in general, the seed-coat is well developed, and must discharge the functions otherwise executed by the fruit.', 'The character of the seed coat bears a definite relation to that of the fruit. They protect the embryo and aid in dissemination; they may also directly promote germination. Among plants with indehiscent fruits, in general, the fruit provides protection for the embryo and secures dissemination. In this case, the seed coat is only slightly developed. If the fruit is dehiscent and the seed is exposed, in general, the seed-coat is well developed, and must discharge the functions otherwise executed by the fruit.', 'The character of the seed coat bears a definite relation to that of the fruit. They protect the embryo and aid in dissemination; they may also directly promote germination. Among plants with indehiscent fruits, in general, the fruit provides protection for the embryo and secures dissemination. In this case, the seed coat is only slightly developed. If the fruit is dehiscent and the seed is exposed, in general, the seed-coat is well developed, and must discharge the functions otherwise executed by the fruit.', 'Agriculture is almost entirely dependent on angiosperms, which provide virtually all plant-based food, and also provide a significant amount of livestock feed. Of all the families of plants, the Poaceae, or grass family (grains), is by far the most important, providing the bulk of all feedstocks (rice, corn — maize, wheat, barley, rye, oats, pearl millet, sugar cane, sorghum). The Fabaceae, or legume family, comes in second place. Also of high importance are the Solanaceae, or nightshade family (potatoes, tomatoes, and peppers, among others), the Cucurbitaceae, or gourd family (also including pumpkins and melons), the Brassicaceae, or mustard plant family (including rapeseed and the innumerable varieties of the cabbage species Brassica oleracea), and the Apiaceae, or parsley family. Many of our fruits come from the Rutaceae, or rue family (including oranges, lemons, grapefruits, etc.), and the Rosaceae, or rose family (including apples, pears, cherries, apricots, plums, etc.).', 'Agriculture is almost entirely dependent on angiosperms, which provide virtually all plant-based food, and also provide a significant amount of livestock feed. Of all the families of plants, the Poaceae, or grass family (grains), is by far the most important, providing the bulk of all feedstocks (rice, corn — maize, wheat, barley, rye, oats, pearl millet, sugar cane, sorghum). The Fabaceae, or legume family, comes in second place. Also of high importance are the Solanaceae, or nightshade family (potatoes, tomatoes, and peppers, among others), the Cucurbitaceae, or gourd family (also including pumpkins and melons), the Brassicaceae, or mustard plant family (including rapeseed and the innumerable varieties of the cabbage species Brassica oleracea), and the Apiaceae, or parsley family. Many of our fruits come from the Rutaceae, or rue family (including oranges, lemons, grapefruits, etc.), and the Rosaceae, or rose family (including apples, pears, cherries, apricots, plums, etc.).', 'Agriculture is almost entirely dependent on angiosperms, which provide virtually all plant-based food, and also provide a significant amount of livestock feed. Of all the families of plants, the Poaceae, or grass family (grains), is by far the most important, providing the bulk of all feedstocks (rice, corn — maize, wheat, barley, rye, oats, pearl millet, sugar cane, sorghum). The Fabaceae, or legume family, comes in second place. Also of high importance are the Solanaceae, or nightshade family (potatoes, tomatoes, and peppers, among others), the Cucurbitaceae, or gourd family (also including pumpkins and melons), the Brassicaceae, or mustard plant family (including rapeseed and the innumerable varieties of the cabbage species Brassica oleracea), and the Apiaceae, or parsley family. Many of our fruits come from the Rutaceae, or rue family (including oranges, lemons, grapefruits, etc.), and the Rosaceae, or rose family (including apples, pears, cherries, apricots, plums, etc.).', 'Agriculture is almost entirely dependent on angiosperms, which provide virtually all plant-based food, and also provide a significant amount of livestock feed. Of all the families of plants, the Poaceae, or grass family (grains), is by far the most important, providing the bulk of all feedstocks (rice, corn — maize, wheat, barley, rye, oats, pearl millet, sugar cane, sorghum). The Fabaceae, or legume family, comes in second place. Also of high importance are the Solanaceae, or nightshade family (potatoes, tomatoes, and peppers, among others), the Cucurbitaceae, or gourd family (also including pumpkins and melons), the Brassicaceae, or mustard plant family (including rapeseed and the innumerable varieties of the cabbage species Brassica oleracea), and the Apiaceae, or parsley family. Many of our fruits come from the Rutaceae, or rue family (including oranges, lemons, grapefruits, etc.), and the Rosaceae, or rose family (including apples, pears, cherries, apricots, plums, etc.).', 'Agriculture is almost entirely dependent on angiosperms, which provide virtually all plant-based food, and also provide a significant amount of livestock feed. Of all the families of plants, the Poaceae, or grass family (grains), is by far the most important, providing the bulk of all feedstocks (rice, corn — maize, wheat, barley, rye, oats, pearl millet, sugar cane, sorghum). The Fabaceae, or legume family, comes in second place. Also of high importance are the Solanaceae, or nightshade family (potatoes, tomatoes, and peppers, among others), the Cucurbitaceae, or gourd family (also including pumpkins and melons), the Brassicaceae, or mustard plant family (including rapeseed and the innumerable varieties of the cabbage species Brassica oleracea), and the Apiaceae, or parsley family. Many of our fruits come from the Rutaceae, or rue family (including oranges, lemons, grapefruits, etc.), and the Rosaceae, or rose family (including apples, pears, cherries, apricots, plums, etc.).', "Hyderabad (i/ˈhaɪdərəˌbæd/ HY-dər-ə-bad; often /ˈhaɪdrəˌbæd/) is the capital of the southern Indian state of Telangana and de jure capital of Andhra Pradesh.[A] Occupying 650 square kilometres (250 sq mi) along the banks of the Musi River, it has a population of about 6.7 million and a metropolitan population of about 7.75 million, making it the fourth most populous city and sixth most populous urban agglomeration in India. At an average altitude of 542 metres (1,778 ft), much of Hyderabad is situated on hilly terrain around artificial lakes, including Hussain Sagar—predating the city's founding—north of the city centre.", "Hyderabad (i/ˈhaɪdərəˌbæd/ HY-dər-ə-bad; often /ˈhaɪdrəˌbæd/) is the capital of the southern Indian state of Telangana and de jure capital of Andhra Pradesh.[A] Occupying 650 square kilometres (250 sq mi) along the banks of the Musi River, it has a population of about 6.7 million and a metropolitan population of about 7.75 million, making it the fourth most populous city and sixth most populous urban agglomeration in India. At an average altitude of 542 metres (1,778 ft), much of Hyderabad is situated on hilly terrain around artificial lakes, including Hussain Sagar—predating the city's founding—north of the city centre.", "Hyderabad (i/ˈhaɪdərəˌbæd/ HY-dər-ə-bad; often /ˈhaɪdrəˌbæd/) is the capital of the southern Indian state of Telangana and de jure capital of Andhra Pradesh.[A] Occupying 650 square kilometres (250 sq mi) along the banks of the Musi River, it has a population of about 6.7 million and a metropolitan population of about 7.75 million, making it the fourth most populous city and sixth most populous urban agglomeration in India. At an average altitude of 542 metres (1,778 ft), much of Hyderabad is situated on hilly terrain around artificial lakes, including Hussain Sagar—predating the city's founding—north of the city centre.", "Hyderabad (i/ˈhaɪdərəˌbæd/ HY-dər-ə-bad; often /ˈhaɪdrəˌbæd/) is the capital of the southern Indian state of Telangana and de jure capital of Andhra Pradesh.[A] Occupying 650 square kilometres (250 sq mi) along the banks of the Musi River, it has a population of about 6.7 million and a metropolitan population of about 7.75 million, making it the fourth most populous city and sixth most populous urban agglomeration in India. At an average altitude of 542 metres (1,778 ft), much of Hyderabad is situated on hilly terrain around artificial lakes, including Hussain Sagar—predating the city's founding—north of the city centre.", "Hyderabad (i/ˈhaɪdərəˌbæd/ HY-dər-ə-bad; often /ˈhaɪdrəˌbæd/) is the capital of the southern Indian state of Telangana and de jure capital of Andhra Pradesh.[A] Occupying 650 square kilometres (250 sq mi) along the banks of the Musi River, it has a population of about 6.7 million and a metropolitan population of about 7.75 million, making it the fourth most populous city and sixth most populous urban agglomeration in India. At an average altitude of 542 metres (1,778 ft), much of Hyderabad is situated on hilly terrain around artificial lakes, including Hussain Sagar—predating the city's founding—north of the city centre.", "Established in 1591 by Muhammad Quli Qutb Shah, Hyderabad remained under the rule of the Qutb Shahi dynasty for nearly a century before the Mughals captured the region. In 1724, Mughal viceroy Asif Jah I declared his sovereignty and created his own dynasty, known as the Nizams of Hyderabad. The Nizam's dominions became a princely state during the British Raj, and remained so for 150 years, with the city serving as its capital. The Nizami influence can still be seen in the culture of the Hyderabadi Muslims. The city continued as the capital of Hyderabad State after it was brought into the Indian Union in 1948, and became the capital of Andhra Pradesh after the States Reorganisation Act, 1956. Since 1956, Rashtrapati Nilayam in the city has been the winter office of the President of India. In 2014, the newly formed state of Telangana split from Andhra Pradesh and the city became joint capital of the two states, a transitional arrangement scheduled to end by 2025.", "Established in 1591 by Muhammad Quli Qutb Shah, Hyderabad remained under the rule of the Qutb Shahi dynasty for nearly a century before the Mughals captured the region. In 1724, Mughal viceroy Asif Jah I declared his sovereignty and created his own dynasty, known as the Nizams of Hyderabad. The Nizam's dominions became a princely state during the British Raj, and remained so for 150 years, with the city serving as its capital. The Nizami influence can still be seen in the culture of the Hyderabadi Muslims. The city continued as the capital of Hyderabad State after it was brought into the Indian Union in 1948, and became the capital of Andhra Pradesh after the States Reorganisation Act, 1956. Since 1956, Rashtrapati Nilayam in the city has been the winter office of the President of India. In 2014, the newly formed state of Telangana split from Andhra Pradesh and the city became joint capital of the two states, a transitional arrangement scheduled to end by 2025.", "Established in 1591 by Muhammad Quli Qutb Shah, Hyderabad remained under the rule of the Qutb Shahi dynasty for nearly a century before the Mughals captured the region. In 1724, Mughal viceroy Asif Jah I declared his sovereignty and created his own dynasty, known as the Nizams of Hyderabad. The Nizam's dominions became a princely state during the British Raj, and remained so for 150 years, with the city serving as its capital. The Nizami influence can still be seen in the culture of the Hyderabadi Muslims. The city continued as the capital of Hyderabad State after it was brought into the Indian Union in 1948, and became the capital of Andhra Pradesh after the States Reorganisation Act, 1956. Since 1956, Rashtrapati Nilayam in the city has been the winter office of the President of India. In 2014, the newly formed state of Telangana split from Andhra Pradesh and the city became joint capital of the two states, a transitional arrangement scheduled to end by 2025.", "Established in 1591 by Muhammad Quli Qutb Shah, Hyderabad remained under the rule of the Qutb Shahi dynasty for nearly a century before the Mughals captured the region. In 1724, Mughal viceroy Asif Jah I declared his sovereignty and created his own dynasty, known as the Nizams of Hyderabad. The Nizam's dominions became a princely state during the British Raj, and remained so for 150 years, with the city serving as its capital. The Nizami influence can still be seen in the culture of the Hyderabadi Muslims. The city continued as the capital of Hyderabad State after it was brought into the Indian Union in 1948, and became the capital of Andhra Pradesh after the States Reorganisation Act, 1956. Since 1956, Rashtrapati Nilayam in the city has been the winter office of the President of India. In 2014, the newly formed state of Telangana split from Andhra Pradesh and the city became joint capital of the two states, a transitional arrangement scheduled to end by 2025.", "Established in 1591 by Muhammad Quli Qutb Shah, Hyderabad remained under the rule of the Qutb Shahi dynasty for nearly a century before the Mughals captured the region. In 1724, Mughal viceroy Asif Jah I declared his sovereignty and created his own dynasty, known as the Nizams of Hyderabad. The Nizam's dominions became a princely state during the British Raj, and remained so for 150 years, with the city serving as its capital. The Nizami influence can still be seen in the culture of the Hyderabadi Muslims. The city continued as the capital of Hyderabad State after it was brought into the Indian Union in 1948, and became the capital of Andhra Pradesh after the States Reorganisation Act, 1956. Since 1956, Rashtrapati Nilayam in the city has been the winter office of the President of India. In 2014, the newly formed state of Telangana split from Andhra Pradesh and the city became joint capital of the two states, a transitional arrangement scheduled to end by 2025.", "Relics of Qutb Shahi and Nizam rule remain visible today, with the Charminar—commissioned by Muhammad Quli Qutb Shah—coming to symbolise Hyderabad. Golconda fort is another major landmark. The influence of Mughlai culture is also evident in the city's distinctive cuisine, which includes Hyderabadi biryani and Hyderabadi haleem. The Qutb Shahis and Nizams established Hyderabad as a cultural hub, attracting men of letters from different parts of the world. Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire in the mid-19th century, with artists migrating to the city from the rest of the Indian subcontinent. While Hyderabad is losing its cultural pre-eminence, it is today, due to the Telugu film industry, the country's second-largest producer of motion pictures.", "Relics of Qutb Shahi and Nizam rule remain visible today, with the Charminar—commissioned by Muhammad Quli Qutb Shah—coming to symbolise Hyderabad. Golconda fort is another major landmark. The influence of Mughlai culture is also evident in the city's distinctive cuisine, which includes Hyderabadi biryani and Hyderabadi haleem. The Qutb Shahis and Nizams established Hyderabad as a cultural hub, attracting men of letters from different parts of the world. Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire in the mid-19th century, with artists migrating to the city from the rest of the Indian subcontinent. While Hyderabad is losing its cultural pre-eminence, it is today, due to the Telugu film industry, the country's second-largest producer of motion pictures.", "Relics of Qutb Shahi and Nizam rule remain visible today, with the Charminar—commissioned by Muhammad Quli Qutb Shah—coming to symbolise Hyderabad. Golconda fort is another major landmark. The influence of Mughlai culture is also evident in the city's distinctive cuisine, which includes Hyderabadi biryani and Hyderabadi haleem. The Qutb Shahis and Nizams established Hyderabad as a cultural hub, attracting men of letters from different parts of the world. Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire in the mid-19th century, with artists migrating to the city from the rest of the Indian subcontinent. While Hyderabad is losing its cultural pre-eminence, it is today, due to the Telugu film industry, the country's second-largest producer of motion pictures.", "Relics of Qutb Shahi and Nizam rule remain visible today, with the Charminar—commissioned by Muhammad Quli Qutb Shah—coming to symbolise Hyderabad. Golconda fort is another major landmark. The influence of Mughlai culture is also evident in the city's distinctive cuisine, which includes Hyderabadi biryani and Hyderabadi haleem. The Qutb Shahis and Nizams established Hyderabad as a cultural hub, attracting men of letters from different parts of the world. Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire in the mid-19th century, with artists migrating to the city from the rest of the Indian subcontinent. While Hyderabad is losing its cultural pre-eminence, it is today, due to the Telugu film industry, the country's second-largest producer of motion pictures.", "Relics of Qutb Shahi and Nizam rule remain visible today, with the Charminar—commissioned by Muhammad Quli Qutb Shah—coming to symbolise Hyderabad. Golconda fort is another major landmark. The influence of Mughlai culture is also evident in the city's distinctive cuisine, which includes Hyderabadi biryani and Hyderabadi haleem. The Qutb Shahis and Nizams established Hyderabad as a cultural hub, attracting men of letters from different parts of the world. Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire in the mid-19th century, with artists migrating to the city from the rest of the Indian subcontinent. While Hyderabad is losing its cultural pre-eminence, it is today, due to the Telugu film industry, the country's second-largest producer of motion pictures.", 'Hyderabad was historically known as a pearl and diamond trading centre, and it continues to be known as the City of Pearls. Many of the city\'s traditional bazaars, including Laad Bazaar, Begum Bazaar and Sultan Bazaar, have remained open for centuries. However, industrialisation throughout the 20th century attracted major Indian manufacturing, research and financial institutions, including Bharat Heavy Electricals Limited, the National Geophysical Research Institute and the Centre for Cellular and Molecular Biology. Special economic zones dedicated to information technology have encouraged companies from across India and around the world to set up operations and the emergence of pharmaceutical and biotechnology industries in the 1990s led to the area\'s naming as India\'s "Genome Valley". With an output of US$74 billion, Hyderabad is the fifth-largest contributor to India\'s overall gross domestic product.', 'Hyderabad was historically known as a pearl and diamond trading centre, and it continues to be known as the City of Pearls. Many of the city\'s traditional bazaars, including Laad Bazaar, Begum Bazaar and Sultan Bazaar, have remained open for centuries. However, industrialisation throughout the 20th century attracted major Indian manufacturing, research and financial institutions, including Bharat Heavy Electricals Limited, the National Geophysical Research Institute and the Centre for Cellular and Molecular Biology. Special economic zones dedicated to information technology have encouraged companies from across India and around the world to set up operations and the emergence of pharmaceutical and biotechnology industries in the 1990s led to the area\'s naming as India\'s "Genome Valley". With an output of US$74 billion, Hyderabad is the fifth-largest contributor to India\'s overall gross domestic product.', 'Hyderabad was historically known as a pearl and diamond trading centre, and it continues to be known as the City of Pearls. Many of the city\'s traditional bazaars, including Laad Bazaar, Begum Bazaar and Sultan Bazaar, have remained open for centuries. However, industrialisation throughout the 20th century attracted major Indian manufacturing, research and financial institutions, including Bharat Heavy Electricals Limited, the National Geophysical Research Institute and the Centre for Cellular and Molecular Biology. Special economic zones dedicated to information technology have encouraged companies from across India and around the world to set up operations and the emergence of pharmaceutical and biotechnology industries in the 1990s led to the area\'s naming as India\'s "Genome Valley". With an output of US$74 billion, Hyderabad is the fifth-largest contributor to India\'s overall gross domestic product.', 'Hyderabad was historically known as a pearl and diamond trading centre, and it continues to be known as the City of Pearls. Many of the city\'s traditional bazaars, including Laad Bazaar, Begum Bazaar and Sultan Bazaar, have remained open for centuries. However, industrialisation throughout the 20th century attracted major Indian manufacturing, research and financial institutions, including Bharat Heavy Electricals Limited, the National Geophysical Research Institute and the Centre for Cellular and Molecular Biology. Special economic zones dedicated to information technology have encouraged companies from across India and around the world to set up operations and the emergence of pharmaceutical and biotechnology industries in the 1990s led to the area\'s naming as India\'s "Genome Valley". With an output of US$74 billion, Hyderabad is the fifth-largest contributor to India\'s overall gross domestic product.', 'Hyderabad was historically known as a pearl and diamond trading centre, and it continues to be known as the City of Pearls. Many of the city\'s traditional bazaars, including Laad Bazaar, Begum Bazaar and Sultan Bazaar, have remained open for centuries. However, industrialisation throughout the 20th century attracted major Indian manufacturing, research and financial institutions, including Bharat Heavy Electricals Limited, the National Geophysical Research Institute and the Centre for Cellular and Molecular Biology. Special economic zones dedicated to information technology have encouraged companies from across India and around the world to set up operations and the emergence of pharmaceutical and biotechnology industries in the 1990s led to the area\'s naming as India\'s "Genome Valley". With an output of US$74 billion, Hyderabad is the fifth-largest contributor to India\'s overall gross domestic product.', 'According to John Everett-Heath, the author of Oxford Concise Dictionary of World Place Names, Hyderabad means "Haydar\'s city" or "lion city", from haydar (lion) and ābād (city). It was named to honour the Caliph Ali Ibn Abi Talib, who was also known as Haydar because of his lion-like valour in battles. Andrew Petersen, a scholar of Islamic architecture, says the city was originally called Baghnagar (city of gardens). One popular theory suggests that Muhammad Quli Qutb Shah, the founder of the city, named it "Bhagyanagar" or "Bhāgnagar" after Bhagmati, a local nautch (dancing) girl with whom he had fallen in love. She converted to Islam and adopted the title Hyder Mahal. The city was renamed Hyderabad in her honour. According to another source, the city was named after Haidar, the son of Quli Qutb Shah.', 'According to John Everett-Heath, the author of Oxford Concise Dictionary of World Place Names, Hyderabad means "Haydar\'s city" or "lion city", from haydar (lion) and ābād (city). It was named to honour the Caliph Ali Ibn Abi Talib, who was also known as Haydar because of his lion-like valour in battles. Andrew Petersen, a scholar of Islamic architecture, says the city was originally called Baghnagar (city of gardens). One popular theory suggests that Muhammad Quli Qutb Shah, the founder of the city, named it "Bhagyanagar" or "Bhāgnagar" after Bhagmati, a local nautch (dancing) girl with whom he had fallen in love. She converted to Islam and adopted the title Hyder Mahal. The city was renamed Hyderabad in her honour. According to another source, the city was named after Haidar, the son of Quli Qutb Shah.', 'According to John Everett-Heath, the author of Oxford Concise Dictionary of World Place Names, Hyderabad means "Haydar\'s city" or "lion city", from haydar (lion) and ābād (city). It was named to honour the Caliph Ali Ibn Abi Talib, who was also known as Haydar because of his lion-like valour in battles. Andrew Petersen, a scholar of Islamic architecture, says the city was originally called Baghnagar (city of gardens). One popular theory suggests that Muhammad Quli Qutb Shah, the founder of the city, named it "Bhagyanagar" or "Bhāgnagar" after Bhagmati, a local nautch (dancing) girl with whom he had fallen in love. She converted to Islam and adopted the title Hyder Mahal. The city was renamed Hyderabad in her honour. According to another source, the city was named after Haidar, the son of Quli Qutb Shah.', 'According to John Everett-Heath, the author of Oxford Concise Dictionary of World Place Names, Hyderabad means "Haydar\'s city" or "lion city", from haydar (lion) and ābād (city). It was named to honour the Caliph Ali Ibn Abi Talib, who was also known as Haydar because of his lion-like valour in battles. Andrew Petersen, a scholar of Islamic architecture, says the city was originally called Baghnagar (city of gardens). One popular theory suggests that Muhammad Quli Qutb Shah, the founder of the city, named it "Bhagyanagar" or "Bhāgnagar" after Bhagmati, a local nautch (dancing) girl with whom he had fallen in love. She converted to Islam and adopted the title Hyder Mahal. The city was renamed Hyderabad in her honour. According to another source, the city was named after Haidar, the son of Quli Qutb Shah.', 'According to John Everett-Heath, the author of Oxford Concise Dictionary of World Place Names, Hyderabad means "Haydar\'s city" or "lion city", from haydar (lion) and ābād (city). It was named to honour the Caliph Ali Ibn Abi Talib, who was also known as Haydar because of his lion-like valour in battles. Andrew Petersen, a scholar of Islamic architecture, says the city was originally called Baghnagar (city of gardens). One popular theory suggests that Muhammad Quli Qutb Shah, the founder of the city, named it "Bhagyanagar" or "Bhāgnagar" after Bhagmati, a local nautch (dancing) girl with whom he had fallen in love. She converted to Islam and adopted the title Hyder Mahal. The city was renamed Hyderabad in her honour. According to another source, the city was named after Haidar, the son of Quli Qutb Shah.', 'Archaeologists excavating near the city have unearthed Iron Age sites that may date from 500 BCE. The region comprising modern Hyderabad and its surroundings was known as Golkonda (Golla Konda-"shepherd\'s hill"), and was ruled by the Chalukya dynasty from 624 CE to 1075 CE. Following the dissolution of the Chalukya empire into four parts in the 11th century, Golkonda came under the control of the Kakatiya dynasty from 1158, whose seat of power was at Warangal, 148 km (92 mi) northeast of modern Hyderabad.', 'Archaeologists excavating near the city have unearthed Iron Age sites that may date from 500 BCE. The region comprising modern Hyderabad and its surroundings was known as Golkonda (Golla Konda-"shepherd\'s hill"), and was ruled by the Chalukya dynasty from 624 CE to 1075 CE. Following the dissolution of the Chalukya empire into four parts in the 11th century, Golkonda came under the control of the Kakatiya dynasty from 1158, whose seat of power was at Warangal, 148 km (92 mi) northeast of modern Hyderabad.', 'Archaeologists excavating near the city have unearthed Iron Age sites that may date from 500 BCE. The region comprising modern Hyderabad and its surroundings was known as Golkonda (Golla Konda-"shepherd\'s hill"), and was ruled by the Chalukya dynasty from 624 CE to 1075 CE. Following the dissolution of the Chalukya empire into four parts in the 11th century, Golkonda came under the control of the Kakatiya dynasty from 1158, whose seat of power was at Warangal, 148 km (92 mi) northeast of modern Hyderabad.', 'Archaeologists excavating near the city have unearthed Iron Age sites that may date from 500 BCE. The region comprising modern Hyderabad and its surroundings was known as Golkonda (Golla Konda-"shepherd\'s hill"), and was ruled by the Chalukya dynasty from 624 CE to 1075 CE. Following the dissolution of the Chalukya empire into four parts in the 11th century, Golkonda came under the control of the Kakatiya dynasty from 1158, whose seat of power was at Warangal, 148 km (92 mi) northeast of modern Hyderabad.', 'Archaeologists excavating near the city have unearthed Iron Age sites that may date from 500 BCE. The region comprising modern Hyderabad and its surroundings was known as Golkonda (Golla Konda-"shepherd\'s hill"), and was ruled by the Chalukya dynasty from 624 CE to 1075 CE. Following the dissolution of the Chalukya empire into four parts in the 11th century, Golkonda came under the control of the Kakatiya dynasty from 1158, whose seat of power was at Warangal, 148 km (92 mi) northeast of modern Hyderabad.', "The Kakatiya dynasty was reduced to a vassal of the Khilji dynasty in 1310 after its defeat by Sultan Alauddin Khilji of the Delhi Sultanate. This lasted until 1321, when the Kakatiya dynasty was annexed by Malik Kafur, Allaudin Khilji's general. During this period, Alauddin Khilji took the Koh-i-Noor diamond, which is said to have been mined from the Kollur Mines of Golkonda, to Delhi. Muhammad bin Tughluq succeeded to the Delhi sultanate in 1325, bringing Warangal under the rule of the Tughlaq dynasty until 1347 when Ala-ud-Din Bahman Shah, a governor under bin Tughluq, rebelled against Delhi and established the Bahmani Sultanate in the Deccan Plateau, with Gulbarga, 200 km (124 mi) west of Hyderabad, as its capital. The Bahmani kings ruled the region until 1518 and were the first independent Muslim rulers of the Deccan.", "The Kakatiya dynasty was reduced to a vassal of the Khilji dynasty in 1310 after its defeat by Sultan Alauddin Khilji of the Delhi Sultanate. This lasted until 1321, when the Kakatiya dynasty was annexed by Malik Kafur, Allaudin Khilji's general. During this period, Alauddin Khilji took the Koh-i-Noor diamond, which is said to have been mined from the Kollur Mines of Golkonda, to Delhi. Muhammad bin Tughluq succeeded to the Delhi sultanate in 1325, bringing Warangal under the rule of the Tughlaq dynasty until 1347 when Ala-ud-Din Bahman Shah, a governor under bin Tughluq, rebelled against Delhi and established the Bahmani Sultanate in the Deccan Plateau, with Gulbarga, 200 km (124 mi) west of Hyderabad, as its capital. The Bahmani kings ruled the region until 1518 and were the first independent Muslim rulers of the Deccan.", "The Kakatiya dynasty was reduced to a vassal of the Khilji dynasty in 1310 after its defeat by Sultan Alauddin Khilji of the Delhi Sultanate. This lasted until 1321, when the Kakatiya dynasty was annexed by Malik Kafur, Allaudin Khilji's general. During this period, Alauddin Khilji took the Koh-i-Noor diamond, which is said to have been mined from the Kollur Mines of Golkonda, to Delhi. Muhammad bin Tughluq succeeded to the Delhi sultanate in 1325, bringing Warangal under the rule of the Tughlaq dynasty until 1347 when Ala-ud-Din Bahman Shah, a governor under bin Tughluq, rebelled against Delhi and established the Bahmani Sultanate in the Deccan Plateau, with Gulbarga, 200 km (124 mi) west of Hyderabad, as its capital. The Bahmani kings ruled the region until 1518 and were the first independent Muslim rulers of the Deccan.", "The Kakatiya dynasty was reduced to a vassal of the Khilji dynasty in 1310 after its defeat by Sultan Alauddin Khilji of the Delhi Sultanate. This lasted until 1321, when the Kakatiya dynasty was annexed by Malik Kafur, Allaudin Khilji's general. During this period, Alauddin Khilji took the Koh-i-Noor diamond, which is said to have been mined from the Kollur Mines of Golkonda, to Delhi. Muhammad bin Tughluq succeeded to the Delhi sultanate in 1325, bringing Warangal under the rule of the Tughlaq dynasty until 1347 when Ala-ud-Din Bahman Shah, a governor under bin Tughluq, rebelled against Delhi and established the Bahmani Sultanate in the Deccan Plateau, with Gulbarga, 200 km (124 mi) west of Hyderabad, as its capital. The Bahmani kings ruled the region until 1518 and were the first independent Muslim rulers of the Deccan.", "The Kakatiya dynasty was reduced to a vassal of the Khilji dynasty in 1310 after its defeat by Sultan Alauddin Khilji of the Delhi Sultanate. This lasted until 1321, when the Kakatiya dynasty was annexed by Malik Kafur, Allaudin Khilji's general. During this period, Alauddin Khilji took the Koh-i-Noor diamond, which is said to have been mined from the Kollur Mines of Golkonda, to Delhi. Muhammad bin Tughluq succeeded to the Delhi sultanate in 1325, bringing Warangal under the rule of the Tughlaq dynasty until 1347 when Ala-ud-Din Bahman Shah, a governor under bin Tughluq, rebelled against Delhi and established the Bahmani Sultanate in the Deccan Plateau, with Gulbarga, 200 km (124 mi) west of Hyderabad, as its capital. The Bahmani kings ruled the region until 1518 and were the first independent Muslim rulers of the Deccan.", 'Sultan Quli, a governor of Golkonda, revolted against the Bahmani Sultanate and established the Qutb Shahi dynasty in 1518; he rebuilt the mud-fort of Golconda and named the city "Muhammad nagar". The fifth sultan, Muhammad Quli Qutb Shah, established Hyderabad on the banks of the Musi River in 1591, to avoid the water shortages experienced at Golkonda. During his rule, he had the Charminar and Mecca Masjid built in the city. On 21 September 1687, the Golkonda Sultanate came under the rule of the Mughal emperor Aurangzeb after a year-long siege of the Golkonda fort. The annexed area was renamed Deccan Suba (Deccan province) and the capital was moved from Golkonda to Aurangabad, about 550 km (342 mi) northwest of Hyderabad.', 'Sultan Quli, a governor of Golkonda, revolted against the Bahmani Sultanate and established the Qutb Shahi dynasty in 1518; he rebuilt the mud-fort of Golconda and named the city "Muhammad nagar". The fifth sultan, Muhammad Quli Qutb Shah, established Hyderabad on the banks of the Musi River in 1591, to avoid the water shortages experienced at Golkonda. During his rule, he had the Charminar and Mecca Masjid built in the city. On 21 September 1687, the Golkonda Sultanate came under the rule of the Mughal emperor Aurangzeb after a year-long siege of the Golkonda fort. The annexed area was renamed Deccan Suba (Deccan province) and the capital was moved from Golkonda to Aurangabad, about 550 km (342 mi) northwest of Hyderabad.', 'Sultan Quli, a governor of Golkonda, revolted against the Bahmani Sultanate and established the Qutb Shahi dynasty in 1518; he rebuilt the mud-fort of Golconda and named the city "Muhammad nagar". The fifth sultan, Muhammad Quli Qutb Shah, established Hyderabad on the banks of the Musi River in 1591, to avoid the water shortages experienced at Golkonda. During his rule, he had the Charminar and Mecca Masjid built in the city. On 21 September 1687, the Golkonda Sultanate came under the rule of the Mughal emperor Aurangzeb after a year-long siege of the Golkonda fort. The annexed area was renamed Deccan Suba (Deccan province) and the capital was moved from Golkonda to Aurangabad, about 550 km (342 mi) northwest of Hyderabad.', 'Sultan Quli, a governor of Golkonda, revolted against the Bahmani Sultanate and established the Qutb Shahi dynasty in 1518; he rebuilt the mud-fort of Golconda and named the city "Muhammad nagar". The fifth sultan, Muhammad Quli Qutb Shah, established Hyderabad on the banks of the Musi River in 1591, to avoid the water shortages experienced at Golkonda. During his rule, he had the Charminar and Mecca Masjid built in the city. On 21 September 1687, the Golkonda Sultanate came under the rule of the Mughal emperor Aurangzeb after a year-long siege of the Golkonda fort. The annexed area was renamed Deccan Suba (Deccan province) and the capital was moved from Golkonda to Aurangabad, about 550 km (342 mi) northwest of Hyderabad.', 'Sultan Quli, a governor of Golkonda, revolted against the Bahmani Sultanate and established the Qutb Shahi dynasty in 1518; he rebuilt the mud-fort of Golconda and named the city "Muhammad nagar". The fifth sultan, Muhammad Quli Qutb Shah, established Hyderabad on the banks of the Musi River in 1591, to avoid the water shortages experienced at Golkonda. During his rule, he had the Charminar and Mecca Masjid built in the city. On 21 September 1687, the Golkonda Sultanate came under the rule of the Mughal emperor Aurangzeb after a year-long siege of the Golkonda fort. The annexed area was renamed Deccan Suba (Deccan province) and the capital was moved from Golkonda to Aurangabad, about 550 km (342 mi) northwest of Hyderabad.', 'In 1713 Farrukhsiyar, the Mughal emperor, appointed Asif Jah I to be Viceroy of the Deccan, with the title Nizam-ul-Mulk (Administrator of the Realm). In 1724, Asif Jah I defeated Mubariz Khan to establish autonomy over the Deccan Suba, named the region Hyderabad Deccan, and started what came to be known as the Asif Jahi dynasty. Subsequent rulers retained the title Nizam ul-Mulk and were referred to as Asif Jahi Nizams, or Nizams of Hyderabad. The death of Asif Jah I in 1748 resulted in a period of political unrest as his sons, backed by opportunistic neighbouring states and colonial foreign forces, contended for the throne. The accession of Asif Jah II, who reigned from 1762 to 1803, ended the instability. In 1768 he signed the treaty of Masulipatnam, surrendering the coastal region to the East India Company in return for a fixed annual rent.', 'In 1713 Farrukhsiyar, the Mughal emperor, appointed Asif Jah I to be Viceroy of the Deccan, with the title Nizam-ul-Mulk (Administrator of the Realm). In 1724, Asif Jah I defeated Mubariz Khan to establish autonomy over the Deccan Suba, named the region Hyderabad Deccan, and started what came to be known as the Asif Jahi dynasty. Subsequent rulers retained the title Nizam ul-Mulk and were referred to as Asif Jahi Nizams, or Nizams of Hyderabad. The death of Asif Jah I in 1748 resulted in a period of political unrest as his sons, backed by opportunistic neighbouring states and colonial foreign forces, contended for the throne. The accession of Asif Jah II, who reigned from 1762 to 1803, ended the instability. In 1768 he signed the treaty of Masulipatnam, surrendering the coastal region to the East India Company in return for a fixed annual rent.', 'In 1713 Farrukhsiyar, the Mughal emperor, appointed Asif Jah I to be Viceroy of the Deccan, with the title Nizam-ul-Mulk (Administrator of the Realm). In 1724, Asif Jah I defeated Mubariz Khan to establish autonomy over the Deccan Suba, named the region Hyderabad Deccan, and started what came to be known as the Asif Jahi dynasty. Subsequent rulers retained the title Nizam ul-Mulk and were referred to as Asif Jahi Nizams, or Nizams of Hyderabad. The death of Asif Jah I in 1748 resulted in a period of political unrest as his sons, backed by opportunistic neighbouring states and colonial foreign forces, contended for the throne. The accession of Asif Jah II, who reigned from 1762 to 1803, ended the instability. In 1768 he signed the treaty of Masulipatnam, surrendering the coastal region to the East India Company in return for a fixed annual rent.', 'In 1713 Farrukhsiyar, the Mughal emperor, appointed Asif Jah I to be Viceroy of the Deccan, with the title Nizam-ul-Mulk (Administrator of the Realm). In 1724, Asif Jah I defeated Mubariz Khan to establish autonomy over the Deccan Suba, named the region Hyderabad Deccan, and started what came to be known as the Asif Jahi dynasty. Subsequent rulers retained the title Nizam ul-Mulk and were referred to as Asif Jahi Nizams, or Nizams of Hyderabad. The death of Asif Jah I in 1748 resulted in a period of political unrest as his sons, backed by opportunistic neighbouring states and colonial foreign forces, contended for the throne. The accession of Asif Jah II, who reigned from 1762 to 1803, ended the instability. In 1768 he signed the treaty of Masulipatnam, surrendering the coastal region to the East India Company in return for a fixed annual rent.', 'In 1713 Farrukhsiyar, the Mughal emperor, appointed Asif Jah I to be Viceroy of the Deccan, with the title Nizam-ul-Mulk (Administrator of the Realm). In 1724, Asif Jah I defeated Mubariz Khan to establish autonomy over the Deccan Suba, named the region Hyderabad Deccan, and started what came to be known as the Asif Jahi dynasty. Subsequent rulers retained the title Nizam ul-Mulk and were referred to as Asif Jahi Nizams, or Nizams of Hyderabad. The death of Asif Jah I in 1748 resulted in a period of political unrest as his sons, backed by opportunistic neighbouring states and colonial foreign forces, contended for the throne. The accession of Asif Jah II, who reigned from 1762 to 1803, ended the instability. In 1768 he signed the treaty of Masulipatnam, surrendering the coastal region to the East India Company in return for a fixed annual rent.', "In 1769 Hyderabad city became the formal capital of the Nizams. In response to regular threats from Hyder Ali (Dalwai of Mysore), Baji Rao I (Peshwa of the Maratha Empire), and Basalath Jung (Asif Jah II's elder brother, who was supported by the Marquis de Bussy-Castelnau), the Nizam signed a subsidiary alliance with the East India Company in 1798, allowing the British Indian Army to occupy Bolarum (modern Secunderabad) to protect the state's borders, for which the Nizams paid an annual maintenance to the British.", "In 1769 Hyderabad city became the formal capital of the Nizams. In response to regular threats from Hyder Ali (Dalwai of Mysore), Baji Rao I (Peshwa of the Maratha Empire), and Basalath Jung (Asif Jah II's elder brother, who was supported by the Marquis de Bussy-Castelnau), the Nizam signed a subsidiary alliance with the East India Company in 1798, allowing the British Indian Army to occupy Bolarum (modern Secunderabad) to protect the state's borders, for which the Nizams paid an annual maintenance to the British.", "In 1769 Hyderabad city became the formal capital of the Nizams. In response to regular threats from Hyder Ali (Dalwai of Mysore), Baji Rao I (Peshwa of the Maratha Empire), and Basalath Jung (Asif Jah II's elder brother, who was supported by the Marquis de Bussy-Castelnau), the Nizam signed a subsidiary alliance with the East India Company in 1798, allowing the British Indian Army to occupy Bolarum (modern Secunderabad) to protect the state's borders, for which the Nizams paid an annual maintenance to the British.", "In 1769 Hyderabad city became the formal capital of the Nizams. In response to regular threats from Hyder Ali (Dalwai of Mysore), Baji Rao I (Peshwa of the Maratha Empire), and Basalath Jung (Asif Jah II's elder brother, who was supported by the Marquis de Bussy-Castelnau), the Nizam signed a subsidiary alliance with the East India Company in 1798, allowing the British Indian Army to occupy Bolarum (modern Secunderabad) to protect the state's borders, for which the Nizams paid an annual maintenance to the British.", "In 1769 Hyderabad city became the formal capital of the Nizams. In response to regular threats from Hyder Ali (Dalwai of Mysore), Baji Rao I (Peshwa of the Maratha Empire), and Basalath Jung (Asif Jah II's elder brother, who was supported by the Marquis de Bussy-Castelnau), the Nizam signed a subsidiary alliance with the East India Company in 1798, allowing the British Indian Army to occupy Bolarum (modern Secunderabad) to protect the state's borders, for which the Nizams paid an annual maintenance to the British.", 'After India gained independence, the Nizam declared his intention to remain independent rather than become part of the Indian Union. The Hyderabad State Congress, with the support of the Indian National Congress and the Communist Party of India, began agitating against Nizam VII in 1948. On 17 September that year, the Indian Army took control of Hyderabad State after an invasion codenamed Operation Polo. With the defeat of his forces, Nizam VII capitulated to the Indian Union by signing an Instrument of Accession, which made him the Rajpramukh (Princely Governor) of the state until 31 October 1956. Between 1946 and 1951, the Communist Party of India fomented the Telangana uprising against the feudal lords of the Telangana region. The Constitution of India, which became effective on 26 January 1950, made Hyderabad State one of the part B states of India, with Hyderabad city continuing to be the capital. In his 1955 report Thoughts on Linguistic States, B. R. Ambedkar, then chairman of the Drafting Committee of the Indian Constitution, proposed designating the city of Hyderabad as the second capital of India because of its amenities and strategic central location. Since 1956, the Rashtrapati Nilayam in Hyderabad has been the second official residence and business office of the President of India; the President stays once a year in winter and conducts official business particularly relating to Southern India.', 'After India gained independence, the Nizam declared his intention to remain independent rather than become part of the Indian Union. The Hyderabad State Congress, with the support of the Indian National Congress and the Communist Party of India, began agitating against Nizam VII in 1948. On 17 September that year, the Indian Army took control of Hyderabad State after an invasion codenamed Operation Polo. With the defeat of his forces, Nizam VII capitulated to the Indian Union by signing an Instrument of Accession, which made him the Rajpramukh (Princely Governor) of the state until 31 October 1956. Between 1946 and 1951, the Communist Party of India fomented the Telangana uprising against the feudal lords of the Telangana region. The Constitution of India, which became effective on 26 January 1950, made Hyderabad State one of the part B states of India, with Hyderabad city continuing to be the capital. In his 1955 report Thoughts on Linguistic States, B. R. Ambedkar, then chairman of the Drafting Committee of the Indian Constitution, proposed designating the city of Hyderabad as the second capital of India because of its amenities and strategic central location. Since 1956, the Rashtrapati Nilayam in Hyderabad has been the second official residence and business office of the President of India; the President stays once a year in winter and conducts official business particularly relating to Southern India.', 'After India gained independence, the Nizam declared his intention to remain independent rather than become part of the Indian Union. The Hyderabad State Congress, with the support of the Indian National Congress and the Communist Party of India, began agitating against Nizam VII in 1948. On 17 September that year, the Indian Army took control of Hyderabad State after an invasion codenamed Operation Polo. With the defeat of his forces, Nizam VII capitulated to the Indian Union by signing an Instrument of Accession, which made him the Rajpramukh (Princely Governor) of the state until 31 October 1956. Between 1946 and 1951, the Communist Party of India fomented the Telangana uprising against the feudal lords of the Telangana region. The Constitution of India, which became effective on 26 January 1950, made Hyderabad State one of the part B states of India, with Hyderabad city continuing to be the capital. In his 1955 report Thoughts on Linguistic States, B. R. Ambedkar, then chairman of the Drafting Committee of the Indian Constitution, proposed designating the city of Hyderabad as the second capital of India because of its amenities and strategic central location. Since 1956, the Rashtrapati Nilayam in Hyderabad has been the second official residence and business office of the President of India; the President stays once a year in winter and conducts official business particularly relating to Southern India.', 'After India gained independence, the Nizam declared his intention to remain independent rather than become part of the Indian Union. The Hyderabad State Congress, with the support of the Indian National Congress and the Communist Party of India, began agitating against Nizam VII in 1948. On 17 September that year, the Indian Army took control of Hyderabad State after an invasion codenamed Operation Polo. With the defeat of his forces, Nizam VII capitulated to the Indian Union by signing an Instrument of Accession, which made him the Rajpramukh (Princely Governor) of the state until 31 October 1956. Between 1946 and 1951, the Communist Party of India fomented the Telangana uprising against the feudal lords of the Telangana region. The Constitution of India, which became effective on 26 January 1950, made Hyderabad State one of the part B states of India, with Hyderabad city continuing to be the capital. In his 1955 report Thoughts on Linguistic States, B. R. Ambedkar, then chairman of the Drafting Committee of the Indian Constitution, proposed designating the city of Hyderabad as the second capital of India because of its amenities and strategic central location. Since 1956, the Rashtrapati Nilayam in Hyderabad has been the second official residence and business office of the President of India; the President stays once a year in winter and conducts official business particularly relating to Southern India.', 'After India gained independence, the Nizam declared his intention to remain independent rather than become part of the Indian Union. The Hyderabad State Congress, with the support of the Indian National Congress and the Communist Party of India, began agitating against Nizam VII in 1948. On 17 September that year, the Indian Army took control of Hyderabad State after an invasion codenamed Operation Polo. With the defeat of his forces, Nizam VII capitulated to the Indian Union by signing an Instrument of Accession, which made him the Rajpramukh (Princely Governor) of the state until 31 October 1956. Between 1946 and 1951, the Communist Party of India fomented the Telangana uprising against the feudal lords of the Telangana region. The Constitution of India, which became effective on 26 January 1950, made Hyderabad State one of the part B states of India, with Hyderabad city continuing to be the capital. In his 1955 report Thoughts on Linguistic States, B. R. Ambedkar, then chairman of the Drafting Committee of the Indian Constitution, proposed designating the city of Hyderabad as the second capital of India because of its amenities and strategic central location. Since 1956, the Rashtrapati Nilayam in Hyderabad has been the second official residence and business office of the President of India; the President stays once a year in winter and conducts official business particularly relating to Southern India.', 'On 1 November 1956 the states of India were reorganised by language. Hyderabad state was split into three parts, which were merged with neighbouring states to form the modern states of Maharashtra, Karnataka and Andhra Pradesh. The nine Telugu- and Urdu-speaking districts of Hyderabad State in the Telangana region were merged with the Telugu-speaking Andhra State to create Andhra Pradesh, with Hyderabad as its capital. Several protests, known collectively as the Telangana movement, attempted to invalidate the merger and demanded the creation of a new Telangana state. Major actions took place in 1969 and 1972, and a third began in 2010. The city suffered several explosions: one at Dilsukhnagar in 2002 claimed two lives; terrorist bombs in May and August 2007 caused communal tension and riots; and two bombs exploded in February 2013. On 30 July 2013 the government of India declared that part of Andhra Pradesh would be split off to form a new Telangana state, and that Hyderabad city would be the capital city and part of Telangana, while the city would also remain the capital of Andhra Pradesh for no more than ten years. On 3 October 2013 the Union Cabinet approved the proposal, and in February 2014 both houses of Parliament passed the Telangana Bill. With the final assent of the President of India in June 2014, Telangana state was formed.', 'On 1 November 1956 the states of India were reorganised by language. Hyderabad state was split into three parts, which were merged with neighbouring states to form the modern states of Maharashtra, Karnataka and Andhra Pradesh. The nine Telugu- and Urdu-speaking districts of Hyderabad State in the Telangana region were merged with the Telugu-speaking Andhra State to create Andhra Pradesh, with Hyderabad as its capital. Several protests, known collectively as the Telangana movement, attempted to invalidate the merger and demanded the creation of a new Telangana state. Major actions took place in 1969 and 1972, and a third began in 2010. The city suffered several explosions: one at Dilsukhnagar in 2002 claimed two lives; terrorist bombs in May and August 2007 caused communal tension and riots; and two bombs exploded in February 2013. On 30 July 2013 the government of India declared that part of Andhra Pradesh would be split off to form a new Telangana state, and that Hyderabad city would be the capital city and part of Telangana, while the city would also remain the capital of Andhra Pradesh for no more than ten years. On 3 October 2013 the Union Cabinet approved the proposal, and in February 2014 both houses of Parliament passed the Telangana Bill. With the final assent of the President of India in June 2014, Telangana state was formed.', 'On 1 November 1956 the states of India were reorganised by language. Hyderabad state was split into three parts, which were merged with neighbouring states to form the modern states of Maharashtra, Karnataka and Andhra Pradesh. The nine Telugu- and Urdu-speaking districts of Hyderabad State in the Telangana region were merged with the Telugu-speaking Andhra State to create Andhra Pradesh, with Hyderabad as its capital. Several protests, known collectively as the Telangana movement, attempted to invalidate the merger and demanded the creation of a new Telangana state. Major actions took place in 1969 and 1972, and a third began in 2010. The city suffered several explosions: one at Dilsukhnagar in 2002 claimed two lives; terrorist bombs in May and August 2007 caused communal tension and riots; and two bombs exploded in February 2013. On 30 July 2013 the government of India declared that part of Andhra Pradesh would be split off to form a new Telangana state, and that Hyderabad city would be the capital city and part of Telangana, while the city would also remain the capital of Andhra Pradesh for no more than ten years. On 3 October 2013 the Union Cabinet approved the proposal, and in February 2014 both houses of Parliament passed the Telangana Bill. With the final assent of the President of India in June 2014, Telangana state was formed.', 'On 1 November 1956 the states of India were reorganised by language. Hyderabad state was split into three parts, which were merged with neighbouring states to form the modern states of Maharashtra, Karnataka and Andhra Pradesh. The nine Telugu- and Urdu-speaking districts of Hyderabad State in the Telangana region were merged with the Telugu-speaking Andhra State to create Andhra Pradesh, with Hyderabad as its capital. Several protests, known collectively as the Telangana movement, attempted to invalidate the merger and demanded the creation of a new Telangana state. Major actions took place in 1969 and 1972, and a third began in 2010. The city suffered several explosions: one at Dilsukhnagar in 2002 claimed two lives; terrorist bombs in May and August 2007 caused communal tension and riots; and two bombs exploded in February 2013. On 30 July 2013 the government of India declared that part of Andhra Pradesh would be split off to form a new Telangana state, and that Hyderabad city would be the capital city and part of Telangana, while the city would also remain the capital of Andhra Pradesh for no more than ten years. On 3 October 2013 the Union Cabinet approved the proposal, and in February 2014 both houses of Parliament passed the Telangana Bill. With the final assent of the President of India in June 2014, Telangana state was formed.', 'On 1 November 1956 the states of India were reorganised by language. Hyderabad state was split into three parts, which were merged with neighbouring states to form the modern states of Maharashtra, Karnataka and Andhra Pradesh. The nine Telugu- and Urdu-speaking districts of Hyderabad State in the Telangana region were merged with the Telugu-speaking Andhra State to create Andhra Pradesh, with Hyderabad as its capital. Several protests, known collectively as the Telangana movement, attempted to invalidate the merger and demanded the creation of a new Telangana state. Major actions took place in 1969 and 1972, and a third began in 2010. The city suffered several explosions: one at Dilsukhnagar in 2002 claimed two lives; terrorist bombs in May and August 2007 caused communal tension and riots; and two bombs exploded in February 2013. On 30 July 2013 the government of India declared that part of Andhra Pradesh would be split off to form a new Telangana state, and that Hyderabad city would be the capital city and part of Telangana, while the city would also remain the capital of Andhra Pradesh for no more than ten years. On 3 October 2013 the Union Cabinet approved the proposal, and in February 2014 both houses of Parliament passed the Telangana Bill. With the final assent of the President of India in June 2014, Telangana state was formed.', 'Situated in the southern part of Telangana in southeastern India, Hyderabad is 1,566 kilometres (973 mi) south of Delhi, 699 kilometres (434 mi) southeast of Mumbai, and 570 kilometres (350 mi) north of Bangalore by road. It lies on the banks of the Musi River, in the northern part of the Deccan Plateau. Greater Hyderabad covers 650 km2 (250 sq mi), making it one of the largest metropolitan areas in India. With an average altitude of 542 metres (1,778 ft), Hyderabad lies on predominantly sloping terrain of grey and pink granite, dotted with small hills, the highest being Banjara Hills at 672 metres (2,205 ft). The city has numerous lakes referred to as sagar, meaning "sea". Examples include artificial lakes created by dams on the Musi, such as Hussain Sagar (built in 1562 near the city centre), Osman Sagar and Himayat Sagar. As of 1996, the city had 140 lakes and 834 water tanks (ponds).', 'Situated in the southern part of Telangana in southeastern India, Hyderabad is 1,566 kilometres (973 mi) south of Delhi, 699 kilometres (434 mi) southeast of Mumbai, and 570 kilometres (350 mi) north of Bangalore by road. It lies on the banks of the Musi River, in the northern part of the Deccan Plateau. Greater Hyderabad covers 650 km2 (250 sq mi), making it one of the largest metropolitan areas in India. With an average altitude of 542 metres (1,778 ft), Hyderabad lies on predominantly sloping terrain of grey and pink granite, dotted with small hills, the highest being Banjara Hills at 672 metres (2,205 ft). The city has numerous lakes referred to as sagar, meaning "sea". Examples include artificial lakes created by dams on the Musi, such as Hussain Sagar (built in 1562 near the city centre), Osman Sagar and Himayat Sagar. As of 1996, the city had 140 lakes and 834 water tanks (ponds).', 'Situated in the southern part of Telangana in southeastern India, Hyderabad is 1,566 kilometres (973 mi) south of Delhi, 699 kilometres (434 mi) southeast of Mumbai, and 570 kilometres (350 mi) north of Bangalore by road. It lies on the banks of the Musi River, in the northern part of the Deccan Plateau. Greater Hyderabad covers 650 km2 (250 sq mi), making it one of the largest metropolitan areas in India. With an average altitude of 542 metres (1,778 ft), Hyderabad lies on predominantly sloping terrain of grey and pink granite, dotted with small hills, the highest being Banjara Hills at 672 metres (2,205 ft). The city has numerous lakes referred to as sagar, meaning "sea". Examples include artificial lakes created by dams on the Musi, such as Hussain Sagar (built in 1562 near the city centre), Osman Sagar and Himayat Sagar. As of 1996, the city had 140 lakes and 834 water tanks (ponds).', 'Situated in the southern part of Telangana in southeastern India, Hyderabad is 1,566 kilometres (973 mi) south of Delhi, 699 kilometres (434 mi) southeast of Mumbai, and 570 kilometres (350 mi) north of Bangalore by road. It lies on the banks of the Musi River, in the northern part of the Deccan Plateau. Greater Hyderabad covers 650 km2 (250 sq mi), making it one of the largest metropolitan areas in India. With an average altitude of 542 metres (1,778 ft), Hyderabad lies on predominantly sloping terrain of grey and pink granite, dotted with small hills, the highest being Banjara Hills at 672 metres (2,205 ft). The city has numerous lakes referred to as sagar, meaning "sea". Examples include artificial lakes created by dams on the Musi, such as Hussain Sagar (built in 1562 near the city centre), Osman Sagar and Himayat Sagar. As of 1996, the city had 140 lakes and 834 water tanks (ponds).', 'Situated in the southern part of Telangana in southeastern India, Hyderabad is 1,566 kilometres (973 mi) south of Delhi, 699 kilometres (434 mi) southeast of Mumbai, and 570 kilometres (350 mi) north of Bangalore by road. It lies on the banks of the Musi River, in the northern part of the Deccan Plateau. Greater Hyderabad covers 650 km2 (250 sq mi), making it one of the largest metropolitan areas in India. With an average altitude of 542 metres (1,778 ft), Hyderabad lies on predominantly sloping terrain of grey and pink granite, dotted with small hills, the highest being Banjara Hills at 672 metres (2,205 ft). The city has numerous lakes referred to as sagar, meaning "sea". Examples include artificial lakes created by dams on the Musi, such as Hussain Sagar (built in 1562 near the city centre), Osman Sagar and Himayat Sagar. As of 1996, the city had 140 lakes and 834 water tanks (ponds).', 'Hyderabad has a tropical wet and dry climate (Köppen Aw) bordering on a hot semi-arid climate (Köppen BSh). The annual mean temperature is 26.6 °C (79.9 °F); monthly mean temperatures are 21–33 °C (70–91 °F). Summers (March–June) are hot and humid, with average highs in the mid-to-high 30s Celsius; maximum temperatures often exceed 40 °C (104 °F) between April and June. The coolest temperatures occur in December and January, when the lowest temperature occasionally dips to 10 °C (50 °F). May is the hottest month, when daily temperatures range from 26 to 39 °C (79–102 °F); December, the coldest, has temperatures varying from 14.5 to 28 °C (57–82 °F).', 'Hyderabad has a tropical wet and dry climate (Köppen Aw) bordering on a hot semi-arid climate (Köppen BSh). The annual mean temperature is 26.6 °C (79.9 °F); monthly mean temperatures are 21–33 °C (70–91 °F). Summers (March–June) are hot and humid, with average highs in the mid-to-high 30s Celsius; maximum temperatures often exceed 40 °C (104 °F) between April and June. The coolest temperatures occur in December and January, when the lowest temperature occasionally dips to 10 °C (50 °F). May is the hottest month, when daily temperatures range from 26 to 39 °C (79–102 °F); December, the coldest, has temperatures varying from 14.5 to 28 °C (57–82 °F).', 'Hyderabad has a tropical wet and dry climate (Köppen Aw) bordering on a hot semi-arid climate (Köppen BSh). The annual mean temperature is 26.6 °C (79.9 °F); monthly mean temperatures are 21–33 °C (70–91 °F). Summers (March–June) are hot and humid, with average highs in the mid-to-high 30s Celsius; maximum temperatures often exceed 40 °C (104 °F) between April and June. The coolest temperatures occur in December and January, when the lowest temperature occasionally dips to 10 °C (50 °F). May is the hottest month, when daily temperatures range from 26 to 39 °C (79–102 °F); December, the coldest, has temperatures varying from 14.5 to 28 °C (57–82 °F).', 'Hyderabad has a tropical wet and dry climate (Köppen Aw) bordering on a hot semi-arid climate (Köppen BSh). The annual mean temperature is 26.6 °C (79.9 °F); monthly mean temperatures are 21–33 °C (70–91 °F). Summers (March–June) are hot and humid, with average highs in the mid-to-high 30s Celsius; maximum temperatures often exceed 40 °C (104 °F) between April and June. The coolest temperatures occur in December and January, when the lowest temperature occasionally dips to 10 °C (50 °F). May is the hottest month, when daily temperatures range from 26 to 39 °C (79–102 °F); December, the coldest, has temperatures varying from 14.5 to 28 °C (57–82 °F).', 'Hyderabad has a tropical wet and dry climate (Köppen Aw) bordering on a hot semi-arid climate (Köppen BSh). The annual mean temperature is 26.6 °C (79.9 °F); monthly mean temperatures are 21–33 °C (70–91 °F). Summers (March–June) are hot and humid, with average highs in the mid-to-high 30s Celsius; maximum temperatures often exceed 40 °C (104 °F) between April and June. The coolest temperatures occur in December and January, when the lowest temperature occasionally dips to 10 °C (50 °F). May is the hottest month, when daily temperatures range from 26 to 39 °C (79–102 °F); December, the coldest, has temperatures varying from 14.5 to 28 °C (57–82 °F).', "Hyderabad's lakes and the sloping terrain of its low-lying hills provide habitat for an assortment of flora and fauna. The forest region in and around the city encompasses areas of ecological and biological importance, which are preserved in the form of national parks, zoos, mini-zoos and a wildlife sanctuary. Nehru Zoological Park, the city's one large zoo, is the first in India to have a lion and tiger safari park. Hyderabad has three national parks (Mrugavani National Park, Mahavir Harina Vanasthali National Park and Kasu Brahmananda Reddy National Park), and the Manjira Wildlife Sanctuary is about 50 km (31 mi) from the city. Hyderabad's other environmental reserves are: Kotla Vijayabhaskara Reddy Botanical Gardens, Shamirpet Lake, Hussain Sagar, Fox Sagar Lake, Mir Alam Tank and Patancheru Lake, which is home to regional birds and attracts seasonal migratory birds from different parts of the world. Organisations engaged in environmental and wildlife preservation include the Telangana Forest Department, Indian Council of Forestry Research and Education, the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT), the Animal Welfare Board of India, the Blue Cross of Hyderabad and the University of Hyderabad.", "Hyderabad's lakes and the sloping terrain of its low-lying hills provide habitat for an assortment of flora and fauna. The forest region in and around the city encompasses areas of ecological and biological importance, which are preserved in the form of national parks, zoos, mini-zoos and a wildlife sanctuary. Nehru Zoological Park, the city's one large zoo, is the first in India to have a lion and tiger safari park. Hyderabad has three national parks (Mrugavani National Park, Mahavir Harina Vanasthali National Park and Kasu Brahmananda Reddy National Park), and the Manjira Wildlife Sanctuary is about 50 km (31 mi) from the city. Hyderabad's other environmental reserves are: Kotla Vijayabhaskara Reddy Botanical Gardens, Shamirpet Lake, Hussain Sagar, Fox Sagar Lake, Mir Alam Tank and Patancheru Lake, which is home to regional birds and attracts seasonal migratory birds from different parts of the world. Organisations engaged in environmental and wildlife preservation include the Telangana Forest Department, Indian Council of Forestry Research and Education, the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT), the Animal Welfare Board of India, the Blue Cross of Hyderabad and the University of Hyderabad.", "Hyderabad's lakes and the sloping terrain of its low-lying hills provide habitat for an assortment of flora and fauna. The forest region in and around the city encompasses areas of ecological and biological importance, which are preserved in the form of national parks, zoos, mini-zoos and a wildlife sanctuary. Nehru Zoological Park, the city's one large zoo, is the first in India to have a lion and tiger safari park. Hyderabad has three national parks (Mrugavani National Park, Mahavir Harina Vanasthali National Park and Kasu Brahmananda Reddy National Park), and the Manjira Wildlife Sanctuary is about 50 km (31 mi) from the city. Hyderabad's other environmental reserves are: Kotla Vijayabhaskara Reddy Botanical Gardens, Shamirpet Lake, Hussain Sagar, Fox Sagar Lake, Mir Alam Tank and Patancheru Lake, which is home to regional birds and attracts seasonal migratory birds from different parts of the world. Organisations engaged in environmental and wildlife preservation include the Telangana Forest Department, Indian Council of Forestry Research and Education, the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT), the Animal Welfare Board of India, the Blue Cross of Hyderabad and the University of Hyderabad.", "Hyderabad's lakes and the sloping terrain of its low-lying hills provide habitat for an assortment of flora and fauna. The forest region in and around the city encompasses areas of ecological and biological importance, which are preserved in the form of national parks, zoos, mini-zoos and a wildlife sanctuary. Nehru Zoological Park, the city's one large zoo, is the first in India to have a lion and tiger safari park. Hyderabad has three national parks (Mrugavani National Park, Mahavir Harina Vanasthali National Park and Kasu Brahmananda Reddy National Park), and the Manjira Wildlife Sanctuary is about 50 km (31 mi) from the city. Hyderabad's other environmental reserves are: Kotla Vijayabhaskara Reddy Botanical Gardens, Shamirpet Lake, Hussain Sagar, Fox Sagar Lake, Mir Alam Tank and Patancheru Lake, which is home to regional birds and attracts seasonal migratory birds from different parts of the world. Organisations engaged in environmental and wildlife preservation include the Telangana Forest Department, Indian Council of Forestry Research and Education, the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT), the Animal Welfare Board of India, the Blue Cross of Hyderabad and the University of Hyderabad.", "Hyderabad's lakes and the sloping terrain of its low-lying hills provide habitat for an assortment of flora and fauna. The forest region in and around the city encompasses areas of ecological and biological importance, which are preserved in the form of national parks, zoos, mini-zoos and a wildlife sanctuary. Nehru Zoological Park, the city's one large zoo, is the first in India to have a lion and tiger safari park. Hyderabad has three national parks (Mrugavani National Park, Mahavir Harina Vanasthali National Park and Kasu Brahmananda Reddy National Park), and the Manjira Wildlife Sanctuary is about 50 km (31 mi) from the city. Hyderabad's other environmental reserves are: Kotla Vijayabhaskara Reddy Botanical Gardens, Shamirpet Lake, Hussain Sagar, Fox Sagar Lake, Mir Alam Tank and Patancheru Lake, which is home to regional birds and attracts seasonal migratory birds from different parts of the world. Organisations engaged in environmental and wildlife preservation include the Telangana Forest Department, Indian Council of Forestry Research and Education, the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT), the Animal Welfare Board of India, the Blue Cross of Hyderabad and the University of Hyderabad.", 'The Greater Hyderabad Municipal Corporation (GHMC) oversees the civic infrastructure of the city\'s 18 "circles", which together encompass 150 municipal wards. Each ward is represented by a corporator, elected by popular vote. The corporators elect the Mayor, who is the titular head of GHMC; executive powers rest with the Municipal Commissioner, appointed by the state government. The GHMC carries out the city\'s infrastructural work such as building and maintenance of roads and drains, town planning including construction regulation, maintenance of municipal markets and parks, solid waste management, the issuing of birth and death certificates, the issuing of trade licences, collection of property tax, and community welfare services such as mother and child healthcare, and pre-school and non-formal education. The GHMC was formed in April 2007 by merging the Municipal Corporation of Hyderabad (MCH) with 12 municipalities of the Hyderabad, Ranga Reddy and Medak districts covering a total area of 650 km2 (250 sq mi).:3 In the 2016 municipal election, the Telangana Rashtra Samithi formed the majority and the present Mayor is Bonthu Ram Mohan. The Secunderabad Cantonment Board is a civic administration agency overseeing an area of 40.1 km2 (15.5 sq mi),:93 where there are several military camps.:2 The Osmania University campus is administered independently by the university authority.:93', 'The Greater Hyderabad Municipal Corporation (GHMC) oversees the civic infrastructure of the city\'s 18 "circles", which together encompass 150 municipal wards. Each ward is represented by a corporator, elected by popular vote. The corporators elect the Mayor, who is the titular head of GHMC; executive powers rest with the Municipal Commissioner, appointed by the state government. The GHMC carries out the city\'s infrastructural work such as building and maintenance of roads and drains, town planning including construction regulation, maintenance of municipal markets and parks, solid waste management, the issuing of birth and death certificates, the issuing of trade licences, collection of property tax, and community welfare services such as mother and child healthcare, and pre-school and non-formal education. The GHMC was formed in April 2007 by merging the Municipal Corporation of Hyderabad (MCH) with 12 municipalities of the Hyderabad, Ranga Reddy and Medak districts covering a total area of 650 km2 (250 sq mi).:3 In the 2016 municipal election, the Telangana Rashtra Samithi formed the majority and the present Mayor is Bonthu Ram Mohan. The Secunderabad Cantonment Board is a civic administration agency overseeing an area of 40.1 km2 (15.5 sq mi),:93 where there are several military camps.:2 The Osmania University campus is administered independently by the university authority.:93', 'The Greater Hyderabad Municipal Corporation (GHMC) oversees the civic infrastructure of the city\'s 18 "circles", which together encompass 150 municipal wards. Each ward is represented by a corporator, elected by popular vote. The corporators elect the Mayor, who is the titular head of GHMC; executive powers rest with the Municipal Commissioner, appointed by the state government. The GHMC carries out the city\'s infrastructural work such as building and maintenance of roads and drains, town planning including construction regulation, maintenance of municipal markets and parks, solid waste management, the issuing of birth and death certificates, the issuing of trade licences, collection of property tax, and community welfare services such as mother and child healthcare, and pre-school and non-formal education. The GHMC was formed in April 2007 by merging the Municipal Corporation of Hyderabad (MCH) with 12 municipalities of the Hyderabad, Ranga Reddy and Medak districts covering a total area of 650 km2 (250 sq mi).:3 In the 2016 municipal election, the Telangana Rashtra Samithi formed the majority and the present Mayor is Bonthu Ram Mohan. The Secunderabad Cantonment Board is a civic administration agency overseeing an area of 40.1 km2 (15.5 sq mi),:93 where there are several military camps.:2 The Osmania University campus is administered independently by the university authority.:93', 'The Greater Hyderabad Municipal Corporation (GHMC) oversees the civic infrastructure of the city\'s 18 "circles", which together encompass 150 municipal wards. Each ward is represented by a corporator, elected by popular vote. The corporators elect the Mayor, who is the titular head of GHMC; executive powers rest with the Municipal Commissioner, appointed by the state government. The GHMC carries out the city\'s infrastructural work such as building and maintenance of roads and drains, town planning including construction regulation, maintenance of municipal markets and parks, solid waste management, the issuing of birth and death certificates, the issuing of trade licences, collection of property tax, and community welfare services such as mother and child healthcare, and pre-school and non-formal education. The GHMC was formed in April 2007 by merging the Municipal Corporation of Hyderabad (MCH) with 12 municipalities of the Hyderabad, Ranga Reddy and Medak districts covering a total area of 650 km2 (250 sq mi).:3 In the 2016 municipal election, the Telangana Rashtra Samithi formed the majority and the present Mayor is Bonthu Ram Mohan. The Secunderabad Cantonment Board is a civic administration agency overseeing an area of 40.1 km2 (15.5 sq mi),:93 where there are several military camps.:2 The Osmania University campus is administered independently by the university authority.:93', 'The Greater Hyderabad Municipal Corporation (GHMC) oversees the civic infrastructure of the city\'s 18 "circles", which together encompass 150 municipal wards. Each ward is represented by a corporator, elected by popular vote. The corporators elect the Mayor, who is the titular head of GHMC; executive powers rest with the Municipal Commissioner, appointed by the state government. The GHMC carries out the city\'s infrastructural work such as building and maintenance of roads and drains, town planning including construction regulation, maintenance of municipal markets and parks, solid waste management, the issuing of birth and death certificates, the issuing of trade licences, collection of property tax, and community welfare services such as mother and child healthcare, and pre-school and non-formal education. The GHMC was formed in April 2007 by merging the Municipal Corporation of Hyderabad (MCH) with 12 municipalities of the Hyderabad, Ranga Reddy and Medak districts covering a total area of 650 km2 (250 sq mi).:3 In the 2016 municipal election, the Telangana Rashtra Samithi formed the majority and the present Mayor is Bonthu Ram Mohan. The Secunderabad Cantonment Board is a civic administration agency overseeing an area of 40.1 km2 (15.5 sq mi),:93 where there are several military camps.:2 The Osmania University campus is administered independently by the university authority.:93', 'The jurisdictions of the city\'s administrative agencies are, in ascending order of size: the Hyderabad Police area, Hyderabad district, the GHMC area ("Hyderabad city") and the area under the Hyderabad Metropolitan Development Authority (HMDA). The HMDA is an apolitical urban planning agency that covers the GHMC and its suburbs, extending to 54 mandals in five districts encircling the city. It coordinates the development activities of GHMC and suburban municipalities and manages the administration of bodies such as the Hyderabad Metropolitan Water Supply and Sewerage Board (HMWSSB).', 'The jurisdictions of the city\'s administrative agencies are, in ascending order of size: the Hyderabad Police area, Hyderabad district, the GHMC area ("Hyderabad city") and the area under the Hyderabad Metropolitan Development Authority (HMDA). The HMDA is an apolitical urban planning agency that covers the GHMC and its suburbs, extending to 54 mandals in five districts encircling the city. It coordinates the development activities of GHMC and suburban municipalities and manages the administration of bodies such as the Hyderabad Metropolitan Water Supply and Sewerage Board (HMWSSB).', 'The jurisdictions of the city\'s administrative agencies are, in ascending order of size: the Hyderabad Police area, Hyderabad district, the GHMC area ("Hyderabad city") and the area under the Hyderabad Metropolitan Development Authority (HMDA). The HMDA is an apolitical urban planning agency that covers the GHMC and its suburbs, extending to 54 mandals in five districts encircling the city. It coordinates the development activities of GHMC and suburban municipalities and manages the administration of bodies such as the Hyderabad Metropolitan Water Supply and Sewerage Board (HMWSSB).', 'The jurisdictions of the city\'s administrative agencies are, in ascending order of size: the Hyderabad Police area, Hyderabad district, the GHMC area ("Hyderabad city") and the area under the Hyderabad Metropolitan Development Authority (HMDA). The HMDA is an apolitical urban planning agency that covers the GHMC and its suburbs, extending to 54 mandals in five districts encircling the city. It coordinates the development activities of GHMC and suburban municipalities and manages the administration of bodies such as the Hyderabad Metropolitan Water Supply and Sewerage Board (HMWSSB).', 'The jurisdictions of the city\'s administrative agencies are, in ascending order of size: the Hyderabad Police area, Hyderabad district, the GHMC area ("Hyderabad city") and the area under the Hyderabad Metropolitan Development Authority (HMDA). The HMDA is an apolitical urban planning agency that covers the GHMC and its suburbs, extending to 54 mandals in five districts encircling the city. It coordinates the development activities of GHMC and suburban municipalities and manages the administration of bodies such as the Hyderabad Metropolitan Water Supply and Sewerage Board (HMWSSB).', 'The HMWSSB regulates rainwater harvesting, sewerage services and water supply, which is sourced from several dams located in the suburbs. In 2005, the HMWSSB started operating a 116-kilometre-long (72 mi) water supply pipeline from Nagarjuna Sagar Dam to meet increasing demand. The Telangana Southern Power Distribution Company Limited manages electricity supply. As of October 2014, there were 15 fire stations in the city, operated by the Telangana State Disaster and Fire Response Department. The government-owned India Post has five head post offices and many sub-post offices in Hyderabad, which are complemented by private courier services.', 'The HMWSSB regulates rainwater harvesting, sewerage services and water supply, which is sourced from several dams located in the suburbs. In 2005, the HMWSSB started operating a 116-kilometre-long (72 mi) water supply pipeline from Nagarjuna Sagar Dam to meet increasing demand. The Telangana Southern Power Distribution Company Limited manages electricity supply. As of October 2014, there were 15 fire stations in the city, operated by the Telangana State Disaster and Fire Response Department. The government-owned India Post has five head post offices and many sub-post offices in Hyderabad, which are complemented by private courier services.', 'The HMWSSB regulates rainwater harvesting, sewerage services and water supply, which is sourced from several dams located in the suburbs. In 2005, the HMWSSB started operating a 116-kilometre-long (72 mi) water supply pipeline from Nagarjuna Sagar Dam to meet increasing demand. The Telangana Southern Power Distribution Company Limited manages electricity supply. As of October 2014, there were 15 fire stations in the city, operated by the Telangana State Disaster and Fire Response Department. The government-owned India Post has five head post offices and many sub-post offices in Hyderabad, which are complemented by private courier services.', 'The HMWSSB regulates rainwater harvesting, sewerage services and water supply, which is sourced from several dams located in the suburbs. In 2005, the HMWSSB started operating a 116-kilometre-long (72 mi) water supply pipeline from Nagarjuna Sagar Dam to meet increasing demand. The Telangana Southern Power Distribution Company Limited manages electricity supply. As of October 2014, there were 15 fire stations in the city, operated by the Telangana State Disaster and Fire Response Department. The government-owned India Post has five head post offices and many sub-post offices in Hyderabad, which are complemented by private courier services.', 'The HMWSSB regulates rainwater harvesting, sewerage services and water supply, which is sourced from several dams located in the suburbs. In 2005, the HMWSSB started operating a 116-kilometre-long (72 mi) water supply pipeline from Nagarjuna Sagar Dam to meet increasing demand. The Telangana Southern Power Distribution Company Limited manages electricity supply. As of October 2014, there were 15 fire stations in the city, operated by the Telangana State Disaster and Fire Response Department. The government-owned India Post has five head post offices and many sub-post offices in Hyderabad, which are complemented by private courier services.', 'Hyderabad produces around 4,500 tonnes of solid waste daily, which is transported from collection units in Imlibun, Yousufguda and Lower Tank Bund to the dumpsite in Jawaharnagar. Disposal is managed by the Integrated Solid Waste Management project which was started by the GHMC in 2010. Rapid urbanisation and increased economic activity has also led to increased industrial waste, air, noise and water pollution, which is regulated by the Telangana Pollution Control Board (TPCB). The contribution of different sources to air pollution in 2006 was: 20–50% from vehicles, 40–70% from a combination of vehicle discharge and road dust, 10–30% from industrial discharges and 3–10% from the burning of household rubbish. Deaths resulting from atmospheric particulate matter are estimated at 1,700–3,000 each year. Ground water around Hyderabad, which has a hardness of up to 1000 ppm, around three times higher than is desirable, is the main source of drinking water but the increasing population and consequent increase in demand has led to a decline in not only ground water but also river and lake levels. This shortage is further exacerbated by inadequately treated effluent discharged from industrial treatment plants polluting the water sources of the city.', 'Hyderabad produces around 4,500 tonnes of solid waste daily, which is transported from collection units in Imlibun, Yousufguda and Lower Tank Bund to the dumpsite in Jawaharnagar. Disposal is managed by the Integrated Solid Waste Management project which was started by the GHMC in 2010. Rapid urbanisation and increased economic activity has also led to increased industrial waste, air, noise and water pollution, which is regulated by the Telangana Pollution Control Board (TPCB). The contribution of different sources to air pollution in 2006 was: 20–50% from vehicles, 40–70% from a combination of vehicle discharge and road dust, 10–30% from industrial discharges and 3–10% from the burning of household rubbish. Deaths resulting from atmospheric particulate matter are estimated at 1,700–3,000 each year. Ground water around Hyderabad, which has a hardness of up to 1000 ppm, around three times higher than is desirable, is the main source of drinking water but the increasing population and consequent increase in demand has led to a decline in not only ground water but also river and lake levels. This shortage is further exacerbated by inadequately treated effluent discharged from industrial treatment plants polluting the water sources of the city.', 'Hyderabad produces around 4,500 tonnes of solid waste daily, which is transported from collection units in Imlibun, Yousufguda and Lower Tank Bund to the dumpsite in Jawaharnagar. Disposal is managed by the Integrated Solid Waste Management project which was started by the GHMC in 2010. Rapid urbanisation and increased economic activity has also led to increased industrial waste, air, noise and water pollution, which is regulated by the Telangana Pollution Control Board (TPCB). The contribution of different sources to air pollution in 2006 was: 20–50% from vehicles, 40–70% from a combination of vehicle discharge and road dust, 10–30% from industrial discharges and 3–10% from the burning of household rubbish. Deaths resulting from atmospheric particulate matter are estimated at 1,700–3,000 each year. Ground water around Hyderabad, which has a hardness of up to 1000 ppm, around three times higher than is desirable, is the main source of drinking water but the increasing population and consequent increase in demand has led to a decline in not only ground water but also river and lake levels. This shortage is further exacerbated by inadequately treated effluent discharged from industrial treatment plants polluting the water sources of the city.', 'Hyderabad produces around 4,500 tonnes of solid waste daily, which is transported from collection units in Imlibun, Yousufguda and Lower Tank Bund to the dumpsite in Jawaharnagar. Disposal is managed by the Integrated Solid Waste Management project which was started by the GHMC in 2010. Rapid urbanisation and increased economic activity has also led to increased industrial waste, air, noise and water pollution, which is regulated by the Telangana Pollution Control Board (TPCB). The contribution of different sources to air pollution in 2006 was: 20–50% from vehicles, 40–70% from a combination of vehicle discharge and road dust, 10–30% from industrial discharges and 3–10% from the burning of household rubbish. Deaths resulting from atmospheric particulate matter are estimated at 1,700–3,000 each year. Ground water around Hyderabad, which has a hardness of up to 1000 ppm, around three times higher than is desirable, is the main source of drinking water but the increasing population and consequent increase in demand has led to a decline in not only ground water but also river and lake levels. This shortage is further exacerbated by inadequately treated effluent discharged from industrial treatment plants polluting the water sources of the city.', 'Hyderabad produces around 4,500 tonnes of solid waste daily, which is transported from collection units in Imlibun, Yousufguda and Lower Tank Bund to the dumpsite in Jawaharnagar. Disposal is managed by the Integrated Solid Waste Management project which was started by the GHMC in 2010. Rapid urbanisation and increased economic activity has also led to increased industrial waste, air, noise and water pollution, which is regulated by the Telangana Pollution Control Board (TPCB). The contribution of different sources to air pollution in 2006 was: 20–50% from vehicles, 40–70% from a combination of vehicle discharge and road dust, 10–30% from industrial discharges and 3–10% from the burning of household rubbish. Deaths resulting from atmospheric particulate matter are estimated at 1,700–3,000 each year. Ground water around Hyderabad, which has a hardness of up to 1000 ppm, around three times higher than is desirable, is the main source of drinking water but the increasing population and consequent increase in demand has led to a decline in not only ground water but also river and lake levels. This shortage is further exacerbated by inadequately treated effluent discharged from industrial treatment plants polluting the water sources of the city.', "The Commissionerate of Health and Family Welfare is responsible for planning, implementation and monitoring of all facilities related to health and preventive services. As of 2010[update]–11, the city had 50 government hospitals, 300 private and charity hospitals and 194 nursing homes providing around 12,000 hospital beds, fewer than half the required 25,000. For every 10,000 people in the city, there are 17.6 hospital beds, 9 specialist doctors, 14 nurses and 6 physicians. The city also has about 4,000 individual clinics and 500 medical diagnostic centres. Private clinics are preferred by many residents because of the distance to, poor quality of care at and long waiting times in government facilities,:60–61 despite the high proportion of the city's residents being covered by government health insurance: 24% according to a National Family Health Survey in 2005.:41 As of 2012[update], many new private hospitals of various sizes were opened or being built. Hyderabad also has outpatient and inpatient facilities that use Unani, homeopathic and Ayurvedic treatments.", "The Commissionerate of Health and Family Welfare is responsible for planning, implementation and monitoring of all facilities related to health and preventive services. As of 2010[update]–11, the city had 50 government hospitals, 300 private and charity hospitals and 194 nursing homes providing around 12,000 hospital beds, fewer than half the required 25,000. For every 10,000 people in the city, there are 17.6 hospital beds, 9 specialist doctors, 14 nurses and 6 physicians. The city also has about 4,000 individual clinics and 500 medical diagnostic centres. Private clinics are preferred by many residents because of the distance to, poor quality of care at and long waiting times in government facilities,:60–61 despite the high proportion of the city's residents being covered by government health insurance: 24% according to a National Family Health Survey in 2005.:41 As of 2012[update], many new private hospitals of various sizes were opened or being built. Hyderabad also has outpatient and inpatient facilities that use Unani, homeopathic and Ayurvedic treatments.", "The Commissionerate of Health and Family Welfare is responsible for planning, implementation and monitoring of all facilities related to health and preventive services. As of 2010[update]–11, the city had 50 government hospitals, 300 private and charity hospitals and 194 nursing homes providing around 12,000 hospital beds, fewer than half the required 25,000. For every 10,000 people in the city, there are 17.6 hospital beds, 9 specialist doctors, 14 nurses and 6 physicians. The city also has about 4,000 individual clinics and 500 medical diagnostic centres. Private clinics are preferred by many residents because of the distance to, poor quality of care at and long waiting times in government facilities,:60–61 despite the high proportion of the city's residents being covered by government health insurance: 24% according to a National Family Health Survey in 2005.:41 As of 2012[update], many new private hospitals of various sizes were opened or being built. Hyderabad also has outpatient and inpatient facilities that use Unani, homeopathic and Ayurvedic treatments.", "The Commissionerate of Health and Family Welfare is responsible for planning, implementation and monitoring of all facilities related to health and preventive services. As of 2010[update]–11, the city had 50 government hospitals, 300 private and charity hospitals and 194 nursing homes providing around 12,000 hospital beds, fewer than half the required 25,000. For every 10,000 people in the city, there are 17.6 hospital beds, 9 specialist doctors, 14 nurses and 6 physicians. The city also has about 4,000 individual clinics and 500 medical diagnostic centres. Private clinics are preferred by many residents because of the distance to, poor quality of care at and long waiting times in government facilities,:60–61 despite the high proportion of the city's residents being covered by government health insurance: 24% according to a National Family Health Survey in 2005.:41 As of 2012[update], many new private hospitals of various sizes were opened or being built. Hyderabad also has outpatient and inpatient facilities that use Unani, homeopathic and Ayurvedic treatments.", "The Commissionerate of Health and Family Welfare is responsible for planning, implementation and monitoring of all facilities related to health and preventive services. As of 2010[update]–11, the city had 50 government hospitals, 300 private and charity hospitals and 194 nursing homes providing around 12,000 hospital beds, fewer than half the required 25,000. For every 10,000 people in the city, there are 17.6 hospital beds, 9 specialist doctors, 14 nurses and 6 physicians. The city also has about 4,000 individual clinics and 500 medical diagnostic centres. Private clinics are preferred by many residents because of the distance to, poor quality of care at and long waiting times in government facilities,:60–61 despite the high proportion of the city's residents being covered by government health insurance: 24% according to a National Family Health Survey in 2005.:41 As of 2012[update], many new private hospitals of various sizes were opened or being built. Hyderabad also has outpatient and inpatient facilities that use Unani, homeopathic and Ayurvedic treatments.", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "In the 2005 National Family Health Survey, it was reported that the city's total fertility rate is 1.8,:47 which is below the replacement rate. Only 61% of children had been provided with all basic vaccines (BCG, measles and full courses of polio and DPT), fewer than in all other surveyed cities except Meerut.:98 The infant mortality rate was 35 per 1,000 live births, and the mortality rate for children under five was 41 per 1,000 live births.:97 The survey also reported that a third of women and a quarter of men are overweight or obese, 49% of children below 5 years are anaemic, and up to 20% of children are underweight,:44, 55–56 while more than 2% of women and 3% of men suffer from diabetes.:57", "When the GHMC was created in 2007, the area occupied by the municipality increased from 175 km2 (68 sq mi) to 650 km2 (250 sq mi). Consequently, the population increased by 87%, from 3,637,483 in the 2001 census to 6,809,970 in the 2011 census, 24% of which are migrants from elsewhere in India,:2 making Hyderabad the nation's fourth most populous city. As of 2011[update], the population density is 18,480/km2 (47,900/sq mi). At the same 2011 census, the Hyderabad Urban Agglomeration had a population of 7,749,334, making it the sixth most populous urban agglomeration in the country. The population of the Hyderabad urban agglomeration has since been estimated by electoral officials to be 9.1 million as of early 2013 but is expected to exceed 10 million by the end of the year. There are 3,500,802 male and 3,309,168 female citizens—a sex ratio of 945 females per 1000 males, higher than the national average of 926 per 1000. Among children aged 0–6 years, 373,794 are boys and 352,022 are girls—a ratio of 942 per 1000. Literacy stands at 82.96% (male 85.96%; female 79.79%), higher than the national average of 74.04%. The socio-economic strata consist of 20% upper class, 50% middle class and 30% working class.", "When the GHMC was created in 2007, the area occupied by the municipality increased from 175 km2 (68 sq mi) to 650 km2 (250 sq mi). Consequently, the population increased by 87%, from 3,637,483 in the 2001 census to 6,809,970 in the 2011 census, 24% of which are migrants from elsewhere in India,:2 making Hyderabad the nation's fourth most populous city. As of 2011[update], the population density is 18,480/km2 (47,900/sq mi). At the same 2011 census, the Hyderabad Urban Agglomeration had a population of 7,749,334, making it the sixth most populous urban agglomeration in the country. The population of the Hyderabad urban agglomeration has since been estimated by electoral officials to be 9.1 million as of early 2013 but is expected to exceed 10 million by the end of the year. There are 3,500,802 male and 3,309,168 female citizens—a sex ratio of 945 females per 1000 males, higher than the national average of 926 per 1000. Among children aged 0–6 years, 373,794 are boys and 352,022 are girls—a ratio of 942 per 1000. Literacy stands at 82.96% (male 85.96%; female 79.79%), higher than the national average of 74.04%. The socio-economic strata consist of 20% upper class, 50% middle class and 30% working class.", "When the GHMC was created in 2007, the area occupied by the municipality increased from 175 km2 (68 sq mi) to 650 km2 (250 sq mi). Consequently, the population increased by 87%, from 3,637,483 in the 2001 census to 6,809,970 in the 2011 census, 24% of which are migrants from elsewhere in India,:2 making Hyderabad the nation's fourth most populous city. As of 2011[update], the population density is 18,480/km2 (47,900/sq mi). At the same 2011 census, the Hyderabad Urban Agglomeration had a population of 7,749,334, making it the sixth most populous urban agglomeration in the country. The population of the Hyderabad urban agglomeration has since been estimated by electoral officials to be 9.1 million as of early 2013 but is expected to exceed 10 million by the end of the year. There are 3,500,802 male and 3,309,168 female citizens—a sex ratio of 945 females per 1000 males, higher than the national average of 926 per 1000. Among children aged 0–6 years, 373,794 are boys and 352,022 are girls—a ratio of 942 per 1000. Literacy stands at 82.96% (male 85.96%; female 79.79%), higher than the national average of 74.04%. The socio-economic strata consist of 20% upper class, 50% middle class and 30% working class.", "When the GHMC was created in 2007, the area occupied by the municipality increased from 175 km2 (68 sq mi) to 650 km2 (250 sq mi). Consequently, the population increased by 87%, from 3,637,483 in the 2001 census to 6,809,970 in the 2011 census, 24% of which are migrants from elsewhere in India,:2 making Hyderabad the nation's fourth most populous city. As of 2011[update], the population density is 18,480/km2 (47,900/sq mi). At the same 2011 census, the Hyderabad Urban Agglomeration had a population of 7,749,334, making it the sixth most populous urban agglomeration in the country. The population of the Hyderabad urban agglomeration has since been estimated by electoral officials to be 9.1 million as of early 2013 but is expected to exceed 10 million by the end of the year. There are 3,500,802 male and 3,309,168 female citizens—a sex ratio of 945 females per 1000 males, higher than the national average of 926 per 1000. Among children aged 0–6 years, 373,794 are boys and 352,022 are girls—a ratio of 942 per 1000. Literacy stands at 82.96% (male 85.96%; female 79.79%), higher than the national average of 74.04%. The socio-economic strata consist of 20% upper class, 50% middle class and 30% working class.", "When the GHMC was created in 2007, the area occupied by the municipality increased from 175 km2 (68 sq mi) to 650 km2 (250 sq mi). Consequently, the population increased by 87%, from 3,637,483 in the 2001 census to 6,809,970 in the 2011 census, 24% of which are migrants from elsewhere in India,:2 making Hyderabad the nation's fourth most populous city. As of 2011[update], the population density is 18,480/km2 (47,900/sq mi). At the same 2011 census, the Hyderabad Urban Agglomeration had a population of 7,749,334, making it the sixth most populous urban agglomeration in the country. The population of the Hyderabad urban agglomeration has since been estimated by electoral officials to be 9.1 million as of early 2013 but is expected to exceed 10 million by the end of the year. There are 3,500,802 male and 3,309,168 female citizens—a sex ratio of 945 females per 1000 males, higher than the national average of 926 per 1000. Among children aged 0–6 years, 373,794 are boys and 352,022 are girls—a ratio of 942 per 1000. Literacy stands at 82.96% (male 85.96%; female 79.79%), higher than the national average of 74.04%. The socio-economic strata consist of 20% upper class, 50% middle class and 30% working class.", 'Referred to as "Hyderabadi", the residents of Hyderabad are predominantly Telugu and Urdu speaking people, with minority Bengali, Gujarati (including Memon), Kannada (including Nawayathi), Malayalam, Marathi, Marwari, Odia, Punjabi, Tamil and Uttar Pradeshi communities. Hyderabad is home to a unique dialect of Urdu called Hyderabadi Urdu, which is a type of Dakhini, and is the mother tongue of most Hyderabadi Muslims, a unique community who owe much of their history, language, cuisine, and culture to Hyderabad, and the various dynasties who previously ruled. Hadhrami Arabs, African Arabs, Armenians, Abyssinians, Iranians, Pathans and Turkish people are also present; these communities, of which the Hadhrami are the largest, declined after Hyderabad State became part of the Indian Union, as they lost the patronage of the Nizams.', 'Referred to as "Hyderabadi", the residents of Hyderabad are predominantly Telugu and Urdu speaking people, with minority Bengali, Gujarati (including Memon), Kannada (including Nawayathi), Malayalam, Marathi, Marwari, Odia, Punjabi, Tamil and Uttar Pradeshi communities. Hyderabad is home to a unique dialect of Urdu called Hyderabadi Urdu, which is a type of Dakhini, and is the mother tongue of most Hyderabadi Muslims, a unique community who owe much of their history, language, cuisine, and culture to Hyderabad, and the various dynasties who previously ruled. Hadhrami Arabs, African Arabs, Armenians, Abyssinians, Iranians, Pathans and Turkish people are also present; these communities, of which the Hadhrami are the largest, declined after Hyderabad State became part of the Indian Union, as they lost the patronage of the Nizams.', 'Referred to as "Hyderabadi", the residents of Hyderabad are predominantly Telugu and Urdu speaking people, with minority Bengali, Gujarati (including Memon), Kannada (including Nawayathi), Malayalam, Marathi, Marwari, Odia, Punjabi, Tamil and Uttar Pradeshi communities. Hyderabad is home to a unique dialect of Urdu called Hyderabadi Urdu, which is a type of Dakhini, and is the mother tongue of most Hyderabadi Muslims, a unique community who owe much of their history, language, cuisine, and culture to Hyderabad, and the various dynasties who previously ruled. Hadhrami Arabs, African Arabs, Armenians, Abyssinians, Iranians, Pathans and Turkish people are also present; these communities, of which the Hadhrami are the largest, declined after Hyderabad State became part of the Indian Union, as they lost the patronage of the Nizams.', 'Referred to as "Hyderabadi", the residents of Hyderabad are predominantly Telugu and Urdu speaking people, with minority Bengali, Gujarati (including Memon), Kannada (including Nawayathi), Malayalam, Marathi, Marwari, Odia, Punjabi, Tamil and Uttar Pradeshi communities. Hyderabad is home to a unique dialect of Urdu called Hyderabadi Urdu, which is a type of Dakhini, and is the mother tongue of most Hyderabadi Muslims, a unique community who owe much of their history, language, cuisine, and culture to Hyderabad, and the various dynasties who previously ruled. Hadhrami Arabs, African Arabs, Armenians, Abyssinians, Iranians, Pathans and Turkish people are also present; these communities, of which the Hadhrami are the largest, declined after Hyderabad State became part of the Indian Union, as they lost the patronage of the Nizams.', 'Referred to as "Hyderabadi", the residents of Hyderabad are predominantly Telugu and Urdu speaking people, with minority Bengali, Gujarati (including Memon), Kannada (including Nawayathi), Malayalam, Marathi, Marwari, Odia, Punjabi, Tamil and Uttar Pradeshi communities. Hyderabad is home to a unique dialect of Urdu called Hyderabadi Urdu, which is a type of Dakhini, and is the mother tongue of most Hyderabadi Muslims, a unique community who owe much of their history, language, cuisine, and culture to Hyderabad, and the various dynasties who previously ruled. Hadhrami Arabs, African Arabs, Armenians, Abyssinians, Iranians, Pathans and Turkish people are also present; these communities, of which the Hadhrami are the largest, declined after Hyderabad State became part of the Indian Union, as they lost the patronage of the Nizams.', 'In the greater metropolitan area, 13% of the population live below the poverty line. According to a 2012 report submitted by GHMC to the World Bank, Hyderabad has 1,476 slums with a total population of 1.7 million, of whom 66% live in 985 slums in the "core" of the city (the part that formed Hyderabad before the April 2007 expansion) and the remaining 34% live in 491 suburban tenements. About 22% of the slum-dwelling households had migrated from different parts of India in the last decade of the 20th century, and 63% claimed to have lived in the slums for more than 10 years.:55 Overall literacy in the slums is 60–80% and female literacy is 52–73%. A third of the slums have basic service connections, and the remainder depend on general public services provided by the government. There are 405 government schools, 267 government aided schools, 175 private schools and 528 community halls in the slum areas.:70 According to a 2008 survey by the Centre for Good Governance, 87.6% of the slum-dwelling households are nuclear families, 18% are very poor, with an income up to ₹20000 (US$300) per annum, 73% live below the poverty line (a standard poverty line recognised by the Andhra Pradesh Government is ₹24000 (US$360) per annum), 27% of the chief wage earners (CWE) are casual labour and 38% of the CWE are illiterate. About 3.72% of the slum children aged 5–14 do not go to school and 3.17% work as child labour, of whom 64% are boys and 36% are girls. The largest employers of child labour are street shops and construction sites. Among the working children, 35% are engaged in hazardous jobs.:59', 'In the greater metropolitan area, 13% of the population live below the poverty line. According to a 2012 report submitted by GHMC to the World Bank, Hyderabad has 1,476 slums with a total population of 1.7 million, of whom 66% live in 985 slums in the "core" of the city (the part that formed Hyderabad before the April 2007 expansion) and the remaining 34% live in 491 suburban tenements. About 22% of the slum-dwelling households had migrated from different parts of India in the last decade of the 20th century, and 63% claimed to have lived in the slums for more than 10 years.:55 Overall literacy in the slums is 60–80% and female literacy is 52–73%. A third of the slums have basic service connections, and the remainder depend on general public services provided by the government. There are 405 government schools, 267 government aided schools, 175 private schools and 528 community halls in the slum areas.:70 According to a 2008 survey by the Centre for Good Governance, 87.6% of the slum-dwelling households are nuclear families, 18% are very poor, with an income up to ₹20000 (US$300) per annum, 73% live below the poverty line (a standard poverty line recognised by the Andhra Pradesh Government is ₹24000 (US$360) per annum), 27% of the chief wage earners (CWE) are casual labour and 38% of the CWE are illiterate. About 3.72% of the slum children aged 5–14 do not go to school and 3.17% work as child labour, of whom 64% are boys and 36% are girls. The largest employers of child labour are street shops and construction sites. Among the working children, 35% are engaged in hazardous jobs.:59', 'In the greater metropolitan area, 13% of the population live below the poverty line. According to a 2012 report submitted by GHMC to the World Bank, Hyderabad has 1,476 slums with a total population of 1.7 million, of whom 66% live in 985 slums in the "core" of the city (the part that formed Hyderabad before the April 2007 expansion) and the remaining 34% live in 491 suburban tenements. About 22% of the slum-dwelling households had migrated from different parts of India in the last decade of the 20th century, and 63% claimed to have lived in the slums for more than 10 years.:55 Overall literacy in the slums is 60–80% and female literacy is 52–73%. A third of the slums have basic service connections, and the remainder depend on general public services provided by the government. There are 405 government schools, 267 government aided schools, 175 private schools and 528 community halls in the slum areas.:70 According to a 2008 survey by the Centre for Good Governance, 87.6% of the slum-dwelling households are nuclear families, 18% are very poor, with an income up to ₹20000 (US$300) per annum, 73% live below the poverty line (a standard poverty line recognised by the Andhra Pradesh Government is ₹24000 (US$360) per annum), 27% of the chief wage earners (CWE) are casual labour and 38% of the CWE are illiterate. About 3.72% of the slum children aged 5–14 do not go to school and 3.17% work as child labour, of whom 64% are boys and 36% are girls. The largest employers of child labour are street shops and construction sites. Among the working children, 35% are engaged in hazardous jobs.:59', 'In the greater metropolitan area, 13% of the population live below the poverty line. According to a 2012 report submitted by GHMC to the World Bank, Hyderabad has 1,476 slums with a total population of 1.7 million, of whom 66% live in 985 slums in the "core" of the city (the part that formed Hyderabad before the April 2007 expansion) and the remaining 34% live in 491 suburban tenements. About 22% of the slum-dwelling households had migrated from different parts of India in the last decade of the 20th century, and 63% claimed to have lived in the slums for more than 10 years.:55 Overall literacy in the slums is 60–80% and female literacy is 52–73%. A third of the slums have basic service connections, and the remainder depend on general public services provided by the government. There are 405 government schools, 267 government aided schools, 175 private schools and 528 community halls in the slum areas.:70 According to a 2008 survey by the Centre for Good Governance, 87.6% of the slum-dwelling households are nuclear families, 18% are very poor, with an income up to ₹20000 (US$300) per annum, 73% live below the poverty line (a standard poverty line recognised by the Andhra Pradesh Government is ₹24000 (US$360) per annum), 27% of the chief wage earners (CWE) are casual labour and 38% of the CWE are illiterate. About 3.72% of the slum children aged 5–14 do not go to school and 3.17% work as child labour, of whom 64% are boys and 36% are girls. The largest employers of child labour are street shops and construction sites. Among the working children, 35% are engaged in hazardous jobs.:59', 'In the greater metropolitan area, 13% of the population live below the poverty line. According to a 2012 report submitted by GHMC to the World Bank, Hyderabad has 1,476 slums with a total population of 1.7 million, of whom 66% live in 985 slums in the "core" of the city (the part that formed Hyderabad before the April 2007 expansion) and the remaining 34% live in 491 suburban tenements. About 22% of the slum-dwelling households had migrated from different parts of India in the last decade of the 20th century, and 63% claimed to have lived in the slums for more than 10 years.:55 Overall literacy in the slums is 60–80% and female literacy is 52–73%. A third of the slums have basic service connections, and the remainder depend on general public services provided by the government. There are 405 government schools, 267 government aided schools, 175 private schools and 528 community halls in the slum areas.:70 According to a 2008 survey by the Centre for Good Governance, 87.6% of the slum-dwelling households are nuclear families, 18% are very poor, with an income up to ₹20000 (US$300) per annum, 73% live below the poverty line (a standard poverty line recognised by the Andhra Pradesh Government is ₹24000 (US$360) per annum), 27% of the chief wage earners (CWE) are casual labour and 38% of the CWE are illiterate. About 3.72% of the slum children aged 5–14 do not go to school and 3.17% work as child labour, of whom 64% are boys and 36% are girls. The largest employers of child labour are street shops and construction sites. Among the working children, 35% are engaged in hazardous jobs.:59', "Many historic and tourist sites lie in south central Hyderabad, such as the Charminar, the Mecca Masjid, the Salar Jung Museum, the Nizam's Museum, the Falaknuma Palace, and the traditional retail corridor comprising the Pearl Market, Laad Bazaar and Madina Circle. North of the river are hospitals, colleges, major railway stations and business areas such as Begum Bazaar, Koti, Abids, Sultan Bazaar and Moazzam Jahi Market, along with administrative and recreational establishments such as the Reserve Bank of India, the Telangana Secretariat, the Hyderabad Mint, the Telangana Legislature, the Public Gardens, the Nizam Club, the Ravindra Bharathi, the State Museum, the Birla Temple and the Birla Planetarium.", "Many historic and tourist sites lie in south central Hyderabad, such as the Charminar, the Mecca Masjid, the Salar Jung Museum, the Nizam's Museum, the Falaknuma Palace, and the traditional retail corridor comprising the Pearl Market, Laad Bazaar and Madina Circle. North of the river are hospitals, colleges, major railway stations and business areas such as Begum Bazaar, Koti, Abids, Sultan Bazaar and Moazzam Jahi Market, along with administrative and recreational establishments such as the Reserve Bank of India, the Telangana Secretariat, the Hyderabad Mint, the Telangana Legislature, the Public Gardens, the Nizam Club, the Ravindra Bharathi, the State Museum, the Birla Temple and the Birla Planetarium.", "Many historic and tourist sites lie in south central Hyderabad, such as the Charminar, the Mecca Masjid, the Salar Jung Museum, the Nizam's Museum, the Falaknuma Palace, and the traditional retail corridor comprising the Pearl Market, Laad Bazaar and Madina Circle. North of the river are hospitals, colleges, major railway stations and business areas such as Begum Bazaar, Koti, Abids, Sultan Bazaar and Moazzam Jahi Market, along with administrative and recreational establishments such as the Reserve Bank of India, the Telangana Secretariat, the Hyderabad Mint, the Telangana Legislature, the Public Gardens, the Nizam Club, the Ravindra Bharathi, the State Museum, the Birla Temple and the Birla Planetarium.", 'North of central Hyderabad lie Hussain Sagar, Tank Bund Road, Rani Gunj and the Secunderabad Railway Station. Most of the city\'s parks and recreational centres, such as Sanjeevaiah Park, Indira Park, Lumbini Park, NTR Gardens, the Buddha statue and Tankbund Park are located here. In the northwest part of the city there are upscale residential and commercial areas such as Banjara Hills, Jubilee Hills, Begumpet, Khairatabad and Miyapur. The northern end contains industrial areas such as Sanathnagar, Moosapet, Balanagar, Patancheru and Chanda Nagar. The northeast end is dotted with residential areas. In the eastern part of the city lie many defence research centres and Ramoji Film City. The "Cyberabad" area in the southwest and west of the city has grown rapidly since the 1990s. It is home to information technology and bio-pharmaceutical companies and to landmarks such as Hyderabad Airport, Osman Sagar, Himayath Sagar and Kasu Brahmananda Reddy National Park.', 'North of central Hyderabad lie Hussain Sagar, Tank Bund Road, Rani Gunj and the Secunderabad Railway Station. Most of the city\'s parks and recreational centres, such as Sanjeevaiah Park, Indira Park, Lumbini Park, NTR Gardens, the Buddha statue and Tankbund Park are located here. In the northwest part of the city there are upscale residential and commercial areas such as Banjara Hills, Jubilee Hills, Begumpet, Khairatabad and Miyapur. The northern end contains industrial areas such as Sanathnagar, Moosapet, Balanagar, Patancheru and Chanda Nagar. The northeast end is dotted with residential areas. In the eastern part of the city lie many defence research centres and Ramoji Film City. The "Cyberabad" area in the southwest and west of the city has grown rapidly since the 1990s. It is home to information technology and bio-pharmaceutical companies and to landmarks such as Hyderabad Airport, Osman Sagar, Himayath Sagar and Kasu Brahmananda Reddy National Park.', 'North of central Hyderabad lie Hussain Sagar, Tank Bund Road, Rani Gunj and the Secunderabad Railway Station. Most of the city\'s parks and recreational centres, such as Sanjeevaiah Park, Indira Park, Lumbini Park, NTR Gardens, the Buddha statue and Tankbund Park are located here. In the northwest part of the city there are upscale residential and commercial areas such as Banjara Hills, Jubilee Hills, Begumpet, Khairatabad and Miyapur. The northern end contains industrial areas such as Sanathnagar, Moosapet, Balanagar, Patancheru and Chanda Nagar. The northeast end is dotted with residential areas. In the eastern part of the city lie many defence research centres and Ramoji Film City. The "Cyberabad" area in the southwest and west of the city has grown rapidly since the 1990s. It is home to information technology and bio-pharmaceutical companies and to landmarks such as Hyderabad Airport, Osman Sagar, Himayath Sagar and Kasu Brahmananda Reddy National Park.', 'North of central Hyderabad lie Hussain Sagar, Tank Bund Road, Rani Gunj and the Secunderabad Railway Station. Most of the city\'s parks and recreational centres, such as Sanjeevaiah Park, Indira Park, Lumbini Park, NTR Gardens, the Buddha statue and Tankbund Park are located here. In the northwest part of the city there are upscale residential and commercial areas such as Banjara Hills, Jubilee Hills, Begumpet, Khairatabad and Miyapur. The northern end contains industrial areas such as Sanathnagar, Moosapet, Balanagar, Patancheru and Chanda Nagar. The northeast end is dotted with residential areas. In the eastern part of the city lie many defence research centres and Ramoji Film City. The "Cyberabad" area in the southwest and west of the city has grown rapidly since the 1990s. It is home to information technology and bio-pharmaceutical companies and to landmarks such as Hyderabad Airport, Osman Sagar, Himayath Sagar and Kasu Brahmananda Reddy National Park.', 'North of central Hyderabad lie Hussain Sagar, Tank Bund Road, Rani Gunj and the Secunderabad Railway Station. Most of the city\'s parks and recreational centres, such as Sanjeevaiah Park, Indira Park, Lumbini Park, NTR Gardens, the Buddha statue and Tankbund Park are located here. In the northwest part of the city there are upscale residential and commercial areas such as Banjara Hills, Jubilee Hills, Begumpet, Khairatabad and Miyapur. The northern end contains industrial areas such as Sanathnagar, Moosapet, Balanagar, Patancheru and Chanda Nagar. The northeast end is dotted with residential areas. In the eastern part of the city lie many defence research centres and Ramoji Film City. The "Cyberabad" area in the southwest and west of the city has grown rapidly since the 1990s. It is home to information technology and bio-pharmaceutical companies and to landmarks such as Hyderabad Airport, Osman Sagar, Himayath Sagar and Kasu Brahmananda Reddy National Park.', 'Heritage buildings constructed during the Qutb Shahi and Nizam eras showcase Indo-Islamic architecture influenced by Medieval, Mughal and European styles. After the 1908 flooding of the Musi River, the city was expanded and civic monuments constructed, particularly during the rule of Mir Osman Ali Khan (the VIIth Nizam), whose patronage of architecture led to him being referred to as the maker of modern Hyderabad. In 2012, the government of India declared Hyderabad the first "Best heritage city of India".', 'Heritage buildings constructed during the Qutb Shahi and Nizam eras showcase Indo-Islamic architecture influenced by Medieval, Mughal and European styles. After the 1908 flooding of the Musi River, the city was expanded and civic monuments constructed, particularly during the rule of Mir Osman Ali Khan (the VIIth Nizam), whose patronage of architecture led to him being referred to as the maker of modern Hyderabad. In 2012, the government of India declared Hyderabad the first "Best heritage city of India".', 'Heritage buildings constructed during the Qutb Shahi and Nizam eras showcase Indo-Islamic architecture influenced by Medieval, Mughal and European styles. After the 1908 flooding of the Musi River, the city was expanded and civic monuments constructed, particularly during the rule of Mir Osman Ali Khan (the VIIth Nizam), whose patronage of architecture led to him being referred to as the maker of modern Hyderabad. In 2012, the government of India declared Hyderabad the first "Best heritage city of India".', 'Heritage buildings constructed during the Qutb Shahi and Nizam eras showcase Indo-Islamic architecture influenced by Medieval, Mughal and European styles. After the 1908 flooding of the Musi River, the city was expanded and civic monuments constructed, particularly during the rule of Mir Osman Ali Khan (the VIIth Nizam), whose patronage of architecture led to him being referred to as the maker of modern Hyderabad. In 2012, the government of India declared Hyderabad the first "Best heritage city of India".', 'Heritage buildings constructed during the Qutb Shahi and Nizam eras showcase Indo-Islamic architecture influenced by Medieval, Mughal and European styles. After the 1908 flooding of the Musi River, the city was expanded and civic monuments constructed, particularly during the rule of Mir Osman Ali Khan (the VIIth Nizam), whose patronage of architecture led to him being referred to as the maker of modern Hyderabad. In 2012, the government of India declared Hyderabad the first "Best heritage city of India".', 'Qutb Shahi architecture of the 16th and early 17th centuries followed classical Persian architecture featuring domes and colossal arches. The oldest surviving Qutb Shahi structure in Hyderabad is the ruins of Golconda fort built in the 16th century. The Charminar, Mecca Masjid, Charkaman and Qutb Shahi tombs are other existing structures of this period. Among these the Charminar has become an icon of the city; located in the centre of old Hyderabad, it is a square structure with sides 20 m (66 ft) long and four grand arches each facing a road. At each corner stands a 56 m (184 ft)-high minaret. Most of the historical bazaars that still exist were constructed on the street north of Charminar towards Golconda fort. The Charminar, Qutb Shahi tombs and Golconda fort are considered to be monuments of national importance in India; in 2010 the Indian government proposed that the sites be listed for UNESCO World Heritage status.:11–18', 'Qutb Shahi architecture of the 16th and early 17th centuries followed classical Persian architecture featuring domes and colossal arches. The oldest surviving Qutb Shahi structure in Hyderabad is the ruins of Golconda fort built in the 16th century. The Charminar, Mecca Masjid, Charkaman and Qutb Shahi tombs are other existing structures of this period. Among these the Charminar has become an icon of the city; located in the centre of old Hyderabad, it is a square structure with sides 20 m (66 ft) long and four grand arches each facing a road. At each corner stands a 56 m (184 ft)-high minaret. Most of the historical bazaars that still exist were constructed on the street north of Charminar towards Golconda fort. The Charminar, Qutb Shahi tombs and Golconda fort are considered to be monuments of national importance in India; in 2010 the Indian government proposed that the sites be listed for UNESCO World Heritage status.:11–18', 'Qutb Shahi architecture of the 16th and early 17th centuries followed classical Persian architecture featuring domes and colossal arches. The oldest surviving Qutb Shahi structure in Hyderabad is the ruins of Golconda fort built in the 16th century. The Charminar, Mecca Masjid, Charkaman and Qutb Shahi tombs are other existing structures of this period. Among these the Charminar has become an icon of the city; located in the centre of old Hyderabad, it is a square structure with sides 20 m (66 ft) long and four grand arches each facing a road. At each corner stands a 56 m (184 ft)-high minaret. Most of the historical bazaars that still exist were constructed on the street north of Charminar towards Golconda fort. The Charminar, Qutb Shahi tombs and Golconda fort are considered to be monuments of national importance in India; in 2010 the Indian government proposed that the sites be listed for UNESCO World Heritage status.:11–18', 'Qutb Shahi architecture of the 16th and early 17th centuries followed classical Persian architecture featuring domes and colossal arches. The oldest surviving Qutb Shahi structure in Hyderabad is the ruins of Golconda fort built in the 16th century. The Charminar, Mecca Masjid, Charkaman and Qutb Shahi tombs are other existing structures of this period. Among these the Charminar has become an icon of the city; located in the centre of old Hyderabad, it is a square structure with sides 20 m (66 ft) long and four grand arches each facing a road. At each corner stands a 56 m (184 ft)-high minaret. Most of the historical bazaars that still exist were constructed on the street north of Charminar towards Golconda fort. The Charminar, Qutb Shahi tombs and Golconda fort are considered to be monuments of national importance in India; in 2010 the Indian government proposed that the sites be listed for UNESCO World Heritage status.:11–18', 'Qutb Shahi architecture of the 16th and early 17th centuries followed classical Persian architecture featuring domes and colossal arches. The oldest surviving Qutb Shahi structure in Hyderabad is the ruins of Golconda fort built in the 16th century. The Charminar, Mecca Masjid, Charkaman and Qutb Shahi tombs are other existing structures of this period. Among these the Charminar has become an icon of the city; located in the centre of old Hyderabad, it is a square structure with sides 20 m (66 ft) long and four grand arches each facing a road. At each corner stands a 56 m (184 ft)-high minaret. Most of the historical bazaars that still exist were constructed on the street north of Charminar towards Golconda fort. The Charminar, Qutb Shahi tombs and Golconda fort are considered to be monuments of national importance in India; in 2010 the Indian government proposed that the sites be listed for UNESCO World Heritage status.:11–18', "Among the oldest surviving examples of Nizam architecture in Hyderabad is the Chowmahalla Palace, which was the seat of royal power. It showcases a diverse array of architectural styles, from the Baroque Harem to its Neoclassical royal court. The other palaces include Falaknuma Palace (inspired by the style of Andrea Palladio), Purani Haveli, King Kothi and Bella Vista Palace all of which were built at the peak of Nizam rule in the 19th century. During Mir Osman Ali Khan's rule, European styles, along with Indo-Islamic, became prominent. These styles are reflected in the Falaknuma Palace and many civic monuments such as the Hyderabad High Court, Osmania Hospital, Osmania University, the State Central Library, City College, the Telangana Legislature, the State Archaeology Museum, Jubilee Hall, and Hyderabad and Kachiguda railway stations. Other landmarks of note are Paigah Palace, Asman Garh Palace, Basheer Bagh Palace, Errum Manzil and the Spanish Mosque, all constructed by the Paigah family.:16–17", "Among the oldest surviving examples of Nizam architecture in Hyderabad is the Chowmahalla Palace, which was the seat of royal power. It showcases a diverse array of architectural styles, from the Baroque Harem to its Neoclassical royal court. The other palaces include Falaknuma Palace (inspired by the style of Andrea Palladio), Purani Haveli, King Kothi and Bella Vista Palace all of which were built at the peak of Nizam rule in the 19th century. During Mir Osman Ali Khan's rule, European styles, along with Indo-Islamic, became prominent. These styles are reflected in the Falaknuma Palace and many civic monuments such as the Hyderabad High Court, Osmania Hospital, Osmania University, the State Central Library, City College, the Telangana Legislature, the State Archaeology Museum, Jubilee Hall, and Hyderabad and Kachiguda railway stations. Other landmarks of note are Paigah Palace, Asman Garh Palace, Basheer Bagh Palace, Errum Manzil and the Spanish Mosque, all constructed by the Paigah family.:16–17", "Among the oldest surviving examples of Nizam architecture in Hyderabad is the Chowmahalla Palace, which was the seat of royal power. It showcases a diverse array of architectural styles, from the Baroque Harem to its Neoclassical royal court. The other palaces include Falaknuma Palace (inspired by the style of Andrea Palladio), Purani Haveli, King Kothi and Bella Vista Palace all of which were built at the peak of Nizam rule in the 19th century. During Mir Osman Ali Khan's rule, European styles, along with Indo-Islamic, became prominent. These styles are reflected in the Falaknuma Palace and many civic monuments such as the Hyderabad High Court, Osmania Hospital, Osmania University, the State Central Library, City College, the Telangana Legislature, the State Archaeology Museum, Jubilee Hall, and Hyderabad and Kachiguda railway stations. Other landmarks of note are Paigah Palace, Asman Garh Palace, Basheer Bagh Palace, Errum Manzil and the Spanish Mosque, all constructed by the Paigah family.:16–17", "Among the oldest surviving examples of Nizam architecture in Hyderabad is the Chowmahalla Palace, which was the seat of royal power. It showcases a diverse array of architectural styles, from the Baroque Harem to its Neoclassical royal court. The other palaces include Falaknuma Palace (inspired by the style of Andrea Palladio), Purani Haveli, King Kothi and Bella Vista Palace all of which were built at the peak of Nizam rule in the 19th century. During Mir Osman Ali Khan's rule, European styles, along with Indo-Islamic, became prominent. These styles are reflected in the Falaknuma Palace and many civic monuments such as the Hyderabad High Court, Osmania Hospital, Osmania University, the State Central Library, City College, the Telangana Legislature, the State Archaeology Museum, Jubilee Hall, and Hyderabad and Kachiguda railway stations. Other landmarks of note are Paigah Palace, Asman Garh Palace, Basheer Bagh Palace, Errum Manzil and the Spanish Mosque, all constructed by the Paigah family.:16–17", "Among the oldest surviving examples of Nizam architecture in Hyderabad is the Chowmahalla Palace, which was the seat of royal power. It showcases a diverse array of architectural styles, from the Baroque Harem to its Neoclassical royal court. The other palaces include Falaknuma Palace (inspired by the style of Andrea Palladio), Purani Haveli, King Kothi and Bella Vista Palace all of which were built at the peak of Nizam rule in the 19th century. During Mir Osman Ali Khan's rule, European styles, along with Indo-Islamic, became prominent. These styles are reflected in the Falaknuma Palace and many civic monuments such as the Hyderabad High Court, Osmania Hospital, Osmania University, the State Central Library, City College, the Telangana Legislature, the State Archaeology Museum, Jubilee Hall, and Hyderabad and Kachiguda railway stations. Other landmarks of note are Paigah Palace, Asman Garh Palace, Basheer Bagh Palace, Errum Manzil and the Spanish Mosque, all constructed by the Paigah family.:16–17", "Hyderabad is the largest contributor to the gross domestic product (GDP), tax and other revenues, of Telangana, and the sixth largest deposit centre and fourth largest credit centre nationwide, as ranked by the Reserve Bank of India (RBI) in June 2012. Its US$74 billion GDP made it the fifth-largest contributor city to India's overall GDP in 2011–12. Its per capita annual income in 2011 was ₹44300 (US$660). As of 2006[update], the largest employers in the city were the governments of Andhra Pradesh (113,098 employees) and India (85,155). According to a 2005 survey, 77% of males and 19% of females in the city were employed. The service industry remains dominant in the city, and 90% of the employed workforce is engaged in this sector.", "Hyderabad is the largest contributor to the gross domestic product (GDP), tax and other revenues, of Telangana, and the sixth largest deposit centre and fourth largest credit centre nationwide, as ranked by the Reserve Bank of India (RBI) in June 2012. Its US$74 billion GDP made it the fifth-largest contributor city to India's overall GDP in 2011–12. Its per capita annual income in 2011 was ₹44300 (US$660). As of 2006[update], the largest employers in the city were the governments of Andhra Pradesh (113,098 employees) and India (85,155). According to a 2005 survey, 77% of males and 19% of females in the city were employed. The service industry remains dominant in the city, and 90% of the employed workforce is engaged in this sector.", "Hyderabad is the largest contributor to the gross domestic product (GDP), tax and other revenues, of Telangana, and the sixth largest deposit centre and fourth largest credit centre nationwide, as ranked by the Reserve Bank of India (RBI) in June 2012. Its US$74 billion GDP made it the fifth-largest contributor city to India's overall GDP in 2011–12. Its per capita annual income in 2011 was ₹44300 (US$660). As of 2006[update], the largest employers in the city were the governments of Andhra Pradesh (113,098 employees) and India (85,155). According to a 2005 survey, 77% of males and 19% of females in the city were employed. The service industry remains dominant in the city, and 90% of the employed workforce is engaged in this sector.", "Hyderabad is the largest contributor to the gross domestic product (GDP), tax and other revenues, of Telangana, and the sixth largest deposit centre and fourth largest credit centre nationwide, as ranked by the Reserve Bank of India (RBI) in June 2012. Its US$74 billion GDP made it the fifth-largest contributor city to India's overall GDP in 2011–12. Its per capita annual income in 2011 was ₹44300 (US$660). As of 2006[update], the largest employers in the city were the governments of Andhra Pradesh (113,098 employees) and India (85,155). According to a 2005 survey, 77% of males and 19% of females in the city were employed. The service industry remains dominant in the city, and 90% of the employed workforce is engaged in this sector.", "Hyderabad is the largest contributor to the gross domestic product (GDP), tax and other revenues, of Telangana, and the sixth largest deposit centre and fourth largest credit centre nationwide, as ranked by the Reserve Bank of India (RBI) in June 2012. Its US$74 billion GDP made it the fifth-largest contributor city to India's overall GDP in 2011–12. Its per capita annual income in 2011 was ₹44300 (US$660). As of 2006[update], the largest employers in the city were the governments of Andhra Pradesh (113,098 employees) and India (85,155). According to a 2005 survey, 77% of males and 19% of females in the city were employed. The service industry remains dominant in the city, and 90% of the employed workforce is engaged in this sector.", 'Hyderabad\'s role in the pearl trade has given it the name "City of Pearls" and up until the 18th century, the city was also the only global trading centre for large diamonds. Industrialisation began under the Nizams in the late 19th century, helped by railway expansion that connected the city with major ports. From the 1950s to the 1970s, Indian enterprises, such as Bharat Heavy Electricals Limited (BHEL), Nuclear Fuel Complex (NFC), National Mineral Development Corporation (NMDC), Bharat Electronics (BEL), Electronics Corporation of India Limited (ECIL), Defence Research and Development Organisation (DRDO), Hindustan Aeronautics Limited (HAL), Centre for Cellular and Molecular Biology (CCMB), Centre for DNA Fingerprinting and Diagnostics (CDFD), State Bank of Hyderabad (SBH) and Andhra Bank (AB) were established in the city. The city is home to Hyderabad Securities formerly known as Hyderabad Stock Exchange (HSE), and houses the regional office of the Securities and Exchange Board of India (SEBI). In 2013, the Bombay Stock Exchange (BSE) facility in Hyderabad was forecast to provide operations and transactions services to BSE-Mumbai by the end of 2014. The growth of the financial services sector has helped Hyderabad evolve from a traditional manufacturing city to a cosmopolitan industrial service centre. Since the 1990s, the growth of information technology (IT), IT-enabled services (ITES), insurance and financial institutions has expanded the service sector, and these primary economic activities have boosted the ancillary sectors of trade and commerce, transport, storage, communication, real estate and retail.', 'Hyderabad\'s role in the pearl trade has given it the name "City of Pearls" and up until the 18th century, the city was also the only global trading centre for large diamonds. Industrialisation began under the Nizams in the late 19th century, helped by railway expansion that connected the city with major ports. From the 1950s to the 1970s, Indian enterprises, such as Bharat Heavy Electricals Limited (BHEL), Nuclear Fuel Complex (NFC), National Mineral Development Corporation (NMDC), Bharat Electronics (BEL), Electronics Corporation of India Limited (ECIL), Defence Research and Development Organisation (DRDO), Hindustan Aeronautics Limited (HAL), Centre for Cellular and Molecular Biology (CCMB), Centre for DNA Fingerprinting and Diagnostics (CDFD), State Bank of Hyderabad (SBH) and Andhra Bank (AB) were established in the city. The city is home to Hyderabad Securities formerly known as Hyderabad Stock Exchange (HSE), and houses the regional office of the Securities and Exchange Board of India (SEBI). In 2013, the Bombay Stock Exchange (BSE) facility in Hyderabad was forecast to provide operations and transactions services to BSE-Mumbai by the end of 2014. The growth of the financial services sector has helped Hyderabad evolve from a traditional manufacturing city to a cosmopolitan industrial service centre. Since the 1990s, the growth of information technology (IT), IT-enabled services (ITES), insurance and financial institutions has expanded the service sector, and these primary economic activities have boosted the ancillary sectors of trade and commerce, transport, storage, communication, real estate and retail.', 'Hyderabad\'s role in the pearl trade has given it the name "City of Pearls" and up until the 18th century, the city was also the only global trading centre for large diamonds. Industrialisation began under the Nizams in the late 19th century, helped by railway expansion that connected the city with major ports. From the 1950s to the 1970s, Indian enterprises, such as Bharat Heavy Electricals Limited (BHEL), Nuclear Fuel Complex (NFC), National Mineral Development Corporation (NMDC), Bharat Electronics (BEL), Electronics Corporation of India Limited (ECIL), Defence Research and Development Organisation (DRDO), Hindustan Aeronautics Limited (HAL), Centre for Cellular and Molecular Biology (CCMB), Centre for DNA Fingerprinting and Diagnostics (CDFD), State Bank of Hyderabad (SBH) and Andhra Bank (AB) were established in the city. The city is home to Hyderabad Securities formerly known as Hyderabad Stock Exchange (HSE), and houses the regional office of the Securities and Exchange Board of India (SEBI). In 2013, the Bombay Stock Exchange (BSE) facility in Hyderabad was forecast to provide operations and transactions services to BSE-Mumbai by the end of 2014. The growth of the financial services sector has helped Hyderabad evolve from a traditional manufacturing city to a cosmopolitan industrial service centre. Since the 1990s, the growth of information technology (IT), IT-enabled services (ITES), insurance and financial institutions has expanded the service sector, and these primary economic activities have boosted the ancillary sectors of trade and commerce, transport, storage, communication, real estate and retail.', 'Hyderabad\'s role in the pearl trade has given it the name "City of Pearls" and up until the 18th century, the city was also the only global trading centre for large diamonds. Industrialisation began under the Nizams in the late 19th century, helped by railway expansion that connected the city with major ports. From the 1950s to the 1970s, Indian enterprises, such as Bharat Heavy Electricals Limited (BHEL), Nuclear Fuel Complex (NFC), National Mineral Development Corporation (NMDC), Bharat Electronics (BEL), Electronics Corporation of India Limited (ECIL), Defence Research and Development Organisation (DRDO), Hindustan Aeronautics Limited (HAL), Centre for Cellular and Molecular Biology (CCMB), Centre for DNA Fingerprinting and Diagnostics (CDFD), State Bank of Hyderabad (SBH) and Andhra Bank (AB) were established in the city. The city is home to Hyderabad Securities formerly known as Hyderabad Stock Exchange (HSE), and houses the regional office of the Securities and Exchange Board of India (SEBI). In 2013, the Bombay Stock Exchange (BSE) facility in Hyderabad was forecast to provide operations and transactions services to BSE-Mumbai by the end of 2014. The growth of the financial services sector has helped Hyderabad evolve from a traditional manufacturing city to a cosmopolitan industrial service centre. Since the 1990s, the growth of information technology (IT), IT-enabled services (ITES), insurance and financial institutions has expanded the service sector, and these primary economic activities have boosted the ancillary sectors of trade and commerce, transport, storage, communication, real estate and retail.', 'Hyderabad\'s role in the pearl trade has given it the name "City of Pearls" and up until the 18th century, the city was also the only global trading centre for large diamonds. Industrialisation began under the Nizams in the late 19th century, helped by railway expansion that connected the city with major ports. From the 1950s to the 1970s, Indian enterprises, such as Bharat Heavy Electricals Limited (BHEL), Nuclear Fuel Complex (NFC), National Mineral Development Corporation (NMDC), Bharat Electronics (BEL), Electronics Corporation of India Limited (ECIL), Defence Research and Development Organisation (DRDO), Hindustan Aeronautics Limited (HAL), Centre for Cellular and Molecular Biology (CCMB), Centre for DNA Fingerprinting and Diagnostics (CDFD), State Bank of Hyderabad (SBH) and Andhra Bank (AB) were established in the city. The city is home to Hyderabad Securities formerly known as Hyderabad Stock Exchange (HSE), and houses the regional office of the Securities and Exchange Board of India (SEBI). In 2013, the Bombay Stock Exchange (BSE) facility in Hyderabad was forecast to provide operations and transactions services to BSE-Mumbai by the end of 2014. The growth of the financial services sector has helped Hyderabad evolve from a traditional manufacturing city to a cosmopolitan industrial service centre. Since the 1990s, the growth of information technology (IT), IT-enabled services (ITES), insurance and financial institutions has expanded the service sector, and these primary economic activities have boosted the ancillary sectors of trade and commerce, transport, storage, communication, real estate and retail.', 'The establishment of Indian Drugs and Pharmaceuticals Limited (IDPL), a public sector undertaking, in 1961 was followed over the decades by many national and global companies opening manufacturing and research facilities in the city. As of 2010[update], the city manufactured one third of India\'s bulk drugs and 16% of biotechnology products, contributing to its reputation as "India\'s pharmaceutical capital" and the "Genome Valley of India". Hyderabad is a global centre of information technology, for which it is known as Cyberabad (Cyber City). As of 2013[update], it contributed 15% of India\'s and 98% of Andhra Pradesh\'s exports in IT and ITES sectors and 22% of NASSCOM\'s total membership is from the city. The development of HITEC City, a township with extensive technological infrastructure, prompted multinational companies to establish facilities in Hyderabad. The city is home to more than 1300 IT and ITES firms, including global conglomerates such as Microsoft (operating its largest R&D campus outside the US), Google, IBM, Yahoo!, Dell, Facebook,:3 and major Indian firms including Tech Mahindra, Infosys, Tata Consultancy Services (TCS), Polaris and Wipro.:3 In 2009 the World Bank Group ranked the city as the second best Indian city for doing business. The city and its suburbs contain the highest number of special economic zones of any Indian city.', 'The establishment of Indian Drugs and Pharmaceuticals Limited (IDPL), a public sector undertaking, in 1961 was followed over the decades by many national and global companies opening manufacturing and research facilities in the city. As of 2010[update], the city manufactured one third of India\'s bulk drugs and 16% of biotechnology products, contributing to its reputation as "India\'s pharmaceutical capital" and the "Genome Valley of India". Hyderabad is a global centre of information technology, for which it is known as Cyberabad (Cyber City). As of 2013[update], it contributed 15% of India\'s and 98% of Andhra Pradesh\'s exports in IT and ITES sectors and 22% of NASSCOM\'s total membership is from the city. The development of HITEC City, a township with extensive technological infrastructure, prompted multinational companies to establish facilities in Hyderabad. The city is home to more than 1300 IT and ITES firms, including global conglomerates such as Microsoft (operating its largest R&D campus outside the US), Google, IBM, Yahoo!, Dell, Facebook,:3 and major Indian firms including Tech Mahindra, Infosys, Tata Consultancy Services (TCS), Polaris and Wipro.:3 In 2009 the World Bank Group ranked the city as the second best Indian city for doing business. The city and its suburbs contain the highest number of special economic zones of any Indian city.', 'The establishment of Indian Drugs and Pharmaceuticals Limited (IDPL), a public sector undertaking, in 1961 was followed over the decades by many national and global companies opening manufacturing and research facilities in the city. As of 2010[update], the city manufactured one third of India\'s bulk drugs and 16% of biotechnology products, contributing to its reputation as "India\'s pharmaceutical capital" and the "Genome Valley of India". Hyderabad is a global centre of information technology, for which it is known as Cyberabad (Cyber City). As of 2013[update], it contributed 15% of India\'s and 98% of Andhra Pradesh\'s exports in IT and ITES sectors and 22% of NASSCOM\'s total membership is from the city. The development of HITEC City, a township with extensive technological infrastructure, prompted multinational companies to establish facilities in Hyderabad. The city is home to more than 1300 IT and ITES firms, including global conglomerates such as Microsoft (operating its largest R&D campus outside the US), Google, IBM, Yahoo!, Dell, Facebook,:3 and major Indian firms including Tech Mahindra, Infosys, Tata Consultancy Services (TCS), Polaris and Wipro.:3 In 2009 the World Bank Group ranked the city as the second best Indian city for doing business. The city and its suburbs contain the highest number of special economic zones of any Indian city.', 'The establishment of Indian Drugs and Pharmaceuticals Limited (IDPL), a public sector undertaking, in 1961 was followed over the decades by many national and global companies opening manufacturing and research facilities in the city. As of 2010[update], the city manufactured one third of India\'s bulk drugs and 16% of biotechnology products, contributing to its reputation as "India\'s pharmaceutical capital" and the "Genome Valley of India". Hyderabad is a global centre of information technology, for which it is known as Cyberabad (Cyber City). As of 2013[update], it contributed 15% of India\'s and 98% of Andhra Pradesh\'s exports in IT and ITES sectors and 22% of NASSCOM\'s total membership is from the city. The development of HITEC City, a township with extensive technological infrastructure, prompted multinational companies to establish facilities in Hyderabad. The city is home to more than 1300 IT and ITES firms, including global conglomerates such as Microsoft (operating its largest R&D campus outside the US), Google, IBM, Yahoo!, Dell, Facebook,:3 and major Indian firms including Tech Mahindra, Infosys, Tata Consultancy Services (TCS), Polaris and Wipro.:3 In 2009 the World Bank Group ranked the city as the second best Indian city for doing business. The city and its suburbs contain the highest number of special economic zones of any Indian city.', 'The establishment of Indian Drugs and Pharmaceuticals Limited (IDPL), a public sector undertaking, in 1961 was followed over the decades by many national and global companies opening manufacturing and research facilities in the city. As of 2010[update], the city manufactured one third of India\'s bulk drugs and 16% of biotechnology products, contributing to its reputation as "India\'s pharmaceutical capital" and the "Genome Valley of India". Hyderabad is a global centre of information technology, for which it is known as Cyberabad (Cyber City). As of 2013[update], it contributed 15% of India\'s and 98% of Andhra Pradesh\'s exports in IT and ITES sectors and 22% of NASSCOM\'s total membership is from the city. The development of HITEC City, a township with extensive technological infrastructure, prompted multinational companies to establish facilities in Hyderabad. The city is home to more than 1300 IT and ITES firms, including global conglomerates such as Microsoft (operating its largest R&D campus outside the US), Google, IBM, Yahoo!, Dell, Facebook,:3 and major Indian firms including Tech Mahindra, Infosys, Tata Consultancy Services (TCS), Polaris and Wipro.:3 In 2009 the World Bank Group ranked the city as the second best Indian city for doing business. The city and its suburbs contain the highest number of special economic zones of any Indian city.', 'Like the rest of India, Hyderabad has a large informal economy that employs 30% of the labour force.:71 According to a survey published in 2007, it had 40–50,000 street vendors, and their numbers were increasing.:9 Among the street vendors, 84% are male and 16% female,:12 and four fifths are "stationary vendors" operating from a fixed pitch, often with their own stall.:15–16 Most are financed through personal savings; only 8% borrow from moneylenders.:19 Vendor earnings vary from ₹50 (74¢ US) to ₹800 (US$12) per day.:25 Other unorganised economic sectors include dairy, poultry farming, brick manufacturing, casual labour and domestic help. Those involved in the informal economy constitute a major portion of urban poor.:71', 'Like the rest of India, Hyderabad has a large informal economy that employs 30% of the labour force.:71 According to a survey published in 2007, it had 40–50,000 street vendors, and their numbers were increasing.:9 Among the street vendors, 84% are male and 16% female,:12 and four fifths are "stationary vendors" operating from a fixed pitch, often with their own stall.:15–16 Most are financed through personal savings; only 8% borrow from moneylenders.:19 Vendor earnings vary from ₹50 (74¢ US) to ₹800 (US$12) per day.:25 Other unorganised economic sectors include dairy, poultry farming, brick manufacturing, casual labour and domestic help. Those involved in the informal economy constitute a major portion of urban poor.:71', 'Like the rest of India, Hyderabad has a large informal economy that employs 30% of the labour force.:71 According to a survey published in 2007, it had 40–50,000 street vendors, and their numbers were increasing.:9 Among the street vendors, 84% are male and 16% female,:12 and four fifths are "stationary vendors" operating from a fixed pitch, often with their own stall.:15–16 Most are financed through personal savings; only 8% borrow from moneylenders.:19 Vendor earnings vary from ₹50 (74¢ US) to ₹800 (US$12) per day.:25 Other unorganised economic sectors include dairy, poultry farming, brick manufacturing, casual labour and domestic help. Those involved in the informal economy constitute a major portion of urban poor.:71', 'Like the rest of India, Hyderabad has a large informal economy that employs 30% of the labour force.:71 According to a survey published in 2007, it had 40–50,000 street vendors, and their numbers were increasing.:9 Among the street vendors, 84% are male and 16% female,:12 and four fifths are "stationary vendors" operating from a fixed pitch, often with their own stall.:15–16 Most are financed through personal savings; only 8% borrow from moneylenders.:19 Vendor earnings vary from ₹50 (74¢ US) to ₹800 (US$12) per day.:25 Other unorganised economic sectors include dairy, poultry farming, brick manufacturing, casual labour and domestic help. Those involved in the informal economy constitute a major portion of urban poor.:71', 'Like the rest of India, Hyderabad has a large informal economy that employs 30% of the labour force.:71 According to a survey published in 2007, it had 40–50,000 street vendors, and their numbers were increasing.:9 Among the street vendors, 84% are male and 16% female,:12 and four fifths are "stationary vendors" operating from a fixed pitch, often with their own stall.:15–16 Most are financed through personal savings; only 8% borrow from moneylenders.:19 Vendor earnings vary from ₹50 (74¢ US) to ₹800 (US$12) per day.:25 Other unorganised economic sectors include dairy, poultry farming, brick manufacturing, casual labour and domestic help. Those involved in the informal economy constitute a major portion of urban poor.:71', 'Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire. After the fall of Delhi in 1857, the migration of performing artists to the city particularly from the north and west of the Indian sub continent, under the patronage of the Nizam, enriched the cultural milieu. This migration resulted in a mingling of North and South Indian languages, cultures and religions, which has since led to a co-existence of Hindu and Muslim traditions, for which the city has become noted.:viii A further consequence of this north–south mix is that both Telugu and Urdu are official languages of Telangana. The mixing of religions has also resulted in many festivals being celebrated in Hyderabad such as Ganesh Chaturthi, Diwali and Bonalu of Hindu tradition and Eid ul-Fitr and Eid al-Adha by Muslims.', 'Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire. After the fall of Delhi in 1857, the migration of performing artists to the city particularly from the north and west of the Indian sub continent, under the patronage of the Nizam, enriched the cultural milieu. This migration resulted in a mingling of North and South Indian languages, cultures and religions, which has since led to a co-existence of Hindu and Muslim traditions, for which the city has become noted.:viii A further consequence of this north–south mix is that both Telugu and Urdu are official languages of Telangana. The mixing of religions has also resulted in many festivals being celebrated in Hyderabad such as Ganesh Chaturthi, Diwali and Bonalu of Hindu tradition and Eid ul-Fitr and Eid al-Adha by Muslims.', 'Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire. After the fall of Delhi in 1857, the migration of performing artists to the city particularly from the north and west of the Indian sub continent, under the patronage of the Nizam, enriched the cultural milieu. This migration resulted in a mingling of North and South Indian languages, cultures and religions, which has since led to a co-existence of Hindu and Muslim traditions, for which the city has become noted.:viii A further consequence of this north–south mix is that both Telugu and Urdu are official languages of Telangana. The mixing of religions has also resulted in many festivals being celebrated in Hyderabad such as Ganesh Chaturthi, Diwali and Bonalu of Hindu tradition and Eid ul-Fitr and Eid al-Adha by Muslims.', 'Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire. After the fall of Delhi in 1857, the migration of performing artists to the city particularly from the north and west of the Indian sub continent, under the patronage of the Nizam, enriched the cultural milieu. This migration resulted in a mingling of North and South Indian languages, cultures and religions, which has since led to a co-existence of Hindu and Muslim traditions, for which the city has become noted.:viii A further consequence of this north–south mix is that both Telugu and Urdu are official languages of Telangana. The mixing of religions has also resulted in many festivals being celebrated in Hyderabad such as Ganesh Chaturthi, Diwali and Bonalu of Hindu tradition and Eid ul-Fitr and Eid al-Adha by Muslims.', 'Hyderabad emerged as the foremost centre of culture in India with the decline of the Mughal Empire. After the fall of Delhi in 1857, the migration of performing artists to the city particularly from the north and west of the Indian sub continent, under the patronage of the Nizam, enriched the cultural milieu. This migration resulted in a mingling of North and South Indian languages, cultures and religions, which has since led to a co-existence of Hindu and Muslim traditions, for which the city has become noted.:viii A further consequence of this north–south mix is that both Telugu and Urdu are official languages of Telangana. The mixing of religions has also resulted in many festivals being celebrated in Hyderabad such as Ganesh Chaturthi, Diwali and Bonalu of Hindu tradition and Eid ul-Fitr and Eid al-Adha by Muslims.', 'In the past, Qutb Shahi rulers and Nizams attracted artists, architects and men of letters from different parts of the world through patronage. The resulting ethnic mix popularised cultural events such as mushairas (poetic symposia). The Qutb Shahi dynasty particularly encouraged the growth of Deccani Urdu literature leading to works such as the Deccani Masnavi and Diwan poetry, which are among the earliest available manuscripts in Urdu. Lazzat Un Nisa, a book compiled in the 15th century at Qutb Shahi courts, contains erotic paintings with diagrams for secret medicines and stimulants in the eastern form of ancient sexual arts. The reign of the Nizams saw many literary reforms and the introduction of Urdu as a language of court, administration and education. In 1824, a collection of Urdu Ghazal poetry, named Gulzar-e-Mahlaqa, authored by Mah Laqa Bai—the first female Urdu poet to produce a Diwan—was published in Hyderabad.', 'In the past, Qutb Shahi rulers and Nizams attracted artists, architects and men of letters from different parts of the world through patronage. The resulting ethnic mix popularised cultural events such as mushairas (poetic symposia). The Qutb Shahi dynasty particularly encouraged the growth of Deccani Urdu literature leading to works such as the Deccani Masnavi and Diwan poetry, which are among the earliest available manuscripts in Urdu. Lazzat Un Nisa, a book compiled in the 15th century at Qutb Shahi courts, contains erotic paintings with diagrams for secret medicines and stimulants in the eastern form of ancient sexual arts. The reign of the Nizams saw many literary reforms and the introduction of Urdu as a language of court, administration and education. In 1824, a collection of Urdu Ghazal poetry, named Gulzar-e-Mahlaqa, authored by Mah Laqa Bai—the first female Urdu poet to produce a Diwan—was published in Hyderabad.', 'In the past, Qutb Shahi rulers and Nizams attracted artists, architects and men of letters from different parts of the world through patronage. The resulting ethnic mix popularised cultural events such as mushairas (poetic symposia). The Qutb Shahi dynasty particularly encouraged the growth of Deccani Urdu literature leading to works such as the Deccani Masnavi and Diwan poetry, which are among the earliest available manuscripts in Urdu. Lazzat Un Nisa, a book compiled in the 15th century at Qutb Shahi courts, contains erotic paintings with diagrams for secret medicines and stimulants in the eastern form of ancient sexual arts. The reign of the Nizams saw many literary reforms and the introduction of Urdu as a language of court, administration and education. In 1824, a collection of Urdu Ghazal poetry, named Gulzar-e-Mahlaqa, authored by Mah Laqa Bai—the first female Urdu poet to produce a Diwan—was published in Hyderabad.', 'In the past, Qutb Shahi rulers and Nizams attracted artists, architects and men of letters from different parts of the world through patronage. The resulting ethnic mix popularised cultural events such as mushairas (poetic symposia). The Qutb Shahi dynasty particularly encouraged the growth of Deccani Urdu literature leading to works such as the Deccani Masnavi and Diwan poetry, which are among the earliest available manuscripts in Urdu. Lazzat Un Nisa, a book compiled in the 15th century at Qutb Shahi courts, contains erotic paintings with diagrams for secret medicines and stimulants in the eastern form of ancient sexual arts. The reign of the Nizams saw many literary reforms and the introduction of Urdu as a language of court, administration and education. In 1824, a collection of Urdu Ghazal poetry, named Gulzar-e-Mahlaqa, authored by Mah Laqa Bai—the first female Urdu poet to produce a Diwan—was published in Hyderabad.', 'In the past, Qutb Shahi rulers and Nizams attracted artists, architects and men of letters from different parts of the world through patronage. The resulting ethnic mix popularised cultural events such as mushairas (poetic symposia). The Qutb Shahi dynasty particularly encouraged the growth of Deccani Urdu literature leading to works such as the Deccani Masnavi and Diwan poetry, which are among the earliest available manuscripts in Urdu. Lazzat Un Nisa, a book compiled in the 15th century at Qutb Shahi courts, contains erotic paintings with diagrams for secret medicines and stimulants in the eastern form of ancient sexual arts. The reign of the Nizams saw many literary reforms and the introduction of Urdu as a language of court, administration and education. In 1824, a collection of Urdu Ghazal poetry, named Gulzar-e-Mahlaqa, authored by Mah Laqa Bai—the first female Urdu poet to produce a Diwan—was published in Hyderabad.', "Hyderabad has continued with these traditions in its annual Hyderabad Literary Festival, held since 2010, showcasing the city's literary and cultural creativity. Organisations engaged in the advancement of literature include the Sahitya Akademi, the Urdu Academy, the Telugu Academy, the National Council for Promotion of Urdu Language, the Comparative Literature Association of India, and the Andhra Saraswata Parishad. Literary development is further aided by state institutions such as the State Central Library, the largest public library in the state which was established in 1891, and other major libraries including the Sri Krishna Devaraya Andhra Bhasha Nilayam, the British Library and the Sundarayya Vignana Kendram.", "Hyderabad has continued with these traditions in its annual Hyderabad Literary Festival, held since 2010, showcasing the city's literary and cultural creativity. Organisations engaged in the advancement of literature include the Sahitya Akademi, the Urdu Academy, the Telugu Academy, the National Council for Promotion of Urdu Language, the Comparative Literature Association of India, and the Andhra Saraswata Parishad. Literary development is further aided by state institutions such as the State Central Library, the largest public library in the state which was established in 1891, and other major libraries including the Sri Krishna Devaraya Andhra Bhasha Nilayam, the British Library and the Sundarayya Vignana Kendram.", "Hyderabad has continued with these traditions in its annual Hyderabad Literary Festival, held since 2010, showcasing the city's literary and cultural creativity. Organisations engaged in the advancement of literature include the Sahitya Akademi, the Urdu Academy, the Telugu Academy, the National Council for Promotion of Urdu Language, the Comparative Literature Association of India, and the Andhra Saraswata Parishad. Literary development is further aided by state institutions such as the State Central Library, the largest public library in the state which was established in 1891, and other major libraries including the Sri Krishna Devaraya Andhra Bhasha Nilayam, the British Library and the Sundarayya Vignana Kendram.", "Hyderabad has continued with these traditions in its annual Hyderabad Literary Festival, held since 2010, showcasing the city's literary and cultural creativity. Organisations engaged in the advancement of literature include the Sahitya Akademi, the Urdu Academy, the Telugu Academy, the National Council for Promotion of Urdu Language, the Comparative Literature Association of India, and the Andhra Saraswata Parishad. Literary development is further aided by state institutions such as the State Central Library, the largest public library in the state which was established in 1891, and other major libraries including the Sri Krishna Devaraya Andhra Bhasha Nilayam, the British Library and the Sundarayya Vignana Kendram.", 'South Indian music and dances such as the Kuchipudi and Bharatanatyam styles are popular in the Deccan region. As a result of their culture policies, North Indian music and dance gained popularity during the rule of the Mughals and Nizams, and it was also during their reign that it became a tradition among the nobility to associate themselves with tawaif (courtesans). These courtesans were revered as the epitome of etiquette and culture, and were appointed to teach singing, poetry and classical dance to many children of the aristocracy. This gave rise to certain styles of court music, dance and poetry. Besides western and Indian popular music genres such as filmi music, the residents of Hyderabad play city-based marfa music, dholak ke geet (household songs based on local Folklore), and qawwali, especially at weddings, festivals and other celebratory events. The state government organises the Golconda Music and Dance Festival, the Taramati Music Festival and the Premavathi Dance Festival to further encourage the development of music.', 'South Indian music and dances such as the Kuchipudi and Bharatanatyam styles are popular in the Deccan region. As a result of their culture policies, North Indian music and dance gained popularity during the rule of the Mughals and Nizams, and it was also during their reign that it became a tradition among the nobility to associate themselves with tawaif (courtesans). These courtesans were revered as the epitome of etiquette and culture, and were appointed to teach singing, poetry and classical dance to many children of the aristocracy. This gave rise to certain styles of court music, dance and poetry. Besides western and Indian popular music genres such as filmi music, the residents of Hyderabad play city-based marfa music, dholak ke geet (household songs based on local Folklore), and qawwali, especially at weddings, festivals and other celebratory events. The state government organises the Golconda Music and Dance Festival, the Taramati Music Festival and the Premavathi Dance Festival to further encourage the development of music.', 'South Indian music and dances such as the Kuchipudi and Bharatanatyam styles are popular in the Deccan region. As a result of their culture policies, North Indian music and dance gained popularity during the rule of the Mughals and Nizams, and it was also during their reign that it became a tradition among the nobility to associate themselves with tawaif (courtesans). These courtesans were revered as the epitome of etiquette and culture, and were appointed to teach singing, poetry and classical dance to many children of the aristocracy. This gave rise to certain styles of court music, dance and poetry. Besides western and Indian popular music genres such as filmi music, the residents of Hyderabad play city-based marfa music, dholak ke geet (household songs based on local Folklore), and qawwali, especially at weddings, festivals and other celebratory events. The state government organises the Golconda Music and Dance Festival, the Taramati Music Festival and the Premavathi Dance Festival to further encourage the development of music.', 'South Indian music and dances such as the Kuchipudi and Bharatanatyam styles are popular in the Deccan region. As a result of their culture policies, North Indian music and dance gained popularity during the rule of the Mughals and Nizams, and it was also during their reign that it became a tradition among the nobility to associate themselves with tawaif (courtesans). These courtesans were revered as the epitome of etiquette and culture, and were appointed to teach singing, poetry and classical dance to many children of the aristocracy. This gave rise to certain styles of court music, dance and poetry. Besides western and Indian popular music genres such as filmi music, the residents of Hyderabad play city-based marfa music, dholak ke geet (household songs based on local Folklore), and qawwali, especially at weddings, festivals and other celebratory events. The state government organises the Golconda Music and Dance Festival, the Taramati Music Festival and the Premavathi Dance Festival to further encourage the development of music.', 'South Indian music and dances such as the Kuchipudi and Bharatanatyam styles are popular in the Deccan region. As a result of their culture policies, North Indian music and dance gained popularity during the rule of the Mughals and Nizams, and it was also during their reign that it became a tradition among the nobility to associate themselves with tawaif (courtesans). These courtesans were revered as the epitome of etiquette and culture, and were appointed to teach singing, poetry and classical dance to many children of the aristocracy. This gave rise to certain styles of court music, dance and poetry. Besides western and Indian popular music genres such as filmi music, the residents of Hyderabad play city-based marfa music, dholak ke geet (household songs based on local Folklore), and qawwali, especially at weddings, festivals and other celebratory events. The state government organises the Golconda Music and Dance Festival, the Taramati Music Festival and the Premavathi Dance Festival to further encourage the development of music.', "Although the city is not particularly noted for theatre and drama, the state government promotes theatre with multiple programmes and festivals in such venues as the Ravindra Bharati, Shilpakala Vedika and Lalithakala Thoranam. Although not a purely music oriented event, Numaish, a popular annual exhibition of local and national consumer products, does feature some musical performances. The city is home to the Telugu film industry, popularly known as Tollywood and as of 2012[update], produces the second largest number of films in India behind Bollywood. Films in the local Hyderabadi dialect are also produced and have been gaining popularity since 2005. The city has also hosted international film festivals such as the International Children's Film Festival and the Hyderabad International Film Festival. In 2005, Guinness World Records declared Ramoji Film City to be the world's largest film studio.", "Although the city is not particularly noted for theatre and drama, the state government promotes theatre with multiple programmes and festivals in such venues as the Ravindra Bharati, Shilpakala Vedika and Lalithakala Thoranam. Although not a purely music oriented event, Numaish, a popular annual exhibition of local and national consumer products, does feature some musical performances. The city is home to the Telugu film industry, popularly known as Tollywood and as of 2012[update], produces the second largest number of films in India behind Bollywood. Films in the local Hyderabadi dialect are also produced and have been gaining popularity since 2005. The city has also hosted international film festivals such as the International Children's Film Festival and the Hyderabad International Film Festival. In 2005, Guinness World Records declared Ramoji Film City to be the world's largest film studio.", "Although the city is not particularly noted for theatre and drama, the state government promotes theatre with multiple programmes and festivals in such venues as the Ravindra Bharati, Shilpakala Vedika and Lalithakala Thoranam. Although not a purely music oriented event, Numaish, a popular annual exhibition of local and national consumer products, does feature some musical performances. The city is home to the Telugu film industry, popularly known as Tollywood and as of 2012[update], produces the second largest number of films in India behind Bollywood. Films in the local Hyderabadi dialect are also produced and have been gaining popularity since 2005. The city has also hosted international film festivals such as the International Children's Film Festival and the Hyderabad International Film Festival. In 2005, Guinness World Records declared Ramoji Film City to be the world's largest film studio.", "Although the city is not particularly noted for theatre and drama, the state government promotes theatre with multiple programmes and festivals in such venues as the Ravindra Bharati, Shilpakala Vedika and Lalithakala Thoranam. Although not a purely music oriented event, Numaish, a popular annual exhibition of local and national consumer products, does feature some musical performances. The city is home to the Telugu film industry, popularly known as Tollywood and as of 2012[update], produces the second largest number of films in India behind Bollywood. Films in the local Hyderabadi dialect are also produced and have been gaining popularity since 2005. The city has also hosted international film festivals such as the International Children's Film Festival and the Hyderabad International Film Festival. In 2005, Guinness World Records declared Ramoji Film City to be the world's largest film studio.", "Although the city is not particularly noted for theatre and drama, the state government promotes theatre with multiple programmes and festivals in such venues as the Ravindra Bharati, Shilpakala Vedika and Lalithakala Thoranam. Although not a purely music oriented event, Numaish, a popular annual exhibition of local and national consumer products, does feature some musical performances. The city is home to the Telugu film industry, popularly known as Tollywood and as of 2012[update], produces the second largest number of films in India behind Bollywood. Films in the local Hyderabadi dialect are also produced and have been gaining popularity since 2005. The city has also hosted international film festivals such as the International Children's Film Festival and the Hyderabad International Film Festival. In 2005, Guinness World Records declared Ramoji Film City to be the world's largest film studio.", 'The region is well known for its Golconda and Hyderabad painting styles which are branches of Deccani painting. Developed during the 16th century, the Golconda style is a native style blending foreign techniques and bears some similarity to the Vijayanagara paintings of neighbouring Mysore. A significant use of luminous gold and white colours is generally found in the Golconda style. The Hyderabad style originated in the 17th century under the Nizams. Highly influenced by Mughal painting, this style makes use of bright colours and mostly depicts regional landscape, culture, costumes and jewellery.', 'The region is well known for its Golconda and Hyderabad painting styles which are branches of Deccani painting. Developed during the 16th century, the Golconda style is a native style blending foreign techniques and bears some similarity to the Vijayanagara paintings of neighbouring Mysore. A significant use of luminous gold and white colours is generally found in the Golconda style. The Hyderabad style originated in the 17th century under the Nizams. Highly influenced by Mughal painting, this style makes use of bright colours and mostly depicts regional landscape, culture, costumes and jewellery.', 'The region is well known for its Golconda and Hyderabad painting styles which are branches of Deccani painting. Developed during the 16th century, the Golconda style is a native style blending foreign techniques and bears some similarity to the Vijayanagara paintings of neighbouring Mysore. A significant use of luminous gold and white colours is generally found in the Golconda style. The Hyderabad style originated in the 17th century under the Nizams. Highly influenced by Mughal painting, this style makes use of bright colours and mostly depicts regional landscape, culture, costumes and jewellery.', 'The region is well known for its Golconda and Hyderabad painting styles which are branches of Deccani painting. Developed during the 16th century, the Golconda style is a native style blending foreign techniques and bears some similarity to the Vijayanagara paintings of neighbouring Mysore. A significant use of luminous gold and white colours is generally found in the Golconda style. The Hyderabad style originated in the 17th century under the Nizams. Highly influenced by Mughal painting, this style makes use of bright colours and mostly depicts regional landscape, culture, costumes and jewellery.', 'The region is well known for its Golconda and Hyderabad painting styles which are branches of Deccani painting. Developed during the 16th century, the Golconda style is a native style blending foreign techniques and bears some similarity to the Vijayanagara paintings of neighbouring Mysore. A significant use of luminous gold and white colours is generally found in the Golconda style. The Hyderabad style originated in the 17th century under the Nizams. Highly influenced by Mughal painting, this style makes use of bright colours and mostly depicts regional landscape, culture, costumes and jewellery.', 'Although not a centre for handicrafts itself, the patronage of the arts by the Mughals and Nizams attracted artisans from the region to Hyderabad. Such crafts include: Bidriware, a metalwork handicraft from neighbouring Karnataka, which was popularised during the 18th century and has since been granted a Geographical Indication (GI) tag under the auspices of the WTO act; and Zari and Zardozi, embroidery works on textile that involve making elaborate designs using gold, silver and other metal threads. Another example of a handicraft drawn to Hyderabad is Kalamkari, a hand-painted or block-printed cotton textile that comes from cities in Andhra Pradesh. This craft is distinguished in having both a Hindu style, known as Srikalahasti and entirely done by hand, and an Islamic style, known as Machilipatnam that uses both hand and block techniques. Examples of Hyderabad\'s arts and crafts are housed in various museums including the Salar Jung Museum (housing "one of the largest one-man-collections in the world"), the AP State Archaeology Museum, the Nizam Museum, the City Museum and the Birla Science Museum.', 'Although not a centre for handicrafts itself, the patronage of the arts by the Mughals and Nizams attracted artisans from the region to Hyderabad. Such crafts include: Bidriware, a metalwork handicraft from neighbouring Karnataka, which was popularised during the 18th century and has since been granted a Geographical Indication (GI) tag under the auspices of the WTO act; and Zari and Zardozi, embroidery works on textile that involve making elaborate designs using gold, silver and other metal threads. Another example of a handicraft drawn to Hyderabad is Kalamkari, a hand-painted or block-printed cotton textile that comes from cities in Andhra Pradesh. This craft is distinguished in having both a Hindu style, known as Srikalahasti and entirely done by hand, and an Islamic style, known as Machilipatnam that uses both hand and block techniques. Examples of Hyderabad\'s arts and crafts are housed in various museums including the Salar Jung Museum (housing "one of the largest one-man-collections in the world"), the AP State Archaeology Museum, the Nizam Museum, the City Museum and the Birla Science Museum.', 'Although not a centre for handicrafts itself, the patronage of the arts by the Mughals and Nizams attracted artisans from the region to Hyderabad. Such crafts include: Bidriware, a metalwork handicraft from neighbouring Karnataka, which was popularised during the 18th century and has since been granted a Geographical Indication (GI) tag under the auspices of the WTO act; and Zari and Zardozi, embroidery works on textile that involve making elaborate designs using gold, silver and other metal threads. Another example of a handicraft drawn to Hyderabad is Kalamkari, a hand-painted or block-printed cotton textile that comes from cities in Andhra Pradesh. This craft is distinguished in having both a Hindu style, known as Srikalahasti and entirely done by hand, and an Islamic style, known as Machilipatnam that uses both hand and block techniques. Examples of Hyderabad\'s arts and crafts are housed in various museums including the Salar Jung Museum (housing "one of the largest one-man-collections in the world"), the AP State Archaeology Museum, the Nizam Museum, the City Museum and the Birla Science Museum.', 'Although not a centre for handicrafts itself, the patronage of the arts by the Mughals and Nizams attracted artisans from the region to Hyderabad. Such crafts include: Bidriware, a metalwork handicraft from neighbouring Karnataka, which was popularised during the 18th century and has since been granted a Geographical Indication (GI) tag under the auspices of the WTO act; and Zari and Zardozi, embroidery works on textile that involve making elaborate designs using gold, silver and other metal threads. Another example of a handicraft drawn to Hyderabad is Kalamkari, a hand-painted or block-printed cotton textile that comes from cities in Andhra Pradesh. This craft is distinguished in having both a Hindu style, known as Srikalahasti and entirely done by hand, and an Islamic style, known as Machilipatnam that uses both hand and block techniques. Examples of Hyderabad\'s arts and crafts are housed in various museums including the Salar Jung Museum (housing "one of the largest one-man-collections in the world"), the AP State Archaeology Museum, the Nizam Museum, the City Museum and the Birla Science Museum.', 'Although not a centre for handicrafts itself, the patronage of the arts by the Mughals and Nizams attracted artisans from the region to Hyderabad. Such crafts include: Bidriware, a metalwork handicraft from neighbouring Karnataka, which was popularised during the 18th century and has since been granted a Geographical Indication (GI) tag under the auspices of the WTO act; and Zari and Zardozi, embroidery works on textile that involve making elaborate designs using gold, silver and other metal threads. Another example of a handicraft drawn to Hyderabad is Kalamkari, a hand-painted or block-printed cotton textile that comes from cities in Andhra Pradesh. This craft is distinguished in having both a Hindu style, known as Srikalahasti and entirely done by hand, and an Islamic style, known as Machilipatnam that uses both hand and block techniques. Examples of Hyderabad\'s arts and crafts are housed in various museums including the Salar Jung Museum (housing "one of the largest one-man-collections in the world"), the AP State Archaeology Museum, the Nizam Museum, the City Museum and the Birla Science Museum.', 'Hyderabadi cuisine comprises a broad repertoire of rice, wheat and meat dishes and the skilled use of various spices. Hyderabadi biryani and Hyderabadi haleem, with their blend of Mughlai and Arab cuisines, have become iconic dishes of India. Hyderabadi cuisine is highly influenced by Mughlai and to some extent by French, Arabic, Turkish, Iranian and native Telugu and Marathwada cuisines. Other popular native dishes include nihari, chakna, baghara baingan and the desserts qubani ka meetha, double ka meetha and kaddu ki kheer (a sweet porridge made with sweet gourd).', 'Hyderabadi cuisine comprises a broad repertoire of rice, wheat and meat dishes and the skilled use of various spices. Hyderabadi biryani and Hyderabadi haleem, with their blend of Mughlai and Arab cuisines, have become iconic dishes of India. Hyderabadi cuisine is highly influenced by Mughlai and to some extent by French, Arabic, Turkish, Iranian and native Telugu and Marathwada cuisines. Other popular native dishes include nihari, chakna, baghara baingan and the desserts qubani ka meetha, double ka meetha and kaddu ki kheer (a sweet porridge made with sweet gourd).', 'Hyderabadi cuisine comprises a broad repertoire of rice, wheat and meat dishes and the skilled use of various spices. Hyderabadi biryani and Hyderabadi haleem, with their blend of Mughlai and Arab cuisines, have become iconic dishes of India. Hyderabadi cuisine is highly influenced by Mughlai and to some extent by French, Arabic, Turkish, Iranian and native Telugu and Marathwada cuisines. Other popular native dishes include nihari, chakna, baghara baingan and the desserts qubani ka meetha, double ka meetha and kaddu ki kheer (a sweet porridge made with sweet gourd).', 'Hyderabadi cuisine comprises a broad repertoire of rice, wheat and meat dishes and the skilled use of various spices. Hyderabadi biryani and Hyderabadi haleem, with their blend of Mughlai and Arab cuisines, have become iconic dishes of India. Hyderabadi cuisine is highly influenced by Mughlai and to some extent by French, Arabic, Turkish, Iranian and native Telugu and Marathwada cuisines. Other popular native dishes include nihari, chakna, baghara baingan and the desserts qubani ka meetha, double ka meetha and kaddu ki kheer (a sweet porridge made with sweet gourd).', "One of Hyderabad's earliest newspapers, The Deccan Times, was established in the 1780s. In modern times, the major Telugu dailies published in Hyderabad are Eenadu, Andhra Jyothy, Sakshi and Namaste Telangana, while the major English papers are The Times of India, The Hindu and The Deccan Chronicle. The major Urdu papers include The Siasat Daily, The Munsif Daily and Etemaad. Many coffee table magazines, professional magazines and research journals are also regularly published. The Secunderabad Cantonment Board established the first radio station in Hyderabad State around 1919. Deccan Radio was the first radio public broadcast station in the city starting on 3 February 1935, with FM broadcasting beginning in 2000. The available channels in Hyderabad include All India Radio, Radio Mirchi, Radio City, Red FM and Big FM.", "One of Hyderabad's earliest newspapers, The Deccan Times, was established in the 1780s. In modern times, the major Telugu dailies published in Hyderabad are Eenadu, Andhra Jyothy, Sakshi and Namaste Telangana, while the major English papers are The Times of India, The Hindu and The Deccan Chronicle. The major Urdu papers include The Siasat Daily, The Munsif Daily and Etemaad. Many coffee table magazines, professional magazines and research journals are also regularly published. The Secunderabad Cantonment Board established the first radio station in Hyderabad State around 1919. Deccan Radio was the first radio public broadcast station in the city starting on 3 February 1935, with FM broadcasting beginning in 2000. The available channels in Hyderabad include All India Radio, Radio Mirchi, Radio City, Red FM and Big FM.", "One of Hyderabad's earliest newspapers, The Deccan Times, was established in the 1780s. In modern times, the major Telugu dailies published in Hyderabad are Eenadu, Andhra Jyothy, Sakshi and Namaste Telangana, while the major English papers are The Times of India, The Hindu and The Deccan Chronicle. The major Urdu papers include The Siasat Daily, The Munsif Daily and Etemaad. Many coffee table magazines, professional magazines and research journals are also regularly published. The Secunderabad Cantonment Board established the first radio station in Hyderabad State around 1919. Deccan Radio was the first radio public broadcast station in the city starting on 3 February 1935, with FM broadcasting beginning in 2000. The available channels in Hyderabad include All India Radio, Radio Mirchi, Radio City, Red FM and Big FM.", "One of Hyderabad's earliest newspapers, The Deccan Times, was established in the 1780s. In modern times, the major Telugu dailies published in Hyderabad are Eenadu, Andhra Jyothy, Sakshi and Namaste Telangana, while the major English papers are The Times of India, The Hindu and The Deccan Chronicle. The major Urdu papers include The Siasat Daily, The Munsif Daily and Etemaad. Many coffee table magazines, professional magazines and research journals are also regularly published. The Secunderabad Cantonment Board established the first radio station in Hyderabad State around 1919. Deccan Radio was the first radio public broadcast station in the city starting on 3 February 1935, with FM broadcasting beginning in 2000. The available channels in Hyderabad include All India Radio, Radio Mirchi, Radio City, Red FM and Big FM.", "One of Hyderabad's earliest newspapers, The Deccan Times, was established in the 1780s. In modern times, the major Telugu dailies published in Hyderabad are Eenadu, Andhra Jyothy, Sakshi and Namaste Telangana, while the major English papers are The Times of India, The Hindu and The Deccan Chronicle. The major Urdu papers include The Siasat Daily, The Munsif Daily and Etemaad. Many coffee table magazines, professional magazines and research journals are also regularly published. The Secunderabad Cantonment Board established the first radio station in Hyderabad State around 1919. Deccan Radio was the first radio public broadcast station in the city starting on 3 February 1935, with FM broadcasting beginning in 2000. The available channels in Hyderabad include All India Radio, Radio Mirchi, Radio City, Red FM and Big FM.", "Television broadcasting in Hyderabad began in 1974 with the launch of Doordarshan, the Government of India's public service broadcaster, which transmits two free-to-air terrestrial television channels and one satellite channel. Private satellite channels started in July 1992 with the launch of Star TV. Satellite TV channels are accessible via cable subscription, direct-broadcast satellite services or internet-based television. Hyderabad's first dial-up internet access became available in the early 1990s and was limited to software development companies. The first public internet access service began in 1995, with the first private sector internet service provider (ISP) starting operations in 1998. In 2015, high-speed public WiFi was introduced in parts of the city.", "Television broadcasting in Hyderabad began in 1974 with the launch of Doordarshan, the Government of India's public service broadcaster, which transmits two free-to-air terrestrial television channels and one satellite channel. Private satellite channels started in July 1992 with the launch of Star TV. Satellite TV channels are accessible via cable subscription, direct-broadcast satellite services or internet-based television. Hyderabad's first dial-up internet access became available in the early 1990s and was limited to software development companies. The first public internet access service began in 1995, with the first private sector internet service provider (ISP) starting operations in 1998. In 2015, high-speed public WiFi was introduced in parts of the city.", "Television broadcasting in Hyderabad began in 1974 with the launch of Doordarshan, the Government of India's public service broadcaster, which transmits two free-to-air terrestrial television channels and one satellite channel. Private satellite channels started in July 1992 with the launch of Star TV. Satellite TV channels are accessible via cable subscription, direct-broadcast satellite services or internet-based television. Hyderabad's first dial-up internet access became available in the early 1990s and was limited to software development companies. The first public internet access service began in 1995, with the first private sector internet service provider (ISP) starting operations in 1998. In 2015, high-speed public WiFi was introduced in parts of the city.", "Television broadcasting in Hyderabad began in 1974 with the launch of Doordarshan, the Government of India's public service broadcaster, which transmits two free-to-air terrestrial television channels and one satellite channel. Private satellite channels started in July 1992 with the launch of Star TV. Satellite TV channels are accessible via cable subscription, direct-broadcast satellite services or internet-based television. Hyderabad's first dial-up internet access became available in the early 1990s and was limited to software development companies. The first public internet access service began in 1995, with the first private sector internet service provider (ISP) starting operations in 1998. In 2015, high-speed public WiFi was introduced in parts of the city.", "Television broadcasting in Hyderabad began in 1974 with the launch of Doordarshan, the Government of India's public service broadcaster, which transmits two free-to-air terrestrial television channels and one satellite channel. Private satellite channels started in July 1992 with the launch of Star TV. Satellite TV channels are accessible via cable subscription, direct-broadcast satellite services or internet-based television. Hyderabad's first dial-up internet access became available in the early 1990s and was limited to software development companies. The first public internet access service began in 1995, with the first private sector internet service provider (ISP) starting operations in 1998. In 2015, high-speed public WiFi was introduced in parts of the city.", 'Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan. About two-thirds of pupils attend privately run institutions. Languages of instruction include English, Hindi, Telugu and Urdu. Depending on the institution, students are required to sit the Secondary School Certificate or the Indian Certificate of Secondary Education. After completing secondary education, students enroll in schools or junior colleges with a higher secondary facility. Admission to professional graduation colleges in Hyderabad, many of which are affiliated with either Jawaharlal Nehru Technological University Hyderabad (JNTUH) or Osmania University (OU), is through the Engineering Agricultural and Medical Common Entrance Test (EAM-CET).', 'Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan. About two-thirds of pupils attend privately run institutions. Languages of instruction include English, Hindi, Telugu and Urdu. Depending on the institution, students are required to sit the Secondary School Certificate or the Indian Certificate of Secondary Education. After completing secondary education, students enroll in schools or junior colleges with a higher secondary facility. Admission to professional graduation colleges in Hyderabad, many of which are affiliated with either Jawaharlal Nehru Technological University Hyderabad (JNTUH) or Osmania University (OU), is through the Engineering Agricultural and Medical Common Entrance Test (EAM-CET).', 'Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan. About two-thirds of pupils attend privately run institutions. Languages of instruction include English, Hindi, Telugu and Urdu. Depending on the institution, students are required to sit the Secondary School Certificate or the Indian Certificate of Secondary Education. After completing secondary education, students enroll in schools or junior colleges with a higher secondary facility. Admission to professional graduation colleges in Hyderabad, many of which are affiliated with either Jawaharlal Nehru Technological University Hyderabad (JNTUH) or Osmania University (OU), is through the Engineering Agricultural and Medical Common Entrance Test (EAM-CET).', 'Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan. About two-thirds of pupils attend privately run institutions. Languages of instruction include English, Hindi, Telugu and Urdu. Depending on the institution, students are required to sit the Secondary School Certificate or the Indian Certificate of Secondary Education. After completing secondary education, students enroll in schools or junior colleges with a higher secondary facility. Admission to professional graduation colleges in Hyderabad, many of which are affiliated with either Jawaharlal Nehru Technological University Hyderabad (JNTUH) or Osmania University (OU), is through the Engineering Agricultural and Medical Common Entrance Test (EAM-CET).', 'Public and private schools in Hyderabad are governed by the Central Board of Secondary Education and follow a "10+2+3" plan. About two-thirds of pupils attend privately run institutions. Languages of instruction include English, Hindi, Telugu and Urdu. Depending on the institution, students are required to sit the Secondary School Certificate or the Indian Certificate of Secondary Education. After completing secondary education, students enroll in schools or junior colleges with a higher secondary facility. Admission to professional graduation colleges in Hyderabad, many of which are affiliated with either Jawaharlal Nehru Technological University Hyderabad (JNTUH) or Osmania University (OU), is through the Engineering Agricultural and Medical Common Entrance Test (EAM-CET).', "There are 13 universities in Hyderabad: two private universities, two deemed universities, six state universities and three central universities. The central universities are the University of Hyderabad, Maulana Azad National Urdu University and the English and Foreign Languages University. Osmania University, established in 1918, was the first university in Hyderabad and as of 2012[update] is India's second most popular institution for international students. The Dr. B. R. Ambedkar Open University, established in 1982, is the first distance learning open university in India.", "There are 13 universities in Hyderabad: two private universities, two deemed universities, six state universities and three central universities. The central universities are the University of Hyderabad, Maulana Azad National Urdu University and the English and Foreign Languages University. Osmania University, established in 1918, was the first university in Hyderabad and as of 2012[update] is India's second most popular institution for international students. The Dr. B. R. Ambedkar Open University, established in 1982, is the first distance learning open university in India.", "There are 13 universities in Hyderabad: two private universities, two deemed universities, six state universities and three central universities. The central universities are the University of Hyderabad, Maulana Azad National Urdu University and the English and Foreign Languages University. Osmania University, established in 1918, was the first university in Hyderabad and as of 2012[update] is India's second most popular institution for international students. The Dr. B. R. Ambedkar Open University, established in 1982, is the first distance learning open university in India.", "There are 13 universities in Hyderabad: two private universities, two deemed universities, six state universities and three central universities. The central universities are the University of Hyderabad, Maulana Azad National Urdu University and the English and Foreign Languages University. Osmania University, established in 1918, was the first university in Hyderabad and as of 2012[update] is India's second most popular institution for international students. The Dr. B. R. Ambedkar Open University, established in 1982, is the first distance learning open university in India.", "There are 13 universities in Hyderabad: two private universities, two deemed universities, six state universities and three central universities. The central universities are the University of Hyderabad, Maulana Azad National Urdu University and the English and Foreign Languages University. Osmania University, established in 1918, was the first university in Hyderabad and as of 2012[update] is India's second most popular institution for international students. The Dr. B. R. Ambedkar Open University, established in 1982, is the first distance learning open university in India.", "Hyderabad is also home to a number of centres specialising in particular fields such as biomedical sciences, biotechnology and pharmaceuticals, such as the National Institute of Pharmaceutical Education and Research (NIPER) and National Institute of Nutrition (NIN). Hyderabad has five major medical schools—Osmania Medical College, Gandhi Medical College, Nizam's Institute of Medical Sciences, Deccan College of Medical Sciences and Shadan Institute of Medical Sciences—and many affiliated teaching hospitals. The Government Nizamia Tibbi College is a college of Unani medicine. Hyderabad is also the headquarters of the Indian Heart Association, a non-profit foundation for cardiovascular education.", "Hyderabad is also home to a number of centres specialising in particular fields such as biomedical sciences, biotechnology and pharmaceuticals, such as the National Institute of Pharmaceutical Education and Research (NIPER) and National Institute of Nutrition (NIN). Hyderabad has five major medical schools—Osmania Medical College, Gandhi Medical College, Nizam's Institute of Medical Sciences, Deccan College of Medical Sciences and Shadan Institute of Medical Sciences—and many affiliated teaching hospitals. The Government Nizamia Tibbi College is a college of Unani medicine. Hyderabad is also the headquarters of the Indian Heart Association, a non-profit foundation for cardiovascular education.", "Hyderabad is also home to a number of centres specialising in particular fields such as biomedical sciences, biotechnology and pharmaceuticals, such as the National Institute of Pharmaceutical Education and Research (NIPER) and National Institute of Nutrition (NIN). Hyderabad has five major medical schools—Osmania Medical College, Gandhi Medical College, Nizam's Institute of Medical Sciences, Deccan College of Medical Sciences and Shadan Institute of Medical Sciences—and many affiliated teaching hospitals. The Government Nizamia Tibbi College is a college of Unani medicine. Hyderabad is also the headquarters of the Indian Heart Association, a non-profit foundation for cardiovascular education.", 'Institutes in Hyderabad include the National Institute of Rural Development, the Indian School of Business, the Institute of Public Enterprise, the Administrative Staff College of India and the Sardar Vallabhbhai Patel National Police Academy. Technical and engineering schools include the International Institute of Information Technology, Hyderabad (IIITH), Birla Institute of Technology and Science, Pilani – Hyderabad (BITS Hyderabad) and Indian Institute of Technology, Hyderabad (IIT-H) as well as agricultural engineering institutes such as the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT) and the Acharya N. G. Ranga Agricultural University. Hyderabad also has schools of fashion design including Raffles Millennium International, NIFT Hyderabad and Wigan and Leigh College. The National Institute of Design, Hyderabad (NID-H), will offer undergraduate and postgraduate courses from 2015.', 'Institutes in Hyderabad include the National Institute of Rural Development, the Indian School of Business, the Institute of Public Enterprise, the Administrative Staff College of India and the Sardar Vallabhbhai Patel National Police Academy. Technical and engineering schools include the International Institute of Information Technology, Hyderabad (IIITH), Birla Institute of Technology and Science, Pilani – Hyderabad (BITS Hyderabad) and Indian Institute of Technology, Hyderabad (IIT-H) as well as agricultural engineering institutes such as the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT) and the Acharya N. G. Ranga Agricultural University. Hyderabad also has schools of fashion design including Raffles Millennium International, NIFT Hyderabad and Wigan and Leigh College. The National Institute of Design, Hyderabad (NID-H), will offer undergraduate and postgraduate courses from 2015.', 'Institutes in Hyderabad include the National Institute of Rural Development, the Indian School of Business, the Institute of Public Enterprise, the Administrative Staff College of India and the Sardar Vallabhbhai Patel National Police Academy. Technical and engineering schools include the International Institute of Information Technology, Hyderabad (IIITH), Birla Institute of Technology and Science, Pilani – Hyderabad (BITS Hyderabad) and Indian Institute of Technology, Hyderabad (IIT-H) as well as agricultural engineering institutes such as the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT) and the Acharya N. G. Ranga Agricultural University. Hyderabad also has schools of fashion design including Raffles Millennium International, NIFT Hyderabad and Wigan and Leigh College. The National Institute of Design, Hyderabad (NID-H), will offer undergraduate and postgraduate courses from 2015.', 'Institutes in Hyderabad include the National Institute of Rural Development, the Indian School of Business, the Institute of Public Enterprise, the Administrative Staff College of India and the Sardar Vallabhbhai Patel National Police Academy. Technical and engineering schools include the International Institute of Information Technology, Hyderabad (IIITH), Birla Institute of Technology and Science, Pilani – Hyderabad (BITS Hyderabad) and Indian Institute of Technology, Hyderabad (IIT-H) as well as agricultural engineering institutes such as the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT) and the Acharya N. G. Ranga Agricultural University. Hyderabad also has schools of fashion design including Raffles Millennium International, NIFT Hyderabad and Wigan and Leigh College. The National Institute of Design, Hyderabad (NID-H), will offer undergraduate and postgraduate courses from 2015.', "The most popular sports played in Hyderabad are cricket and association football. At the professional level, the city has hosted national and international sports events such as the 2002 National Games of India, the 2003 Afro-Asian Games, the 2004 AP Tourism Hyderabad Open women's tennis tournament, the 2007 Military World Games, the 2009 World Badminton Championships and the 2009 IBSF World Snooker Championship. The city hosts a number of venues suitable for professional competition such as the Swarnandhra Pradesh Sports Complex for field hockey, the G. M. C. Balayogi Stadium in Gachibowli for athletics and football, and for cricket, the Lal Bahadur Shastri Stadium and Rajiv Gandhi International Cricket Stadium, home ground of the Hyderabad Cricket Association. Hyderabad has hosted many international cricket matches, including matches in the 1987 and the 1996 ICC Cricket World Cups. The Hyderabad cricket team represents the city in the Ranji Trophy—a first-class cricket tournament among India's states and cities. Hyderabad is also home to the Indian Premier League franchise Sunrisers Hyderabad. A previous franchise was the Deccan Chargers, which won the 2009 Indian Premier League held in South Africa.", "The most popular sports played in Hyderabad are cricket and association football. At the professional level, the city has hosted national and international sports events such as the 2002 National Games of India, the 2003 Afro-Asian Games, the 2004 AP Tourism Hyderabad Open women's tennis tournament, the 2007 Military World Games, the 2009 World Badminton Championships and the 2009 IBSF World Snooker Championship. The city hosts a number of venues suitable for professional competition such as the Swarnandhra Pradesh Sports Complex for field hockey, the G. M. C. Balayogi Stadium in Gachibowli for athletics and football, and for cricket, the Lal Bahadur Shastri Stadium and Rajiv Gandhi International Cricket Stadium, home ground of the Hyderabad Cricket Association. Hyderabad has hosted many international cricket matches, including matches in the 1987 and the 1996 ICC Cricket World Cups. The Hyderabad cricket team represents the city in the Ranji Trophy—a first-class cricket tournament among India's states and cities. Hyderabad is also home to the Indian Premier League franchise Sunrisers Hyderabad. A previous franchise was the Deccan Chargers, which won the 2009 Indian Premier League held in South Africa.", "The most popular sports played in Hyderabad are cricket and association football. At the professional level, the city has hosted national and international sports events such as the 2002 National Games of India, the 2003 Afro-Asian Games, the 2004 AP Tourism Hyderabad Open women's tennis tournament, the 2007 Military World Games, the 2009 World Badminton Championships and the 2009 IBSF World Snooker Championship. The city hosts a number of venues suitable for professional competition such as the Swarnandhra Pradesh Sports Complex for field hockey, the G. M. C. Balayogi Stadium in Gachibowli for athletics and football, and for cricket, the Lal Bahadur Shastri Stadium and Rajiv Gandhi International Cricket Stadium, home ground of the Hyderabad Cricket Association. Hyderabad has hosted many international cricket matches, including matches in the 1987 and the 1996 ICC Cricket World Cups. The Hyderabad cricket team represents the city in the Ranji Trophy—a first-class cricket tournament among India's states and cities. Hyderabad is also home to the Indian Premier League franchise Sunrisers Hyderabad. A previous franchise was the Deccan Chargers, which won the 2009 Indian Premier League held in South Africa.", "The most popular sports played in Hyderabad are cricket and association football. At the professional level, the city has hosted national and international sports events such as the 2002 National Games of India, the 2003 Afro-Asian Games, the 2004 AP Tourism Hyderabad Open women's tennis tournament, the 2007 Military World Games, the 2009 World Badminton Championships and the 2009 IBSF World Snooker Championship. The city hosts a number of venues suitable for professional competition such as the Swarnandhra Pradesh Sports Complex for field hockey, the G. M. C. Balayogi Stadium in Gachibowli for athletics and football, and for cricket, the Lal Bahadur Shastri Stadium and Rajiv Gandhi International Cricket Stadium, home ground of the Hyderabad Cricket Association. Hyderabad has hosted many international cricket matches, including matches in the 1987 and the 1996 ICC Cricket World Cups. The Hyderabad cricket team represents the city in the Ranji Trophy—a first-class cricket tournament among India's states and cities. Hyderabad is also home to the Indian Premier League franchise Sunrisers Hyderabad. A previous franchise was the Deccan Chargers, which won the 2009 Indian Premier League held in South Africa.", "The most popular sports played in Hyderabad are cricket and association football. At the professional level, the city has hosted national and international sports events such as the 2002 National Games of India, the 2003 Afro-Asian Games, the 2004 AP Tourism Hyderabad Open women's tennis tournament, the 2007 Military World Games, the 2009 World Badminton Championships and the 2009 IBSF World Snooker Championship. The city hosts a number of venues suitable for professional competition such as the Swarnandhra Pradesh Sports Complex for field hockey, the G. M. C. Balayogi Stadium in Gachibowli for athletics and football, and for cricket, the Lal Bahadur Shastri Stadium and Rajiv Gandhi International Cricket Stadium, home ground of the Hyderabad Cricket Association. Hyderabad has hosted many international cricket matches, including matches in the 1987 and the 1996 ICC Cricket World Cups. The Hyderabad cricket team represents the city in the Ranji Trophy—a first-class cricket tournament among India's states and cities. Hyderabad is also home to the Indian Premier League franchise Sunrisers Hyderabad. A previous franchise was the Deccan Chargers, which won the 2009 Indian Premier League held in South Africa.", 'During British rule, Secunderabad became a well-known sporting centre and many race courses, parade grounds and polo fields were built.:18 Many elite clubs formed by the Nizams and the British such as the Secunderabad Club, the Nizam Club and the Hyderabad Race Club, which is known for its horse racing especially the annual Deccan derby, still exist. In more recent times, motorsports has become popular with the Andhra Pradesh Motor Sports Club organising popular events such as the Deccan ¼ Mile Drag, TSD Rallies and 4x4 off-road rallying.', 'During British rule, Secunderabad became a well-known sporting centre and many race courses, parade grounds and polo fields were built.:18 Many elite clubs formed by the Nizams and the British such as the Secunderabad Club, the Nizam Club and the Hyderabad Race Club, which is known for its horse racing especially the annual Deccan derby, still exist. In more recent times, motorsports has become popular with the Andhra Pradesh Motor Sports Club organising popular events such as the Deccan ¼ Mile Drag, TSD Rallies and 4x4 off-road rallying.', 'During British rule, Secunderabad became a well-known sporting centre and many race courses, parade grounds and polo fields were built.:18 Many elite clubs formed by the Nizams and the British such as the Secunderabad Club, the Nizam Club and the Hyderabad Race Club, which is known for its horse racing especially the annual Deccan derby, still exist. In more recent times, motorsports has become popular with the Andhra Pradesh Motor Sports Club organising popular events such as the Deccan ¼ Mile Drag, TSD Rallies and 4x4 off-road rallying.', 'During British rule, Secunderabad became a well-known sporting centre and many race courses, parade grounds and polo fields were built.:18 Many elite clubs formed by the Nizams and the British such as the Secunderabad Club, the Nizam Club and the Hyderabad Race Club, which is known for its horse racing especially the annual Deccan derby, still exist. In more recent times, motorsports has become popular with the Andhra Pradesh Motor Sports Club organising popular events such as the Deccan ¼ Mile Drag, TSD Rallies and 4x4 off-road rallying.', 'During British rule, Secunderabad became a well-known sporting centre and many race courses, parade grounds and polo fields were built.:18 Many elite clubs formed by the Nizams and the British such as the Secunderabad Club, the Nizam Club and the Hyderabad Race Club, which is known for its horse racing especially the annual Deccan derby, still exist. In more recent times, motorsports has become popular with the Andhra Pradesh Motor Sports Club organising popular events such as the Deccan ¼ Mile Drag, TSD Rallies and 4x4 off-road rallying.', 'International-level sportspeople from Hyderabad include: cricketers Ghulam Ahmed, M. L. Jaisimha, Mohammed Azharuddin, V. V. S. Laxman, Venkatapathy Raju, Shivlal Yadav, Arshad Ayub, Syed Abid Ali and Noel David; football players Syed Abdul Rahim, Syed Nayeemuddin and Shabbir Ali; tennis player Sania Mirza; badminton players S. M. Arif, Pullela Gopichand, Saina Nehwal, P. V. Sindhu, Jwala Gutta and Chetan Anand; hockey players Syed Mohammad Hadi and Mukesh Kumar; rifle shooters Gagan Narang and Asher Noria and bodybuilder Mir Mohtesham Ali Khan.', 'International-level sportspeople from Hyderabad include: cricketers Ghulam Ahmed, M. L. Jaisimha, Mohammed Azharuddin, V. V. S. Laxman, Venkatapathy Raju, Shivlal Yadav, Arshad Ayub, Syed Abid Ali and Noel David; football players Syed Abdul Rahim, Syed Nayeemuddin and Shabbir Ali; tennis player Sania Mirza; badminton players S. M. Arif, Pullela Gopichand, Saina Nehwal, P. V. Sindhu, Jwala Gutta and Chetan Anand; hockey players Syed Mohammad Hadi and Mukesh Kumar; rifle shooters Gagan Narang and Asher Noria and bodybuilder Mir Mohtesham Ali Khan.', 'International-level sportspeople from Hyderabad include: cricketers Ghulam Ahmed, M. L. Jaisimha, Mohammed Azharuddin, V. V. S. Laxman, Venkatapathy Raju, Shivlal Yadav, Arshad Ayub, Syed Abid Ali and Noel David; football players Syed Abdul Rahim, Syed Nayeemuddin and Shabbir Ali; tennis player Sania Mirza; badminton players S. M. Arif, Pullela Gopichand, Saina Nehwal, P. V. Sindhu, Jwala Gutta and Chetan Anand; hockey players Syed Mohammad Hadi and Mukesh Kumar; rifle shooters Gagan Narang and Asher Noria and bodybuilder Mir Mohtesham Ali Khan.', 'International-level sportspeople from Hyderabad include: cricketers Ghulam Ahmed, M. L. Jaisimha, Mohammed Azharuddin, V. V. S. Laxman, Venkatapathy Raju, Shivlal Yadav, Arshad Ayub, Syed Abid Ali and Noel David; football players Syed Abdul Rahim, Syed Nayeemuddin and Shabbir Ali; tennis player Sania Mirza; badminton players S. M. Arif, Pullela Gopichand, Saina Nehwal, P. V. Sindhu, Jwala Gutta and Chetan Anand; hockey players Syed Mohammad Hadi and Mukesh Kumar; rifle shooters Gagan Narang and Asher Noria and bodybuilder Mir Mohtesham Ali Khan.', 'International-level sportspeople from Hyderabad include: cricketers Ghulam Ahmed, M. L. Jaisimha, Mohammed Azharuddin, V. V. S. Laxman, Venkatapathy Raju, Shivlal Yadav, Arshad Ayub, Syed Abid Ali and Noel David; football players Syed Abdul Rahim, Syed Nayeemuddin and Shabbir Ali; tennis player Sania Mirza; badminton players S. M. Arif, Pullela Gopichand, Saina Nehwal, P. V. Sindhu, Jwala Gutta and Chetan Anand; hockey players Syed Mohammad Hadi and Mukesh Kumar; rifle shooters Gagan Narang and Asher Noria and bodybuilder Mir Mohtesham Ali Khan.', "The most commonly used forms of medium distance transport in Hyderabad include government owned services such as light railways and buses, as well as privately operated taxis and auto rickshaws. Bus services operate from the Mahatma Gandhi Bus Station in the city centre and carry over 130 million passengers daily across the entire network.:76 Hyderabad's light rail transportation system, the Multi-Modal Transport System (MMTS), is a three line suburban rail service used by over 160,000 passengers daily. Complementing these government services are minibus routes operated by Setwin (Society for Employment Promotion & Training in Twin Cities). Intercity rail services also operate from Hyderabad; the main, and largest, station is Secunderabad Railway Station, which serves as Indian Railways' South Central Railway zone headquarters and a hub for both buses and MMTS light rail services connecting Secunderabad and Hyderabad. Other major railway stations in Hyderabad are Hyderabad Deccan Station, Kachiguda Railway Station, Begumpet Railway Station, Malkajgiri Railway Station and Lingampally Railway Station. The Hyderabad Metro, a new rapid transit system, is to be added to the existing public transport infrastructure and is scheduled to operate three lines by 2015.", "The most commonly used forms of medium distance transport in Hyderabad include government owned services such as light railways and buses, as well as privately operated taxis and auto rickshaws. Bus services operate from the Mahatma Gandhi Bus Station in the city centre and carry over 130 million passengers daily across the entire network.:76 Hyderabad's light rail transportation system, the Multi-Modal Transport System (MMTS), is a three line suburban rail service used by over 160,000 passengers daily. Complementing these government services are minibus routes operated by Setwin (Society for Employment Promotion & Training in Twin Cities). Intercity rail services also operate from Hyderabad; the main, and largest, station is Secunderabad Railway Station, which serves as Indian Railways' South Central Railway zone headquarters and a hub for both buses and MMTS light rail services connecting Secunderabad and Hyderabad. Other major railway stations in Hyderabad are Hyderabad Deccan Station, Kachiguda Railway Station, Begumpet Railway Station, Malkajgiri Railway Station and Lingampally Railway Station. The Hyderabad Metro, a new rapid transit system, is to be added to the existing public transport infrastructure and is scheduled to operate three lines by 2015.", "The most commonly used forms of medium distance transport in Hyderabad include government owned services such as light railways and buses, as well as privately operated taxis and auto rickshaws. Bus services operate from the Mahatma Gandhi Bus Station in the city centre and carry over 130 million passengers daily across the entire network.:76 Hyderabad's light rail transportation system, the Multi-Modal Transport System (MMTS), is a three line suburban rail service used by over 160,000 passengers daily. Complementing these government services are minibus routes operated by Setwin (Society for Employment Promotion & Training in Twin Cities). Intercity rail services also operate from Hyderabad; the main, and largest, station is Secunderabad Railway Station, which serves as Indian Railways' South Central Railway zone headquarters and a hub for both buses and MMTS light rail services connecting Secunderabad and Hyderabad. Other major railway stations in Hyderabad are Hyderabad Deccan Station, Kachiguda Railway Station, Begumpet Railway Station, Malkajgiri Railway Station and Lingampally Railway Station. The Hyderabad Metro, a new rapid transit system, is to be added to the existing public transport infrastructure and is scheduled to operate three lines by 2015.", "The most commonly used forms of medium distance transport in Hyderabad include government owned services such as light railways and buses, as well as privately operated taxis and auto rickshaws. Bus services operate from the Mahatma Gandhi Bus Station in the city centre and carry over 130 million passengers daily across the entire network.:76 Hyderabad's light rail transportation system, the Multi-Modal Transport System (MMTS), is a three line suburban rail service used by over 160,000 passengers daily. Complementing these government services are minibus routes operated by Setwin (Society for Employment Promotion & Training in Twin Cities). Intercity rail services also operate from Hyderabad; the main, and largest, station is Secunderabad Railway Station, which serves as Indian Railways' South Central Railway zone headquarters and a hub for both buses and MMTS light rail services connecting Secunderabad and Hyderabad. Other major railway stations in Hyderabad are Hyderabad Deccan Station, Kachiguda Railway Station, Begumpet Railway Station, Malkajgiri Railway Station and Lingampally Railway Station. The Hyderabad Metro, a new rapid transit system, is to be added to the existing public transport infrastructure and is scheduled to operate three lines by 2015.", "The most commonly used forms of medium distance transport in Hyderabad include government owned services such as light railways and buses, as well as privately operated taxis and auto rickshaws. Bus services operate from the Mahatma Gandhi Bus Station in the city centre and carry over 130 million passengers daily across the entire network.:76 Hyderabad's light rail transportation system, the Multi-Modal Transport System (MMTS), is a three line suburban rail service used by over 160,000 passengers daily. Complementing these government services are minibus routes operated by Setwin (Society for Employment Promotion & Training in Twin Cities). Intercity rail services also operate from Hyderabad; the main, and largest, station is Secunderabad Railway Station, which serves as Indian Railways' South Central Railway zone headquarters and a hub for both buses and MMTS light rail services connecting Secunderabad and Hyderabad. Other major railway stations in Hyderabad are Hyderabad Deccan Station, Kachiguda Railway Station, Begumpet Railway Station, Malkajgiri Railway Station and Lingampally Railway Station. The Hyderabad Metro, a new rapid transit system, is to be added to the existing public transport infrastructure and is scheduled to operate three lines by 2015.", 'As of 2012[update], there are over 3.5 million vehicles operating in the city, of which 74% are two-wheelers, 15% cars and 3% three-wheelers. The remaining 8% include buses, goods vehicles and taxis. The large number of vehicles coupled with relatively low road coverage—roads occupy only 9.5% of the total city area:79—has led to widespread traffic congestion especially since 80% of passengers and 60% of freight are transported by road.:3 The Inner Ring Road, the Outer Ring Road, the Hyderabad Elevated Expressway, the longest flyover in India, and various interchanges, overpasses and underpasses were built to ease the congestion. Maximum speed limits within the city are 50 km/h (31 mph) for two-wheelers and cars, 35 km/h (22 mph) for auto rickshaws and 40 km/h (25 mph) for light commercial vehicles and buses.', 'As of 2012[update], there are over 3.5 million vehicles operating in the city, of which 74% are two-wheelers, 15% cars and 3% three-wheelers. The remaining 8% include buses, goods vehicles and taxis. The large number of vehicles coupled with relatively low road coverage—roads occupy only 9.5% of the total city area:79—has led to widespread traffic congestion especially since 80% of passengers and 60% of freight are transported by road.:3 The Inner Ring Road, the Outer Ring Road, the Hyderabad Elevated Expressway, the longest flyover in India, and various interchanges, overpasses and underpasses were built to ease the congestion. Maximum speed limits within the city are 50 km/h (31 mph) for two-wheelers and cars, 35 km/h (22 mph) for auto rickshaws and 40 km/h (25 mph) for light commercial vehicles and buses.', 'As of 2012[update], there are over 3.5 million vehicles operating in the city, of which 74% are two-wheelers, 15% cars and 3% three-wheelers. The remaining 8% include buses, goods vehicles and taxis. The large number of vehicles coupled with relatively low road coverage—roads occupy only 9.5% of the total city area:79—has led to widespread traffic congestion especially since 80% of passengers and 60% of freight are transported by road.:3 The Inner Ring Road, the Outer Ring Road, the Hyderabad Elevated Expressway, the longest flyover in India, and various interchanges, overpasses and underpasses were built to ease the congestion. Maximum speed limits within the city are 50 km/h (31 mph) for two-wheelers and cars, 35 km/h (22 mph) for auto rickshaws and 40 km/h (25 mph) for light commercial vehicles and buses.', 'As of 2012[update], there are over 3.5 million vehicles operating in the city, of which 74% are two-wheelers, 15% cars and 3% three-wheelers. The remaining 8% include buses, goods vehicles and taxis. The large number of vehicles coupled with relatively low road coverage—roads occupy only 9.5% of the total city area:79—has led to widespread traffic congestion especially since 80% of passengers and 60% of freight are transported by road.:3 The Inner Ring Road, the Outer Ring Road, the Hyderabad Elevated Expressway, the longest flyover in India, and various interchanges, overpasses and underpasses were built to ease the congestion. Maximum speed limits within the city are 50 km/h (31 mph) for two-wheelers and cars, 35 km/h (22 mph) for auto rickshaws and 40 km/h (25 mph) for light commercial vehicles and buses.', 'As of 2012[update], there are over 3.5 million vehicles operating in the city, of which 74% are two-wheelers, 15% cars and 3% three-wheelers. The remaining 8% include buses, goods vehicles and taxis. The large number of vehicles coupled with relatively low road coverage—roads occupy only 9.5% of the total city area:79—has led to widespread traffic congestion especially since 80% of passengers and 60% of freight are transported by road.:3 The Inner Ring Road, the Outer Ring Road, the Hyderabad Elevated Expressway, the longest flyover in India, and various interchanges, overpasses and underpasses were built to ease the congestion. Maximum speed limits within the city are 50 km/h (31 mph) for two-wheelers and cars, 35 km/h (22 mph) for auto rickshaws and 40 km/h (25 mph) for light commercial vehicles and buses.', 'Hyderabad sits at the junction of three National Highways linking it to six other states: NH-7 runs 2,369 km (1,472 mi) from Varanasi, Uttar Pradesh, in the north to Kanyakumari, Tamil Nadu, in the south; NH-9, runs 841 km (523 mi) east-west between Machilipatnam, Andhra Pradesh, and Pune, Maharashtra; and the 280 km (174 mi) NH-163 links Hyderabad to Bhopalpatnam, Chhattisgarh NH-765 links Hyderabad to Srisailam. Five state highways, SH-1, SH-2, SH-4, SH-5 and SH-6, either start from, or pass through, Hyderabad.:58', 'Hyderabad sits at the junction of three National Highways linking it to six other states: NH-7 runs 2,369 km (1,472 mi) from Varanasi, Uttar Pradesh, in the north to Kanyakumari, Tamil Nadu, in the south; NH-9, runs 841 km (523 mi) east-west between Machilipatnam, Andhra Pradesh, and Pune, Maharashtra; and the 280 km (174 mi) NH-163 links Hyderabad to Bhopalpatnam, Chhattisgarh NH-765 links Hyderabad to Srisailam. Five state highways, SH-1, SH-2, SH-4, SH-5 and SH-6, either start from, or pass through, Hyderabad.:58', 'Hyderabad sits at the junction of three National Highways linking it to six other states: NH-7 runs 2,369 km (1,472 mi) from Varanasi, Uttar Pradesh, in the north to Kanyakumari, Tamil Nadu, in the south; NH-9, runs 841 km (523 mi) east-west between Machilipatnam, Andhra Pradesh, and Pune, Maharashtra; and the 280 km (174 mi) NH-163 links Hyderabad to Bhopalpatnam, Chhattisgarh NH-765 links Hyderabad to Srisailam. Five state highways, SH-1, SH-2, SH-4, SH-5 and SH-6, either start from, or pass through, Hyderabad.:58', 'Hyderabad sits at the junction of three National Highways linking it to six other states: NH-7 runs 2,369 km (1,472 mi) from Varanasi, Uttar Pradesh, in the north to Kanyakumari, Tamil Nadu, in the south; NH-9, runs 841 km (523 mi) east-west between Machilipatnam, Andhra Pradesh, and Pune, Maharashtra; and the 280 km (174 mi) NH-163 links Hyderabad to Bhopalpatnam, Chhattisgarh NH-765 links Hyderabad to Srisailam. Five state highways, SH-1, SH-2, SH-4, SH-5 and SH-6, either start from, or pass through, Hyderabad.:58', 'Hyderabad sits at the junction of three National Highways linking it to six other states: NH-7 runs 2,369 km (1,472 mi) from Varanasi, Uttar Pradesh, in the north to Kanyakumari, Tamil Nadu, in the south; NH-9, runs 841 km (523 mi) east-west between Machilipatnam, Andhra Pradesh, and Pune, Maharashtra; and the 280 km (174 mi) NH-163 links Hyderabad to Bhopalpatnam, Chhattisgarh NH-765 links Hyderabad to Srisailam. Five state highways, SH-1, SH-2, SH-4, SH-5 and SH-6, either start from, or pass through, Hyderabad.:58', "Santa Monica is a beachfront city in western Los Angeles County, California, United States. The city is named after the Christian saint, Monica. Situated on Santa Monica Bay, it is bordered on three sides by the city of Los Angeles – Pacific Palisades to the north, Brentwood on the northeast, Sawtelle on the east, Mar Vista on the southeast, and Venice on the south. Santa Monica is well known for its affluent single-family neighborhoods but also has many neighborhoods consisting primarily of condominiums and apartments. Over two-thirds of Santa Monica's residents are renters. The Census Bureau population for Santa Monica in 2010 was 89,736.", "Santa Monica is a beachfront city in western Los Angeles County, California, United States. The city is named after the Christian saint, Monica. Situated on Santa Monica Bay, it is bordered on three sides by the city of Los Angeles – Pacific Palisades to the north, Brentwood on the northeast, Sawtelle on the east, Mar Vista on the southeast, and Venice on the south. Santa Monica is well known for its affluent single-family neighborhoods but also has many neighborhoods consisting primarily of condominiums and apartments. Over two-thirds of Santa Monica's residents are renters. The Census Bureau population for Santa Monica in 2010 was 89,736.", "Santa Monica is a beachfront city in western Los Angeles County, California, United States. The city is named after the Christian saint, Monica. Situated on Santa Monica Bay, it is bordered on three sides by the city of Los Angeles – Pacific Palisades to the north, Brentwood on the northeast, Sawtelle on the east, Mar Vista on the southeast, and Venice on the south. Santa Monica is well known for its affluent single-family neighborhoods but also has many neighborhoods consisting primarily of condominiums and apartments. Over two-thirds of Santa Monica's residents are renters. The Census Bureau population for Santa Monica in 2010 was 89,736.", "Santa Monica is a beachfront city in western Los Angeles County, California, United States. The city is named after the Christian saint, Monica. Situated on Santa Monica Bay, it is bordered on three sides by the city of Los Angeles – Pacific Palisades to the north, Brentwood on the northeast, Sawtelle on the east, Mar Vista on the southeast, and Venice on the south. Santa Monica is well known for its affluent single-family neighborhoods but also has many neighborhoods consisting primarily of condominiums and apartments. Over two-thirds of Santa Monica's residents are renters. The Census Bureau population for Santa Monica in 2010 was 89,736.", "Santa Monica is a beachfront city in western Los Angeles County, California, United States. The city is named after the Christian saint, Monica. Situated on Santa Monica Bay, it is bordered on three sides by the city of Los Angeles – Pacific Palisades to the north, Brentwood on the northeast, Sawtelle on the east, Mar Vista on the southeast, and Venice on the south. Santa Monica is well known for its affluent single-family neighborhoods but also has many neighborhoods consisting primarily of condominiums and apartments. Over two-thirds of Santa Monica's residents are renters. The Census Bureau population for Santa Monica in 2010 was 89,736.", "Santa Monica was long inhabited by the Tongva people. Santa Monica was called Kecheek in the Tongva language. The first non-indigenous group to set foot in the area was the party of explorer Gaspar de Portolà, who camped near the present day intersection of Barrington and Ohio Avenues on August 3, 1769. There are two different versions of the naming of the city. One says that it was named in honor of the feast day of Saint Monica (mother of Saint Augustine), but her feast day is actually May 4. Another version says that it was named by Juan Crespí on account of a pair of springs, the Kuruvungna Springs (Serra Springs), that were reminiscent of the tears that Saint Monica shed over her son's early impiety.", "Santa Monica was long inhabited by the Tongva people. Santa Monica was called Kecheek in the Tongva language. The first non-indigenous group to set foot in the area was the party of explorer Gaspar de Portolà, who camped near the present day intersection of Barrington and Ohio Avenues on August 3, 1769. There are two different versions of the naming of the city. One says that it was named in honor of the feast day of Saint Monica (mother of Saint Augustine), but her feast day is actually May 4. Another version says that it was named by Juan Crespí on account of a pair of springs, the Kuruvungna Springs (Serra Springs), that were reminiscent of the tears that Saint Monica shed over her son's early impiety.", "Santa Monica was long inhabited by the Tongva people. Santa Monica was called Kecheek in the Tongva language. The first non-indigenous group to set foot in the area was the party of explorer Gaspar de Portolà, who camped near the present day intersection of Barrington and Ohio Avenues on August 3, 1769. There are two different versions of the naming of the city. One says that it was named in honor of the feast day of Saint Monica (mother of Saint Augustine), but her feast day is actually May 4. Another version says that it was named by Juan Crespí on account of a pair of springs, the Kuruvungna Springs (Serra Springs), that were reminiscent of the tears that Saint Monica shed over her son's early impiety.", "Santa Monica was long inhabited by the Tongva people. Santa Monica was called Kecheek in the Tongva language. The first non-indigenous group to set foot in the area was the party of explorer Gaspar de Portolà, who camped near the present day intersection of Barrington and Ohio Avenues on August 3, 1769. There are two different versions of the naming of the city. One says that it was named in honor of the feast day of Saint Monica (mother of Saint Augustine), but her feast day is actually May 4. Another version says that it was named by Juan Crespí on account of a pair of springs, the Kuruvungna Springs (Serra Springs), that were reminiscent of the tears that Saint Monica shed over her son's early impiety.", "Santa Monica was long inhabited by the Tongva people. Santa Monica was called Kecheek in the Tongva language. The first non-indigenous group to set foot in the area was the party of explorer Gaspar de Portolà, who camped near the present day intersection of Barrington and Ohio Avenues on August 3, 1769. There are two different versions of the naming of the city. One says that it was named in honor of the feast day of Saint Monica (mother of Saint Augustine), but her feast day is actually May 4. Another version says that it was named by Juan Crespí on account of a pair of springs, the Kuruvungna Springs (Serra Springs), that were reminiscent of the tears that Saint Monica shed over her son's early impiety.", 'Around the start of the 20th century, a growing population of Asian Americans lived in or near Santa Monica and Venice. A Japanese fishing village was located near the Long Wharf while small numbers of Chinese lived or worked in both Santa Monica and Venice. The two ethnic minorities were often viewed differently by White Americans who were often well-disposed towards the Japanese but condescending towards the Chinese. The Japanese village fishermen were an integral economic part of the Santa Monica Bay community.', 'Around the start of the 20th century, a growing population of Asian Americans lived in or near Santa Monica and Venice. A Japanese fishing village was located near the Long Wharf while small numbers of Chinese lived or worked in both Santa Monica and Venice. The two ethnic minorities were often viewed differently by White Americans who were often well-disposed towards the Japanese but condescending towards the Chinese. The Japanese village fishermen were an integral economic part of the Santa Monica Bay community.', 'Around the start of the 20th century, a growing population of Asian Americans lived in or near Santa Monica and Venice. A Japanese fishing village was located near the Long Wharf while small numbers of Chinese lived or worked in both Santa Monica and Venice. The two ethnic minorities were often viewed differently by White Americans who were often well-disposed towards the Japanese but condescending towards the Chinese. The Japanese village fishermen were an integral economic part of the Santa Monica Bay community.', 'Around the start of the 20th century, a growing population of Asian Americans lived in or near Santa Monica and Venice. A Japanese fishing village was located near the Long Wharf while small numbers of Chinese lived or worked in both Santa Monica and Venice. The two ethnic minorities were often viewed differently by White Americans who were often well-disposed towards the Japanese but condescending towards the Chinese. The Japanese village fishermen were an integral economic part of the Santa Monica Bay community.', 'Around the start of the 20th century, a growing population of Asian Americans lived in or near Santa Monica and Venice. A Japanese fishing village was located near the Long Wharf while small numbers of Chinese lived or worked in both Santa Monica and Venice. The two ethnic minorities were often viewed differently by White Americans who were often well-disposed towards the Japanese but condescending towards the Chinese. The Japanese village fishermen were an integral economic part of the Santa Monica Bay community.', 'Donald Wills Douglas, Sr. built a plant in 1922 at Clover Field (Santa Monica Airport) for the Douglas Aircraft Company. In 1924, four Douglas-built planes took off from Clover Field to attempt the first aerial circumnavigation of the world. Two planes made it back, after having covered 27,553 miles (44,342 km) in 175 days, and were greeted on their return September 23, 1924, by a crowd of 200,000 (generously estimated). The Douglas Company (later McDonnell Douglas) kept facilities in the city until the 1960s.', 'Donald Wills Douglas, Sr. built a plant in 1922 at Clover Field (Santa Monica Airport) for the Douglas Aircraft Company. In 1924, four Douglas-built planes took off from Clover Field to attempt the first aerial circumnavigation of the world. Two planes made it back, after having covered 27,553 miles (44,342 km) in 175 days, and were greeted on their return September 23, 1924, by a crowd of 200,000 (generously estimated). The Douglas Company (later McDonnell Douglas) kept facilities in the city until the 1960s.', 'Donald Wills Douglas, Sr. built a plant in 1922 at Clover Field (Santa Monica Airport) for the Douglas Aircraft Company. In 1924, four Douglas-built planes took off from Clover Field to attempt the first aerial circumnavigation of the world. Two planes made it back, after having covered 27,553 miles (44,342 km) in 175 days, and were greeted on their return September 23, 1924, by a crowd of 200,000 (generously estimated). The Douglas Company (later McDonnell Douglas) kept facilities in the city until the 1960s.', 'Donald Wills Douglas, Sr. built a plant in 1922 at Clover Field (Santa Monica Airport) for the Douglas Aircraft Company. In 1924, four Douglas-built planes took off from Clover Field to attempt the first aerial circumnavigation of the world. Two planes made it back, after having covered 27,553 miles (44,342 km) in 175 days, and were greeted on their return September 23, 1924, by a crowd of 200,000 (generously estimated). The Douglas Company (later McDonnell Douglas) kept facilities in the city until the 1960s.', 'Donald Wills Douglas, Sr. built a plant in 1922 at Clover Field (Santa Monica Airport) for the Douglas Aircraft Company. In 1924, four Douglas-built planes took off from Clover Field to attempt the first aerial circumnavigation of the world. Two planes made it back, after having covered 27,553 miles (44,342 km) in 175 days, and were greeted on their return September 23, 1924, by a crowd of 200,000 (generously estimated). The Douglas Company (later McDonnell Douglas) kept facilities in the city until the 1960s.', "Douglas's business grew astronomically with the onset of World War II, employing as many as 44,000 people in 1943. To defend against air attack set designers from the Warner Brothers Studios prepared elaborate camouflage that disguised the factory and airfield. The RAND Corporation began as a project of the Douglas Company in 1945, and spun off into an independent think tank on May 14, 1948. RAND eventually acquired a 15-acre (61,000 m²) campus centrally located between the Civic Center and the pier entrance.", "Douglas's business grew astronomically with the onset of World War II, employing as many as 44,000 people in 1943. To defend against air attack set designers from the Warner Brothers Studios prepared elaborate camouflage that disguised the factory and airfield. The RAND Corporation began as a project of the Douglas Company in 1945, and spun off into an independent think tank on May 14, 1948. RAND eventually acquired a 15-acre (61,000 m²) campus centrally located between the Civic Center and the pier entrance.", "Douglas's business grew astronomically with the onset of World War II, employing as many as 44,000 people in 1943. To defend against air attack set designers from the Warner Brothers Studios prepared elaborate camouflage that disguised the factory and airfield. The RAND Corporation began as a project of the Douglas Company in 1945, and spun off into an independent think tank on May 14, 1948. RAND eventually acquired a 15-acre (61,000 m²) campus centrally located between the Civic Center and the pier entrance.", "Douglas's business grew astronomically with the onset of World War II, employing as many as 44,000 people in 1943. To defend against air attack set designers from the Warner Brothers Studios prepared elaborate camouflage that disguised the factory and airfield. The RAND Corporation began as a project of the Douglas Company in 1945, and spun off into an independent think tank on May 14, 1948. RAND eventually acquired a 15-acre (61,000 m²) campus centrally located between the Civic Center and the pier entrance.", "Douglas's business grew astronomically with the onset of World War II, employing as many as 44,000 people in 1943. To defend against air attack set designers from the Warner Brothers Studios prepared elaborate camouflage that disguised the factory and airfield. The RAND Corporation began as a project of the Douglas Company in 1945, and spun off into an independent think tank on May 14, 1948. RAND eventually acquired a 15-acre (61,000 m²) campus centrally located between the Civic Center and the pier entrance.", "The Santa Monica Looff Hippodrome (carousel) is a National Historic Landmark. It sits on the Santa Monica Pier, which was built in 1909. The La Monica Ballroom on the pier was once the largest ballroom in the US and the source for many New Year's Eve national network broadcasts. The Santa Monica Civic Auditorium was an important music venue for several decades and hosted the Academy Awards in the 1960s. McCabe's Guitar Shop is still a leading acoustic performance space as well as retail outlet. Bergamot Station is a city-owned art gallery compound that includes the Santa Monica Museum of Art. The city is also home to the California Heritage Museum and the Angels Attic dollhouse and toy museum.", "The Santa Monica Looff Hippodrome (carousel) is a National Historic Landmark. It sits on the Santa Monica Pier, which was built in 1909. The La Monica Ballroom on the pier was once the largest ballroom in the US and the source for many New Year's Eve national network broadcasts. The Santa Monica Civic Auditorium was an important music venue for several decades and hosted the Academy Awards in the 1960s. McCabe's Guitar Shop is still a leading acoustic performance space as well as retail outlet. Bergamot Station is a city-owned art gallery compound that includes the Santa Monica Museum of Art. The city is also home to the California Heritage Museum and the Angels Attic dollhouse and toy museum.", "The Santa Monica Looff Hippodrome (carousel) is a National Historic Landmark. It sits on the Santa Monica Pier, which was built in 1909. The La Monica Ballroom on the pier was once the largest ballroom in the US and the source for many New Year's Eve national network broadcasts. The Santa Monica Civic Auditorium was an important music venue for several decades and hosted the Academy Awards in the 1960s. McCabe's Guitar Shop is still a leading acoustic performance space as well as retail outlet. Bergamot Station is a city-owned art gallery compound that includes the Santa Monica Museum of Art. The city is also home to the California Heritage Museum and the Angels Attic dollhouse and toy museum.", "The Santa Monica Looff Hippodrome (carousel) is a National Historic Landmark. It sits on the Santa Monica Pier, which was built in 1909. The La Monica Ballroom on the pier was once the largest ballroom in the US and the source for many New Year's Eve national network broadcasts. The Santa Monica Civic Auditorium was an important music venue for several decades and hosted the Academy Awards in the 1960s. McCabe's Guitar Shop is still a leading acoustic performance space as well as retail outlet. Bergamot Station is a city-owned art gallery compound that includes the Santa Monica Museum of Art. The city is also home to the California Heritage Museum and the Angels Attic dollhouse and toy museum.", "The Santa Monica Looff Hippodrome (carousel) is a National Historic Landmark. It sits on the Santa Monica Pier, which was built in 1909. The La Monica Ballroom on the pier was once the largest ballroom in the US and the source for many New Year's Eve national network broadcasts. The Santa Monica Civic Auditorium was an important music venue for several decades and hosted the Academy Awards in the 1960s. McCabe's Guitar Shop is still a leading acoustic performance space as well as retail outlet. Bergamot Station is a city-owned art gallery compound that includes the Santa Monica Museum of Art. The city is also home to the California Heritage Museum and the Angels Attic dollhouse and toy museum.", "The Downtown District is the home of the Third Street Promenade, a major outdoor pedestrian-only shopping district that stretches for three blocks between Wilshire Blvd. and Broadway (not the same Broadway in downtown and south Los Angeles). Third Street is closed to vehicles for those three blocks to allow people to stroll, congregate, shop and enjoy street performers. Santa Monica Place, featuring Bloomingdale's and Nordstrom in a three-level outdoor environment, is located at the south end of the Promenade. After a period of redevelopment, the mall reopened in the fall of 2010 as a modern shopping, entertainment and dining complex with more outdoor space.", "The Downtown District is the home of the Third Street Promenade, a major outdoor pedestrian-only shopping district that stretches for three blocks between Wilshire Blvd. and Broadway (not the same Broadway in downtown and south Los Angeles). Third Street is closed to vehicles for those three blocks to allow people to stroll, congregate, shop and enjoy street performers. Santa Monica Place, featuring Bloomingdale's and Nordstrom in a three-level outdoor environment, is located at the south end of the Promenade. After a period of redevelopment, the mall reopened in the fall of 2010 as a modern shopping, entertainment and dining complex with more outdoor space.", "The Downtown District is the home of the Third Street Promenade, a major outdoor pedestrian-only shopping district that stretches for three blocks between Wilshire Blvd. and Broadway (not the same Broadway in downtown and south Los Angeles). Third Street is closed to vehicles for those three blocks to allow people to stroll, congregate, shop and enjoy street performers. Santa Monica Place, featuring Bloomingdale's and Nordstrom in a three-level outdoor environment, is located at the south end of the Promenade. After a period of redevelopment, the mall reopened in the fall of 2010 as a modern shopping, entertainment and dining complex with more outdoor space.", "The Downtown District is the home of the Third Street Promenade, a major outdoor pedestrian-only shopping district that stretches for three blocks between Wilshire Blvd. and Broadway (not the same Broadway in downtown and south Los Angeles). Third Street is closed to vehicles for those three blocks to allow people to stroll, congregate, shop and enjoy street performers. Santa Monica Place, featuring Bloomingdale's and Nordstrom in a three-level outdoor environment, is located at the south end of the Promenade. After a period of redevelopment, the mall reopened in the fall of 2010 as a modern shopping, entertainment and dining complex with more outdoor space.", "The Downtown District is the home of the Third Street Promenade, a major outdoor pedestrian-only shopping district that stretches for three blocks between Wilshire Blvd. and Broadway (not the same Broadway in downtown and south Los Angeles). Third Street is closed to vehicles for those three blocks to allow people to stroll, congregate, shop and enjoy street performers. Santa Monica Place, featuring Bloomingdale's and Nordstrom in a three-level outdoor environment, is located at the south end of the Promenade. After a period of redevelopment, the mall reopened in the fall of 2010 as a modern shopping, entertainment and dining complex with more outdoor space.", 'Every fall the Santa Monica Chamber of Commerce hosts The Taste of Santa Monica on the Santa Monica Pier. Visitors can sample food and drinks from Santa Monica restaurants. Other annual events include the Business and Consumer Expo, Sustainable Quality Awards, Santa Monica Cares Health and Wellness Festival, and the State of the City. The swanky Shutters on the Beach Hotel offers a trip to the famous Santa Monica Farmers Market to select and influence the materials that will become that evening\'s special "Market Dinner."', 'Every fall the Santa Monica Chamber of Commerce hosts The Taste of Santa Monica on the Santa Monica Pier. Visitors can sample food and drinks from Santa Monica restaurants. Other annual events include the Business and Consumer Expo, Sustainable Quality Awards, Santa Monica Cares Health and Wellness Festival, and the State of the City. The swanky Shutters on the Beach Hotel offers a trip to the famous Santa Monica Farmers Market to select and influence the materials that will become that evening\'s special "Market Dinner."', 'Every fall the Santa Monica Chamber of Commerce hosts The Taste of Santa Monica on the Santa Monica Pier. Visitors can sample food and drinks from Santa Monica restaurants. Other annual events include the Business and Consumer Expo, Sustainable Quality Awards, Santa Monica Cares Health and Wellness Festival, and the State of the City. The swanky Shutters on the Beach Hotel offers a trip to the famous Santa Monica Farmers Market to select and influence the materials that will become that evening\'s special "Market Dinner."', 'Every fall the Santa Monica Chamber of Commerce hosts The Taste of Santa Monica on the Santa Monica Pier. Visitors can sample food and drinks from Santa Monica restaurants. Other annual events include the Business and Consumer Expo, Sustainable Quality Awards, Santa Monica Cares Health and Wellness Festival, and the State of the City. The swanky Shutters on the Beach Hotel offers a trip to the famous Santa Monica Farmers Market to select and influence the materials that will become that evening\'s special "Market Dinner."', 'Every fall the Santa Monica Chamber of Commerce hosts The Taste of Santa Monica on the Santa Monica Pier. Visitors can sample food and drinks from Santa Monica restaurants. Other annual events include the Business and Consumer Expo, Sustainable Quality Awards, Santa Monica Cares Health and Wellness Festival, and the State of the City. The swanky Shutters on the Beach Hotel offers a trip to the famous Santa Monica Farmers Market to select and influence the materials that will become that evening\'s special "Market Dinner."', 'Classified as a Subtropical Mediterranean climate (Köppen Csb), Santa Monica enjoys an average of 310 days of sunshine a year. It is located in USDA plant hardiness zone 11a. Because of its location, nestled on the vast and open Santa Monica Bay, morning fog is a common phenomenon in May, June and early July (caused by ocean temperature variations and currents). Like other inhabitants of the greater Los Angeles area, residents have a particular terminology for this phenomenon: the "May Gray" and the "June Gloom". Overcast skies are common during June mornings, but usually the strong sun burns the fog off by noon. In the late winter/early summer, daily fog is a phenomenon too. It happens suddenly and it may last some hours or past sunset time. Nonetheless, it will sometimes stay cloudy and cool all day during June, even as other parts of the Los Angeles area enjoy sunny skies and warmer temperatures. At times, the sun can be shining east of 20th Street, while the beach area is overcast. As a general rule, the beach temperature is from 5 to 10 degrees Fahrenheit (3 to 6 degrees Celsius) cooler than it is inland during summer days, and 5–10 degrees warmer during winter nights.', 'Classified as a Subtropical Mediterranean climate (Köppen Csb), Santa Monica enjoys an average of 310 days of sunshine a year. It is located in USDA plant hardiness zone 11a. Because of its location, nestled on the vast and open Santa Monica Bay, morning fog is a common phenomenon in May, June and early July (caused by ocean temperature variations and currents). Like other inhabitants of the greater Los Angeles area, residents have a particular terminology for this phenomenon: the "May Gray" and the "June Gloom". Overcast skies are common during June mornings, but usually the strong sun burns the fog off by noon. In the late winter/early summer, daily fog is a phenomenon too. It happens suddenly and it may last some hours or past sunset time. Nonetheless, it will sometimes stay cloudy and cool all day during June, even as other parts of the Los Angeles area enjoy sunny skies and warmer temperatures. At times, the sun can be shining east of 20th Street, while the beach area is overcast. As a general rule, the beach temperature is from 5 to 10 degrees Fahrenheit (3 to 6 degrees Celsius) cooler than it is inland during summer days, and 5–10 degrees warmer during winter nights.', 'Classified as a Subtropical Mediterranean climate (Köppen Csb), Santa Monica enjoys an average of 310 days of sunshine a year. It is located in USDA plant hardiness zone 11a. Because of its location, nestled on the vast and open Santa Monica Bay, morning fog is a common phenomenon in May, June and early July (caused by ocean temperature variations and currents). Like other inhabitants of the greater Los Angeles area, residents have a particular terminology for this phenomenon: the "May Gray" and the "June Gloom". Overcast skies are common during June mornings, but usually the strong sun burns the fog off by noon. In the late winter/early summer, daily fog is a phenomenon too. It happens suddenly and it may last some hours or past sunset time. Nonetheless, it will sometimes stay cloudy and cool all day during June, even as other parts of the Los Angeles area enjoy sunny skies and warmer temperatures. At times, the sun can be shining east of 20th Street, while the beach area is overcast. As a general rule, the beach temperature is from 5 to 10 degrees Fahrenheit (3 to 6 degrees Celsius) cooler than it is inland during summer days, and 5–10 degrees warmer during winter nights.', 'Classified as a Subtropical Mediterranean climate (Köppen Csb), Santa Monica enjoys an average of 310 days of sunshine a year. It is located in USDA plant hardiness zone 11a. Because of its location, nestled on the vast and open Santa Monica Bay, morning fog is a common phenomenon in May, June and early July (caused by ocean temperature variations and currents). Like other inhabitants of the greater Los Angeles area, residents have a particular terminology for this phenomenon: the "May Gray" and the "June Gloom". Overcast skies are common during June mornings, but usually the strong sun burns the fog off by noon. In the late winter/early summer, daily fog is a phenomenon too. It happens suddenly and it may last some hours or past sunset time. Nonetheless, it will sometimes stay cloudy and cool all day during June, even as other parts of the Los Angeles area enjoy sunny skies and warmer temperatures. At times, the sun can be shining east of 20th Street, while the beach area is overcast. As a general rule, the beach temperature is from 5 to 10 degrees Fahrenheit (3 to 6 degrees Celsius) cooler than it is inland during summer days, and 5–10 degrees warmer during winter nights.', 'Classified as a Subtropical Mediterranean climate (Köppen Csb), Santa Monica enjoys an average of 310 days of sunshine a year. It is located in USDA plant hardiness zone 11a. Because of its location, nestled on the vast and open Santa Monica Bay, morning fog is a common phenomenon in May, June and early July (caused by ocean temperature variations and currents). Like other inhabitants of the greater Los Angeles area, residents have a particular terminology for this phenomenon: the "May Gray" and the "June Gloom". Overcast skies are common during June mornings, but usually the strong sun burns the fog off by noon. In the late winter/early summer, daily fog is a phenomenon too. It happens suddenly and it may last some hours or past sunset time. Nonetheless, it will sometimes stay cloudy and cool all day during June, even as other parts of the Los Angeles area enjoy sunny skies and warmer temperatures. At times, the sun can be shining east of 20th Street, while the beach area is overcast. As a general rule, the beach temperature is from 5 to 10 degrees Fahrenheit (3 to 6 degrees Celsius) cooler than it is inland during summer days, and 5–10 degrees warmer during winter nights.', "Santa Monica is one of the most environmentally activist municipalities in the nation. The city first proposed its Sustainable City Plan in 1992 and in 1994, was one of the first cities in the nation to formally adopt a comprehensive sustainability plan, setting waste reduction and water conservation policies for both public and private sector through its Office of Sustainability and the Environment. Eighty-two percent of the city's public works vehicles now run on alternative fuels, including nearly 100% of the municipal bus system, making it among the largest such fleets in the country. Santa Monica fleet vehicles and Buses now source their natural gas from Redeem, a Southern California-based supplier of renewable and sustainable natural gas obtained from non-fracked methane biogas generated from organic landfill waste.", "Santa Monica is one of the most environmentally activist municipalities in the nation. The city first proposed its Sustainable City Plan in 1992 and in 1994, was one of the first cities in the nation to formally adopt a comprehensive sustainability plan, setting waste reduction and water conservation policies for both public and private sector through its Office of Sustainability and the Environment. Eighty-two percent of the city's public works vehicles now run on alternative fuels, including nearly 100% of the municipal bus system, making it among the largest such fleets in the country. Santa Monica fleet vehicles and Buses now source their natural gas from Redeem, a Southern California-based supplier of renewable and sustainable natural gas obtained from non-fracked methane biogas generated from organic landfill waste.", "Santa Monica is one of the most environmentally activist municipalities in the nation. The city first proposed its Sustainable City Plan in 1992 and in 1994, was one of the first cities in the nation to formally adopt a comprehensive sustainability plan, setting waste reduction and water conservation policies for both public and private sector through its Office of Sustainability and the Environment. Eighty-two percent of the city's public works vehicles now run on alternative fuels, including nearly 100% of the municipal bus system, making it among the largest such fleets in the country. Santa Monica fleet vehicles and Buses now source their natural gas from Redeem, a Southern California-based supplier of renewable and sustainable natural gas obtained from non-fracked methane biogas generated from organic landfill waste.", "Santa Monica is one of the most environmentally activist municipalities in the nation. The city first proposed its Sustainable City Plan in 1992 and in 1994, was one of the first cities in the nation to formally adopt a comprehensive sustainability plan, setting waste reduction and water conservation policies for both public and private sector through its Office of Sustainability and the Environment. Eighty-two percent of the city's public works vehicles now run on alternative fuels, including nearly 100% of the municipal bus system, making it among the largest such fleets in the country. Santa Monica fleet vehicles and Buses now source their natural gas from Redeem, a Southern California-based supplier of renewable and sustainable natural gas obtained from non-fracked methane biogas generated from organic landfill waste.", "Santa Monica is one of the most environmentally activist municipalities in the nation. The city first proposed its Sustainable City Plan in 1992 and in 1994, was one of the first cities in the nation to formally adopt a comprehensive sustainability plan, setting waste reduction and water conservation policies for both public and private sector through its Office of Sustainability and the Environment. Eighty-two percent of the city's public works vehicles now run on alternative fuels, including nearly 100% of the municipal bus system, making it among the largest such fleets in the country. Santa Monica fleet vehicles and Buses now source their natural gas from Redeem, a Southern California-based supplier of renewable and sustainable natural gas obtained from non-fracked methane biogas generated from organic landfill waste.", "An urban runoff facility (SMURFF), the first of its kind in the US, catches and treats 3.5 million US gallons (13,000 m3) of water each week that would otherwise flow into the bay via storm-drains and sells it back to end-users within the city for reuse as gray-water, while bio-swales throughout the city allow rainwater to percolate into and replenish the groundwater supply. The groundwater supply in turn plays an important role in the city's Sustainable Water Master Plan, whereby Santa Monica has set a goal of attaining 100% water independence by 2020. The city has numerous programs designed to promote water conservation among residents, including a rebate of $1.50 per square foot for those who convert water intensive lawns to more local drought-tolerant gardens that require less water.", "An urban runoff facility (SMURFF), the first of its kind in the US, catches and treats 3.5 million US gallons (13,000 m3) of water each week that would otherwise flow into the bay via storm-drains and sells it back to end-users within the city for reuse as gray-water, while bio-swales throughout the city allow rainwater to percolate into and replenish the groundwater supply. The groundwater supply in turn plays an important role in the city's Sustainable Water Master Plan, whereby Santa Monica has set a goal of attaining 100% water independence by 2020. The city has numerous programs designed to promote water conservation among residents, including a rebate of $1.50 per square foot for those who convert water intensive lawns to more local drought-tolerant gardens that require less water.", "An urban runoff facility (SMURFF), the first of its kind in the US, catches and treats 3.5 million US gallons (13,000 m3) of water each week that would otherwise flow into the bay via storm-drains and sells it back to end-users within the city for reuse as gray-water, while bio-swales throughout the city allow rainwater to percolate into and replenish the groundwater supply. The groundwater supply in turn plays an important role in the city's Sustainable Water Master Plan, whereby Santa Monica has set a goal of attaining 100% water independence by 2020. The city has numerous programs designed to promote water conservation among residents, including a rebate of $1.50 per square foot for those who convert water intensive lawns to more local drought-tolerant gardens that require less water.", "An urban runoff facility (SMURFF), the first of its kind in the US, catches and treats 3.5 million US gallons (13,000 m3) of water each week that would otherwise flow into the bay via storm-drains and sells it back to end-users within the city for reuse as gray-water, while bio-swales throughout the city allow rainwater to percolate into and replenish the groundwater supply. The groundwater supply in turn plays an important role in the city's Sustainable Water Master Plan, whereby Santa Monica has set a goal of attaining 100% water independence by 2020. The city has numerous programs designed to promote water conservation among residents, including a rebate of $1.50 per square foot for those who convert water intensive lawns to more local drought-tolerant gardens that require less water.", "An urban runoff facility (SMURFF), the first of its kind in the US, catches and treats 3.5 million US gallons (13,000 m3) of water each week that would otherwise flow into the bay via storm-drains and sells it back to end-users within the city for reuse as gray-water, while bio-swales throughout the city allow rainwater to percolate into and replenish the groundwater supply. The groundwater supply in turn plays an important role in the city's Sustainable Water Master Plan, whereby Santa Monica has set a goal of attaining 100% water independence by 2020. The city has numerous programs designed to promote water conservation among residents, including a rebate of $1.50 per square foot for those who convert water intensive lawns to more local drought-tolerant gardens that require less water.", "The city is also in the process of implementing a 5-year and 20 year Bike Action Plan with a goal of attaining 14 to 35% bicycle transportation mode share by 2030 through the installation of enhanced bicycle infrastructure throughout the city. Other environmentally focused initiatives include curbside recycling, curbside composting bins (in addition to trash, yard-waste, and recycle bins), farmers' markets, community gardens, garden-share, an urban forest initiative, a hazardous materials home-collection service, green business certification, and a municipal bus system which is currently being revamped to integrate with the soon-to-open Expo Line.", "The city is also in the process of implementing a 5-year and 20 year Bike Action Plan with a goal of attaining 14 to 35% bicycle transportation mode share by 2030 through the installation of enhanced bicycle infrastructure throughout the city. Other environmentally focused initiatives include curbside recycling, curbside composting bins (in addition to trash, yard-waste, and recycle bins), farmers' markets, community gardens, garden-share, an urban forest initiative, a hazardous materials home-collection service, green business certification, and a municipal bus system which is currently being revamped to integrate with the soon-to-open Expo Line.", "The city is also in the process of implementing a 5-year and 20 year Bike Action Plan with a goal of attaining 14 to 35% bicycle transportation mode share by 2030 through the installation of enhanced bicycle infrastructure throughout the city. Other environmentally focused initiatives include curbside recycling, curbside composting bins (in addition to trash, yard-waste, and recycle bins), farmers' markets, community gardens, garden-share, an urban forest initiative, a hazardous materials home-collection service, green business certification, and a municipal bus system which is currently being revamped to integrate with the soon-to-open Expo Line.", "The city is also in the process of implementing a 5-year and 20 year Bike Action Plan with a goal of attaining 14 to 35% bicycle transportation mode share by 2030 through the installation of enhanced bicycle infrastructure throughout the city. Other environmentally focused initiatives include curbside recycling, curbside composting bins (in addition to trash, yard-waste, and recycle bins), farmers' markets, community gardens, garden-share, an urban forest initiative, a hazardous materials home-collection service, green business certification, and a municipal bus system which is currently being revamped to integrate with the soon-to-open Expo Line.", "The city is also in the process of implementing a 5-year and 20 year Bike Action Plan with a goal of attaining 14 to 35% bicycle transportation mode share by 2030 through the installation of enhanced bicycle infrastructure throughout the city. Other environmentally focused initiatives include curbside recycling, curbside composting bins (in addition to trash, yard-waste, and recycle bins), farmers' markets, community gardens, garden-share, an urban forest initiative, a hazardous materials home-collection service, green business certification, and a municipal bus system which is currently being revamped to integrate with the soon-to-open Expo Line.", 'There were 46,917 households, out of which 7,835 (16.7%) had children under the age of 18 living in them, 13,092 (27.9%) were opposite-sex married couples living together, 3,510 (7.5%) had a female householder with no husband present, 1,327 (2.8%) had a male householder with no wife present. There were 2,867 (6.1%) unmarried opposite-sex partnerships, and 416 (0.9%) same-sex married couples or partnerships. 22,716 households (48.4%) were made up of individuals and 5,551 (11.8%) had someone living alone who was 65 years of age or older. The average household size was 1.87. There were 17,929 families (38.2% of all households); the average family size was 2.79.', 'There were 46,917 households, out of which 7,835 (16.7%) had children under the age of 18 living in them, 13,092 (27.9%) were opposite-sex married couples living together, 3,510 (7.5%) had a female householder with no husband present, 1,327 (2.8%) had a male householder with no wife present. There were 2,867 (6.1%) unmarried opposite-sex partnerships, and 416 (0.9%) same-sex married couples or partnerships. 22,716 households (48.4%) were made up of individuals and 5,551 (11.8%) had someone living alone who was 65 years of age or older. The average household size was 1.87. There were 17,929 families (38.2% of all households); the average family size was 2.79.', 'There were 46,917 households, out of which 7,835 (16.7%) had children under the age of 18 living in them, 13,092 (27.9%) were opposite-sex married couples living together, 3,510 (7.5%) had a female householder with no husband present, 1,327 (2.8%) had a male householder with no wife present. There were 2,867 (6.1%) unmarried opposite-sex partnerships, and 416 (0.9%) same-sex married couples or partnerships. 22,716 households (48.4%) were made up of individuals and 5,551 (11.8%) had someone living alone who was 65 years of age or older. The average household size was 1.87. There were 17,929 families (38.2% of all households); the average family size was 2.79.', 'There were 46,917 households, out of which 7,835 (16.7%) had children under the age of 18 living in them, 13,092 (27.9%) were opposite-sex married couples living together, 3,510 (7.5%) had a female householder with no husband present, 1,327 (2.8%) had a male householder with no wife present. There were 2,867 (6.1%) unmarried opposite-sex partnerships, and 416 (0.9%) same-sex married couples or partnerships. 22,716 households (48.4%) were made up of individuals and 5,551 (11.8%) had someone living alone who was 65 years of age or older. The average household size was 1.87. There were 17,929 families (38.2% of all households); the average family size was 2.79.', 'There were 46,917 households, out of which 7,835 (16.7%) had children under the age of 18 living in them, 13,092 (27.9%) were opposite-sex married couples living together, 3,510 (7.5%) had a female householder with no husband present, 1,327 (2.8%) had a male householder with no wife present. There were 2,867 (6.1%) unmarried opposite-sex partnerships, and 416 (0.9%) same-sex married couples or partnerships. 22,716 households (48.4%) were made up of individuals and 5,551 (11.8%) had someone living alone who was 65 years of age or older. The average household size was 1.87. There were 17,929 families (38.2% of all households); the average family size was 2.79.', 'As of the census of 2000, there are 84,084 people, 44,497 households, and 16,775 families in the city. The population density is 10,178.7 inhabitants per square mile (3,930.4/km²). There are 47,863 housing units at an average density of 5,794.0 per square mile (2,237.3/km²). The racial makeup of the city is 78.29% White, 7.25% Asian, 3.78% African American, 0.47% Native American, 0.10% Pacific Islander, 5.97% from other races, and 4.13% from two or more races. 13.44% of the population are Hispanic or Latino of any race. There are 44,497 households, out of which 15.8% have children under the age of 18, 27.5% are married couples living together, 7.5% have a female householder with no husband present, and 62.3% are non-families. 51.2% of all households are made up of individuals and 10.6% have someone living alone who is 65 years of age or older. The average household size is 1.83 and the average family size is 2.80.', 'As of the census of 2000, there are 84,084 people, 44,497 households, and 16,775 families in the city. The population density is 10,178.7 inhabitants per square mile (3,930.4/km²). There are 47,863 housing units at an average density of 5,794.0 per square mile (2,237.3/km²). The racial makeup of the city is 78.29% White, 7.25% Asian, 3.78% African American, 0.47% Native American, 0.10% Pacific Islander, 5.97% from other races, and 4.13% from two or more races. 13.44% of the population are Hispanic or Latino of any race. There are 44,497 households, out of which 15.8% have children under the age of 18, 27.5% are married couples living together, 7.5% have a female householder with no husband present, and 62.3% are non-families. 51.2% of all households are made up of individuals and 10.6% have someone living alone who is 65 years of age or older. The average household size is 1.83 and the average family size is 2.80.', 'As of the census of 2000, there are 84,084 people, 44,497 households, and 16,775 families in the city. The population density is 10,178.7 inhabitants per square mile (3,930.4/km²). There are 47,863 housing units at an average density of 5,794.0 per square mile (2,237.3/km²). The racial makeup of the city is 78.29% White, 7.25% Asian, 3.78% African American, 0.47% Native American, 0.10% Pacific Islander, 5.97% from other races, and 4.13% from two or more races. 13.44% of the population are Hispanic or Latino of any race. There are 44,497 households, out of which 15.8% have children under the age of 18, 27.5% are married couples living together, 7.5% have a female householder with no husband present, and 62.3% are non-families. 51.2% of all households are made up of individuals and 10.6% have someone living alone who is 65 years of age or older. The average household size is 1.83 and the average family size is 2.80.', 'As of the census of 2000, there are 84,084 people, 44,497 households, and 16,775 families in the city. The population density is 10,178.7 inhabitants per square mile (3,930.4/km²). There are 47,863 housing units at an average density of 5,794.0 per square mile (2,237.3/km²). The racial makeup of the city is 78.29% White, 7.25% Asian, 3.78% African American, 0.47% Native American, 0.10% Pacific Islander, 5.97% from other races, and 4.13% from two or more races. 13.44% of the population are Hispanic or Latino of any race. There are 44,497 households, out of which 15.8% have children under the age of 18, 27.5% are married couples living together, 7.5% have a female householder with no husband present, and 62.3% are non-families. 51.2% of all households are made up of individuals and 10.6% have someone living alone who is 65 years of age or older. The average household size is 1.83 and the average family size is 2.80.', 'As of the census of 2000, there are 84,084 people, 44,497 households, and 16,775 families in the city. The population density is 10,178.7 inhabitants per square mile (3,930.4/km²). There are 47,863 housing units at an average density of 5,794.0 per square mile (2,237.3/km²). The racial makeup of the city is 78.29% White, 7.25% Asian, 3.78% African American, 0.47% Native American, 0.10% Pacific Islander, 5.97% from other races, and 4.13% from two or more races. 13.44% of the population are Hispanic or Latino of any race. There are 44,497 households, out of which 15.8% have children under the age of 18, 27.5% are married couples living together, 7.5% have a female householder with no husband present, and 62.3% are non-families. 51.2% of all households are made up of individuals and 10.6% have someone living alone who is 65 years of age or older. The average household size is 1.83 and the average family size is 2.80.', "Santa Monica College is a junior college originally founded in 1929. Many SMC graduates transfer to the University of California system. It occupies 35 acres (14 hectares) and enrolls 30,000 students annually. The Frederick S. Pardee RAND Graduate School, associated with the RAND Corporation, is the U.S.'s largest producer of public policy PhDs. The Art Institute of California – Los Angeles is also located in Santa Monica near the Santa Monica Airport. Universities and colleges within a 22-mile (35 km) radius from Santa Monica include Santa Monica College, Antioch University Los Angeles, Loyola Marymount University, Mount St. Mary's College, Pepperdine University, California State University, Northridge, California State University, Los Angeles, UCLA, USC, West Los Angeles College, California Institute of Technology (Caltech), Occidental College (Oxy), Los Angeles City College, Los Angeles Southwest College, Los Angeles Valley College, and Emperor's College of Traditional Oriental Medicine.", "Santa Monica College is a junior college originally founded in 1929. Many SMC graduates transfer to the University of California system. It occupies 35 acres (14 hectares) and enrolls 30,000 students annually. The Frederick S. Pardee RAND Graduate School, associated with the RAND Corporation, is the U.S.'s largest producer of public policy PhDs. The Art Institute of California – Los Angeles is also located in Santa Monica near the Santa Monica Airport. Universities and colleges within a 22-mile (35 km) radius from Santa Monica include Santa Monica College, Antioch University Los Angeles, Loyola Marymount University, Mount St. Mary's College, Pepperdine University, California State University, Northridge, California State University, Los Angeles, UCLA, USC, West Los Angeles College, California Institute of Technology (Caltech), Occidental College (Oxy), Los Angeles City College, Los Angeles Southwest College, Los Angeles Valley College, and Emperor's College of Traditional Oriental Medicine.", "Santa Monica College is a junior college originally founded in 1929. Many SMC graduates transfer to the University of California system. It occupies 35 acres (14 hectares) and enrolls 30,000 students annually. The Frederick S. Pardee RAND Graduate School, associated with the RAND Corporation, is the U.S.'s largest producer of public policy PhDs. The Art Institute of California – Los Angeles is also located in Santa Monica near the Santa Monica Airport. Universities and colleges within a 22-mile (35 km) radius from Santa Monica include Santa Monica College, Antioch University Los Angeles, Loyola Marymount University, Mount St. Mary's College, Pepperdine University, California State University, Northridge, California State University, Los Angeles, UCLA, USC, West Los Angeles College, California Institute of Technology (Caltech), Occidental College (Oxy), Los Angeles City College, Los Angeles Southwest College, Los Angeles Valley College, and Emperor's College of Traditional Oriental Medicine.", "Santa Monica College is a junior college originally founded in 1929. Many SMC graduates transfer to the University of California system. It occupies 35 acres (14 hectares) and enrolls 30,000 students annually. The Frederick S. Pardee RAND Graduate School, associated with the RAND Corporation, is the U.S.'s largest producer of public policy PhDs. The Art Institute of California – Los Angeles is also located in Santa Monica near the Santa Monica Airport. Universities and colleges within a 22-mile (35 km) radius from Santa Monica include Santa Monica College, Antioch University Los Angeles, Loyola Marymount University, Mount St. Mary's College, Pepperdine University, California State University, Northridge, California State University, Los Angeles, UCLA, USC, West Los Angeles College, California Institute of Technology (Caltech), Occidental College (Oxy), Los Angeles City College, Los Angeles Southwest College, Los Angeles Valley College, and Emperor's College of Traditional Oriental Medicine.", "Santa Monica College is a junior college originally founded in 1929. Many SMC graduates transfer to the University of California system. It occupies 35 acres (14 hectares) and enrolls 30,000 students annually. The Frederick S. Pardee RAND Graduate School, associated with the RAND Corporation, is the U.S.'s largest producer of public policy PhDs. The Art Institute of California – Los Angeles is also located in Santa Monica near the Santa Monica Airport. Universities and colleges within a 22-mile (35 km) radius from Santa Monica include Santa Monica College, Antioch University Los Angeles, Loyola Marymount University, Mount St. Mary's College, Pepperdine University, California State University, Northridge, California State University, Los Angeles, UCLA, USC, West Los Angeles College, California Institute of Technology (Caltech), Occidental College (Oxy), Los Angeles City College, Los Angeles Southwest College, Los Angeles Valley College, and Emperor's College of Traditional Oriental Medicine.", 'Santa Monica has a bike action plan and recently launched a Bicycle sharing system in November 2015. The city is traversed by the Marvin Braude Bike Trail. Santa Monica has received the Bicycle Friendly Community Award (Bronze in 2009, Silver in 2013) by the League of American Bicyclists. Local bicycle advocacy organizations include Santa Monica Spoke, a local chapter of the Los Angeles County Bicycle Coalition. Santa Monica is thought to be one of the leaders for bicycle infrastructure and programming in Los Angeles County.[citation needed]', 'Santa Monica has a bike action plan and recently launched a Bicycle sharing system in November 2015. The city is traversed by the Marvin Braude Bike Trail. Santa Monica has received the Bicycle Friendly Community Award (Bronze in 2009, Silver in 2013) by the League of American Bicyclists. Local bicycle advocacy organizations include Santa Monica Spoke, a local chapter of the Los Angeles County Bicycle Coalition. Santa Monica is thought to be one of the leaders for bicycle infrastructure and programming in Los Angeles County.[citation needed]', 'Santa Monica has a bike action plan and recently launched a Bicycle sharing system in November 2015. The city is traversed by the Marvin Braude Bike Trail. Santa Monica has received the Bicycle Friendly Community Award (Bronze in 2009, Silver in 2013) by the League of American Bicyclists. Local bicycle advocacy organizations include Santa Monica Spoke, a local chapter of the Los Angeles County Bicycle Coalition. Santa Monica is thought to be one of the leaders for bicycle infrastructure and programming in Los Angeles County.[citation needed]', 'Santa Monica has a bike action plan and recently launched a Bicycle sharing system in November 2015. The city is traversed by the Marvin Braude Bike Trail. Santa Monica has received the Bicycle Friendly Community Award (Bronze in 2009, Silver in 2013) by the League of American Bicyclists. Local bicycle advocacy organizations include Santa Monica Spoke, a local chapter of the Los Angeles County Bicycle Coalition. Santa Monica is thought to be one of the leaders for bicycle infrastructure and programming in Los Angeles County.[citation needed]', 'Santa Monica has a bike action plan and recently launched a Bicycle sharing system in November 2015. The city is traversed by the Marvin Braude Bike Trail. Santa Monica has received the Bicycle Friendly Community Award (Bronze in 2009, Silver in 2013) by the League of American Bicyclists. Local bicycle advocacy organizations include Santa Monica Spoke, a local chapter of the Los Angeles County Bicycle Coalition. Santa Monica is thought to be one of the leaders for bicycle infrastructure and programming in Los Angeles County.[citation needed]', 'The Santa Monica Freeway (Interstate 10) begins in Santa Monica near the Pacific Ocean and heads east. The Santa Monica Freeway between Santa Monica and downtown Los Angeles has the distinction of being one of the busiest highways in all of North America. After traversing Los Angeles County, I-10 crosses seven more states, terminating at Jacksonville, Florida. In Santa Monica, there is a road sign designating this route as the Christopher Columbus Transcontinental Highway. State Route 2 (Santa Monica Boulevard) begins in Santa Monica, barely grazing State Route 1 at Lincoln Boulevard, and continues northeast across Los Angeles County, through the Angeles National Forest, crossing the San Gabriel Mountains as the Angeles Crest Highway, ending in Wrightwood. Santa Monica is also the western (Pacific) terminus of historic U.S. Route 66. Close to the eastern boundary of Santa Monica, Sepulveda Boulevard reaches from Long Beach at the south, to the northern end of the San Fernando Valley. Just east of Santa Monica is Interstate 405, the "San Diego Freeway", a major north-south route in Los Angeles County and Orange County, California.', 'The Santa Monica Freeway (Interstate 10) begins in Santa Monica near the Pacific Ocean and heads east. The Santa Monica Freeway between Santa Monica and downtown Los Angeles has the distinction of being one of the busiest highways in all of North America. After traversing Los Angeles County, I-10 crosses seven more states, terminating at Jacksonville, Florida. In Santa Monica, there is a road sign designating this route as the Christopher Columbus Transcontinental Highway. State Route 2 (Santa Monica Boulevard) begins in Santa Monica, barely grazing State Route 1 at Lincoln Boulevard, and continues northeast across Los Angeles County, through the Angeles National Forest, crossing the San Gabriel Mountains as the Angeles Crest Highway, ending in Wrightwood. Santa Monica is also the western (Pacific) terminus of historic U.S. Route 66. Close to the eastern boundary of Santa Monica, Sepulveda Boulevard reaches from Long Beach at the south, to the northern end of the San Fernando Valley. Just east of Santa Monica is Interstate 405, the "San Diego Freeway", a major north-south route in Los Angeles County and Orange County, California.', 'The Santa Monica Freeway (Interstate 10) begins in Santa Monica near the Pacific Ocean and heads east. The Santa Monica Freeway between Santa Monica and downtown Los Angeles has the distinction of being one of the busiest highways in all of North America. After traversing Los Angeles County, I-10 crosses seven more states, terminating at Jacksonville, Florida. In Santa Monica, there is a road sign designating this route as the Christopher Columbus Transcontinental Highway. State Route 2 (Santa Monica Boulevard) begins in Santa Monica, barely grazing State Route 1 at Lincoln Boulevard, and continues northeast across Los Angeles County, through the Angeles National Forest, crossing the San Gabriel Mountains as the Angeles Crest Highway, ending in Wrightwood. Santa Monica is also the western (Pacific) terminus of historic U.S. Route 66. Close to the eastern boundary of Santa Monica, Sepulveda Boulevard reaches from Long Beach at the south, to the northern end of the San Fernando Valley. Just east of Santa Monica is Interstate 405, the "San Diego Freeway", a major north-south route in Los Angeles County and Orange County, California.', 'The Santa Monica Freeway (Interstate 10) begins in Santa Monica near the Pacific Ocean and heads east. The Santa Monica Freeway between Santa Monica and downtown Los Angeles has the distinction of being one of the busiest highways in all of North America. After traversing Los Angeles County, I-10 crosses seven more states, terminating at Jacksonville, Florida. In Santa Monica, there is a road sign designating this route as the Christopher Columbus Transcontinental Highway. State Route 2 (Santa Monica Boulevard) begins in Santa Monica, barely grazing State Route 1 at Lincoln Boulevard, and continues northeast across Los Angeles County, through the Angeles National Forest, crossing the San Gabriel Mountains as the Angeles Crest Highway, ending in Wrightwood. Santa Monica is also the western (Pacific) terminus of historic U.S. Route 66. Close to the eastern boundary of Santa Monica, Sepulveda Boulevard reaches from Long Beach at the south, to the northern end of the San Fernando Valley. Just east of Santa Monica is Interstate 405, the "San Diego Freeway", a major north-south route in Los Angeles County and Orange County, California.', 'The Santa Monica Freeway (Interstate 10) begins in Santa Monica near the Pacific Ocean and heads east. The Santa Monica Freeway between Santa Monica and downtown Los Angeles has the distinction of being one of the busiest highways in all of North America. After traversing Los Angeles County, I-10 crosses seven more states, terminating at Jacksonville, Florida. In Santa Monica, there is a road sign designating this route as the Christopher Columbus Transcontinental Highway. State Route 2 (Santa Monica Boulevard) begins in Santa Monica, barely grazing State Route 1 at Lincoln Boulevard, and continues northeast across Los Angeles County, through the Angeles National Forest, crossing the San Gabriel Mountains as the Angeles Crest Highway, ending in Wrightwood. Santa Monica is also the western (Pacific) terminus of historic U.S. Route 66. Close to the eastern boundary of Santa Monica, Sepulveda Boulevard reaches from Long Beach at the south, to the northern end of the San Fernando Valley. Just east of Santa Monica is Interstate 405, the "San Diego Freeway", a major north-south route in Los Angeles County and Orange County, California.', 'Historical aspects of the Expo line route are noteworthy. It uses the right-of-way for the Santa Monica Air Line that provided electric-powered freight and passenger service between Los Angeles and Santa Monica beginning in the 1920s. Service was discontinued in 1953 but diesel-powered freight deliveries to warehouses along the route continued until March 11, 1988. The abandonment of the line spurred concerns within the community and the entire right-of-way was purchased from Southern Pacific by Los Angeles Metropolitan Transportation Authority. The line was built in 1875 as the steam-powered Los Angeles and Independence Railroad to bring mining ore to ships in Santa Monica harbor and as a passenger excursion train to the beach.', 'Historical aspects of the Expo line route are noteworthy. It uses the right-of-way for the Santa Monica Air Line that provided electric-powered freight and passenger service between Los Angeles and Santa Monica beginning in the 1920s. Service was discontinued in 1953 but diesel-powered freight deliveries to warehouses along the route continued until March 11, 1988. The abandonment of the line spurred concerns within the community and the entire right-of-way was purchased from Southern Pacific by Los Angeles Metropolitan Transportation Authority. The line was built in 1875 as the steam-powered Los Angeles and Independence Railroad to bring mining ore to ships in Santa Monica harbor and as a passenger excursion train to the beach.', 'Historical aspects of the Expo line route are noteworthy. It uses the right-of-way for the Santa Monica Air Line that provided electric-powered freight and passenger service between Los Angeles and Santa Monica beginning in the 1920s. Service was discontinued in 1953 but diesel-powered freight deliveries to warehouses along the route continued until March 11, 1988. The abandonment of the line spurred concerns within the community and the entire right-of-way was purchased from Southern Pacific by Los Angeles Metropolitan Transportation Authority. The line was built in 1875 as the steam-powered Los Angeles and Independence Railroad to bring mining ore to ships in Santa Monica harbor and as a passenger excursion train to the beach.', 'Historical aspects of the Expo line route are noteworthy. It uses the right-of-way for the Santa Monica Air Line that provided electric-powered freight and passenger service between Los Angeles and Santa Monica beginning in the 1920s. Service was discontinued in 1953 but diesel-powered freight deliveries to warehouses along the route continued until March 11, 1988. The abandonment of the line spurred concerns within the community and the entire right-of-way was purchased from Southern Pacific by Los Angeles Metropolitan Transportation Authority. The line was built in 1875 as the steam-powered Los Angeles and Independence Railroad to bring mining ore to ships in Santa Monica harbor and as a passenger excursion train to the beach.', 'Historical aspects of the Expo line route are noteworthy. It uses the right-of-way for the Santa Monica Air Line that provided electric-powered freight and passenger service between Los Angeles and Santa Monica beginning in the 1920s. Service was discontinued in 1953 but diesel-powered freight deliveries to warehouses along the route continued until March 11, 1988. The abandonment of the line spurred concerns within the community and the entire right-of-way was purchased from Southern Pacific by Los Angeles Metropolitan Transportation Authority. The line was built in 1875 as the steam-powered Los Angeles and Independence Railroad to bring mining ore to ships in Santa Monica harbor and as a passenger excursion train to the beach.', "In 2006, crime in Santa Monica affected 4.41% of the population, slightly lower than the national average crime rate that year of 4.48%. The majority of this was property crime, which affected 3.74% of Santa Monica's population in 2006; this was higher than the rates for Los Angeles County (2.76%) and California (3.17%), but lower than the national average (3.91%). These per-capita crime rates are computed based on Santa Monica's full-time population of about 85,000. However, the Santa Monica Police Department has suggested the actual per-capita crime rate is much lower, as tourists, workers, and beachgoers can increase the city's daytime population to between 250,000 and 450,000 people.", "In 2006, crime in Santa Monica affected 4.41% of the population, slightly lower than the national average crime rate that year of 4.48%. The majority of this was property crime, which affected 3.74% of Santa Monica's population in 2006; this was higher than the rates for Los Angeles County (2.76%) and California (3.17%), but lower than the national average (3.91%). These per-capita crime rates are computed based on Santa Monica's full-time population of about 85,000. However, the Santa Monica Police Department has suggested the actual per-capita crime rate is much lower, as tourists, workers, and beachgoers can increase the city's daytime population to between 250,000 and 450,000 people.", "In 2006, crime in Santa Monica affected 4.41% of the population, slightly lower than the national average crime rate that year of 4.48%. The majority of this was property crime, which affected 3.74% of Santa Monica's population in 2006; this was higher than the rates for Los Angeles County (2.76%) and California (3.17%), but lower than the national average (3.91%). These per-capita crime rates are computed based on Santa Monica's full-time population of about 85,000. However, the Santa Monica Police Department has suggested the actual per-capita crime rate is much lower, as tourists, workers, and beachgoers can increase the city's daytime population to between 250,000 and 450,000 people.", "In 2006, crime in Santa Monica affected 4.41% of the population, slightly lower than the national average crime rate that year of 4.48%. The majority of this was property crime, which affected 3.74% of Santa Monica's population in 2006; this was higher than the rates for Los Angeles County (2.76%) and California (3.17%), but lower than the national average (3.91%). These per-capita crime rates are computed based on Santa Monica's full-time population of about 85,000. However, the Santa Monica Police Department has suggested the actual per-capita crime rate is much lower, as tourists, workers, and beachgoers can increase the city's daytime population to between 250,000 and 450,000 people.", "In 2006, crime in Santa Monica affected 4.41% of the population, slightly lower than the national average crime rate that year of 4.48%. The majority of this was property crime, which affected 3.74% of Santa Monica's population in 2006; this was higher than the rates for Los Angeles County (2.76%) and California (3.17%), but lower than the national average (3.91%). These per-capita crime rates are computed based on Santa Monica's full-time population of about 85,000. However, the Santa Monica Police Department has suggested the actual per-capita crime rate is much lower, as tourists, workers, and beachgoers can increase the city's daytime population to between 250,000 and 450,000 people.", 'In 1999, there was a double homicide in the Westside Clothing store on Lincoln Boulevard. During the incident, Culver City gang members David "Puppet" Robles and Jesse "Psycho" Garcia entered the store masked and began opening fire, killing Anthony and Michael Juarez. They then ran outside to a getaway vehicle driven by a third Culver City gang member, who is now also in custody. The clothing store was believed to be a local hang out for Santa Monica gang members. The dead included two men from Northern California who had merely been visiting the store\'s owner, their cousin, to see if they could open a similar store in their area. Police say the incident was in retaliation for a shooting committed by the Santa Monica 13 gang days before the Juarez brothers were gunned down.', 'In 1999, there was a double homicide in the Westside Clothing store on Lincoln Boulevard. During the incident, Culver City gang members David "Puppet" Robles and Jesse "Psycho" Garcia entered the store masked and began opening fire, killing Anthony and Michael Juarez. They then ran outside to a getaway vehicle driven by a third Culver City gang member, who is now also in custody. The clothing store was believed to be a local hang out for Santa Monica gang members. The dead included two men from Northern California who had merely been visiting the store\'s owner, their cousin, to see if they could open a similar store in their area. Police say the incident was in retaliation for a shooting committed by the Santa Monica 13 gang days before the Juarez brothers were gunned down.', 'In 1999, there was a double homicide in the Westside Clothing store on Lincoln Boulevard. During the incident, Culver City gang members David "Puppet" Robles and Jesse "Psycho" Garcia entered the store masked and began opening fire, killing Anthony and Michael Juarez. They then ran outside to a getaway vehicle driven by a third Culver City gang member, who is now also in custody. The clothing store was believed to be a local hang out for Santa Monica gang members. The dead included two men from Northern California who had merely been visiting the store\'s owner, their cousin, to see if they could open a similar store in their area. Police say the incident was in retaliation for a shooting committed by the Santa Monica 13 gang days before the Juarez brothers were gunned down.', 'In 1999, there was a double homicide in the Westside Clothing store on Lincoln Boulevard. During the incident, Culver City gang members David "Puppet" Robles and Jesse "Psycho" Garcia entered the store masked and began opening fire, killing Anthony and Michael Juarez. They then ran outside to a getaway vehicle driven by a third Culver City gang member, who is now also in custody. The clothing store was believed to be a local hang out for Santa Monica gang members. The dead included two men from Northern California who had merely been visiting the store\'s owner, their cousin, to see if they could open a similar store in their area. Police say the incident was in retaliation for a shooting committed by the Santa Monica 13 gang days before the Juarez brothers were gunned down.', 'In 1999, there was a double homicide in the Westside Clothing store on Lincoln Boulevard. During the incident, Culver City gang members David "Puppet" Robles and Jesse "Psycho" Garcia entered the store masked and began opening fire, killing Anthony and Michael Juarez. They then ran outside to a getaway vehicle driven by a third Culver City gang member, who is now also in custody. The clothing store was believed to be a local hang out for Santa Monica gang members. The dead included two men from Northern California who had merely been visiting the store\'s owner, their cousin, to see if they could open a similar store in their area. Police say the incident was in retaliation for a shooting committed by the Santa Monica 13 gang days before the Juarez brothers were gunned down.', 'Hundreds of movies have been shot or set in part within the city of Santa Monica. One of the oldest exterior shots in Santa Monica is Buster Keaton\'s Spite Marriage (1929) which shows much of 2nd Street. The comedy It\'s a Mad, Mad, Mad, Mad World (1963) included several scenes shot in Santa Monica, including those along the California Incline, which led to the movie\'s treasure spot, "The Big W". The Sylvester Stallone film Rocky III (1982) shows Rocky Balboa and Apollo Creed training to fight Clubber Lang by running on the Santa Monica Beach, and Stallone\'s Demolition Man (1993) includes Santa Monica settings. Henry Jaglom\'s indie Someone to Love (1987), the last film in which Orson Welles appeared, takes place in Santa Monica\'s venerable Mayfair Theatre. Heathers (1989) used Santa Monica\'s John Adams Middle School for many exterior shots. The Truth About Cats & Dogs (1996) is set entirely in Santa Monica, particularly the Palisades Park area, and features a radio station that resembles KCRW at Santa Monica College. 17 Again (2009) was shot at Samohi. Other films that show significant exterior shots of Santa Monica include Fletch (1985), Species (1995), Get Shorty (1995), and Ocean\'s Eleven (2001). Richard Rossi\'s biopic Aimee Semple McPherson opens and closes at the beach in Santa Monica. Iron Man features the Santa Monica pier and surrounding communities as Tony Stark tests his experimental flight suit.', 'Hundreds of movies have been shot or set in part within the city of Santa Monica. One of the oldest exterior shots in Santa Monica is Buster Keaton\'s Spite Marriage (1929) which shows much of 2nd Street. The comedy It\'s a Mad, Mad, Mad, Mad World (1963) included several scenes shot in Santa Monica, including those along the California Incline, which led to the movie\'s treasure spot, "The Big W". The Sylvester Stallone film Rocky III (1982) shows Rocky Balboa and Apollo Creed training to fight Clubber Lang by running on the Santa Monica Beach, and Stallone\'s Demolition Man (1993) includes Santa Monica settings. Henry Jaglom\'s indie Someone to Love (1987), the last film in which Orson Welles appeared, takes place in Santa Monica\'s venerable Mayfair Theatre. Heathers (1989) used Santa Monica\'s John Adams Middle School for many exterior shots. The Truth About Cats & Dogs (1996) is set entirely in Santa Monica, particularly the Palisades Park area, and features a radio station that resembles KCRW at Santa Monica College. 17 Again (2009) was shot at Samohi. Other films that show significant exterior shots of Santa Monica include Fletch (1985), Species (1995), Get Shorty (1995), and Ocean\'s Eleven (2001). Richard Rossi\'s biopic Aimee Semple McPherson opens and closes at the beach in Santa Monica. Iron Man features the Santa Monica pier and surrounding communities as Tony Stark tests his experimental flight suit.', 'Hundreds of movies have been shot or set in part within the city of Santa Monica. One of the oldest exterior shots in Santa Monica is Buster Keaton\'s Spite Marriage (1929) which shows much of 2nd Street. The comedy It\'s a Mad, Mad, Mad, Mad World (1963) included several scenes shot in Santa Monica, including those along the California Incline, which led to the movie\'s treasure spot, "The Big W". The Sylvester Stallone film Rocky III (1982) shows Rocky Balboa and Apollo Creed training to fight Clubber Lang by running on the Santa Monica Beach, and Stallone\'s Demolition Man (1993) includes Santa Monica settings. Henry Jaglom\'s indie Someone to Love (1987), the last film in which Orson Welles appeared, takes place in Santa Monica\'s venerable Mayfair Theatre. Heathers (1989) used Santa Monica\'s John Adams Middle School for many exterior shots. The Truth About Cats & Dogs (1996) is set entirely in Santa Monica, particularly the Palisades Park area, and features a radio station that resembles KCRW at Santa Monica College. 17 Again (2009) was shot at Samohi. Other films that show significant exterior shots of Santa Monica include Fletch (1985), Species (1995), Get Shorty (1995), and Ocean\'s Eleven (2001). Richard Rossi\'s biopic Aimee Semple McPherson opens and closes at the beach in Santa Monica. Iron Man features the Santa Monica pier and surrounding communities as Tony Stark tests his experimental flight suit.', 'Hundreds of movies have been shot or set in part within the city of Santa Monica. One of the oldest exterior shots in Santa Monica is Buster Keaton\'s Spite Marriage (1929) which shows much of 2nd Street. The comedy It\'s a Mad, Mad, Mad, Mad World (1963) included several scenes shot in Santa Monica, including those along the California Incline, which led to the movie\'s treasure spot, "The Big W". The Sylvester Stallone film Rocky III (1982) shows Rocky Balboa and Apollo Creed training to fight Clubber Lang by running on the Santa Monica Beach, and Stallone\'s Demolition Man (1993) includes Santa Monica settings. Henry Jaglom\'s indie Someone to Love (1987), the last film in which Orson Welles appeared, takes place in Santa Monica\'s venerable Mayfair Theatre. Heathers (1989) used Santa Monica\'s John Adams Middle School for many exterior shots. The Truth About Cats & Dogs (1996) is set entirely in Santa Monica, particularly the Palisades Park area, and features a radio station that resembles KCRW at Santa Monica College. 17 Again (2009) was shot at Samohi. Other films that show significant exterior shots of Santa Monica include Fletch (1985), Species (1995), Get Shorty (1995), and Ocean\'s Eleven (2001). Richard Rossi\'s biopic Aimee Semple McPherson opens and closes at the beach in Santa Monica. Iron Man features the Santa Monica pier and surrounding communities as Tony Stark tests his experimental flight suit.', 'Hundreds of movies have been shot or set in part within the city of Santa Monica. One of the oldest exterior shots in Santa Monica is Buster Keaton\'s Spite Marriage (1929) which shows much of 2nd Street. The comedy It\'s a Mad, Mad, Mad, Mad World (1963) included several scenes shot in Santa Monica, including those along the California Incline, which led to the movie\'s treasure spot, "The Big W". The Sylvester Stallone film Rocky III (1982) shows Rocky Balboa and Apollo Creed training to fight Clubber Lang by running on the Santa Monica Beach, and Stallone\'s Demolition Man (1993) includes Santa Monica settings. Henry Jaglom\'s indie Someone to Love (1987), the last film in which Orson Welles appeared, takes place in Santa Monica\'s venerable Mayfair Theatre. Heathers (1989) used Santa Monica\'s John Adams Middle School for many exterior shots. The Truth About Cats & Dogs (1996) is set entirely in Santa Monica, particularly the Palisades Park area, and features a radio station that resembles KCRW at Santa Monica College. 17 Again (2009) was shot at Samohi. Other films that show significant exterior shots of Santa Monica include Fletch (1985), Species (1995), Get Shorty (1995), and Ocean\'s Eleven (2001). Richard Rossi\'s biopic Aimee Semple McPherson opens and closes at the beach in Santa Monica. Iron Man features the Santa Monica pier and surrounding communities as Tony Stark tests his experimental flight suit.', "Santa Monica is featured in the video games True Crime: Streets of LA (2003), Vampire: The Masquerade – Bloodlines (2004), Grand Theft Auto San Andreas (2004) as a fictional district - Santa Maria Beach, Destroy All Humans! (2004), Tony Hawk's American Wasteland (2005), L.A. Rush (2005), Midnight Club: Los Angeles (2008), Cars Race-O-Rama (2009), Grand Theft Auto V (2013) as a fictional district – Del Perro, Call of Duty: Ghosts (2013) as a fictional U.S. military base – Fort Santa Monica, The Crew (2014), Need for Speed (2015)", "Santa Monica is featured in the video games True Crime: Streets of LA (2003), Vampire: The Masquerade – Bloodlines (2004), Grand Theft Auto San Andreas (2004) as a fictional district - Santa Maria Beach, Destroy All Humans! (2004), Tony Hawk's American Wasteland (2005), L.A. Rush (2005), Midnight Club: Los Angeles (2008), Cars Race-O-Rama (2009), Grand Theft Auto V (2013) as a fictional district – Del Perro, Call of Duty: Ghosts (2013) as a fictional U.S. military base – Fort Santa Monica, The Crew (2014), Need for Speed (2015)", "Santa Monica is featured in the video games True Crime: Streets of LA (2003), Vampire: The Masquerade – Bloodlines (2004), Grand Theft Auto San Andreas (2004) as a fictional district - Santa Maria Beach, Destroy All Humans! (2004), Tony Hawk's American Wasteland (2005), L.A. Rush (2005), Midnight Club: Los Angeles (2008), Cars Race-O-Rama (2009), Grand Theft Auto V (2013) as a fictional district – Del Perro, Call of Duty: Ghosts (2013) as a fictional U.S. military base – Fort Santa Monica, The Crew (2014), Need for Speed (2015)", "Santa Monica is featured in the video games True Crime: Streets of LA (2003), Vampire: The Masquerade – Bloodlines (2004), Grand Theft Auto San Andreas (2004) as a fictional district - Santa Maria Beach, Destroy All Humans! (2004), Tony Hawk's American Wasteland (2005), L.A. Rush (2005), Midnight Club: Los Angeles (2008), Cars Race-O-Rama (2009), Grand Theft Auto V (2013) as a fictional district – Del Perro, Call of Duty: Ghosts (2013) as a fictional U.S. military base – Fort Santa Monica, The Crew (2014), Need for Speed (2015)", "Santa Monica is featured in the video games True Crime: Streets of LA (2003), Vampire: The Masquerade – Bloodlines (2004), Grand Theft Auto San Andreas (2004) as a fictional district - Santa Maria Beach, Destroy All Humans! (2004), Tony Hawk's American Wasteland (2005), L.A. Rush (2005), Midnight Club: Los Angeles (2008), Cars Race-O-Rama (2009), Grand Theft Auto V (2013) as a fictional district – Del Perro, Call of Duty: Ghosts (2013) as a fictional U.S. military base – Fort Santa Monica, The Crew (2014), Need for Speed (2015)", "Washington University in St. Louis (Wash. U., or WUSTL) is a private research university located in St. Louis, Missouri, United States. Founded in 1853, and named after George Washington, the university has students and faculty from all 50 U.S. states and more than 120 countries. Twenty-five Nobel laureates have been affiliated with Washington University, nine having done the major part of their pioneering research at the university. Washington University's undergraduate program is ranked 15th by U.S. News and World Report. The university is ranked 32nd in the world by the Academic Ranking of World Universities.", "Washington University in St. Louis (Wash. U., or WUSTL) is a private research university located in St. Louis, Missouri, United States. Founded in 1853, and named after George Washington, the university has students and faculty from all 50 U.S. states and more than 120 countries. Twenty-five Nobel laureates have been affiliated with Washington University, nine having done the major part of their pioneering research at the university. Washington University's undergraduate program is ranked 15th by U.S. News and World Report. The university is ranked 32nd in the world by the Academic Ranking of World Universities.", "Washington University in St. Louis (Wash. U., or WUSTL) is a private research university located in St. Louis, Missouri, United States. Founded in 1853, and named after George Washington, the university has students and faculty from all 50 U.S. states and more than 120 countries. Twenty-five Nobel laureates have been affiliated with Washington University, nine having done the major part of their pioneering research at the university. Washington University's undergraduate program is ranked 15th by U.S. News and World Report. The university is ranked 32nd in the world by the Academic Ranking of World Universities.", "Washington University in St. Louis (Wash. U., or WUSTL) is a private research university located in St. Louis, Missouri, United States. Founded in 1853, and named after George Washington, the university has students and faculty from all 50 U.S. states and more than 120 countries. Twenty-five Nobel laureates have been affiliated with Washington University, nine having done the major part of their pioneering research at the university. Washington University's undergraduate program is ranked 15th by U.S. News and World Report. The university is ranked 32nd in the world by the Academic Ranking of World Universities.", "Washington University in St. Louis (Wash. U., or WUSTL) is a private research university located in St. Louis, Missouri, United States. Founded in 1853, and named after George Washington, the university has students and faculty from all 50 U.S. states and more than 120 countries. Twenty-five Nobel laureates have been affiliated with Washington University, nine having done the major part of their pioneering research at the university. Washington University's undergraduate program is ranked 15th by U.S. News and World Report. The university is ranked 32nd in the world by the Academic Ranking of World Universities.", "The university's first chancellor was Joseph Gibson Hoyt. Crow secured the university charter from the Missouri General Assembly in 1853, and Eliot was named President of the Board of Trustees. Early on, Eliot solicited support from members of the local business community, including John O'Fallon, but Eliot failed to secure a permanent endowment. Washington University is unusual among major American universities in not having had a prior financial endowment. The institution had no backing of a religious organization, single wealthy patron, or earmarked government support.", "The university's first chancellor was Joseph Gibson Hoyt. Crow secured the university charter from the Missouri General Assembly in 1853, and Eliot was named President of the Board of Trustees. Early on, Eliot solicited support from members of the local business community, including John O'Fallon, but Eliot failed to secure a permanent endowment. Washington University is unusual among major American universities in not having had a prior financial endowment. The institution had no backing of a religious organization, single wealthy patron, or earmarked government support.", "The university's first chancellor was Joseph Gibson Hoyt. Crow secured the university charter from the Missouri General Assembly in 1853, and Eliot was named President of the Board of Trustees. Early on, Eliot solicited support from members of the local business community, including John O'Fallon, but Eliot failed to secure a permanent endowment. Washington University is unusual among major American universities in not having had a prior financial endowment. The institution had no backing of a religious organization, single wealthy patron, or earmarked government support.", "The university's first chancellor was Joseph Gibson Hoyt. Crow secured the university charter from the Missouri General Assembly in 1853, and Eliot was named President of the Board of Trustees. Early on, Eliot solicited support from members of the local business community, including John O'Fallon, but Eliot failed to secure a permanent endowment. Washington University is unusual among major American universities in not having had a prior financial endowment. The institution had no backing of a religious organization, single wealthy patron, or earmarked government support.", "The university's first chancellor was Joseph Gibson Hoyt. Crow secured the university charter from the Missouri General Assembly in 1853, and Eliot was named President of the Board of Trustees. Early on, Eliot solicited support from members of the local business community, including John O'Fallon, but Eliot failed to secure a permanent endowment. Washington University is unusual among major American universities in not having had a prior financial endowment. The institution had no backing of a religious organization, single wealthy patron, or earmarked government support.", 'During the three years following its inception, the university bore three different names. The board first approved "Eliot Seminary," but William Eliot was uncomfortable with naming a university after himself and objected to the establishment of a seminary, which would implicitly be charged with teaching a religious faith. He favored a nonsectarian university. In 1854, the Board of Trustees changed the name to "Washington Institute" in honor of George Washington. Naming the University after the nation\'s first president, only seven years before the American Civil War and during a time of bitter national division, was no coincidence. During this time of conflict, Americans universally admired George Washington as the father of the United States and a symbol of national unity. The Board of Trustees believed that the university should be a force of unity in a strongly divided Missouri. In 1856, the University amended its name to "Washington University." The university amended its name once more in 1976, when the Board of Trustees voted to add the suffix "in St. Louis" to distinguish the university from the nearly two dozen other universities bearing Washington\'s name.', 'During the three years following its inception, the university bore three different names. The board first approved "Eliot Seminary," but William Eliot was uncomfortable with naming a university after himself and objected to the establishment of a seminary, which would implicitly be charged with teaching a religious faith. He favored a nonsectarian university. In 1854, the Board of Trustees changed the name to "Washington Institute" in honor of George Washington. Naming the University after the nation\'s first president, only seven years before the American Civil War and during a time of bitter national division, was no coincidence. During this time of conflict, Americans universally admired George Washington as the father of the United States and a symbol of national unity. The Board of Trustees believed that the university should be a force of unity in a strongly divided Missouri. In 1856, the University amended its name to "Washington University." The university amended its name once more in 1976, when the Board of Trustees voted to add the suffix "in St. Louis" to distinguish the university from the nearly two dozen other universities bearing Washington\'s name.', 'During the three years following its inception, the university bore three different names. The board first approved "Eliot Seminary," but William Eliot was uncomfortable with naming a university after himself and objected to the establishment of a seminary, which would implicitly be charged with teaching a religious faith. He favored a nonsectarian university. In 1854, the Board of Trustees changed the name to "Washington Institute" in honor of George Washington. Naming the University after the nation\'s first president, only seven years before the American Civil War and during a time of bitter national division, was no coincidence. During this time of conflict, Americans universally admired George Washington as the father of the United States and a symbol of national unity. The Board of Trustees believed that the university should be a force of unity in a strongly divided Missouri. In 1856, the University amended its name to "Washington University." The university amended its name once more in 1976, when the Board of Trustees voted to add the suffix "in St. Louis" to distinguish the university from the nearly two dozen other universities bearing Washington\'s name.', 'During the three years following its inception, the university bore three different names. The board first approved "Eliot Seminary," but William Eliot was uncomfortable with naming a university after himself and objected to the establishment of a seminary, which would implicitly be charged with teaching a religious faith. He favored a nonsectarian university. In 1854, the Board of Trustees changed the name to "Washington Institute" in honor of George Washington. Naming the University after the nation\'s first president, only seven years before the American Civil War and during a time of bitter national division, was no coincidence. During this time of conflict, Americans universally admired George Washington as the father of the United States and a symbol of national unity. The Board of Trustees believed that the university should be a force of unity in a strongly divided Missouri. In 1856, the University amended its name to "Washington University." The university amended its name once more in 1976, when the Board of Trustees voted to add the suffix "in St. Louis" to distinguish the university from the nearly two dozen other universities bearing Washington\'s name.', 'During the three years following its inception, the university bore three different names. The board first approved "Eliot Seminary," but William Eliot was uncomfortable with naming a university after himself and objected to the establishment of a seminary, which would implicitly be charged with teaching a religious faith. He favored a nonsectarian university. In 1854, the Board of Trustees changed the name to "Washington Institute" in honor of George Washington. Naming the University after the nation\'s first president, only seven years before the American Civil War and during a time of bitter national division, was no coincidence. During this time of conflict, Americans universally admired George Washington as the father of the United States and a symbol of national unity. The Board of Trustees believed that the university should be a force of unity in a strongly divided Missouri. In 1856, the University amended its name to "Washington University." The university amended its name once more in 1976, when the Board of Trustees voted to add the suffix "in St. Louis" to distinguish the university from the nearly two dozen other universities bearing Washington\'s name.', 'Although chartered as a university, for many years Washington University functioned primarily as a night school located on 17th Street and Washington Avenue in the heart of downtown St. Louis. Owing to limited financial resources, Washington University initially used public buildings. Classes began on October 22, 1854, at the Benton School building. At first the university paid for the evening classes, but as their popularity grew, their funding was transferred to the St. Louis Public Schools. Eventually the board secured funds for the construction of Academic Hall and a half dozen other buildings. Later the university divided into three departments: the Manual Training School, Smith Academy, and the Mary Institute.', 'Although chartered as a university, for many years Washington University functioned primarily as a night school located on 17th Street and Washington Avenue in the heart of downtown St. Louis. Owing to limited financial resources, Washington University initially used public buildings. Classes began on October 22, 1854, at the Benton School building. At first the university paid for the evening classes, but as their popularity grew, their funding was transferred to the St. Louis Public Schools. Eventually the board secured funds for the construction of Academic Hall and a half dozen other buildings. Later the university divided into three departments: the Manual Training School, Smith Academy, and the Mary Institute.', 'Although chartered as a university, for many years Washington University functioned primarily as a night school located on 17th Street and Washington Avenue in the heart of downtown St. Louis. Owing to limited financial resources, Washington University initially used public buildings. Classes began on October 22, 1854, at the Benton School building. At first the university paid for the evening classes, but as their popularity grew, their funding was transferred to the St. Louis Public Schools. Eventually the board secured funds for the construction of Academic Hall and a half dozen other buildings. Later the university divided into three departments: the Manual Training School, Smith Academy, and the Mary Institute.', 'Although chartered as a university, for many years Washington University functioned primarily as a night school located on 17th Street and Washington Avenue in the heart of downtown St. Louis. Owing to limited financial resources, Washington University initially used public buildings. Classes began on October 22, 1854, at the Benton School building. At first the university paid for the evening classes, but as their popularity grew, their funding was transferred to the St. Louis Public Schools. Eventually the board secured funds for the construction of Academic Hall and a half dozen other buildings. Later the university divided into three departments: the Manual Training School, Smith Academy, and the Mary Institute.', 'Although chartered as a university, for many years Washington University functioned primarily as a night school located on 17th Street and Washington Avenue in the heart of downtown St. Louis. Owing to limited financial resources, Washington University initially used public buildings. Classes began on October 22, 1854, at the Benton School building. At first the university paid for the evening classes, but as their popularity grew, their funding was transferred to the St. Louis Public Schools. Eventually the board secured funds for the construction of Academic Hall and a half dozen other buildings. Later the university divided into three departments: the Manual Training School, Smith Academy, and the Mary Institute.', "In 1867, the university opened the first private nonsectarian law school west of the Mississippi River. By 1882, Washington University had expanded to numerous departments, which were housed in various buildings across St. Louis. Medical classes were first held at Washington University in 1891 after the St. Louis Medical College decided to affiliate with the University, establishing the School of Medicine. During the 1890s, Robert Sommers Brookings, the president of the Board of Trustees, undertook the tasks of reorganizing the university's finances, putting them onto a sound foundation, and buying land for a new campus.", "In 1867, the university opened the first private nonsectarian law school west of the Mississippi River. By 1882, Washington University had expanded to numerous departments, which were housed in various buildings across St. Louis. Medical classes were first held at Washington University in 1891 after the St. Louis Medical College decided to affiliate with the University, establishing the School of Medicine. During the 1890s, Robert Sommers Brookings, the president of the Board of Trustees, undertook the tasks of reorganizing the university's finances, putting them onto a sound foundation, and buying land for a new campus.", "In 1867, the university opened the first private nonsectarian law school west of the Mississippi River. By 1882, Washington University had expanded to numerous departments, which were housed in various buildings across St. Louis. Medical classes were first held at Washington University in 1891 after the St. Louis Medical College decided to affiliate with the University, establishing the School of Medicine. During the 1890s, Robert Sommers Brookings, the president of the Board of Trustees, undertook the tasks of reorganizing the university's finances, putting them onto a sound foundation, and buying land for a new campus.", "In 1867, the university opened the first private nonsectarian law school west of the Mississippi River. By 1882, Washington University had expanded to numerous departments, which were housed in various buildings across St. Louis. Medical classes were first held at Washington University in 1891 after the St. Louis Medical College decided to affiliate with the University, establishing the School of Medicine. During the 1890s, Robert Sommers Brookings, the president of the Board of Trustees, undertook the tasks of reorganizing the university's finances, putting them onto a sound foundation, and buying land for a new campus.", 'Washington University spent its first half century in downtown St. Louis bounded by Washington Ave., Lucas Place, and Locust Street. By the 1890s, owing to the dramatic expansion of the Manual School and a new benefactor in Robert Brookings, the University began to move west. The University Board of Directors began a process to find suitable ground and hired the landscape architecture firm Olmsted, Olmsted & Eliot of Boston. A committee of Robert S. Brookings, Henry Ware Eliot, and William Huse found a site of 103 acres (41.7 ha) just beyond Forest Park, located west of the city limits in St. Louis County. The elevation of the land was thought to resemble the Acropolis and inspired the nickname of "Hilltop" campus, renamed the Danforth campus in 2006 to honor former chancellor William H. Danforth.', 'Washington University spent its first half century in downtown St. Louis bounded by Washington Ave., Lucas Place, and Locust Street. By the 1890s, owing to the dramatic expansion of the Manual School and a new benefactor in Robert Brookings, the University began to move west. The University Board of Directors began a process to find suitable ground and hired the landscape architecture firm Olmsted, Olmsted & Eliot of Boston. A committee of Robert S. Brookings, Henry Ware Eliot, and William Huse found a site of 103 acres (41.7 ha) just beyond Forest Park, located west of the city limits in St. Louis County. The elevation of the land was thought to resemble the Acropolis and inspired the nickname of "Hilltop" campus, renamed the Danforth campus in 2006 to honor former chancellor William H. Danforth.', 'Washington University spent its first half century in downtown St. Louis bounded by Washington Ave., Lucas Place, and Locust Street. By the 1890s, owing to the dramatic expansion of the Manual School and a new benefactor in Robert Brookings, the University began to move west. The University Board of Directors began a process to find suitable ground and hired the landscape architecture firm Olmsted, Olmsted & Eliot of Boston. A committee of Robert S. Brookings, Henry Ware Eliot, and William Huse found a site of 103 acres (41.7 ha) just beyond Forest Park, located west of the city limits in St. Louis County. The elevation of the land was thought to resemble the Acropolis and inspired the nickname of "Hilltop" campus, renamed the Danforth campus in 2006 to honor former chancellor William H. Danforth.', 'Washington University spent its first half century in downtown St. Louis bounded by Washington Ave., Lucas Place, and Locust Street. By the 1890s, owing to the dramatic expansion of the Manual School and a new benefactor in Robert Brookings, the University began to move west. The University Board of Directors began a process to find suitable ground and hired the landscape architecture firm Olmsted, Olmsted & Eliot of Boston. A committee of Robert S. Brookings, Henry Ware Eliot, and William Huse found a site of 103 acres (41.7 ha) just beyond Forest Park, located west of the city limits in St. Louis County. The elevation of the land was thought to resemble the Acropolis and inspired the nickname of "Hilltop" campus, renamed the Danforth campus in 2006 to honor former chancellor William H. Danforth.', 'Washington University spent its first half century in downtown St. Louis bounded by Washington Ave., Lucas Place, and Locust Street. By the 1890s, owing to the dramatic expansion of the Manual School and a new benefactor in Robert Brookings, the University began to move west. The University Board of Directors began a process to find suitable ground and hired the landscape architecture firm Olmsted, Olmsted & Eliot of Boston. A committee of Robert S. Brookings, Henry Ware Eliot, and William Huse found a site of 103 acres (41.7 ha) just beyond Forest Park, located west of the city limits in St. Louis County. The elevation of the land was thought to resemble the Acropolis and inspired the nickname of "Hilltop" campus, renamed the Danforth campus in 2006 to honor former chancellor William H. Danforth.', "In 1899, the university opened a national design contest for the new campus. The renowned Philadelphia firm Cope & Stewardson won unanimously with its plan for a row of Collegiate Gothic quadrangles inspired by Oxford and Cambridge Universities. The cornerstone of the first building, Busch Hall, was laid on October 20, 1900. The construction of Brookings Hall, Ridgley, and Cupples began shortly thereafter. The school delayed occupying these buildings until 1905 to accommodate the 1904 World's Fair and Olympics. The delay allowed the university to construct ten buildings instead of the seven originally planned. This original cluster of buildings set a precedent for the development of the Danforth Campus; Cope & Stewardson’s original plan and its choice of building materials have, with few exceptions, guided the construction and expansion of the Danforth Campus to the present day.", "In 1899, the university opened a national design contest for the new campus. The renowned Philadelphia firm Cope & Stewardson won unanimously with its plan for a row of Collegiate Gothic quadrangles inspired by Oxford and Cambridge Universities. The cornerstone of the first building, Busch Hall, was laid on October 20, 1900. The construction of Brookings Hall, Ridgley, and Cupples began shortly thereafter. The school delayed occupying these buildings until 1905 to accommodate the 1904 World's Fair and Olympics. The delay allowed the university to construct ten buildings instead of the seven originally planned. This original cluster of buildings set a precedent for the development of the Danforth Campus; Cope & Stewardson’s original plan and its choice of building materials have, with few exceptions, guided the construction and expansion of the Danforth Campus to the present day."]

In [9]:
tokenizer.pad_token = tokenizer.eos_token

In [14]:
for q,c in zip(questions,contexts):
    q =q.strip()
    c = c.strip()
    tokenizer(
                q,
                c,
                truncation="only_second" ,
                max_length=384,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length" ,
            )

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       